In [1]:
import pandas as pd
import os
import json
from datetime import datetime
import subprocess

def procesar_excel_y_actualizar_html():
    base_path = r"C:\Users\Jorge Vasquez\Ranking"
    ruta_excel = os.path.join(base_path, "CS_AVANCE DE ASESORES.xlsx")
    ruta_excel_asistencias = os.path.join(base_path, "CS_MARCACIONES.xlsx")
    ruta_html = os.path.join(base_path, "index.html")
    
    # Obtener todas las hojas disponibles del Excel
    excel_file = pd.ExcelFile(ruta_excel)
    hojas_disponibles = excel_file.sheet_names
    
    # Procesar cada hoja
    meses_validos = []
    datos_cumpleaños_global = []
    
    for hoja in hojas_disponibles:
        # Procesar hoja de cumpleaños
        if hoja == "INF":
            print(f"\n🎂 PROCESANDO HOJA DE CUMPLEAÑOS: {hoja}")
            df_inf = pd.read_excel(ruta_excel, sheet_name=hoja, header=None)
            datos_cumpleaños_global = extraer_cumpleaños_de_inf(df_inf)
            continue
        
        # Procesar hojas mensuales
        if '_' in hoja:
            meses_validos.append(hoja)
    
    # Extraer años y meses únicos
    años_unicos = sorted(set(hoja.split('_')[1] for hoja in meses_validos), reverse=True)
    meses_unicos = ['ENERO', 'FEBRERO', 'MARZO', 'ABRIL', 'MAYO', 'JUNIO', 
                   'JULIO', 'AGOSTO', 'SETIEMBRE', 'OCTUBRE', 'NOVIEMBRE', 'DICIEMBRE']
    
    # Procesar datos para cada mes
    todos_los_datos = {}
    datos_supervisores_completos = {}
    todos_los_asesores = set()
    dias_laborables_por_mes = {}
    datos_cumpleaños_por_mes = {}
    base_asesores_analisis = {}
    
    for hoja in meses_validos:
        print(f"\nProcesando hoja: {hoja}")
        df = pd.read_excel(ruta_excel, sheet_name=hoja, header=None)
        
        # Extraer días laborables (posición fija L3)
        dias_laborables = int(df.iloc[2, 11])
        dias_laborables_por_mes[hoja] = dias_laborables
        
        # Extraer datos de la hoja mensual (lo que ya tienes)
        resultado = extraer_datos_completos(df, hoja)
        asesores_data = resultado['asesores']
        datos_supervisores = resultado['supervisores']
    
        # ✅ NUEVO: Base oficial desde A2:I.. (B2 = ASESOR_CRR)
        base_mes = extraer_base_asesores_desde_b2(df)
        base_asesores_analisis[hoja] = base_mes
        
        todos_los_datos[hoja] = asesores_data
        datos_supervisores_completos[hoja] = datos_supervisores
        datos_cumpleaños_por_mes[hoja] = datos_cumpleaños_global
        
        # Agregar asesores a la lista global
        for asesor in asesores_data:
            todos_los_asesores.add(asesor['nombre'])
    
    # Procesar datos por supervisor
    datos_por_supervisor = agrupar_por_supervisor(todos_los_datos, datos_supervisores_completos)
    lista_supervisores = sorted(set(datos_por_supervisor.keys()))
    
    # Procesar datos de asistencias
    datos_asistencias = procesar_asistencias_excel(ruta_excel_asistencias, meses_validos)
    
    # Generar HTML
    generar_html_modular(todos_los_datos, ruta_html, meses_unicos, años_unicos, meses_validos, 
                        sorted(todos_los_asesores), datos_por_supervisor, lista_supervisores,
                        datos_asistencias, dias_laborables_por_mes, datos_cumpleaños_por_mes,
                        base_asesores_analisis)

def extraer_base_asesores_desde_b2(df):
    """
    Lee el bloque A2:I.. donde en B2 está el encabezado ASESOR_CRR.
    Retorna lista de dicts: alias_crr, dni, fecha_ingreso, supervisor
    Columnas (0-based): A=0, B=1, E=4, H=7, I=8
    """
    base = []

    # Validar que en B2 (fila 1, col 1) esté el header
    if df.shape[0] < 3 or df.shape[1] < 9:
        return base

    header_b2 = str(df.iloc[1, 1]).strip() if pd.notna(df.iloc[1, 1]) else ""
    if header_b2.upper() != "ASESOR_CRR":
        return base

    # Data empieza en fila 3 (index 2)
    for i in range(2, len(df)):
        alias = str(df.iloc[i, 1]).strip() if pd.notna(df.iloc[i, 1]) else ""
        if not alias:
            break  # se corta al primer vacío

        supervisor = str(df.iloc[i, 4]).strip() if pd.notna(df.iloc[i, 4]) else ""
        dni = str(df.iloc[i, 7]).strip() if pd.notna(df.iloc[i, 7]) else ""
        fecha_ingreso = str(df.iloc[i, 8]).strip() if pd.notna(df.iloc[i, 8]) else ""
        alcance = df.iloc[i, 3] if df.shape[1] > 3 else None
        recupero = df.iloc[i, 6] if df.shape[1] > 6 else None
        meta = df.iloc[i, 2] if df.shape[1] > 2 else None

        base.append({
            "alias_crr": alias,
            "dni": dni,
            "fecha_ingreso": fecha_ingreso,
            "supervisor": supervisor,
            "alcance": float(alcance) if pd.notna(alcance) else None,
            "recupero": float(recupero) if pd.notna(recupero) else 0,
            "meta": float(meta) if pd.notna(meta) else 0
        })

    return base

def extraer_cumpleaños_de_inf(df_inf):
    """Extrae datos de cumpleaños específicamente de la hoja INF"""
    
    datos_cumpleaños = []
    
    # POSICIONES FIJAS - según tu estructura exacta
    col_alias = 0       # Columna A - Alias (se mantiene para referencia)
    col_puesto_real = 1 # Columna B - PUESTO REAL (NUEVO)
    col_nombre = 3      # Columna D - Nombre completo
    col_puesto = 4      # Columna E - PUESTO (ASESOR/STAFF)
    col_mes = 5         # Columna F - Mes
    col_dia = 6         # Columna G - Día
    
    # Saltar 2 encabezados: grupo y FEC_NAC
    fila_inicio = 2  # Fila 3 en Excel (0-indexed: 0=header1, 1=header2, 2=primer dato)
    
    for idx in range(fila_inicio, len(df_inf)):
        row = df_inf.iloc[idx]
        
        # Extracción directa
        alias = str(row[col_alias])
        puesto_real = str(row[col_puesto_real]) if col_puesto_real < len(row) else ""  # NUEVO
        nombre = str(row[col_nombre])
        puesto = str(row[col_puesto])  # Ya viene como "ASESOR" o "STAFF"
        mes = str(row[col_mes])        # Ya viene como "ENERO", "FEBRERO", etc.
        dia = int(row[col_dia])        # Ya viene como número entero
        
        # Crear registro - AGREGAR puesto_real
        cumple_data = {
            'alias': alias,
            'puesto_real': puesto_real,  # NUEVO
            'nombre': nombre,
            'puesto': puesto,
            'mes': mes,
            'dia': dia
        }
        
        datos_cumpleaños.append(cumple_data)
    
    return datos_cumpleaños

def agrupar_por_supervisor(todos_los_datos, datos_supervisores_completos):
    datos_por_supervisor = {}
    
    for mes, asesores in todos_los_datos.items():
        # Datos de supervisores para este mes (siempre existen)
        datos_supervisores_mes = datos_supervisores_completos[mes]
        
        # Agrupar asesores por supervisor
        supervisores_mes = {}
        for asesor in asesores:
            supervisor = asesor['supervisor']
            
            if supervisor not in supervisores_mes:
                # Datos directos del supervisor (siempre completos)
                sup_data = datos_supervisores_mes[supervisor]
                supervisores_mes[supervisor] = {
                    'recupero_supervisor': sup_data['recupero_supervisor'],
                    'alcance_supervisor': sup_data['alcance_supervisor'],
                    'meta_super': sup_data['meta_super'],
                    'cartera': sup_data['cartera'],
                    'datos_diarios_supervisor': sup_data['datos_diarios_supervisor'],
                    'asesores': []
                }
            
            supervisores_mes[supervisor]['asesores'].append(asesor)
        
        # Procesar cada supervisor
        for supervisor, datos_super in supervisores_mes.items():
            if supervisor not in datos_por_supervisor:
                datos_por_supervisor[supervisor] = {}
            
            # Cálculos directos
            total_meta_asesores = sum(asesor['meta'] for asesor in datos_super['asesores'])
            
            # Datos diarios del supervisor (siempre existen, al menos un día)
            datos_diarios_supervisor = datos_super['datos_diarios_supervisor']
            
            # Calcular alcance acumulado diario (siempre hay datos)
            fechas_ordenadas = sorted(datos_diarios_supervisor.keys(), 
                                     key=lambda x: convertir_fecha_diaria_a_orden(x))
            
            alcance_acumulado_diario = {}
            for fecha in fechas_ordenadas:
                recupero_acumulado = datos_diarios_supervisor[fecha]
                # meta_super siempre > 0
                alcance_acumulado_diario[fecha] = (recupero_acumulado / datos_super['meta_super']) * 100
            
            # Inicializar datos del mes
            datos_por_supervisor[supervisor][mes] = {
                'asesores': datos_super['asesores'],
                'total_recupero': datos_super['recupero_supervisor'],
                'total_meta': total_meta_asesores,
                'meta_super': datos_super['meta_super'],
                'porcentaje_promedio': datos_super['alcance_supervisor'],
                'porcentaje_vs_meta_super': datos_super['alcance_supervisor'],
                'cartera': datos_super['cartera'],
                'cantidad_asesores': len(datos_super['asesores']),
                'distribucion_categorias': {">100%": 0, ">70%": 0, ">40%": 0, ">0%": 0},
                'datos_diarios_supervisor': datos_diarios_supervisor,
                'alcance_acumulado_diario': alcance_acumulado_diario
            }
            
            # Distribución de categorías (siempre válidas)
            datos_mes = datos_por_supervisor[supervisor][mes]
            for asesor in datos_super['asesores']:
                datos_mes['distribucion_categorias'][asesor['clasificacion']] += 1
            
            # Clasificación del supervisor
            porcentaje = datos_mes['porcentaje_vs_meta_super']
            if porcentaje > 100:
                datos_mes['clasificacion'] = ">100%"
            elif porcentaje > 70:
                datos_mes['clasificacion'] = ">70%"
            elif porcentaje > 40:
                datos_mes['clasificacion'] = ">40%"
            else:
                datos_mes['clasificacion'] = ">0%"
    
    return datos_por_supervisor

def convertir_fecha_diaria_a_orden(fecha_str):
    """Convierte fecha en formato '3-Nov' a orden numérico"""
    try:
        # Separar día y mes abreviado
        partes = fecha_str.split('-')
        if len(partes) == 2:
            dia = int(partes[0])
            mes_abrev = partes[1].upper()
            
            # Mapear meses abreviados a números
            meses = {
                'ENE': 1, 'FEB': 2, 'MAR': 3, 'ABR': 4, 'MAY': 5, 'JUN': 6,
                'JUL': 7, 'AGO': 8, 'SEP': 9, 'OCT': 10, 'NOV': 11, 'DIC': 12
            }
            
            # Buscar coincidencias parciales
            for mes_key, mes_num in meses.items():
                if mes_abrev.startswith(mes_key[:3]):
                    return mes_num * 100 + dia
            
        return 0
    except:
        return 0
    
def extraer_datos_completos(df, nombre_hoja):
    """Extrae datos completos con detección inteligente de bloques"""
    
    fila_encabezados = 7
    row_7 = df.iloc[fila_encabezados]
    
    encabezados_columna = {}
    for i in range(len(row_7)):
        valor = row_7[i]
        if pd.notna(valor):
            encabezado = str(valor).strip()
            encabezados_columna[i] = encabezado
    
    # Encontrar separador entre las dos tablas
    col_meta_super_inicio = None
    for i, encabezado in encabezados_columna.items():
        if 'META SUPER' in encabezado.upper():
            col_meta_super_inicio = i
            break
    
    if col_meta_super_inicio is None:
        # Buscar la primera columna vacía como separador
        for i in range(len(row_7)):
            if pd.isna(row_7[i]):
                col_meta_super_inicio = i + 1
                break
    
    if col_meta_super_inicio is None:
        return {'asesores': [], 'supervisores': {}, 'cumpleaños': []}
    
    print(f"\n  📋 MAPEO DE COLUMNAS - TABLA 1 (ASESORES):")
    
    # TABLA 1: COLUMNAS DE ASESORES (primera mitad)
    col_al_ac = None
    col_clasificacion = None
    col_supervisor = None
    col_recupero = None
    col_meta = None
    col_aser = None
    columnas_fechas_asesores = {}  # {fecha_str: col_index}
    
    for i in range(col_meta_super_inicio):
        if i in encabezados_columna:
            encabezado = encabezados_columna[i].upper()
            
            if 'AL_AC' in encabezado or 'ALCANCE' in encabezado:
                col_al_ac = i
                print(f"    ✅ AL_AC/ALCANCE: col {i}")
            elif 'CLASIFICACIÓN' in encabezado or 'CLASIFICACION' in encabezado:
                col_clasificacion = i
                print(f"    ✅ CLASIFICACIÓN: col {i}")
            elif 'SUPERVISOR' in encabezado and 'ALCANCE' not in encabezado:
                col_supervisor = i
                print(f"    ✅ SUPERVISOR: col {i}")
            elif 'RECUPERO' in encabezado and 'SUPERVISOR' not in encabezado:
                col_recupero = i
                print(f"    ✅ RECUPERO: col {i}")
            elif 'META' in encabezado and 'SUPER' not in encabezado:
                col_meta = i
                print(f"    ✅ META: col {i}")
            elif 'ASER_' in encabezado:
                col_aser = i
                print(f"    ✅ ASER_: col {i}")
            elif '-' in encabezado:  # Fechas de asesores (ej: "1-Dic")
                fecha_str = encabezados_columna[i].strip()
                columnas_fechas_asesores[fecha_str] = i
                print(f"    📅 Fecha asesor: '{fecha_str}' (col {i})")
    
    print(f"  📊 Total fechas asesores: {len(columnas_fechas_asesores)}")
    
    print(f"\n  📋 MAPEO DE COLUMNAS - TABLA 2 (SUPERVISORES):")
    
    # TABLA 2: COLUMNAS DE SUPERVISORES (segunda mitad)
    col_meta_super = None
    col_recupero_supervisor = None
    col_alcance_supervisor = None
    col_super = None  # NOMBRE DEL SUPERVISOR
    col_cartera = None
    columnas_fechas_supervisores = {}  # {fecha_str: col_index}
    
    for i in range(col_meta_super_inicio, len(row_7)):
        if i in encabezados_columna:
            encabezado = encabezados_columna[i].upper()
            
            if 'META SUPER' in encabezado:
                col_meta_super = i
                print(f"    ✅ META SUPER: col {i}")
            elif 'RECUPERO SUPERVISOR' in encabezado:
                col_recupero_supervisor = i
                print(f"    ✅ RECUPERO SUPERVISOR: col {i}")
            elif 'ALCANCE SUPERVISOR' in encabezado:
                col_alcance_supervisor = i
                print(f"    ✅ ALCANCE SUPERVISOR: col {i}")
            elif 'SUPER' in encabezado and i != col_meta_super:
                col_super = i
                print(f"    ✅ SUPER (nombre supervisor): col {i}")
            elif 'CARTERA' in encabezado:
                col_cartera = i
                print(f"    ✅ CARTERA: col {i}")
            elif '-' in encabezado:  # Fechas de supervisores (ej: "3-Nov")
                fecha_str = encabezados_columna[i].strip()
                columnas_fechas_supervisores[fecha_str] = i
                print(f"    📅 Fecha supervisor: '{fecha_str}' (col {i})")
    
    print(f"  📊 Total fechas supervisores: {len(columnas_fechas_supervisores)}")
    
    # VALIDAR COLUMNAS ESENCIALES
    columnas_faltantes = []
    if col_aser is None: columnas_faltantes.append("ASER_ (NOMBRE ASESOR)")
    if col_super is None: columnas_faltantes.append("SUPER (NOMBRE SUPERVISOR)")
    
    if columnas_faltantes:
        print(f"  ⚠️  Columnas faltantes: {', '.join(columnas_faltantes)}")
    
    # ========== PASO 1: EXTRAER DATOS DE SUPERVISORES (TABLA 2) ==========
    print(f"\n  🔍 Extrayendo datos de SUPERVISORES (Tabla 2)...")
    
    datos_supervisores = {}
    start_data_row = 8  # Fila 9 en Excel (1-index)
    total_filas = len(df)
    
    for idx in range(start_data_row, total_filas):
        row = df.iloc[idx]
        
        # Verificar si esta fila tiene un NOMBRE DE SUPERVISOR en la columna SUPER
        if col_super is not None and col_super < len(row) and pd.notna(row[col_super]):
            nombre_supervisor = str(row[col_super]).strip()
            
            if nombre_supervisor and nombre_supervisor not in ['', 'NaN', 'None', 'nan']:
                supervisor_data = {}
                
                # Extraer META SUPER
                if col_meta_super is not None and col_meta_super < len(row):
                    try:
                        valor = row[col_meta_super]
                        supervisor_data['meta_super'] = float(valor) if pd.notna(valor) else 0
                    except:
                        supervisor_data['meta_super'] = 0
                else:
                    supervisor_data['meta_super'] = 0
                
                # Extraer RECUPERO SUPERVISOR
                if col_recupero_supervisor is not None and col_recupero_supervisor < len(row):
                    try:
                        valor = row[col_recupero_supervisor]
                        supervisor_data['recupero_supervisor'] = float(valor) if pd.notna(valor) else 0
                    except:
                        supervisor_data['recupero_supervisor'] = 0
                else:
                    supervisor_data['recupero_supervisor'] = 0
                
                # Extraer ALCANCE SUPERVISOR
                if col_alcance_supervisor is not None and col_alcance_supervisor < len(row):
                    try:
                        valor = row[col_alcance_supervisor]
                        supervisor_data['alcance_supervisor'] = convertir_porcentaje_excel(valor)
                    except:
                        supervisor_data['alcance_supervisor'] = 0
                else:
                    supervisor_data['alcance_supervisor'] = 0
                
                # Extraer CARTERA
                if col_cartera is not None and col_cartera < len(row):
                    try:
                        valor = row[col_cartera]
                        supervisor_data['cartera'] = str(valor).strip() if pd.notna(valor) else 'No definida'
                    except:
                        supervisor_data['cartera'] = 'No definida'
                else:
                    supervisor_data['cartera'] = 'No definida'
                
                # EXTRAER DATOS DIARIOS DEL SUPERVISOR (RECUPERO ACUMULADO)
                datos_diarios_supervisor = {}
                for fecha_str, col_idx in columnas_fechas_supervisores.items():
                    if col_idx < len(row):
                        try:
                            valor = row[col_idx]
                            if pd.notna(valor):
                                datos_diarios_supervisor[fecha_str] = float(valor)
                            else:
                                datos_diarios_supervisor[fecha_str] = 0
                        except:
                            datos_diarios_supervisor[fecha_str] = 0
                
                supervisor_data['datos_diarios_supervisor'] = datos_diarios_supervisor
                
                # Guardar datos del supervisor
                datos_supervisores[nombre_supervisor] = supervisor_data
    
    print(f"  📊 Total supervisores encontrados: {len(datos_supervisores)}")
    
    # ========== PASO 2: EXTRAER DATOS DE ASESORES (TABLA 1) ==========
    print(f"\n  🔍 Extrayendo datos de ASESORES (Tabla 1)...")
    
    asesores_data = []
    
    for idx in range(start_data_row, total_filas):
        row = df.iloc[idx]
        
        # Verificar si esta fila tiene un NOMBRE DE ASESOR (columna ASER_)
        if col_aser is not None and col_aser < len(row) and pd.notna(row[col_aser]):
            nombre_asesor = str(row[col_aser]).strip()
            
            if nombre_asesor and nombre_asesor not in ['', 'NaN', 'None', 'nan']:
                asesor_data = {}
                
                # Nombre del asesor
                asesor_data['nombre'] = nombre_asesor
                
                # Porcentaje (AL_AC)
                if col_al_ac is not None and col_al_ac < len(row):
                    porcentaje_valor = convertir_porcentaje_excel(row[col_al_ac])
                else:
                    porcentaje_valor = 0
                
                asesor_data['porcentaje'] = round(porcentaje_valor, 2)
                
                # Clasificación
                asesor_data['clasificacion'] = determinar_clasificacion(asesor_data['porcentaje'])
                asesor_data['alias_crr'] = str(row[1]).strip() if len(row) > 1 and pd.notna(row[1]) else ''
                asesor_data['dni'] = str(row[7]).strip() if len(row) > 7 and pd.notna(row[7]) else ''
                asesor_data['fecha_ingreso'] = str(row[8]).strip() if len(row) > 8 and pd.notna(row[8]) else ''
               
                # Supervisor del asesor
                supervisor_nombre = 'Sin Supervisor'
                if col_supervisor is not None and col_supervisor < len(row):
                    try:
                        valor = row[col_supervisor]
                        if pd.notna(valor):
                            supervisor_nombre = str(valor).strip()
                    except:
                        supervisor_nombre = 'Sin Supervisor'
                
                asesor_data['supervisor'] = supervisor_nombre
                if (asesor_data.get('supervisor') in [None, '', 'Sin Supervisor']) \
                   and len(row) > 4 and pd.notna(row[4]):
                    asesor_data['supervisor'] = str(row[4]).strip()
                
                # Recupero individual del asesor
                if col_recupero is not None and col_recupero < len(row):
                    try:
                        valor = row[col_recupero]
                        asesor_data['recupero'] = float(valor) if pd.notna(valor) else 0
                    except:
                        asesor_data['recupero'] = 0
                else:
                    asesor_data['recupero'] = 0
                
                # Meta individual del asesor
                if col_meta is not None and col_meta < len(row):
                    try:
                        valor = row[col_meta]
                        asesor_data['meta'] = float(valor) if pd.notna(valor) else 0
                    except:
                        asesor_data['meta'] = 0
                else:
                    asesor_data['meta'] = 0
                
                # Asignar datos del supervisor correspondiente (solo datos básicos)
                if supervisor_nombre in datos_supervisores:
                    supervisor_data = datos_supervisores[supervisor_nombre]
                    asesor_data['meta_super'] = supervisor_data.get('meta_super', 0)
                    asesor_data['recupero_supervisor'] = supervisor_data.get('recupero_supervisor', 0)
                    asesor_data['alcance_supervisor'] = supervisor_data.get('alcance_supervisor', 0)
                    asesor_data['cartera'] = supervisor_data.get('cartera', 'No definida')
                else:
                    asesor_data['meta_super'] = 0
                    asesor_data['recupero_supervisor'] = 0
                    asesor_data['alcance_supervisor'] = 0
                    asesor_data['cartera'] = 'No definida'
                
                # Datos diarios del asesor (opcional, para otras funcionalidades)
                datos_diarios_asesor = {}
                for fecha_str, col_idx in columnas_fechas_asesores.items():
                    if col_idx < len(row):
                        try:
                            valor = row[col_idx]
                            datos_diarios_asesor[fecha_str] = float(valor) if pd.notna(valor) else 0
                        except:
                            datos_diarios_asesor[fecha_str] = 0
                
                asesor_data['datos_diarios_asesor'] = datos_diarios_asesor
                
                asesores_data.append(asesor_data)
    
    print(f"  📊 Total asesores encontrados: {len(asesores_data)}")
    
    # ========== RETORNAR AMBAS ESTRUCTURAS SEPARADAS ==========
    return {
        'asesores': asesores_data,
        'supervisores': datos_supervisores,
    }

def determinar_clasificacion(porcentaje):
    """Determina la clasificación según el porcentaje"""
    if porcentaje > 100:
        return ">100%"
    elif porcentaje > 70:
        return ">70%"
    elif porcentaje > 40:
        return ">40%"
    else:
        return ">0%"

def convertir_porcentaje_excel(valor_crudo):
    """Convierte valores de porcentaje del Excel - Formato: 1.25 = 125%"""
    return float(valor_crudo) * 100

def procesar_asistencias_excel(ruta_excel, meses_validos_avance):
    """Procesa el archivo Excel de asistencias con la nueva estructura"""
    datos_asistencias = {}
    
    try:
        # Obtener todas las hojas disponibles del Excel de asistencias
        excel_file = pd.ExcelFile(ruta_excel)
        hojas_asistencias = excel_file.sheet_names
        
        # Filtrar hojas que coincidan con los meses del archivo de avance
        for hoja in hojas_asistencias:
            # Verificar si esta hoja está en los meses válidos del avance
            if any(hoja in mes for mes in meses_validos_avance):
                print(f"\n📊 Procesando asistencias de: {hoja}")
                
                # Leer los datos de la hoja
                df = pd.read_excel(ruta_excel, sheet_name=hoja)
                
                # Limpiar nombres de columnas
                df.columns = df.columns.str.strip()
                
                # Verificar columnas esperadas
                columnas_esperadas = ['EQUIPO', 'alias', 'fecha', 'Hora_Inicio', 
                                     'estadoSecundario', 'SUPERVISOR', 'TARDANZA', 'FECHA DE INGRESO']
                columnas_faltantes = [col for col in columnas_esperadas if col not in df.columns]
                
                if columnas_faltantes:
                    print(f"  ⚠️  Columnas faltantes en {hoja}: {', '.join(columnas_faltantes)}")
                    continue
                
                # Limpiar datos
                df['alias'] = df['alias'].astype(str).str.strip()
                df['EQUIPO'] = df['EQUIPO'].astype(str).str.strip()
                df['estadoSecundario'] = df['estadoSecundario'].astype(str).str.strip()
                df['SUPERVISOR'] = df['SUPERVISOR'].astype(str).str.strip()
                
                # Convertir fechas
                df['fecha'] = pd.to_datetime(df['fecha'], errors='coerce')
                df['FECHA DE INGRESO'] = pd.to_datetime(df['FECHA DE INGRESO'], errors='coerce')
                
                # Filtrar filas con alias válidos
                df = df[df['alias'].notna() & (df['alias'] != 'nan') & (df['alias'] != '')]
                
                # Procesar tardanzas (convertir a segundos totales)
                def tardanza_a_segundos(tiempo_str):
                    try:
                        if pd.isna(tiempo_str) or tiempo_str == '' or tiempo_str == '00:00:00':
                            return 0
                        # Formato HH:MM:SS
                        partes = str(tiempo_str).split(':')
                        if len(partes) >= 3:
                            horas = int(partes[0])
                            minutos = int(partes[1])
                            segundos = int(partes[2])
                            return horas * 3600 + minutos * 60 + segundos
                        return 0
                    except:
                        return 0
                
                df['TARDANZA_SEGUNDOS'] = df['TARDANZA'].apply(tardanza_a_segundos)
                
                # Determinar tipo de persona (asesor o supervisor)
                df['TIPO_PERSONA'] = df['SUPERVISOR'].apply(
                    lambda x: 'SUPERVISOR' if str(x).strip().upper() == 'SI' else 'ASESOR'
                )
                
                # Filtrar solo registros válidos (En la cola para asesores, Disponible para supervisores)
                df = df[
                    ((df['TIPO_PERSONA'] == 'ASESOR') & (df['estadoSecundario'] == 'En la cola')) |
                    ((df['TIPO_PERSONA'] == 'SUPERVISOR') & (df['estadoSecundario'] == 'Disponible'))
                ]
                
                # Agrupar por persona para calcular estadísticas
                asistencias_por_persona = {}
                
                # Obtener fechas únicas laborables (días con registros)
                fechas_unicas = sorted(df['fecha'].dropna().unique())
                dias_laborables_mes = len(fechas_unicas)
                
                for _, row in df.iterrows():
                    alias = row['alias']
                    equipo = row['EQUIPO']
                    fecha = row['fecha']
                    tipo_persona = row['TIPO_PERSONA']
                    fecha_ingreso = row['FECHA DE INGRESO']
                    tardanza_segundos = row['TARDANZA_SEGUNDOS']
                    
                    if pd.isna(fecha):
                        continue
                    
                    fecha_str = fecha.strftime('%Y-%m-%d')
                    
                    if alias not in asistencias_por_persona:
                        # Calcular días que debería haber trabajado (considerando fecha de ingreso)
                        dias_deberia_trabajar = dias_laborables_mes
                        if not pd.isna(fecha_ingreso):
                            # Si ingresó después del inicio del mes, calcular días desde ingreso
                            fecha_inicio_mes = fechas_unicas[0] if len(fechas_unicas) > 0 else fecha
                            if fecha_ingreso > fecha_inicio_mes:
                                # Contar días laborables desde fecha de ingreso
                                dias_desde_ingreso = len([f for f in fechas_unicas if f >= fecha_ingreso])
                                dias_deberia_trabajar = min(dias_desde_ingreso, dias_laborables_mes)
                        
                        asistencias_por_persona[alias] = {
                            'equipo': equipo,
                            'tipo': tipo_persona,
                            'fecha_ingreso': fecha_ingreso,
                            'dias_deberia_trabajar': dias_deberia_trabajar,
                            'dias_asistidos': 0,
                            'dias_faltados': dias_deberia_trabajar,  # Inicialmente todos faltados
                            'tardanza_total_segundos': 0,
                            'cantidad_tardanzas': 0,
                            'asistencias_detalle': {},  # {fecha: {tardanza_segundos, hora_inicio}}
                            'supervisor': equipo  # El equipo es el supervisor
                        }
                    
                    # Registrar asistencia del día
                    if fecha_str not in asistencias_por_persona[alias]['asistencias_detalle']:
                        asistencias_por_persona[alias]['dias_asistidos'] += 1
                        asistencias_por_persona[alias]['dias_faltados'] -= 1
                        asistencias_por_persona[alias]['asistencias_detalle'][fecha_str] = {
                            'tardanza_segundos': tardanza_segundos,
                            'hora_inicio': row['Hora_Inicio']
                        }
                    
                    # Acumular tardanzas
                    asistencias_por_persona[alias]['tardanza_total_segundos'] += tardanza_segundos
                    if tardanza_segundos > 0:
                        asistencias_por_persona[alias]['cantidad_tardanzas'] += 1
                
                datos_asistencias[hoja] = {
                    'fechas_unicas': [f.strftime('%Y-%m-%d') for f in fechas_unicas],
                    'dias_laborables': dias_laborables_mes,
                    'personas': asistencias_por_persona
                }
                
                print(f"  ✅ {len(asistencias_por_persona)} personas procesadas")
                print(f"  Días laborables detectados: {dias_laborables_mes}")
                
    except Exception as e:
        print(f"Error procesando asistencias: {e}")
        import traceback
        traceback.print_exc()
    
    return datos_asistencias

def generar_seccion_evaluacion():
    return """
    <!-- CONTROLES (EVALUACIÓN) -->
    <div class="controles-grafica" id="controlesEvaluacionMeses">

      <button type="button" class="btn-comparacion activo" id="btnEval3Meses">
        3 MESES
      </button>

      <button type="button" class="btn-comparacion" id="btnEval6Meses">
        6 MESES
      </button>

      <label class="checkbox-incluir-mes" id="wrapChkIncluirMesSeleccionadoEval">
        <input type="checkbox" id="chkIncluirMesSeleccionadoEval" />
        <span>INCLUIR MES SELECCIONADO</span>
      </label>

      <button
        type="button"
        class="boton-busqueda"
        id="btnEvalExportarAnalisis"
        style="background: linear-gradient(135deg, #9b59b6, #8e44ad);"
      >
        📊 EXPORTAR ANÁLISIS
      </button>

      <button
          type="button"
          class="boton-busqueda"
          id="btnEvalExportarRecuperoMetas"
          style="background: linear-gradient(135deg, #27ae60, #2ecc71);"
        >
          💾 EXPORTAR RECUPERO Y METAS
      </button>

    </div>


    <div style="padding: 30px; text-align:center; color:#666;">
      Sección en construcción.
    </div>
    """

def generar_html_modular(todos_los_datos, ruta_html, meses_unicos, años_unicos, meses_validos,
                         lista_asesores, datos_por_supervisor, lista_supervisores,
                         datos_asistencias=None, dias_laborables_por_mes=None, datos_cumpleaños_por_mes=None, base_asesores_analisis=None):
    """Genera el HTML con estructura modular de 4 secciones"""
    if base_asesores_analisis is None:
        base_asesores_analisis = {}
    if datos_asistencias is None:
        datos_asistencias = {}
    if dias_laborables_por_mes is None:
        dias_laborables_por_mes = {}
    if datos_cumpleaños_por_mes is None:
        datos_cumpleaños_por_mes = {}
    
    # Determinar mes y año actual más reciente
    mes_actual, año_actual = obtener_mes_año_actual(meses_validos)
    
    # Obtener datos del mes actual
    clave_actual = f"{mes_actual}_{año_actual}"
    asesores_data = todos_los_datos.get(clave_actual, [])
    asesores_data.sort(key=lambda x: x['porcentaje'], reverse=True)
    
    # Contar asesores por categoría
    contadores = {
        ">100%": len([a for a in asesores_data if a['clasificacion'] == ">100%"]),
        ">70%": len([a for a in asesores_data if a['clasificacion'] == ">70%"]),
        ">40%": len([a for a in asesores_data if a['clasificacion'] == ">40%"]),
        ">0%": len([a for a in asesores_data if a['clasificacion'] == ">0%"])
    }

    # Crear opciones de los selectores
    opciones_mes = ""
    for mes in meses_unicos:
        selected = "selected" if mes == mes_actual else ""
        opciones_mes += f'<option value="{mes}" {selected}>{mes.title()}</option>\n'
    
    opciones_año = ""
    for año in años_unicos:
        selected = "selected" if año == año_actual else ""
        opciones_año += f'<option value="{año}" {selected}>{año}</option>\n'
    
    # Crear opciones para el autocompletado de asesores
    opciones_asesores = "".join([f'<option value="{asesor}">{asesor}</option>' for asesor in lista_asesores])
    
    # Crear opciones para el autocompletado de supervisores
    opciones_supervisores = "".join([f'<option value="{supervisor}">' for supervisor in lista_supervisores])

    # Crear una versión serializable de los datos
    datos_para_js = {}
    for mes, asesores in todos_los_datos.items():
        datos_para_js[mes] = []
        for asesor in asesores:
            datos_asesor = {
                'nombre': asesor.get('nombre', ''),
                'porcentaje': asesor.get('porcentaje', 0),
                'clasificacion': asesor.get('clasificacion', '>0%'),
                'supervisor': asesor.get('supervisor', 'Sin Supervisor'),
                'recupero': asesor.get('recupero', 0),
                'meta': asesor.get('meta', 0),
                'meta_super': asesor.get('meta_super', 0),
                'recupero_supervisor': asesor.get('recupero_supervisor', 0),
                'alcance_supervisor': asesor.get('alcance_supervisor', 0),
                'cartera': asesor.get('cartera', 'No definida'),
                'datos_diarios_asesor': asesor.get('datos_diarios_asesor', {}),
                'alias_crr': asesor.get('alias_crr', ''),
                'dni': asesor.get('dni', ''),
                'fecha_ingreso': asesor.get('fecha_ingreso', '')
            }
            datos_para_js[mes].append(datos_asesor)

        
    
    # Convertir a JSON
    datos_json_str = json.dumps(datos_para_js, default=str, ensure_ascii=False)
    datos_supervisores_json_str = json.dumps(datos_por_supervisor, default=str, ensure_ascii=False)
    base_asesores_json = json.dumps(base_asesores_analisis, ensure_ascii=False)
    
    # ESCAPAR para JavaScript
    datos_json_str_escaped = datos_json_str.replace('\\', '\\\\').replace('`', '\\`').replace('${', '\\${')
    datos_supervisores_json_str_escaped = datos_supervisores_json_str.replace('\\', '\\\\').replace('`', '\\`').replace('${', '\\${')
    base_asesores_json_esc = base_asesores_json.replace('\\', '\\\\').replace("'", "\\'")

    # Serializar datos de cumpleaños
    datos_cumpleaños_json_str = json.dumps(datos_cumpleaños_por_mes, default=str, ensure_ascii=False)
    datos_cumpleaños_json_str_escaped = datos_cumpleaños_json_str.replace('\\', '\\\\').replace('`', '\\`').replace('${{', '\\${{')
    
    # Generar contenido para cada sección
    contenido_ranking = generar_seccion_ranking(asesores_data, contadores, opciones_asesores, 
                                                opciones_supervisores, meses_unicos, años_unicos)
    contenido_asistencias = generar_seccion_asistencias()
    contenido_recuperos = generar_seccion_recuperos(mes_actual, año_actual)
    contenido_evaluacion = generar_seccion_evaluacion()
    
    # Primero, pasar datos_asistencias al JavaScript:
    datos_asistencias_json_str = json.dumps(datos_asistencias, default=str, ensure_ascii=False)
    datos_asistencias_json_str_escaped = datos_asistencias_json_str.replace('\\', '\\\\').replace('`', '\\`').replace('${{', '\\${{')

    # Serializar días laborables para JavaScript
    dias_laborables_json_str = json.dumps(dias_laborables_por_mes, default=str, ensure_ascii=False)
    dias_laborables_json_str_escaped = dias_laborables_json_str.replace('\\', '\\\\').replace('`', '\\`').replace('${{', '\\${{')
    
    html_content = f'''<!DOCTYPE html>
<html lang="es">
<head>
    <script src="https://cdn.jsdelivr.net/npm/chart.js"></script>
    <script src="https://cdnjs.cloudflare.com/ajax/libs/exceljs/4.4.0/exceljs.min.js"></script>
    <script src="https://cdnjs.cloudflare.com/ajax/libs/FileSaver.js/2.0.5/FileSaver.min.js"></script>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Sistema de Gestión de Desempeño</title>
    <style>
        :root {{
            --verde-oscuro: #1b5e20;
            --verde-claro: #c8e6c9;
            --amarillo-oscuro: #FCCF10;
            --amarillo-claro: #FEF6D2;
            --naranja-oscuro: #e65100;
            --naranja-claro: #ffe0b2;
            --rojo-oscuro: #b71c1c;
            --rojo-claro: #ffcdd2;
            --azul-oscuro: #1565c0;
            --azul-claro: #e3f2fd;
            --gris: #f5f5f5;
            --azul-principal: #2c3e50;
            --azul-secundario: #34495e;
            --verde-principal: #27ae60;
            --naranja-principal: #e67e22;
            --purpura-principal: #9b59b6;
        }}
        
        * {{
            margin: 0;
            padding: 0;
            box-sizing: border-box;
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
        }}
        
        body {{
            background-color: #E7E9ED;
            color: #333;
            padding: 20px;
        }}
        
        .container {{
            max-width: 1400px;
            margin: 0 auto;
            background: transparent;
        }}
        
        /* ========== HEADER ========== */
        header {{
            text-align: center;
            margin-bottom: 30px;
            padding: 25px;
            background: linear-gradient(135deg, var(--azul-principal), var(--azul-secundario));
            color: white;
            border-radius: 15px;
            box-shadow: 0 6px 15px rgba(0, 0, 0, 0.15);
        }}
        
        h1 {{
            font-size: 2.8rem;
            margin-bottom: 15px;
            letter-spacing: 1px;
        }}
        
        .selectores-periodo {{
            margin: 25px 0;
            text-align: center;
        }}
        
        .selectores-periodo select {{
            padding: 12px 20px;
            font-size: 1.2rem;
            border: 2px solid #4a6491;
            border-radius: 10px;
            background-color: white;
            cursor: pointer;
            margin: 0 15px;
            min-width: 180px;
        }}
        
        .fecha-actualizacion {{
            font-size: 1.1rem;
            margin-bottom: 40px;
            opacity: 0.95;
            background: rgba(255, 255, 255, 0.1);
            padding: 10px 20px;
            border-radius: 25px;
            display: inline-block;
        }}
        
        /* ========== MENÚ PRINCIPAL ========== */
        .menu-principal {{
            display: grid;
            grid-template-columns: repeat(4, 1fr);
            gap: 10px;
            margin: 10px 0;
            max-width: 1300px;
            margin-left: auto;
            margin-right: auto;
            padding: 0 20px;
        }}
        
        .boton-modulo {{
            background: white;
            border: none;
            border-radius: 15px;
            padding: 20px 10px;
            cursor: pointer;
            transition: all 0.3s ease;
            display: flex;
            flex-direction: column;
            align-items: center;
            justify-content: center;
            box-shadow: 0 5px 15px rgba(0, 0, 0, 0.1);
            position: relative;
            overflow: hidden;
            width: 100%;
            max-width: 350px;
            margin: 0 auto;
            transform-origin: center;
            will-change: transform;
        }}

        .boton-modulo::before {{
            content: '';
            position: absolute;
            top: 0;
            left: 0;
            right: 0;
            height: 5px;
            background: linear-gradient(90deg, var(--verde-principal), var(--azul-principal), var(--naranja-principal), var(--purpura-principal));
        }}
        
        .boton-modulo:hover {{
            transform: translateY(-10px);
            box-shadow: 0 15px 30px rgba(0, 0, 0, 0.2);
        }}
        
        .boton-modulo.activo {{
            background: linear-gradient(135deg, var(--azul-principal), var(--azul-secundario));
            color: white;
            transform: translateY(-6px) scale(1.04);
            box-shadow: 0 18px 38px rgba(0, 0, 0, 0.28);
            animation: pulsoModuloActivo 1.8s ease-in-out infinite;
        }}

        .boton-modulo.activo::after {{
            content: '';
            position: absolute;
            inset: 8px;
            border-radius: 12px;
            border: 2px solid rgba(255, 255, 255, 0.45);
            pointer-events: none;
        }}
        
        .icono-modulo {{
            font-size: 3.5rem;
            margin-bottom: 20px;
        }}
        
        .titulo-modulo {{
            font-size: 1.8rem;
            font-weight: 700;
            margin-bottom: 10px;
        }}
        
        .descripcion-modulo {{
            font-size: 1rem;
            opacity: 0.8;
            text-align: center;
            max-width: 250px;
        }}
        
        /* COLORES POR MÓDULO */
        .boton-modulo.ranking {{
            border-top: 5px solid var(--verde-principal);
        }}
        
        .boton-modulo.ranking.activo {{
            background: linear-gradient(135deg, var(--verde-principal), #2ecc71);
        }}

        .boton-modulo.recuperos {{
            border-top: 5px solid var(--purpura-principal);
        }}
        
        .boton-modulo.recuperos.activo {{
            background: linear-gradient(135deg, var(--purpura-principal), #8e44ad);
        }}
        
        .boton-modulo.asistencias {{
            border-top: 5px solid var(--azul-principal);
        }}
        
        .boton-modulo.asistencias.activo {{
            background: linear-gradient(135deg, var(--azul-principal), #3498db);
        }}

        .boton-modulo.evaluacion {{
            border-top: 5px solid var(--naranja-principal);
        }}
        
        .boton-modulo.evaluacion.activo {{
            background: linear-gradient(135deg, var(--naranja-principal), #f39c12);
        }}

        /* ========== SECCIONES DE CONTENIDO ========== */
        .seccion-contenido {{
            display: none;
            background: #F8F9FA;
            padding: 10px;
            border-radius: 15px;
            box-shadow: 0 8px 30px rgba(0, 0, 0, 0.35);
            margin-top: 30px;
            animation: fadeIn 0.5s ease;
        }}
        
        .seccion-contenido.activa {{
            display: block;
        }}
        
        @keyframes fadeIn {{
            from {{ opacity: 0; transform: translateY(20px); }}
            to {{ opacity: 1; transform: translateY(0); }}
        }}
        
        /* ========== ESTILOS ESPECÍFICOS POR SECCIÓN ========== */
        
        /* RANKING */
        .clasificaciones {{
            display: grid;
            grid-template-columns: repeat(auto-fit, minmax(280px, 1fr));
            gap: 25px;
            margin-bottom: 40px;
        }}
        
        .clasificacion {{
            border-radius: 12px;
            overflow: hidden;
            box-shadow: 0 5px 15px rgba(0, 0, 0, 0.1);
        }}
        
        .clasificacion-header {{
            padding: 20px;
            color: white;
            text-align: center;
            font-weight: bold;
            font-size: 1.3rem;
        }}
        
        .clasificacion-100 {{ background-color: var(--verde-oscuro); }}
        .clasificacion-70 {{ background-color: var(--amarillo-oscuro); }}
        .clasificacion-40 {{ background-color: var(--naranja-oscuro); }}
        .clasificacion-0 {{ background-color: var(--rojo-oscuro); }}
        
        .asesores-lista {{
            max-height: 400px;
            overflow-y: auto;
            background-color: white;
        }}
        
        .asesor-item {{
            padding: 15px;
            border-bottom: 1px solid #eee;
            display: flex;
            justify-content: space-between;
            align-items: center;
            transition: background-color 0.2s;
        }}
        
        .asesor-item:hover {{
            background-color: #f8f9fa;
        }}
        
        .asesor-nombre {{
            font-weight: 500;
            font-size: 1.1rem;
        }}
        
        .asesor-porcentaje {{
            font-weight: bold;
            padding: 6px 12px;
            border-radius: 20px;
            color: white;
            font-size: 1rem;
            min-width: 80px;
            text-align: center;
        }}
        
        .gradiente-100 {{ background: linear-gradient(to right, var(--verde-claro), white); }}
        .gradiente-70 {{ background: linear-gradient(to right, var(--amarillo-claro), white); }}
        .gradiente-40 {{ background: linear-gradient(to right, var(--naranja-claro), white); }}
        .gradiente-0 {{ background: linear-gradient(to right, var(--rojo-claro), white); }}
        
        .porcentaje-100 {{ background-color: var(--verde-oscuro); }}
        .porcentaje-70 {{ background-color: var(--amarillo-oscuro); }}
        .porcentaje-40 {{ background-color: var(--naranja-oscuro); }}
        .porcentaje-0 {{ background-color: var(--rojo-oscuro); }}
        
        .boton-supervisor.activo {{
            background: linear-gradient(135deg, #2ecc71, #27ae60);
            box-shadow: 0 0 0 3px white, 0 0 0 6px #2ecc71;
        }}
        
        /* Secciones de búsqueda */
        .seccion-busqueda {{
            background: white;
            padding: 30px;
            border-radius: 12px;
            box-shadow: 0 4px 10px rgba(0, 0, 0, 0.08);
            margin: 30px 0;
            border-left: 5px solid #3498db;
        }}
        
        .seccion-busqueda h2 {{
            color: #2c3e50;
            margin-bottom: 25px;
            text-align: center;
            font-size: 1.8rem;
        }}
        
        .controles-busqueda {{
            display: flex;
            gap: 15px;
            align-items: center;
            flex-wrap: wrap;
            justify-content: center;
            margin-bottom: 20px;
        }}
        
        .input-busqueda {{
            flex: 1;
            min-width: 300px;
            padding: 14px 20px;
            font-size: 1.1rem;
            border: 2px solid #ddd;
            border-radius: 10px;
            background-color: white;
            transition: border-color 0.3s;
        }}
        
        .input-busqueda:focus {{
            border-color: #3498db;
            outline: none;
            box-shadow: 0 0 0 3px rgba(52, 152, 219, 0.2);
        }}
        
        .boton-busqueda {{
            padding: 14px 30px;
            font-size: 1.1rem;
            background: var(--azul-principal);
            color: white;
            border: none;
            border-radius: 10px;
            cursor: pointer;
            transition: all 0.3s;
            font-weight: 600;
        }}
        
        .boton-busqueda:hover {{
            background: #0d47a1;
            transform: translateY(-2px);
            box-shadow: 0 4px 8px rgba(0,0,0,0.2);
        }}
        
        .boton-busqueda.verde {{
            background: var(--verde-principal);
        }}
        
        .boton-busqueda.rojo {{
            background: #e74c3c;
        }}
        
        .boton-busqueda.naranja {{
            background: var(--naranja-principal);
        }}
        
        /* Tags de selección */
        .elementos-seleccionados {{
            margin-top: 25px;
            display: flex;
            flex-wrap: wrap;
            gap: 12px;
        }}
        
        .tag-elemento {{
            background: var(--azul-claro);
            color: var(--azul-principal);
            padding: 10px 20px;
            border-radius: 25px;
            display: flex;
            align-items: center;
            gap: 10px;
            font-weight: 600;
            font-size: 1rem;
            box-shadow: 0 2px 5px rgba(0,0,0,0.1);
        }}
        
        .eliminar-elemento {{
            background: none;
            border: none;
            color: var(--azul-principal);
            cursor: pointer;
            font-weight: bold;
            padding: 0;
            width: 24px;
            height: 24px;
            border-radius: 50%;
            display: flex;
            align-items: center;
            justify-content: center;
            transition: all 0.2s;
        }}
        
        .eliminar-elemento:hover {{
            background: var(--azul-principal);
            color: white;
        }}
        
        /* Tablas de comparación */
        .tabla-comparacion {{
            width: 100%;
            border-collapse: collapse;
            margin-top: 20px;
            font-size: 0.95rem;
            box-shadow: 0 3px 10px rgba(0,0,0,0.08);
            border-radius: 10px;
            overflow: hidden;
            border: 1px solid #e0e0e0;
        }}
        
        .tabla-comparacion th {{
            background: var(--azul-principal);
            color: white;
            padding: 15px 10px;
            text-align: center;
            font-weight: 600;
            position: sticky;
            top: 0;
            z-index: 10;
        }}
        
        .tabla-comparacion td {{
            padding: 12px 8px;
            border-bottom: 1px solid #eee;
            text-align: center;
            vertical-align: middle;
        }}
        
        /* Estilo para fila de supervisor */
        .tabla-comparacion tr.fila-supervisor {{
            background-color: #f8f9fa;
            border-top: 1px dashed #ddd;
        }}
        
        .tabla-comparacion tr.fila-supervisor td {{
            padding: 6px 8px;
            font-size: 0.85em;
            color: #666;
            border-bottom: none;
        }}

        .tabla-comparacion tr:first-child td {{
            padding: 8px 8px !important;      /* Padding normal para fila principal */
        }}
        
        /* Estilo para separador entre asesores */

        .fila-supervisor-compacta {{
            background-color: #f8f9fa !important;
            height: 22px !important;
            line-height: 1 !important;
        }}
        
        .fila-supervisor-compacta td {{
            padding: 1px 8px 2px 8px !important;
            line-height: 1 !important;
            vertical-align: middle !important;
            border-bottom: none !important;
            font-size: 0.85em !important;
        }}
        
        .label-supervisor {{
            color: #666 !important;
            font-style: italic !important;
            padding-left: 19px !important;
            padding-right: 0 !important;
            text-align: left !important;
            width: 80px !important;
            min-width: 80px !important;
            max-width: 80px !important;
        }}
        
        /* Estilos para los textos de supervisor */
        .supervisor-normal {{
            display: inline-block;
            color: #3498db;
            font-weight: 500;
            white-space: nowrap;
            line-height: 1.2;
        }}
        
        .supervisor-jefe {{
            display: inline-block;
            color: #2ecc71;
            font-weight: 600;
            white-space: nowrap;
            line-height: 1.2;
        }}
        
        .supervisor-sin {{
            display: inline-block;
            color: #e74c3c;
            font-weight: 500;
            white-space: nowrap;
            line-height: 1.2;
        }}
        
        .supervisor-vacio {{
            color: #bbb !important;
            font-style: italic !important;
            line-height: 1.2 !important;
        }}
        
        /* Separador compacto entre asesores */
        .separador-compacto td {{
            padding: 4px 0 !important;
            background: linear-gradient(to right, #f0f2f5, #e8eaf1, #f0f2f5) !important;
            height: 6px !important;
            line-height: 1 !important;
            border: none !important;
        }}
        
        /* Asegurar consistencia en las celdas */
        .tabla-comparacion td {{
            padding: 10px 8px !important;
            line-height: 1.4 !important;
            vertical-align: middle !important;
            height: auto !important;
            min-height: 36px !important;
            box-sizing: border-box !important;
        }}
        
        /* Forzar consistencia en todas las filas */
        .tabla-comparacion tr {{
            height: auto !important;
            min-height: 36px !important;
            line-height: 1.4 !important;
        }}
        
        .tabla-comparacion tr.separador-asesor td {{
            padding: 5px 0;
            background: linear-gradient(to right, #f0f2f5, #e8eaf1, #f0f2f5);
            height: 10px;
            border: none;
        }}
        
        .tabla-comparacion tr:hover {{
            background-color: #f5f7fa;
        }}
        
        .tabla-comparacion tr.fila-supervisor:hover {{
            background-color: #f0f2f5;
        }}
        
        /* Contenedor con scroll para muchas columnas */
        .contenedor-tabla-scroll {{
            overflow-x: auto;
            max-width: 100%;
            margin: 20px 0;
            border: 1px solid #e0e0e0;
            border-radius: 8px;
        }}
        
        /* Badges para supervisores */
        .badge-supervisor {{
            display: inline-block;
            padding: 3px 10px;
            background-color: #e3f2fd;
            color: #1565c0;
            border-radius: 15px;
            font-size: 0.85em;
            font-weight: 500;
            border: 1px solid #bbdefb;
            white-space: nowrap;
        }}
        
        .badge-supervisor-jefe {{
            background-color: #e8f5e9;
            color: #2e7d32;
            border-color: #c8e6c9;
        }}
        
        .badge-sin-supervisor {{
            background-color: #ffebee;
            color: #c62828;
            border-color: #ffcdd2;
        }}
        
        /* ========== RECUPEROS Y ALCANCES ========== */
        .grafica-container {{
            background: white;
            padding: 25px;
            border-radius: 12px;
            margin: 25px 0;
            box-shadow: 0 4px 15px rgba(0,0,0,0.1);
            border: 1px solid #e0e0e0;
        }}
        
        .grafica-container h3 {{
            color: var(--naranja-principal);
            margin-bottom: 20px;
            text-align: center;
            font-size: 1.6rem;
        }}
        
        .boton-exportar-excel {{
            background: linear-gradient(135deg, #21c45d, #0f9d58);
            color: white;
            border: none;
            padding: 15px 35px;
            font-size: 1.2rem;
            font-weight: 600;
            border-radius: 10px;
            cursor: pointer;
            transition: all 0.3s ease;
            box-shadow: 0 5px 15px rgba(15, 157, 88, 0.3);
            display: inline-flex;
            align-items: center;
            justify-content: center;
            gap: 12px;
            margin: 20px 0;
        }}
        
        .boton-exportar-excel:hover {{
            background: linear-gradient(135deg, #0f9d58, #0a7e46);
            transform: translateY(-3px);
            box-shadow: 0 8px 20px rgba(15, 157, 88, 0.4);
        }}
        
        /* ========== PROYECCIONES ========== */
        .panel-proyecciones {{
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            color: white;
            padding: 30px;
            border-radius: 15px;
            margin: 30px 0;
            box-shadow: 0 10px 30px rgba(102, 126, 234, 0.4);
        }}
        
        .controles-proyeccion {{
            display: flex;
            gap: 15px;
            align-items: center;
            flex-wrap: wrap;
            margin-bottom: 25px;
            justify-content: center;
        }}
        
        .input-proyeccion {{
            padding: 12px 20px;
            font-size: 1.1rem;
            border: 2px solid rgba(255,255,255,0.3);
            border-radius: 10px;
            background: rgba(255,255,255,0.1);
            color: white;
            min-width: 150px;
        }}
        
        .input-proyeccion::placeholder {{
            color: rgba(255,255,255,0.7);
        }}
        
        /* ========== ESTILOS PARA ASISTENCIAS ========== */
        .selector-asistencias {{
            padding: 12px 20px;
            font-size: 1.1rem;
            border: 2px solid #3498db;
            border-radius: 10px;
            background-color: white;
            cursor: pointer;
            min-width: 200px;
        }}
        
        .estadistica-asistencia {{
            background: white;
            padding: 20px;
            border-radius: 12px;
            box-shadow: 0 4px 15px rgba(0,0,0,0.1);
            margin-bottom: 20px;
            border-left: 5px solid #3498db;
        }}
        
        .estadistica-row {{
            display: grid;
            grid-template-columns: repeat(auto-fit, minmax(200px, 1fr));
            gap: 15px;
            margin-top: 15px;
        }}
        
        .estadistica-item {{
            text-align: center;
            padding: 15px;
            border-radius: 8px;
            background: #f8f9fa;
        }}
        
        .estadistica-item.verde {{
            background: #e8f6f3;
            border: 2px solid #27ae60;
        }}
        
        .estadistica-item.naranja {{
            background: #fdebd0;
            border: 2px solid #e67e22;
        }}
        
        .estadistica-item.rojo {{
            background: #fadbd8;
            border: 2px solid #e74c3c;
        }}
        
        .estadistica-valor {{
            font-size: 2rem;
            font-weight: bold;
            margin: 10px 0;
        }}
        
        .estadistica-label {{
            font-size: 0.9rem;
            color: #666;
            text-transform: uppercase;
            letter-spacing: 1px;
        }}
        
        /* Indicadores de estado */
        .indicador-puntual {{
            background-color: #27ae60;
            color: white;
            padding: 3px 10px;
            border-radius: 15px;
            font-size: 0.85em;
            font-weight: 500;
        }}
        
        .indicador-tardanza {{
            background-color: #f39c12;
            color: white;
            padding: 3px 10px;
            border-radius: 15px;
            font-size: 0.85em;
            font-weight: 500;
        }}
        
        .indicador-ausente {{
            background-color: #e74c3c;
            color: white;
            padding: 3px 10px;
            border-radius: 15px;
            font-size: 0.85em;
            font-weight: 500;
        }}
        
        /* Gráficos de asistencias */
        .grafica-asistencias-container {{
            background: white;
            padding: 25px;
            border-radius: 12px;
            margin: 25px 0;
            box-shadow: 0 4px 15px rgba(0,0,0,0.1);
            border: 1px solid #e0e0e0;
        }}
        
        /* ========== FOOTER ========== */
        footer {{
            text-align: center;
            margin-top: 50px;
            padding: 25px;
            color: #666;
            font-size: 0.95rem;
            border-top: 1px solid #ddd;
        }}
        
        /* ========== RESPONSIVE ========== */
        @media (max-width: 1200px) {{
            .menu-principal {{
                grid-template-columns: repeat(3, 1fr);
                max-width: 1000px;
            }}
        }}
        
        @media (max-width: 768px) {{
            .menu-principal {{
                grid-template-columns: 1fr;
                max-width: 500px;
            }}
            
            .selectores-periodo select {{
                display: block;
                margin: 10px auto;
                width: 80%;
            }}
            
            .controles-busqueda {{
                flex-direction: column;
            }}
            
            .input-busqueda {{
                width: 100%;
                min-width: auto;
            }}
        }}

        /* ========== CSS PARA BOTÓN DE 3 MESES ========== */
        .controles-grafica {{
            display: flex;
            justify-content: center;
            gap: 15px;
            padding: 15px;
        }}
            
        .controles-grafica .btn-comparacion {{
            padding: 10px 25px;
            border: 2px solid #3498db;
            background: white;
            color: #3498db;
            border-radius: 8px;
            cursor: pointer;
            font-weight: 600;
            font-size: 1rem;
            transition: all 0.3s;
            min-width: 100px;
        }}
            
        .controles-grafica .btn-comparacion:hover {{
            background: #3498db;
            color: white;
            transform: translateY(-3px);
            box-shadow: 0 5px 15px rgba(52, 152, 219, 0.3);
        }}
            
        .controles-grafica .btn-comparacion.activo {{
            background: #3498db;
            color: white;
            border-color: #2980b9;
            box-shadow: 0 0 0 3px white, 0 0 0 6px #3498db;
        }}

        /* ========== CHECKBOX "INCLUIR MES SELECCIONADO" ========== */
        .checkbox-incluir-mes {{
          display: flex;
          align-items: center;
          gap: 10px;
          cursor: pointer;
          font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
          font-weight: 700;
          font-size: 0.95rem;
          letter-spacing: 0.4px;
          text-transform: uppercase;
        
          color: #2c3e50;
        }}
        
        .checkbox-incluir-mes input[type="checkbox"] {{
          width: 18px;
          height: 18px;
          cursor: pointer;
        }}

        /* ========== ESTILOS PARA TABLA DE COMPARACIÓN DE SUPERVISORES ========== */
        .supervisor-name {{
            font-weight: bold !important;
            color: #2c3e50 !important;
            padding-left: 15px !important;
            background-color: #f8f9fa !important;
            border-left: 4px solid #3498db !important;
            /* MANTENER padding normal para fila principal */
            padding: 8px 8px 8px 15px !important;  /* Cambiado de padding-top/padding-bottom separados */
            font-size: 1em !important;
        }}
        
        .fila-equipo {{
            background-color: #ffffff !important;
            height: 26px !important;
            min-height: 26px !important;
            line-height: 1 !important;             /* AGREGADO: Anula line-height general */
        }}
        
        .fila-equipo:hover {{
            background-color: #f5f7fa !important;
        }}
        
        /* ESTILOS ESPECÍFICOS para celdas de filas de equipo - DEBEN ir DESPUÉS de reglas generales */
        .tabla-comparacion tr.fila-equipo td {{
            padding: 3px 8px !important;           /* Sobreescribe padding general */
            line-height: 1 !important;             /* Sobreescribe line-height general */
            min-height: 26px !important;           /* Sobreescribe min-height general */
            height: 26px !important;               /* Sobreescribe height general */
            vertical-align: middle !important;
            border-bottom: 1px solid #eee;
            text-align: center;
        }}
        
        .equipo-label {{
            padding-left: 30px !important;
            font-style: italic !important;
            color: #666 !important;
            background-color: #ffffff !important;
            text-align: left !important;
            width: 120px !important;
            min-width: 120px !important;
            max-width: 120px !important;
        }}
        
        .equipo-supervisor {{
            text-align: center !important;
            vertical-align: middle !important;
            background-color: #ffffff !important;
            font-size: 0.9em !important;
        }}
        
        .sin-equipo {{
            color: #bbb !important;
            font-style: italic !important;
            text-align: center !important;
            background-color: #ffffff !important;
            font-size: 0.9em !important;
        }}
        
        /* Mejorar el hover para filas de equipo */
        .fila-equipo:hover td {{
            background-color: #f5f7fa !important;
        }}
        
        /* Estilos para colores de segmento */
        .segmento-premier {{
            color: #8e44ad !important;
            background-color: #f4ecf7 !important;
        }}
        
        .segmento-empresarial {{
            color: #3498db !important;
            background-color: #ebf5fb !important;
        }}
        
        .segmento-masivo {{
            color: #27ae60 !important;
            background-color: #eafaf1 !important;
        }}
        
        .segmento-pyme {{
            color: #f39c12 !important;
            background-color: #fef5e7 !important;
        }}
        
        /* REGLA CRÍTICA: Sobreescribir estilos generales para filas específicas */
        .tabla-comparacion tr.fila-equipo {{
            height: 26px !important;
            min-height: 26px !important;
            line-height: 1 !important;
        }}
        
        /* Asegurar que las filas principales mantengan su altura normal */
        .tabla-comparacion tr:first-child:not(.fila-equipo) td {{
            padding: 8px 8px !important;
            min-height: 36px !important;
            line-height: 1.4 !important;
        }}

        /* ========== ESTILOS PARA TABLA DE COMPARACIÓN DE ASESORES ========== */
        .fila-secundaria-asesor {{
            background-color: #f8f9fa !important;
            height: 26px !important;
            min-height: 26px !important;
            line-height: 1 !important;
        }}
        
        /* Sobreescribir estilos generales para filas secundarias de asesores */
        .tabla-comparacion tr.fila-secundaria-asesor td {{
            padding: 3px 8px !important;           /* Altura reducida */
            line-height: 1 !important;             /* Altura reducida */
            min-height: 26px !important;           /* Altura reducida */
            height: 26px !important;               /* Altura reducida */
            vertical-align: middle !important;
            border-bottom: 1px solid #eee;
        }}
        
        .label-secundario-asesor {{
            color: #666 !important;
            font-style: italic !important;
            padding-left: 20px !important;
            text-align: left !important;
            width: 100px !important;
            min-width: 100px !important;
            max-width: 100px !important;
            background-color: #f8f9fa !important;
        }}
        
        .dato-secundario-asesor {{
            text-align: center !important;
            vertical-align: middle !important;
            background-color: #f8f9fa !important;
            font-size: 0.9em !important;
        }}
        
        /* Asegurar que las filas principales de asesores mantengan altura normal */
        .tabla-comparacion tr:first-child:not(.fila-secundaria-asesor) td {{
            padding: 8px 8px !important;
            min-height: 36px !important;
            line-height: 1.4 !important;
        }}
        
        /* Asegurar hover para filas secundarias */
        .fila-secundaria-asesor:hover td {{
            background-color: #f5f7fa !important;
        }}

        /* ========== ESTILOS PARA PERIODO DE PRUEBA ========== */
        .selectores-periodo-prueba {{
            display: grid;
            grid-template-columns: repeat(5, 1fr);
            gap: 15px;
            margin-bottom: 20px;
        }}
        
        .selectores-periodo-prueba select,
        .selectores-periodo-prueba input {{
            padding: 12px 15px;
            font-size: 1rem;
            border: 2px solid #ddd;
            border-radius: 8px;
            background-color: white;
            color: #333;
            transition: all 0.3s;
            width: 100%;
        }}
        
        .selectores-periodo-prueba select:focus,
        .selectores-periodo-prueba input:focus {{
            border-color: #3498db;
            outline: none;
            box-shadow: 0 0 0 3px rgba(52, 152, 219, 0.2);
        }}
        
        .selectores-periodo-prueba select:hover,
        .selectores-periodo-prueba input:hover {{
            border-color: #3498db;
        }}
        
        /* Placeholder específico para periodo de prueba */
        .selectores-periodo-prueba select option[value=""] {{
            color: #999;
            font-style: italic;
        }}
        
        .selectores-periodo-prueba input::placeholder {{
            color: #999;
            font-style: italic;
        }}

        @media (max-width: 1200px) {{
            .selectores-periodo-prueba {{
                grid-template-columns: repeat(3, 1fr);
            }}
        }}
        
        @media (max-width: 768px) {{
            .selectores-periodo-prueba {{
                grid-template-columns: repeat(2, 1fr);
            }}
            
            .selector-label {{
                font-size: 0.85rem;
            }}
        }}
        
        @media (max-width: 576px) {{
            .selectores-periodo-prueba {{
                grid-template-columns: 1fr;
                gap: 12px;
            }}
            
            .selectores-periodo-prueba select,
            .selectores-periodo-prueba input {{
                height: 44px;
                font-size: 0.95rem;
            }}
        }}

        /* ========== ESTILOS MEJORADOS PARA PERIODO DE PRUEBA ========== */
        .selectores-periodo-prueba {{
            display: grid;
            grid-template-columns: repeat(5, 1fr);
            gap: 15px;
            margin-bottom: 20px;
        }}
        
        .selector-item {{
            display: flex;
            flex-direction: column;
            gap: 8px;
        }}
        
        .selector-label {{
            font-size: 0.9rem;
            font-weight: 600;
            color: #2c3e50;
            margin-bottom: 4px;
            display: flex;
            align-items: center;
            gap: 5px;
        }}
        
        .selectores-periodo-prueba select,
        .selectores-periodo-prueba input {{
            padding: 12px 15px;
            font-size: 1rem;
            border: 2px solid #ddd;
            border-radius: 8px;
            background-color: white;
            color: #333;
            transition: all 0.3s;
            width: 100%;
            height: 48px; /* Altura consistente */
        }}
        
        .selectores-periodo-prueba select:focus,
        .selectores-periodo-prueba input:focus {{
            border-color: #3498db;
            outline: none;
            box-shadow: 0 0 0 3px rgba(52, 152, 219, 0.2);
        }}
        
        .selectores-periodo-prueba select:hover,
        .selectores-periodo-prueba input:hover {{
            border-color: #3498db;
        }}
        
        /* Placeholder y opciones vacías */
        .selectores-periodo-prueba select option[value=""] {{
            color: #999;
            font-style: italic;
        }}
        
        .selectores-periodo-prueba input::placeholder {{
            color: #999;
            font-style: italic;
        }}
        
        /* Estilo para el input de porcentaje */
        #porcentajeMinimo {{
            background-color: #fffbf0;
            border-color: #f39c12;
        }}
        
        #porcentajeMinimo:focus {{
            border-color: #e67e22;
            box-shadow: 0 0 0 3px rgba(230, 126, 34, 0.2);
        }}
        
        /* Indicador visual para selects vacíos */
        .selectores-periodo-prueba select:invalid {{
            border-color: #ffcccb;
            background-color: #fff5f5;
        }}
        
        .selectores-periodo-prueba select:invalid + .selector-label::after {{
            content: " *";
            color: #e74c3c;
        }}

        /* ========== ESTILOS PARA DATALIST ========== */
        datalist {{
            position: absolute;
            background-color: white;
            border: 1px solid #ddd;
            border-radius: 8px;
            max-height: 200px;
            overflow-y: auto;
            box-shadow: 0 4px 12px rgba(0, 0, 0, 0.1);
            z-index: 1000;
        }}
        
        datalist option {{
            padding: 10px 15px;
            cursor: pointer;
            transition: background-color 0.2s;
        }}
        
        datalist option:hover {{
            background-color: #f8f9fa;
        }}
        
        /* Mejorar el input con datalist */
        .input-busqueda[list] {{
            background-image: url("data:image/svg+xml,%3Csvg xmlns='http://www.w3.org/2000/svg' width='16' height='16' fill='%233498db' viewBox='0 0 16 16'%3E%3Cpath d='M7.247 11.14 2.451 5.658C1.885 5.013 2.345 4 3.204 4h9.592a1 1 0 0 1 .753 1.659l-4.796 5.48a1 1 0 0 1-1.506 0z'/%3E%3C/svg%3E");
            background-repeat: no-repeat;
            background-position: right 15px center;
            padding-right: 40px;
        }}

        /* ========== ESTILOS PARA TABLA DE PERIODO DE PRUEBA ========== */
        #tablaPeriodo {{
            margin-top: 20px;
            font-size: 0.95rem;
        }}
        
        #tablaPeriodo th {{
            background-color: #2c3e50;
            color: white;
            padding: 12px 10px;
            text-align: center;
            font-weight: 600;
            position: sticky;
            top: 0;
            z-index: 10;
            white-space: nowrap;
        }}
        
        #tablaPeriodo td {{
            padding: 10px 8px;
            border-bottom: 1px solid #eee;
            vertical-align: middle;
            text-align: center;
        }}
        
        #tablaPeriodo tr:hover {{
            background-color: #f5f7fa;
        }}
        
        /* Estilos específicos para columnas numéricas */
        #tablaPeriodo td:nth-child(2),  /* Periodo 1 */
        #tablaPeriodo td:nth-child(3),  /* Periodo 2 */
        #tablaPeriodo td:nth-child(4),  /* Total Recupero */
        #tablaPeriodo td:nth-child(5),  /* Total Meta */
        #tablaPeriodo td:nth-child(8) {{ /* Falta Recuperar */
            font-family: 'Courier New', monospace;
            font-weight: 500;
            letter-spacing: 0.5px;
        }}
        
        /* Colores para estado */
        #tablaPeriodo td:nth-child(7) {{ /* Estado */
            font-weight: 600;
        }}
        
        /* Alinear nombres de asesores a la izquierda */
        #tablaPeriodo td:first-child {{
            text-align: left;
            padding-left: 15px;
        }}
        
        /* Estilo para la fila de totales */
        #tablaPeriodo tr:last-child {{
            background-color: #e8f4fd !important;
            font-size: 1rem;
        }}
        
        #tablaPeriodo tr:last-child td {{
            padding: 14px 8px;
            border-top: 2px solid #2c3e50;
            border-bottom: 2px solid #2c3e50;
        }}
        
        /* Responsive para tabla de periodo de prueba */
        @media (max-width: 1200px) {{
            #resultadosPeriodo .contenedor-tabla-scroll {{
                max-width: 100%;
                overflow-x: auto;
            }}
            
            #tablaPeriodo {{
                min-width: 1000px;
            }}
        }}
        
        /* Mejorar visualización de números grandes */
        .numero-grande {{
            font-size: 1.05em;
            font-weight: 600;
        }}

        //styles para cambio de gráfica recupero//
        .btn-toggle-grafica {{
            padding: 10px 25px;
            border: none;
            border-radius: 25px;
            cursor: pointer;
            font-size: 1rem;
            font-weight: 600;
            transition: all 0.3s ease;
            min-width: 180px;
        }}
        
        .btn-toggle-grafica:hover {{
            transform: translateY(-3px);
            box-shadow: 0 6px 15px rgba(0, 0, 0, 0.2) !important;
        }}
        
        .btn-toggle-grafica.activo {{
            background: linear-gradient(135deg, #2ecc71, #27ae60) !important;
            box-shadow: 0 0 0 3px white, 0 0 0 6px #2ecc71 !important;
            transform: translateY(-2px);
        }}
        
        /* Contenedor de gráfica con posición relativa para los botones */
        .grafica-container {{
            position: relative;
            background: white;
            padding: 25px;
            border-radius: 12px;
            margin: 25px 0;
            box-shadow: 0 4px 15px rgba(0,0,0,0.1);
            border: 1px solid #e0e0e0;
        }}

        /* ========== ESTILOS ESPECÍFICOS PARA GRAFICAS DONA ASISTENCIAS ========== */
        .contenedor-grafica-dona {{
            position: relative;
            width: 100%;
            height: 350px;
            margin: 0 auto;
        }}
        
        .contenedor-grafica-dona canvas {{
            display: block !important;
            width: 500px !important;  /* Más ancho */
            height: 320px !important;
            max-width: 100% !important;
            max-height: 100% !important;
            margin: 0 auto;
        }}
        
        /* Solo para los canvas de asistencias */
        #graficaTopFaltas,
        #graficaTopTardanzaTiempo,
        #graficaTopTardanzas {{
            width: 500px !important;  /* Más ancho */
            height: 320px !important;
            max-width: 100% !important;
        }}
        
        /* Para que los contenedores se ajusten */
        .grafica-container {{
            min-width: 500px !important;
            overflow-x: auto !important;
        }}

        /* Estilos simplificados para histórico */
        .tabla-historico-contenedor {{
            overflow-x: auto;
            max-height: 300px;
            border-top: 1px solid #eee;
            border-bottom: 1px solid #eee;
        }}
        
        .tabla-historico {{
            width: 100%;
            border-collapse: collapse;
            font-size: 0.85rem;
            min-width: 500px;
        }}
        
        .tabla-historico th {{
            padding: 8px 6px;
            text-align: center;
            font-weight: 500;
            color: #2c3e50;
            background-color: #f8f9fa;
            border-bottom: 2px solid #ddd;
            position: sticky;
            top: 0;
            z-index: 10;
            white-space: nowrap;
        }}
        
        .tabla-historico th:first-child {{
            position: sticky;
            left: 0;
            z-index: 20;
            min-width: 120px;
            text-align: left;
            padding-left: 15px;
        }}
        
        .tabla-historico td {{
            padding: 6px 4px;
            text-align: center;
            border-bottom: 1px solid #eee;
            vertical-align: middle;
            min-width: 80px;
        }}
        
        .tabla-historico td:first-child {{
            position: sticky;
            left: 0;
            background: inherit;
            z-index: 5;
            font-weight: 600;
            text-align: left;
            padding-left: 15px;
            min-width: 120px;
            border-right: 1px solid #eee;
        }}
        
        .tabla-historico tr:hover {{
            background-color: #f5f7fa;
        }}
        
        .tabla-historico tr:hover td:first-child {{
            background-color: #f5f7fa;
        }}
        
        /* Scrollbar simple */
        .tabla-historico-contenedor::-webkit-scrollbar {{
            height: 6px;
            width: 6px;
        }}
        
        .tabla-historico-contenedor::-webkit-scrollbar-track {{
            background: #f1f1f1;
        }}
        
        .tabla-historico-contenedor::-webkit-scrollbar-thumb {{
            background: #c1c1c1;
            border-radius: 3px;
        }}
        
        .tabla-historico-contenedor::-webkit-scrollbar-thumb:hover {{
            background: #a8a8a8;
        }}

        /* Estilos para los promedios */
        .promedio-card {{
            background: white;
            padding: 12px;
            border-radius: 6px;
            border-left: 4px solid #ccc;
            transition: transform 0.2s ease;
        }}
        
        .promedio-card:hover {{
            transform: translateY(-2px);
            box-shadow: 0 4px 8px rgba(0,0,0,0.1);
        }}
        
        /* Estilos para etiqueta de evaluación */
        .etiqueta-evaluacion {{
            background: white;
            padding: 15px;
            border-radius: 6px;
            border: 2px solid #2ecc71;
            transition: all 0.3s ease;
        }}
        
        .etiqueta-evaluacion.inestable {{
            border-color: #e74c3c;
            background: linear-gradient(135deg, #fff5f5, #ffeaea);
        }}
        
        .etiqueta-evaluacion.estable {{
            border-color: #2ecc71;
            background: linear-gradient(135deg, #f5fff7, #eaffef);
        }}
        
        .etiqueta-evaluacion.mejora {{
            border-color: #3498db;
            background: linear-gradient(135deg, #f0f8ff, #e6f2ff);
        }}

        /* ========== BARRA DE FILTROS GLOBAL ========== */
        .barra-filtros-supervisores {{
            display: flex;
            flex-wrap: wrap;
            gap: 10px;
            justify-content: center;
            margin: 20px 0;
            padding: 15px;
            background: rgba(255, 255, 255, 0.1);
            border-radius: 12px;
        }}
        
        .filtro-supervisor {{
            padding: 10px 20px;
            background: linear-gradient(135deg, #3498db, #2980b9);
            color: white;
            border: none;
            border-radius: 25px;
            cursor: pointer;
            font-size: 0.95rem;
            font-weight: 600;
            transition: all 0.3s ease;
            box-shadow: 0 3px 8px rgba(0,0,0,0.2);
        }}
        
        .filtro-supervisor:hover {{
            background: linear-gradient(135deg, #2980b9, #1f6396);
            transform: translateY(-3px);
            box-shadow: 0 5px 12px rgba(0,0,0,0.3);
        }}
        
        .filtro-supervisor.activo {{
            background: linear-gradient(135deg, #2ecc71, #27ae60);
            box-shadow: 0 0 0 3px white, 0 0 0 6px #2ecc71;
        }}
        
        .filtro-supervisor.especial {{
            background: linear-gradient(135deg, #9b59b6, #8e44ad);
        }}
        
        .filtro-supervisor.especial:hover {{
            background: linear-gradient(135deg, #8e44ad, #7d3c98);
        }}

    </style>
</head>
<body>
<div class="container">
    <!-- ========== HEADER ========== -->
    <header>
        <h1>🏆 SISTEMA DE GESTIÓN DE DESEMPEÑO</h1>
        <div class="selectores-periodo">
            <select id="selectorAño" onchange="actualizarPeriodo()">
                {opciones_año}
            </select>
            <select id="selectorMes" onchange="actualizarPeriodo()">
                {opciones_mes}
            </select>
        </div>
        <div class="fecha-actualizacion">Última actualización: {datetime.now().strftime("%d de %B de %Y a las %H:%M")}</div>
    
        <!-- ========== MENÚ PRINCIPAL (AHORA 3 BOTONES) ========== -->
        <div class="menu-principal">
            <button class="boton-modulo ranking activo" onclick="mostrarSeccion('ranking')">
                <div class="icono-modulo"></div>
                <div class="titulo-modulo">RANKING</div>
                <div class="descripcion-modulo">Clasificación y comparación de asesores y supervisores</div>
            </button>

            <button class="boton-modulo recuperos" onclick="mostrarSeccion('recuperos')">
                <div class="icono-modulo"></div>
                <div class="titulo-modulo">ALCANCES</div>
                <div class="descripcion-modulo">Gr&aacute;ficos de alcance por equipos y recupero diario</div>
            </button>
            
            <button class="boton-modulo asistencias" onclick="mostrarSeccion('asistencias')">
                <div class="icono-modulo"></div>
                <div class="titulo-modulo">ASISTENCIAS</div>
                <div class="descripcion-modulo">Registro y control de asistencias</div>
            </button>

            <button class="boton-modulo evaluacion" onclick="mostrarSeccion('evaluacion')">
                <div class="icono-modulo"></div>
                <div class="titulo-modulo">EVALUACIÓN</div>
                <div class="descripcion-modulo">Análisis y clasificación de asesores por periodo y alcance promediado</div>
            </button>
        </div>
        <div class="barra-filtros-supervisores" id="barraFiltrosSupervisoresGlobal">
            <!-- Se generará dinámicamente -->
        </div>
    </header>
    <!-- ========== SECCIÓN RANKING ========== -->
    <section id="seccion-ranking" class="seccion-contenido activa">
        {contenido_ranking}
    </section>
    
    <!-- ========== SECCIÓN ASISTENCIAS ========== -->
    <section id="seccion-asistencias" class="seccion-contenido">
        {contenido_asistencias}
    </section>
    
    <!-- ========== SECCIÓN RECUPEROS Y ALCANCES ========== -->
    <section id="seccion-recuperos" class="seccion-contenido">
        {contenido_recuperos}
    </section>

    <!-- ========== SECCIÓN EVALUACIÓN ========== -->
    <section id="seccion-evaluacion" class="seccion-contenido">
        {contenido_evaluacion}
    </section>
    
    <footer>
        <p>Sistema de seguimiento de metas - Actualización diaria según cortes</p>
    </footer>
</div>

<script>

    // ========== VARIABLES GLOBALES ==========
    const asesoresSeleccionados = new Set();
    const supervisoresSeleccionados = new Set();
    const asesoresSeleccionadosPeriodo = new Set();
    let estadoGlobal = {{
    mesActual: '',
    añoActual: '',
    periodoActual: ''
    }};
    let mesesAMostrar = 6;
    let datosAsistencias = JSON.parse(`{datos_asistencias_json_str_escaped}`);
    let diasLaborablesPorMes = JSON.parse(`{dias_laborables_json_str_escaped}`);
    let datosMeses = {datos_json_str_escaped};
    let datosSupervisores = {datos_supervisores_json_str_escaped};
    window.baseAsesoresAnalisis = JSON.parse('{base_asesores_json_esc}');
    let datosCumpleaños = JSON.parse(`{datos_cumpleaños_json_str_escaped}`);
    let chartTop10Modal = null;
    let chartCumpleañosModal = null;
    let chartInstance = null;
    let chartInstanceSupervisores = null;
    let chartAlcanceEquipos = null;
    let chartTendencia = null;
    let chartIncrementoTotal = null;
    let supervisorFiltroActual = 'TODOS';
    let vistaEvaluacion = '3M'; 
    let incluirMesSeleccionadoEval = false;
    
    // ========== FUNCIÓN ÚNICA PARA ACTUALIZAR FILTROS ==========
    function actualizarFiltrosGlobales() {{
        const mesSeleccionado = document.getElementById('selectorMes').value;
        const añoSeleccionado = document.getElementById('selectorAño').value;
        const periodoCompleto = `${{mesSeleccionado}}_${{añoSeleccionado}}`;
        
        if (!datosMeses[periodoCompleto]) return;
        
        const asesores = datosMeses[periodoCompleto];
        const supervisores = [...new Set(asesores.map(a => a.supervisor).filter(s => s))].sort();
        
        let html = `
            <button class="filtro-supervisor activo" data-supervisor="TODOS">
                👥 TODOS LOS EQUIPOS
            </button>
        `;
        
        supervisores.forEach(supervisor => {{
            html += `
                <button class="filtro-supervisor" data-supervisor="${{supervisor}}">
                    ${{supervisor}}
                </button>
            `;
        }});
        
        // Agregar botones especiales
        html += `
            <button class="filtro-supervisor especial" onclick="mostrarTop10()">
                🏆 Top 10
            </button>
            <button class="filtro-supervisor especial" onclick="mostrarModalCumpleaños()">
                🎂 Cumpleaños
            </button>
        `;
        
        document.getElementById('barraFiltrosSupervisoresGlobal').innerHTML = html;
        
        // Event listeners únicos
        document.querySelectorAll('.filtro-supervisor[data-supervisor]').forEach(btn => {{
            btn.addEventListener('click', function() {{
                supervisorFiltroActual = this.getAttribute('data-supervisor');
                aplicarFiltroSupervisor(supervisorFiltroActual);
            }});
        }});
    }}
    
    // ========== FUNCIÓN ÚNICA PARA APLICAR FILTRO ==========
    function aplicarFiltroSupervisor(supervisor) {{
        // Actualizar botón activo
        document.querySelectorAll('.filtro-supervisor').forEach(btn => {{
            btn.classList.remove('activo');
            if (btn.getAttribute('data-supervisor') === supervisor) {{
                btn.classList.add('activo');
            }}
        }});
        
        // Determinar sección activa
        const seccionActiva = document.querySelector('.seccion-contenido.activa');
        
        if (seccionActiva.id === 'seccion-ranking') {{
            aplicarFiltroRanking(supervisor);
        }} else if (seccionActiva.id === 'seccion-recuperos') {{
            aplicarFiltroAlcances(supervisor);
        }} else if (seccionActiva.id === 'seccion-asistencias') {{
            aplicarFiltroAsistencias(supervisor);
        }} else if (seccionActiva.id === 'seccion-evaluacion') {{
            // por ahora no hace nada
        }}
    }}
    
    // ========== FUNCIONES DE NAVEGACIÓN ==========
    function mostrarSeccion(seccion) {{
        // Ocultar todas las secciones
        document.querySelectorAll('.seccion-contenido').forEach(sec => {{
            sec.classList.remove('activa');
        }});
        
        // Remover activo de todos los botones
        document.querySelectorAll('.boton-modulo').forEach(boton => {{
            boton.classList.remove('activo');
        }});
        
        // Mostrar la sección seleccionada
        document.getElementById(`seccion-${{seccion}}`).classList.add('activa');
        document.querySelector(`.boton-modulo.${{seccion}}`).classList.add('activo');
        
        // Aplicar filtro actual a la nueva sección
        setTimeout(() => {{
            aplicarFiltroSupervisor(supervisorFiltroActual);
        }}, 100);
    }}

    function mostrarMensajeSinDatos(canvas, periodoCompleto, supervisor = null) {{
        const ctx = canvas.getContext('2d');
        ctx.clearRect(0, 0, canvas.width, canvas.height);
        
        // Configurar el canvas para mensaje
        canvas.width = 1300;
        canvas.height = 200;
        canvas.style.display = 'block';
        canvas.style.margin = '0 auto';
        canvas.style.maxWidth = '100%';
        
        // Dibujar mensaje
        ctx.fillStyle = '#95a5a6';
        ctx.font = 'bold 24px Arial';
        ctx.textAlign = 'center';
        
        let mensaje = '📊 No hay datos de recupero acumulado disponibles';
        if (supervisor) {{
            mensaje = `📊 No hay datos de recupero para ${{supervisor}}`;
        }}
        
        ctx.fillText(mensaje, canvas.width / 2, canvas.height / 2 - 20);
        
        ctx.fillStyle = '#7f8c8d';
        ctx.font = '18px Arial';
        ctx.fillText(`Periodo: ${{periodoCompleto.replace('_', ' ')}}`, canvas.width / 2, canvas.height / 2 + 20);
    }}
    
    function forzarActualizacionPeriodo(seccionDestino) {{
        // Obtener valores actuales del header
        const mesSeleccionado = document.getElementById('selectorMes').value;
        const añoSeleccionado = document.getElementById('selectorAño').value;
        const periodoCompleto = `${{mesSeleccionado}}_${{añoSeleccionado}}`;
        
        console.log(`🔄 Forzando actualización para ${{seccionDestino}}: ${{periodoCompleto}}`);
        
        if (!datosMeses[periodoCompleto]) {{
            console.warn(`⚠️ No hay datos para ${{periodoCompleto}}`);
            return;
        }}
        
        const asesores = datosMeses[periodoCompleto] || [];
        
        // Aplicar según la sección destino
        if (seccionDestino === 'ranking') {{
            actualizarFiltrosGlobales();  // Nueva función única
            aplicarFiltroRanking(supervisorFiltroActual);
        }}
        
        if (seccionDestino === 'recuperos') {{
            actualizarFiltrosGlobales();
            aplicarFiltroAlcances(supervisorFiltroActual);
        }}
        
        if (seccionDestino === 'asistencias') {{
            actualizarFiltrosGlobales();
            aplicarFiltroAsistencias(supervisorFiltroActual);
        }}
    }}

    function aplicarFiltroAlcances(supervisor) {{
        const mesSeleccionado = document.getElementById('selectorMes').value;
        const añoSeleccionado = document.getElementById('selectorAño').value;
        const periodoCompleto = `${{mesSeleccionado}}_${{añoSeleccionado}}`;
        
        if (supervisor === 'TODOS') {{
            calcularEstadisticasRecuperos(periodoCompleto);
            generarGraficasRecuperos(periodoCompleto);
        }} else {{
            calcularEstadisticasRecuperosPorSupervisor(periodoCompleto, supervisor);
            generarGraficasRecuperosSupervisor(periodoCompleto, supervisor);
        }}
    }}
    
    function generarGraficasRecuperosSupervisor(periodoCompleto, supervisor) {{
        const graficasContainer = document.getElementById('graficas-recuperos');
        if (!graficasContainer) return;
        
        // Determinar título según supervisor
        let tituloGráfica = 'Recupero y Alcance';
        if (supervisor && supervisor !== 'TODOS') {{
            tituloGráfica = `Recupero y Alcance - ${{supervisor}}`;
        }}
        
            graficasContainer.innerHTML = `
              <div style="text-align: center; margin-bottom: 30px; display:flex; justify-content:center; gap:12px; flex-wrap:wrap;">
                  <button
                    class="boton-exportar-excel"
                    onclick="exportarTablaDetalleExcelAvanzado('${{supervisor || ''}}')"
                  >
                      📊 Exportar Recupero Diario ${{supervisor && supervisor !== 'TODOS' ? `- ${{supervisor}}` : ''}}
                  </button>
            
                  <button
                    class="boton-exportar-excel"
                    onclick="exportarRecupero4MesesDiaSupervisor('${{supervisor || ''}}')"
                  >
                      📈 Exportar Recupero 4M (Día x Mes) ${{supervisor && supervisor !== 'TODOS' ? `- ${{supervisor}}` : ''}}
                  </button>
              </div>
    
            <!-- CONTENEDOR PARA LAS CUATRO GRÁFICAS CON TOGGLE -->
            <div class="grafica-container" id="contenedorGraficasToggle" style="position: relative;">
                <!-- BARRA DE CONTROLES CON 4 BOTONES -->
                <div style="display: flex; justify-content: center; gap: 20px; margin-bottom: 20px; position: absolute; top: 15px; left: 0; right: 0; z-index: 10;">
                    <button id="btnRecuperoAcumulado" class="btn-toggle-grafica activo" onclick="cambiarGrafica('acumulado')" style="
                        padding: 10px 25px;
                        background: linear-gradient(135deg, #9b59b6, #8e44ad);
                        color: white;
                        border: none;
                        border-radius: 25px;
                        cursor: pointer;
                        font-size: 1rem;
                        font-weight: 600;
                        transition: all 0.3s ease;
                        box-shadow: 0 4px 10px rgba(52, 152, 219, 0.3);
                    ">
                        📈 Recupero Acumulado
                    </button>
                    
                    <button id="btnRecuperoDiario" class="btn-toggle-grafica" onclick="cambiarGrafica('diario')" style="
                        padding: 10px 25px;
                        background: linear-gradient(135deg, #9b59b6, #8e44ad);
                        color: white;
                        border: none;
                        border-radius: 25px;
                        cursor: pointer;
                        font-size: 1rem;
                        font-weight: 600;
                        transition: all 0.3s ease;
                        box-shadow: 0 4px 10px rgba(155, 89, 182, 0.3);
                    ">
                        📅 Recupero Diario
                    </button>
                    
                    <button id="btnRecuperoEquipos" class="btn-toggle-grafica" onclick="cambiarGrafica('equipos')" style="
                        padding: 10px 25px;
                        background: linear-gradient(135deg, #9b59b6, #8e44ad);
                        color: white;
                        border: none;
                        border-radius: 25px;
                        cursor: pointer;
                        font-size: 1rem;
                        font-weight: 600;
                        transition: all 0.3s ease;
                        box-shadow: 0 4px 10px rgba(46, 204, 113, 0.3);
                    ">
                        👥 Recupero por Equipos
                    </button>
                    
                    <button id="btnRecuperoAnual" class="btn-toggle-grafica" onclick="cambiarGrafica('anual')" style="
                        padding: 10px 25px;
                        background: linear-gradient(135deg, #9b59b6, #8e44ad);
                        color: white;
                        border: none;
                        border-radius: 25px;
                        cursor: pointer;
                        font-size: 1rem;
                        font-weight: 600;
                        transition: all 0.3s ease;
                        box-shadow: 0 4px 10px rgba(230, 126, 34, 0.3);
                    ">
                        📊 Recupero Anual
                    </button>
                </div>
                
                <!-- CANVAS PARA LAS GRÁFICAS (SOLO UNO VISIBLE A LA VEZ) -->
                <div id="graficaAcumuladoContainer" style="display: block;">
                    <canvas id="graficaIncrementoTotal" width="1200" height="700"></canvas>
                </div>
                
                <div id="graficaDiarioContainer" style="display: none;">
                    <canvas id="graficaRecuperoDiario" width="1200" height="700"></canvas>
                </div>
                
                <div id="graficaEquiposContainer" style="display: none;">
                    <canvas id="graficaAlcanceEquipos" width="1200" height="700"></canvas>
                </div>
                
                <div id="graficaAnualContainer" style="display: none;">
                    <canvas id="graficaEvolucionAnualRecuperos" width="1200" height="700"></canvas>
                </div>
            </div>
        `;
        
        // Generar gráficos después de un breve delay
        setTimeout(() => {{
            if (supervisor && supervisor !== 'TODOS') {{
                // Generar gráficas filtradas por supervisor
                generarGraficaIncrementoTotalSupervisor(periodoCompleto, supervisor);
                generarGraficaRecuperoDiarioSupervisor(periodoCompleto, supervisor);
            }} else {{
                // Generar gráficas para todos (comportamiento actual)
                generarGraficaIncrementoTotal(periodoCompleto);
                generarGraficaRecuperoDiario(periodoCompleto);
            }}
    
            generarGraficaAlcanceEquiposRecuperos(periodoCompleto);
            
            // Generar gráfica anual también
            actualizarGraficaEvolucionAnualRecuperos();
        }}, 100);
    }}

    function actualizarGraficaEvolucionAnualRecuperos() {{
        const añoSeleccionado = document.getElementById('selectorAño').value;
        const contenedorGrafica = document.getElementById('graficaAnualContainer');
        const añoElement = document.getElementById('año-grafica-evolucion-recuperos');
        
        if (!añoSeleccionado) {{
            // Mostrar mensaje si no hay año seleccionado
            const canvas = document.getElementById('graficaEvolucionAnualRecuperos');
            if (canvas) {{
                const ctx = canvas.getContext('2d');
                ctx.clearRect(0, 0, canvas.width, canvas.height);
                
                canvas.width = 1200;
                canvas.height = 200;
                canvas.style.display = 'block';
                canvas.style.margin = '0 auto';
                canvas.style.maxWidth = '100%';
                
                ctx.fillStyle = '#95a5a6';
                ctx.font = 'bold 20px Arial';
                ctx.textAlign = 'center';
                ctx.fillText('📊 Seleccione un año en los filtros superiores', canvas.width / 2, canvas.height / 2);
            }}
            return;
        }}
        
        // Mostrar contenedor y actualizar año
        contenedorGrafica.style.display = 'block';
        if (añoElement) {{
            añoElement.textContent = añoSeleccionado;
        }}
        
        // Generar gráfica de tendencia
        generarGraficaEvolucionAnualTendenciaRecuperos(añoSeleccionado);
    }}

    function generarGraficaEvolucionAnualTendenciaRecuperos(año) {{
        const mesesDelAño = obtenerMesesDelAño(año);
        if (mesesDelAño.length === 0) {{
            mostrarMensajeSinDatosAnualRecuperos('graficaEvolucionAnualRecuperos', año);
            return;
        }}
        
        // Calcular alcance total por mes
        const alcancesPorMes = mesesDelAño.map(mes => {{
            const periodoCompleto = `${{mes}}_${{año}}`;
            return calcularAlcanceTotalMes(periodoCompleto);
        }});
        
        const canvas = document.getElementById('graficaEvolucionAnualRecuperos');
        if (!canvas) return;
        
        const ctx = canvas.getContext('2d');
        
        // Destruir gráfica anterior si existe
        if (window.chartEvolucionAnualRecuperos) {{
            window.chartEvolucionAnualRecuperos.destroy();
            window.chartEvolucionAnualRecuperos = null;
        }}
        
        // Limpiar canvas
        ctx.clearRect(0, 0, canvas.width, canvas.height);
        
        canvas.width = 1200;
        canvas.height = 700;
        canvas.style.display = 'block';
        canvas.style.margin = '0 auto';
        canvas.style.maxWidth = '100%';
        
        // Abreviar nombres de meses
        const mesesAbreviados = mesesDelAño.map(mes => {{
            return mes.substring(0, 3).toUpperCase();
        }});
        
        // Calcular línea de tendencia (regresión lineal simple)
        const tendencia = calcularLineaTendencia(alcancesPorMes);
        
        try {{
            window.chartEvolucionAnualRecuperos = new Chart(ctx, {{
                type: 'line',
                data: {{
                    labels: mesesAbreviados,
                    datasets: [
                        {{
                            label: 'Alcance Real',
                            data: alcancesPorMes,
                            borderColor: '#3498db',
                            backgroundColor: 'rgba(52, 152, 219, 0.1)',
                            borderWidth: 3,
                            fill: true,
                            tension: 0.4,
                            pointRadius: 6,
                            pointHoverRadius: 10,
                            pointBackgroundColor: '#3498db',
                            pointBorderColor: '#ffffff',
                            pointBorderWidth: 2
                        }},
                        {{
                            label: 'Línea de Tendencia',
                            data: tendencia,
                            borderColor: '#e74c3c',
                            backgroundColor: 'transparent',
                            borderWidth: 2,
                            borderDash: [5, 5],
                            fill: false,
                            tension: 0,
                            pointRadius: 0
                        }},
                        {{
                            label: 'Meta 100%',
                            data: Array(mesesDelAño.length).fill(100),
                            borderColor: '#2ecc71',
                            backgroundColor: 'transparent',
                            borderWidth: 1,
                            borderDash: [3, 3],
                            fill: false,
                            tension: 0,
                            pointRadius: 0
                        }}
                    ]
                }},
                options: {{
                    responsive: false,
                    maintainAspectRatio: false,
                    plugins: {{
                        title: {{
                            display: true,
                            text: `Tendencia de Alcances Anual - Año ${{año}}`,
                            font: {{ size: 18, weight: 'bold' }},
                            padding: {{ bottom: 20 }}
                        }},
                        tooltip: {{
                            mode: 'index',
                            intersect: false,
                            backgroundColor: 'rgba(0, 0, 0, 0.9)',
                            titleFont: {{ size: 14, weight: 'bold' }},
                            bodyFont: {{ size: 13 }},
                            padding: 12,
                            cornerRadius: 8,
                            callbacks: {{
                                label: function(context) {{
                                    const label = context.dataset.label;
                            
                                    if (label === 'Meta 100%' || label === 'Línea de Tendencia') {{
                                        return null;
                                    }}
                            
                                    let texto = label ? label + ': ' : '';
                            
                                    if (context.parsed.y !== null) {{
                                        texto += context.parsed.y.toFixed(2) + '%';
                                    }}
                            
                                    return texto;
                                }}
                            }}
                        }},
                        legend: {{
                            position: 'top',
                            labels: {{ font: {{ size: 12 }} }}
                        }}
                    }},
                    scales: {{
                        x: {{
                            title: {{ 
                                display: true, 
                                text: 'Meses del Año',
                                font: {{ size: 14, weight: 'bold' }}
                            }},
                            ticks: {{ 
                                font: {{ size: 13 }}
                            }}
                        }},
                        y: {{
                            beginAtZero: true,
                            title: {{ 
                                display: true, 
                                text: 'Alcance Total (%)',
                                font: {{ size: 14, weight: 'bold' }}
                            }},
                            ticks: {{
                                callback: function(value) {{ return value + '%'; }},
                                stepSize: 10,
                                font: {{ size: 13 }}
                            }},
                            min: 0,
                            suggestedMax: function() {{
                                const valoresValidos = alcancesPorMes.filter(v => v !== null);
                                if (valoresValidos.length === 0) return 100;
                                const maxValor = Math.max(...valoresValidos);
                                return Math.max(100, Math.ceil(maxValor / 10) * 10);
                            }}()
                        }}
                    }},
                    interaction: {{
                        mode: 'nearest',
                        axis: 'x',
                        intersect: false
                    }}
                }}
            }});
        }} catch (error) {{
            console.error("❌ Error al crear la gráfica de tendencia para recuperos:", error);
            mostrarMensajeSinDatosAnualRecuperos('graficaEvolucionAnualRecuperos', año);
        }}
    }}

    function generarGraficaRecuperoDiarioSupervisor(periodoCompleto, supervisor) {{
        // Obtener asesores del supervisor específico
        const asesoresPeriodo = datosMeses[periodoCompleto] || [];
        const asesoresSupervisor = asesoresPeriodo.filter(a => a.supervisor === supervisor);
        
        if (asesoresSupervisor.length === 0) {{
            const canvas = document.getElementById('graficaRecuperoDiario');
            canvas.width = 1200;
            canvas.height = 700;
            canvas.style.display = 'block';
            canvas.style.margin = '0 auto';
            canvas.style.maxWidth = '100%';
            const ctx = canvas.getContext('2d');
            ctx.clearRect(0, 0, canvas.width, canvas.height);
            return;
        }}
        
        // Obtener todas las fechas únicas de los datos diarios de los asesores
        const todasFechas = new Set();
        asesoresSupervisor.forEach(asesor => {{
            Object.keys(asesor.datos_diarios_asesor || {{}}).forEach(fecha => {{
                todasFechas.add(fecha);
            }});
        }});
        
        const fechasOrdenadas = Array.from(todasFechas).sort((a, b) => 
            convertirFechaDiaria(a) - convertirFechaDiaria(b)
        );
        
        // Obtener meta del supervisor
        const supervisorData = datosSupervisores[supervisor];
        const datosMes = supervisorData ? supervisorData[periodoCompleto] : null;
        const metaSupervisor = datosMes ? datosMes.meta_super : 0;
        
        // Calcular días laborables y meta diaria
        const diasLaborables = fechasOrdenadas.length;
        const metaDiaria = diasLaborables > 0 ? metaSupervisor / diasLaborables : 0;
        
        // Calcular alcance diario del equipo completo del supervisor
        const alcanceDiario = fechasOrdenadas.map(fecha => {{
            let totalRecuperoDia = 0;
            asesoresSupervisor.forEach(asesor => {{
                totalRecuperoDia += (asesor.datos_diarios_asesor || {{}})[fecha] || 0;
            }});
            
            // Calcular porcentaje de alcance diario
            const porcentajeDiario = metaDiaria > 0 ? (totalRecuperoDia / metaDiaria) * 100 : 0;
            
            return {{
                fecha: fecha,
                recupero: totalRecuperoDia,
                porcentaje: porcentajeDiario
            }};
        }});
        
        const canvas = document.getElementById('graficaRecuperoDiario');
        canvas.width = 1300;
        canvas.height = 700;
        canvas.style.display = 'block';
        canvas.style.margin = '0 auto';
        canvas.style.maxWidth = '100%';
        const ctx = canvas.getContext('2d');

        // Destruir gráfica anterior si existe
        if (chartTendencia) {{
            chartTendencia.destroy();
            chartTendencia = null;
        }}
        
        ctx.clearRect(0, 0, canvas.width, canvas.height);
        
        const porcentajes = alcanceDiario.map(dia => dia.porcentaje);
        
        const coloresBarras = porcentajes.map(porcentaje => {{
            if (porcentaje <= 0) return '#F3E5F5';
            if (porcentaje <= 20) return '#E1BEE7';
            if (porcentaje <= 40) return '#BA68C8';
            if (porcentaje <= 60) return '#8E24AA';
            if (porcentaje <= 80) return '#6A1B9A';
            if (porcentaje <= 100) return '#4A148C';
            return '#38006B';
        }});

        // PLUGIN PARA ETIQUETAS DE DATOS (RECUPERO DIARIO)
        const dataLabelsPluginDiario = {{
            id: 'dataLabelsDiario',
            afterDatasetsDraw(chart, args, options) {{
                const {{ctx}} = chart;
                const meta = chart.getDatasetMeta(0);
                
                meta.data.forEach((bar, index) => {{
                    const value = chart.data.datasets[0].data[index];
                    if (value === null || value === undefined) return;
                    
                    ctx.save();
                    
                    const x = bar.x;
                    const y = bar.y - 20; // Posicionar más arriba de la barra
                    
                    // Obtener el recupero DIARIO (no acumulado)
                    const recuperoDiario = alcanceDiario[index]?.recupero || 0;
                    
                    // Formatear el valor COMPLETO con separador de miles
                    const formattedValue = recuperoDiario.toLocaleString('es-PE', {{
                        minimumFractionDigits: 0,
                        maximumFractionDigits: 0
                    }});
                    
                    // TAMAÑO DE FUENTE AUMENTADO (15px)
                    ctx.font = 'bold 15px Arial';
                    ctx.textAlign = 'center';
                    ctx.textBaseline = 'bottom';
                    
                    // Color de texto según el valor - SIN FONDO, SOLO TEXTO
                    let textColor = '#2c3e50';
                    if (recuperoDiario === 0) textColor = '#95a5a6';
                    else if (recuperoDiario < metaDiaria * 0.5) textColor = '#e74c3c';
                    else if (recuperoDiario < metaDiaria) textColor = '#f39c12';
                    else if (recuperoDiario < metaDiaria * 1.2) textColor = '#27ae60';
                    else textColor = '#2ecc71';
                    
                    ctx.fillStyle = textColor;
                    
                    // Sombra sutil para mejor legibilidad - SIN FONDO BLANCO
                    ctx.shadowColor = 'rgba(255, 255, 255, 0.95)';
                    ctx.shadowBlur = 8;
                    ctx.shadowOffsetX = 0;
                    ctx.shadowOffsetY = 0;
                    
                    // DIBUJAR SOLO EL TEXTO - SIN FONDO NI MARCO
                    ctx.fillText(formattedValue, x, y);
                    
                    ctx.restore();
                }});
            }}
        }};
        chartTendencia = new Chart(ctx, {{
            type: 'bar',
            data: {{
                labels: fechasOrdenadas.map(fecha => {{
                    if (fecha.includes('-')) {{
                        return fecha.split('-')[0];
                    }}
                    return fecha;
                }}),
                datasets: [{{
                    label: `Alcance Diario - ${{supervisor}}`,
                    data: porcentajes,
                    backgroundColor: coloresBarras,
                    borderColor: coloresBarras.map(color => color + 'CC'),
                    borderWidth: 2,
                    borderRadius: 6,
                    borderSkipped: false
                }}]
            }},
            options: {{
                responsive: false,
                maintainAspectRatio: false,
                interaction: {{
                    mode: 'index',
                    intersect: false
                }},
                plugins: {{
                    title: {{
                        display: true,
                        text: ['            ', '            '],
                        font: {{ size: 24, weight: 'bold' }}
                    }},
                    tooltip: {{
                        mode: 'nearest',
                        intersect: false,
                        backgroundColor: 'rgba(0, 0, 0, 0.8)',
                        titleFont: {{ size: 16, weight: 'bold' }},
                        bodyFont: {{ size: 14 }},
                        padding: 15,
                        cornerRadius: 10,
                        callbacks: {{
                            title: function(context) {{
                                const index = context[0].dataIndex;
                                return `Día ${{index + 1}}: ${{fechasOrdenadas[index]}}`;
                            }},
                            label: function(context) {{
                                const index = context.dataIndex;
                                const dia = alcanceDiario[index];
                                
                                return [
                                    `Alcance: ${{dia.porcentaje.toFixed(2)}}%`,
                                    `Recupero: S/ ${{dia.recupero.toLocaleString('es-PE', {{ 
                                        minimumFractionDigits: 2,
                                        maximumFractionDigits: 2 
                                    }})}}`,
                                    `Meta diaria: S/ ${{metaDiaria.toLocaleString('es-PE', {{ 
                                        minimumFractionDigits: 2,
                                        maximumFractionDigits: 2 
                                    }})}}`
                                ];
                            }},
                            afterLabel: function(context) {{
                                const index = context.dataIndex;
                                const dia = alcanceDiario[index];
                                
                                if (dia.porcentaje >= 100) {{
                                    return '✅ Superó la meta diaria';
                                }} else if (dia.porcentaje >= 70) {{
                                    return '🟡 Cerca de la meta';
                                }} else if (dia.porcentaje >= 40) {{
                                    return '🟠 Bajo rendimiento';
                                }} else {{
                                    return '🔴 Rendimiento crítico';
                                }}
                            }}
                        }}
                    }},
                    legend: {{
                        display: false
                    }}
                }},
                scales: {{
                    x: {{
                        title: {{
                            display: false,
                            text: 'Días del Mes',
                            font: {{ size: 14, weight: 'bold' }}
                        }},
                        ticks: {{
                            font: {{ size: 16 }},
                            maxRotation: 0,
                            minRotation: 0
                        }},
                        grid: {{
                            display: false
                        }}
                    }},
                    y: {{
                        type: 'linear',
                        display: true,
                        position: 'left',
                        title: {{
                            display: false,
                            text: 'Alcance Diario (%)',
                            font: {{ size: 14, weight: 'bold' }}
                        }},
                        ticks: {{
                            callback: function(value) {{
                                return value + '%';
                            }},
                            font: {{ size: 16 }},
                            stepSize: 10
                        }},
                        // Ajustar escala según el máximo alcance
                        min: 0,
                        suggestedMax: Math.max(100, Math.ceil(Math.max(...porcentajes) / 10) * 10)
                    }}
                }}
            }},
            plugins: [dataLabelsPluginDiario]  // ← AGREGAR EL PLUGIN AQUÍ
        }});
    }}

    function generarGraficaIncrementoTotalSupervisor(periodoCompleto, supervisor) {{
        const canvas = document.getElementById('graficaIncrementoTotal');
        if (!canvas) return;
        
        const ctx = canvas.getContext('2d');
        
        // Destruir gráfica anterior si existe
        if (window.chartIncrementoTotal) {{
            window.chartIncrementoTotal.destroy();
            window.chartIncrementoTotal = null;
        }}
        
        // Limpiar canvas
        ctx.clearRect(0, 0, canvas.width, canvas.height);
        
        // Obtener datos del supervisor
        const supervisorData = datosSupervisores[supervisor];
        if (!supervisorData || !supervisorData[periodoCompleto]) {{
            mostrarMensajeSinDatos(canvas, periodoCompleto, supervisor);
            return;
        }}
        
        const datosMes = supervisorData[periodoCompleto];
        const metaSupervisor = datosMes.meta_super || 0;
        const datosDiarios = datosMes.datos_diarios_supervisor || {{}};
        
        if (Object.keys(datosDiarios).length === 0) {{
            mostrarMensajeSinDatos(canvas, periodoCompleto, supervisor);
            return;
        }}
        
        // ========== CAMBIO 1: OBTENER TODAS LAS FECHAS (sin filtrar por datos > 0) ==========
        // Ordenar TODAS las fechas cronológicamente
        const todasFechas = Object.keys(datosDiarios).sort((a, b) => 
            convertirFechaDiaria(a) - convertirFechaDiaria(b)
        );
        
        console.log(`📊 Supervisor ${{supervisor}} - Total fechas: ${{todasFechas.length}}`);
        
        // ========== CAMBIO 2: CALCULAR ALCANCE PARA CADA FECHA ==========
        const alcancesPorcentaje = [];
        const recuperoAcumuladoPorFecha = {{}};
        const fechasConDatosReales = [];  // Solo fechas con datos > 0
        const datosValidosParaGrafica = [];  // Datos para la línea
        
        // Inicializar todas las fechas
        todasFechas.forEach(fecha => {{
            const recuperoAcumulado = datosDiarios[fecha] || 0;
            recuperoAcumuladoPorFecha[fecha] = recuperoAcumulado;
            
            const alcanceDia = metaSupervisor > 0 ? (recuperoAcumulado / metaSupervisor) * 100 : 0;
            
            // SOLO agregar a datos válidos si hay datos reales (recupero > 0)
            if (recuperoAcumulado > 0) {{
                fechasConDatosReales.push(fecha);
                datosValidosParaGrafica.push(alcanceDia);
                alcancesPorcentaje.push(alcanceDia);
            }} else {{
                // Para días sin datos, poner null (la línea no se dibujará aquí)
                datosValidosParaGrafica.push(null);
                alcancesPorcentaje.push(0);
            }}
        }});
        
        console.log(`📊 Fechas con datos > 0: ${{fechasConDatosReales.length}}`);
        
        // ========== 3. ENCONTRAR EL ÚLTIMO DÍA CON DATOS REALES ==========
        let ultimoDiaConDatos = -1;
        for (let i = todasFechas.length - 1; i >= 0; i--) {{
            const fecha = todasFechas[i];
            const recuperoDia = recuperoAcumuladoPorFecha[fecha] || 0;
            if (recuperoDia > 0) {{
                ultimoDiaConDatos = i;
                break;
            }}
        }}
        
        if (ultimoDiaConDatos === -1) {{
            ultimoDiaConDatos = todasFechas.length - 1;
        }}
        
        // ========== 4. CREAR LA GRÁFICA DE LÍNEA CON ÁREA ==========
        canvas.width = 1200;
        canvas.height = 700;
        canvas.style.display = 'block';
        canvas.style.margin = '0 auto';
        canvas.style.maxWidth = '100%';
        
        // USAR TODAS LAS FECHAS para las etiquetas (CAMBIO PRINCIPAL)
        const labelsParaGrafica = todasFechas.map(fecha => {{
            if (fecha.includes('-')) {{
                return fecha.split('-')[0];  // Mostrar solo el número del día
            }}
            return fecha;
        }});
        
        // COLORES PARA GRÁFICA DE LÍNEA CON ÁREA (MORADO)
        const colorLinea = '#9b59b6';
        const colorArea = 'rgba(155, 89, 182, 0.2)';
        const colorPuntoInactivo = '#e0e0e0';
        
        // PLUGIN PARA ETIQUETAS DE DATOS (PORCENTAJE DE ALCANCE)
        const dataLabelsPluginAccumulated = {{
            id: 'dataLabelsAccumulated',
            afterDatasetsDraw(chart, args, options) {{
                const {{ctx}} = chart;
                const meta = chart.getDatasetMeta(0);
                
                meta.data.forEach((point, index) => {{
                    const value = chart.data.datasets[0].data[index];
                    if (value === null || value === undefined) return;
                    
                    // Solo mostrar etiqueta si hay datos > 0
                    if (value > 0) {{
                        ctx.save();
                        
                        const x = point.x;
                        const y = point.y - 20;
                        
                        const porcentajeAlcance = value;
                        const formattedValue = porcentajeAlcance.toFixed(1) + '%';
                        
                        ctx.font = 'bold 15px Arial';
                        ctx.textAlign = 'center';
                        ctx.textBaseline = 'bottom';
                        
                        // Color de texto según el porcentaje
                        let textColor = '#2c3e50';
                        if (porcentajeAlcance === 0) textColor = '#95a5a6';
                        else if (porcentajeAlcance < 40) textColor = '#e74c3c';
                        else if (porcentajeAlcance < 70) textColor = '#f39c12';
                        else if (porcentajeAlcance < 100) textColor = '#27ae60';
                        else textColor = '#2ecc71';
                        
                        ctx.fillStyle = textColor;
                        
                        ctx.shadowColor = 'rgba(255, 255, 255, 0.95)';
                        ctx.shadowBlur = 8;
                        ctx.shadowOffsetX = 0;
                        ctx.shadowOffsetY = 0;
                        
                        ctx.fillText(formattedValue, x, y);
                        
                        ctx.restore();
                    }}
                }});
            }}
        }};
        
        try {{
            window.chartIncrementoTotal = new Chart(ctx, {{
                type: 'line',
                data: {{
                    labels: labelsParaGrafica,  // TODAS las fechas
                    datasets: [{{
                        label: `Alcance Acumulado - ${{supervisor}}`,
                        data: datosValidosParaGrafica,  // Datos: válidos donde hay > 0, null donde no
                        backgroundColor: colorArea,
                        borderColor: colorLinea,
                        borderWidth: 3,
                        fill: true,
                        tension: 0.4,
                        pointRadius: function(context) {{
                            // Puntos más grandes para datos válidos, pequeños o invisibles para nulos
                            const value = context.dataset.data[context.dataIndex];
                            return value === null || value === 0 ? 0 : 6;
                        }},
                        pointHoverRadius: function(context) {{
                            const value = context.dataset.data[context.dataIndex];
                            return value === null || value === 0 ? 0 : 8;
                        }},
                        pointBackgroundColor: function(context) {{
                            const value = context.dataset.data[context.dataIndex];
                            return value === null || value === 0 ? 'transparent' : colorLinea;
                        }},
                        pointBorderColor: function(context) {{
                            const value = context.dataset.data[context.dataIndex];
                            return value === null || value === 0 ? 'transparent' : '#ffffff';
                        }},
                        pointBorderWidth: 2,
                        spanGaps: false  // No conectar puntos nulos
                    }}]
                }},
                options: {{
                    responsive: false,
                    maintainAspectRatio: false,
                    plugins: {{
                        title: {{
                            display: true,
                            text: [`            `,`            `],
                            font: {{ size: 22, weight: 'bold' }},
                            padding: {{ bottom: 25 }}
                        }},
                        tooltip: {{
                            mode: 'nearest',
                            intersect: false,
                            backgroundColor: 'rgba(0, 0, 0, 0.9)',
                            titleFont: {{ size: 16, weight: 'bold' }},
                            bodyFont: {{ size: 14 }},
                            padding: 15,
                            cornerRadius: 10,
                            callbacks: {{
                                title: function(context) {{
                                    const index = context[0].dataIndex;
                                    return 'Día ' + (index + 1) + ': ' + todasFechas[index];
                                }},
                                label: function(context) {{
                                    const index = context.dataIndex;
                                    const alcance = context.parsed.y;
                                    const recuperoAcumulado = recuperoAcumuladoPorFecha[todasFechas[index]] || 0;
                                    
                                    // Si no hay datos para este día
                                    if (index > ultimoDiaConDatos || alcance === null) {{
                                        return '📭 Sin datos de recupero';
                                    }}
                                    
                                    // Calcular incremento del día
                                    let incrementoDia = 0;
                                    if (index > 0) {{
                                        const alcanceAnterior = context.chart.data.datasets[0].data[index - 1] || 0;
                                        incrementoDia = (alcance - alcanceAnterior);
                                    }} else {{
                                        incrementoDia = alcance;
                                    }}
                                    
                                    return [
                                        '🎯 Alcance Acumulado: ' + alcance.toFixed(2) + '%',
                                        '📈 Alcance del día: ' + incrementoDia.toFixed(2) + '%',
                                        '💰 Recupero Acumulado: ' + recuperoAcumulado.toLocaleString('es-PE', {{ 
                                            minimumFractionDigits: 0,
                                            maximumFractionDigits: 0 
                                        }})
                                    ];
                                }}
                            }}
                        }},
                        legend: {{ display: false }}
                    }},
                    scales: {{
                        x: {{
                            title: {{ 
                                display: false, 
                                text: 'Días del Mes',
                                font: {{ size: 14, weight: 'bold' }}
                            }},
                            ticks: {{ 
                                font: {{ size: 16 }},
                                maxRotation: 0,
                                minRotation: 0,
                                autoSkip: false,
                                callback: function(value, index, values) {{
                                    // Colores diferentes para días con/sin datos
                                    const tieneDatos = datosValidosParaGrafica[index] !== null && 
                                                      datosValidosParaGrafica[index] > 0;
                                    this.fontColor = tieneDatos ? '#2c3e50' : '#bdc3c7';
                                    return this.getLabelForValue(value);
                                }}
                            }},
                            grid: {{ 
                                display: true,
                                color: 'rgba(0, 0, 0, 0.05)'
                            }}
                        }},
                        y: {{
                            beginAtZero: true,
                            title: {{ 
                                display: false, 
                                text: 'Alcance Acumulado (%)',
                                font: {{ size: 16, weight: 'bold' }}
                            }},
                            ticks: {{
                                callback: function(value) {{ return value + '%'; }},
                                stepSize: 10,
                                font: {{ size: 16 }}
                            }},
                            min: 0,
                            suggestedMax: function() {{
                                // Calcular máximo solo de los datos válidos
                                const valoresValidos = datosValidosParaGrafica.filter(v => v !== null && v > 0);
                                if (valoresValidos.length === 0) return 100;
                                const maxValor = Math.max(...valoresValidos);
                                return Math.max(100, Math.ceil(maxValor / 10) * 10);
                            }}(),
                            grid: {{
                                display: true,
                                color: 'rgba(0, 0, 0, 0.05)'
                            }}
                        }}
                    }},
                    interaction: {{
                        mode: 'nearest',
                        axis: 'x',
                        intersect: false
                    }},
                    elements: {{
                        line: {{
                            tension: 0.4  // Suavizado de la línea
                        }},
                        point: {{
                            hoverBackgroundColor: function(context) {{
                                const value = context.dataset.data[context.dataIndex];
                                return value === null || value === 0 ? 'transparent' : colorLinea;
                            }}
                        }}
                    }}
                }},
                plugins: [dataLabelsPluginAccumulated]
            }});
            
        }} catch (error) {{
            console.error("Error al crear la gráfica:", error);
            // Mostrar mensaje de error en el canvas
            ctx.fillStyle = '#e74c3c';
            ctx.font = 'bold 20px Arial';
            ctx.textAlign = 'center';
            ctx.fillText('Error al generar gráfica', canvas.width / 2, canvas.height / 2);
        }}
    }}
    
    function calcularEstadisticasRecuperosPorSupervisor(periodoCompleto, supervisor) {{
        // 1. OBTENER DATOS DEL SUPERVISOR ESPECÍFICO
        const supervisorData = datosSupervisores[supervisor];
        
        if (!supervisorData || !supervisorData[periodoCompleto]) {{
            console.log(`⚠️ No hay datos para el supervisor: ${{supervisor}} en ${{periodoCompleto}}`);
            mostrarEstadisticasVacias(periodoCompleto, supervisor);
            return;
        }}
        
        const datosMes = supervisorData[periodoCompleto];
        
        // 2. DATOS BÁSICOS DEL SUPERVISOR
        const metaTotalMes = datosMes.meta_super || 0;
        const recuperoTotalActual = datosMes.total_recupero || 0;
        
        // 3. OBTENER DÍAS LABORABLES (de los datos diarios del supervisor)
        const datosDiarios = datosMes.datos_diarios_supervisor || {{}};
        let fechas = Object.keys(datosDiarios);
        
        if (fechas.length === 0) {{
            // Si no hay datos diarios, intentar con alcance acumulado
            const alcanceDiario = datosMes.alcance_acumulado_diario || {{}};
            fechas = Object.keys(alcanceDiario);
        }}
        
        const fechasOrdenadas = fechas.sort((a, b) => 
            convertirFechaDiaria(a) - convertirFechaDiaria(b)
        );
        
        const diasLaborables = fechasOrdenadas.length;
        
        // 4. CALCULAR DÍAS TRABAJADOS (con recupero > 0)
        let diasTrabajados = 0;
        let totalRecuperoDiasTrabajados = 0;
        
        if (fechasOrdenadas.length > 0) {{
            for (let i = 0; i < fechasOrdenadas.length; i++) {{
                const fecha = fechasOrdenadas[i];
                let recuperoDia = 0;
                
                // Calcular recupero del día (diferencia entre acumulados)
                if (datosDiarios[fecha] !== undefined) {{
                    if (i === 0) {{
                        recuperoDia = datosDiarios[fecha] || 0;
                    }} else {{
                        const fechaAnterior = fechasOrdenadas[i-1];
                        const recuperoAnterior = datosDiarios[fechaAnterior] || 0;
                        const recuperoActual = datosDiarios[fecha] || 0;
                        recuperoDia = recuperoActual - recuperoAnterior;
                    }}
                }}
                
                if (recuperoDia > 0) {{
                    diasTrabajados++;
                    totalRecuperoDiasTrabajados += recuperoDia;
                }}
            }}
        }}
        
        // 5. CALCULAR ALCANCE ACTUAL
        const alcanceActual = metaTotalMes > 0 ? 
            parseFloat(((recuperoTotalActual / metaTotalMes) * 100).toFixed(2)) : 0;
        
        // 6. CALCULAR META DIARIA (basada en días laborables)
        const metaDiaria = diasLaborables > 0 ? metaTotalMes / diasLaborables : 0;
        
        // 7. CALCULAR PROMEDIO DIARIO REAL
        const promedioDiarioReal = diasTrabajados > 0 ? totalRecuperoDiasTrabajados / diasTrabajados : 0;
        
        // 8. CALCULAR PROYECCIONES
        const diasRestantes = Math.max(0, diasLaborables - diasTrabajados);
        const recuperoProyectado = promedioDiarioReal * diasRestantes;
        const recuperoTotalProyectado = recuperoTotalActual + recuperoProyectado;
        const eficienciaDiaria = metaTotalMes > 0 ? (recuperoTotalProyectado / metaTotalMes) * 100 : 0;
        
        // 9. Calcular eficiencia vs meta diaria
        const eficienciaVsMetaDiaria = metaDiaria > 0 ? (promedioDiarioReal / metaDiaria) * 100 : 0;
        
        // 10. MOSTRAR EN INTERFAZ
        mostrarEstadisticasUnificada(
            periodoCompleto,
            {{
                metaTotalMes,
                recuperoTotalActual,
                alcanceActual,
                eficienciaDiaria,
                diasLaborables,
                diasTrabajados,
                metaDiaria,
                promedioDiarioReal,
                eficienciaVsMetaDiaria,
                recuperoProyectado,
                recuperoTotalProyectado,
                diasRestantes
            }},
            'supervisor',
            supervisor
        );
    }}

    function mostrarEstadisticasVacias(periodoCompleto, supervisor = null) {{
        const statsElement = document.getElementById('estadisticas-recuperos');
        if (!statsElement) return;
        
        const [mes, año] = periodoCompleto.split('_');
        
        let mensajeSupervisor = '';
        if (supervisor) {{
            mensajeSupervisor = `Supervisor: ${{supervisor}}`;
        }}
        
        statsElement.innerHTML = `
            <div style="text-align: center; padding: 40px; background: #f8f9fa; border-radius: 12px; border: 2px dashed #95a5a6; margin: 20px 0;">
                <div style="font-size: 2.5rem; color: #95a5a6; margin-bottom: 20px;">📭</div>
                <div style="font-size: 1.5rem; font-weight: bold; color: #7f8c8d; margin-bottom: 10px;">
                    No hay datos de supervisores disponibles
                </div>
                <div style="font-size: 1.1rem; color: #95a5a6;">
                    Periodo: ${{mes}} ${{año}}
                </div>
                ${{mensajeSupervisor ? `<div style="font-size: 1.1rem; color: #95a5a6; margin-top: 10px;">${{mensajeSupervisor}}</div>` : ''}}
            </div>
        `;
    }}

    function mostrarEstadisticasUnificada(
        periodoCompleto,
        estadisticas,  // Objeto con todas las métricas
        contexto = 'global',  // 'global' o 'supervisor'
        supervisorNombre = null
    ) {{
        const {{
            metaTotalMes,
            recuperoTotalActual,
            alcanceActual,
            eficienciaDiaria,
            diasLaborables,
            diasTrabajados,
            metaDiaria,
            promedioDiarioReal,
            eficienciaVsMetaDiaria,
            recuperoProyectado,
            recuperoTotalProyectado,
            diasRestantes
        }} = estadisticas;
        
        // CÁLCULOS COMUNES
        const formatoMoneda = (valor) => 'S/ ' + valor.toLocaleString('es-PE', {{minimumFractionDigits: 0}});
        const [mes, año] = periodoCompleto.split('_');
        const porcentajeTiempo = diasLaborables > 0 ? ((diasTrabajados/diasLaborables)*100).toFixed(0) : 0;
        const progresoTiempoPorcentaje = diasLaborables > 0 ? (diasTrabajados/diasLaborables)*100 : 0;
        
        // COLORES SEGÚN VALORES
        const colorAlcanceActual = alcanceActual >= 100 ? '#27ae60' : 
                                  alcanceActual >= 70 ? '#f39c12' : 
                                  alcanceActual >= 40 ? '#e67e22' : '#e74c3c';
        
        const colorEficiencia = eficienciaDiaria >= 100 ? '#27ae60' : 
                               eficienciaDiaria >= 70 ? '#f39c12' : 
                               eficienciaDiaria >= 40 ? '#e67e22' : '#e74c3c';
        
        // TEXTO CONTEXTUAL (sin títulos)
        const textoMeta = contexto === 'supervisor' ? 'Meta del supervisor' : 'Meta total del período';
        const textoRecupero = contexto === 'supervisor' ? 'Total recuperado' : 'Total recuperado por todos';
        
        // GENERAR HTML SIN TÍTULOS
        const html = `
            <!-- PRIMERA FILA: INDICADORES CLAVE -->
            <div style="display: grid; grid-template-columns: repeat(4, 1fr); gap: 15px; margin-bottom: 15px;">
                <!-- DÍAS TRABAJADOS -->
                <div style="text-align: center; padding: 18px; background: #E8F4FD; border-radius: 10px; border: 2px solid #3498db;">
                    <div style="font-size: 0.95rem; color: #3498db; margin-bottom: 5px; font-weight: 600;">DÍAS TRABAJADOS</div>
                    <div style="font-size: 2rem; font-weight: bold; color: #3498db;">${{diasTrabajados}}/${{diasLaborables}}</div>
                    <div style="font-size: 0.85rem; color: #666; margin-top: 5px;">${{porcentajeTiempo}}% del mes</div>
                </div>
                
                <!-- META DEL MES -->
                <div style="text-align: center; padding: 18px; background: #F4ECF7; border-radius: 10px; border: 2px solid #9b59b6;">
                    <div style="font-size: 0.95rem; color: #9b59b6; margin-bottom: 5px; font-weight: 600;">META DEL MES</div>
                    <div style="font-size: 1.5rem; font-weight: bold; color: #9b59b6;">${{formatoMoneda(metaTotalMes)}}</div>
                    <div style="font-size: 0.85rem; color: #666; margin-top: 5px;">${{textoMeta}}</div>
                </div>
                
                <!-- PROMEDIO DIARIO REAL -->
                <div style="text-align: center; padding: 18px; background: #FEF5E7; border-radius: 10px; border: 2px solid #f39c12;">
                    <div style="font-size: 0.95rem; color: #f39c12; margin-bottom: 5px; font-weight: 600;">PROMEDIO DIARIO</div>
                    <div style="font-size: 1.5rem; font-weight: bold; color: #f39c12;">${{formatoMoneda(promedioDiarioReal)}}</div>
                    <div style="font-size: 0.85rem; color: #666; margin-top: 5px;">Meta por día: ${{formatoMoneda(metaDiaria)}}</div>
                </div>
                
                <!-- RECUPERO ACTUAL -->
                <div style="text-align: center; padding: 18px; background: #E8F6F3; border-radius: 10px; border: 2px solid #27ae60;">
                    <div style="font-size: 0.95rem; color: #27ae60; margin-bottom: 5px; font-weight: 600;">RECUPERO ACTUAL</div>
                    <div style="font-size: 1.8rem; font-weight: bold; color: #27ae60;">${{formatoMoneda(recuperoTotalActual)}}</div>
                    <div style="font-size: 0.85rem; color: #666; margin-top: 5px;">${{textoRecupero}}</div>
                </div>
            </div>
            
            <!-- SEGUNDA FILA: ALCANCE vs EFICIENCIA -->
            <div style="display: grid; grid-template-columns: repeat(2, 1fr); gap: 20px; margin-bottom: 25px;">
                <!-- ALCANCE ACTUAL -->
                <div style="text-align: center; padding: 25px; background: linear-gradient(135deg, #D1E9FB, #A9CCE3); border-radius: 10px; border: 3px solid #3498db; box-shadow: 0 4px 12px rgba(52, 152, 219, 0.15);">
                    <div style="font-size: 1.1rem; color: #3498db; margin-bottom: 10px; font-weight: 600;">
                        🎯 ALCANCE ACTUAL
                    </div>
                    <div style="font-size: 3rem; font-weight: bold; color: #3498db; margin: 10px 0;">${{alcanceActual.toFixed(2)}}%</div>
                    <div style="font-size: 1.1rem; color: #2c3e50; margin-top: 5px;">
                        ${{formatoMoneda(recuperoTotalActual)}} / ${{formatoMoneda(metaTotalMes)}}
                    </div>
                    <div style="font-size: 0.9rem; color: #666; margin-top: 10px;">
                        ${{diasTrabajados}} días de ${{diasLaborables}} (${{porcentajeTiempo}}% del tiempo)
                    </div>
                </div>
                
                <!-- EFICIENCIA DIARIA = ALCANCE PROYECTADO -->
                <div style="text-align: center; padding: 25px; background: linear-gradient(135deg, ${{colorEficiencia}}20, ${{colorEficiencia}}40); border-radius: 10px; border: 3px solid ${{colorEficiencia}}; box-shadow: 0 4px 12px rgba(0,0,0,0.1);">
                    <div style="font-size: 1.1rem; color: ${{colorEficiencia}}; margin-bottom: 10px; font-weight: 600;">
                        📊 ALCANCE PROYECTADO
                    </div>
                    <div style="font-size: 3rem; font-weight: bold; color: ${{colorEficiencia}}; margin: 10px 0;">${{eficienciaDiaria.toFixed(2)}}%</div>
                    <div style="font-size: 1.1rem; color: #2c3e50; margin-top: 5px;">
                        ${{formatoMoneda(recuperoTotalProyectado)}} proyectados
                    </div>
                    <div style="font-size: 0.9rem; color: #666; margin-top: 10px;">
                        +${{formatoMoneda(recuperoProyectado)}} en ${{diasRestantes}} días restantes
                    </div>
                </div>
            </div>
            
            <!-- BARRA DE PROGRESO DOBLE: TIEMPO Y ALCANCE -->
            <div style="margin-top: 25px; background: #f8f9fa; padding: 20px; border-radius: 10px; border: 1px solid #ddd;">
                <!-- BARRA 1: PROGRESO DE ALCANCE -->
                <div>
                    <div style="display: flex; justify-content: space-between; margin-bottom: 8px;">
                        <div style="font-weight: 600; color: #2c3e50;">
                            <span style="color: ${{colorAlcanceActual}};">🎯</span> Progreso de Alcance
                        </div>
                        <div style="font-weight: 600; color: ${{colorAlcanceActual}};">
                            ${{alcanceActual.toFixed(2)}}% de ${{metaTotalMes > 0 ? '100%' : '0%'}}
                        </div>
                    </div>
                    <div style="height: 16px; background: #ecf0f1; border-radius: 8px; overflow: hidden; position: relative;">
                        <!-- Fondo de la barra -->
                        <div style="height: 100%; width: 100%; background: #ecf0f1; border-radius: 8px;"></div>
                        
                        <!-- Progreso del alcance actual -->
                        <div style="position: absolute; top: 0; left: 0; height: 100%; width: ${{Math.min(alcanceActual, 100)}}%; background: linear-gradient(90deg, ${{colorAlcanceActual}}, ${{colorAlcanceActual}}CC); border-radius: 8px; transition: width 0.5s ease;"></div>
                        
                        <!-- Línea de meta (100%) -->
                        <div style="position: absolute; left: 100%; top: -2px; width: 3px; height: 20px; background: #2c3e50; border-radius: 1.5px;"></div>
                        
                        <!-- Marcador del alcance actual -->
                        <div style="position: absolute; left: ${{Math.min(alcanceActual, 100)}}%; top: -4px; width: 6px; height: 24px; background: ${{colorAlcanceActual}}; border-radius: 3px; box-shadow: 0 2px 4px rgba(0,0,0,0.2);"></div>
                        
                        <!-- Etiqueta del alcance actual -->
                        <div style="position: absolute; left: ${{Math.min(alcanceActual, 100)}}%; top: -30px; transform: translateX(-50%); background: ${{colorAlcanceActual}}; color: white; padding: 4px 8px; border-radius: 4px; font-size: 0.8rem; font-weight: 600; white-space: nowrap;">
                            ${{alcanceActual.toFixed(2)}}%
                            <div style="position: absolute; bottom: -4px; left: 50%; transform: translateX(-50%); width: 0; height: 0; border-left: 4px solid transparent; border-right: 4px solid transparent; border-top: 4px solid ${{colorAlcanceActual}};"></div>
                        </div>
                    </div>
                    <div style="display: flex; justify-content: space-between; margin-top: 8px; font-size: 0.85rem; color: #666;">
                        <div>0%</div>
                        <div>50%</div>
                        <div style="color: ${{alcanceActual > 100 ? colorAlcanceActual : '#666'}};">
                            ${{alcanceActual > 100 ? alcanceActual.toFixed(2) + '%' : '100%'}}
                        </div>
                    </div>
                </div>
    
                <!-- BARRA 2: PROGRESO DEL TIEMPO -->
                <div style="margin-bottom: 25px;">
                    <div style="display: flex; justify-content: space-between; margin-bottom: 8px;">
                        <div style="font-weight: 600; color: #2c3e50;">
                            <span style="color: #3498db;">⏱️</span> Progreso del Tiempo
                        </div>
                        <div style="font-weight: 600; color: #2c3e50;">
                            ${{diasTrabajados}}/${{diasLaborables}} días (${{Math.round(progresoTiempoPorcentaje)}}%)
                        </div>
                    </div>
                    <div style="height: 16px; background: #ecf0f1; border-radius: 8px; overflow: hidden; position: relative;">
                        <div style="height: 100%; width: ${{progresoTiempoPorcentaje}}%; background: linear-gradient(90deg, #3498db, #2ecc71); border-radius: 8px; transition: width 0.5s ease;"></div>
                        <!-- Marcador de tiempo transcurrido -->
                        <div style="position: absolute; left: 50%; top: -2px; width: 3px; height: 20px; background: rgba(0,0,0,0.3); border-radius: 1.5px;"></div>
                    </div>
                    <div style="display: flex; justify-content: space-between; margin-top: 8px; font-size: 0.85rem; color: #666;">
                        <div>Inicio del mes</div>
                        <div>Mitad del mes</div>
                        <div>Fin del mes</div>
                    </div>
                </div>
            </div>
        `;
        
        // ACTUALIZAR EL ELEMENTO HTML
        const statsElement = document.getElementById('estadisticas-recuperos');
        if (statsElement) {{
            statsElement.innerHTML = html;
        }}
    }}
    
    // FUNCIÓN PARA MOSTRAR ESTADÍSTICAS VACÍAS (CASO ESPECIAL)
    function mostrarEstadisticasVacias(periodoCompleto, supervisor = null) {{
        const statsElement = document.getElementById('estadisticas-recuperos');
        if (!statsElement) return;
        
        const [mes, año] = periodoCompleto.split('_');
        
        let mensajeSupervisor = '';
        if (supervisor) {{
            mensajeSupervisor = `Supervisor: ${{supervisor}}`;
        }}
        
        statsElement.innerHTML = `
            <div style="text-align: center; padding: 40px; background: #f8f9fa; border-radius: 12px; border: 2px dashed #95a5a6; margin: 20px 0;">
                <div style="font-size: 2.5rem; color: #95a5a6; margin-bottom: 20px;">📭</div>
                <div style="font-size: 1.5rem; font-weight: bold; color: #7f8c8d; margin-bottom: 10px;">
                    No hay datos de supervisores disponibles
                </div>
                <div style="font-size: 1.1rem; color: #95a5a6;">
                    Periodo: ${{mes}} ${{año}}
                </div>
                ${{mensajeSupervisor ? `<div style="font-size: 1.1rem; color: #95a5a6; margin-top: 10px;">${{mensajeSupervisor}}</div>` : ''}}
            </div>
        `;
    }}

    function actualizarPeriodo() {{
        const mesSeleccionado = document.getElementById('selectorMes').value;
        const añoSeleccionado = document.getElementById('selectorAño').value;
        const periodoCompleto = `${{mesSeleccionado}}_${{añoSeleccionado}}`;
        
        if (!datosMeses[periodoCompleto]) {{
            return;
        }}
        
        // DETECTAR QUÉ SECCIÓN ESTÁ ACTIVA
        const seccionRanking = document.getElementById('seccion-ranking');
        const seccionRecuperos = document.getElementById('seccion-recuperos');
        const seccionAsistencias = document.getElementById('seccion-asistencias');
        
        if (seccionRanking && seccionRanking.classList.contains('activa')) {{
            // Estamos en RANKING
            console.log(`📅 Cambiando periodo a ${{periodoCompleto}} en RANKING`);
            const asesores = datosMeses[periodoCompleto] || [];
            actualizarLista(asesores, '>100%', 'lista-100');
            actualizarLista(asesores, '>70%', 'lista-70');
            actualizarLista(asesores, '>40%', 'lista-40');
            actualizarLista(asesores, '>0%', 'lista-0');
            actualizarEstadisticas(asesores);
            
            // Si hay comparaciones activas, actualizarlas también
            if (asesoresSeleccionados.size > 0) {{
                actualizarTablaComparacionAsesores();
                actualizarGraficaComparacionAsesores();
            }}
            if (supervisoresSeleccionados.size > 0) {{
                actualizarTablaComparacionSupervisores();
                actualizarGraficaComparacionSupervisores();
            }}
        }}
        else if (seccionRecuperos && seccionRecuperos.classList.contains('activa')) {{
            // Estamos en RECUPEROS
            console.log(`📅 Cambiando periodo a ${{periodoCompleto}} en ALCANCES`);
            const asesores = datosMeses[periodoCompleto] || [];
            calcularEstadisticasRecuperos(periodoCompleto);
            generarGraficasRecuperos(periodoCompleto);
            
            // NUEVO: Actualizar gráfica de evolución anual si está visible
            const containerAnual = document.getElementById('graficaAnualContainer');
            if (containerAnual && containerAnual.style.display !== 'none') {{
                actualizarGraficaEvolucionAnualRecuperos();
            }}
        }}
        else if (seccionAsistencias && seccionAsistencias.classList.contains('activa')) {{
            // Estamos en ASISTENCIAS
            console.log(`📅 Cambiando periodo a ${{periodoCompleto}} en ASISTENCIAS`);
            const asesores = datosMeses[periodoCompleto] || [];
            
            // Determinar supervisor activo
            const botonActivo = document.querySelector('#contenedorBotonesSupervisoresAsistencias .boton-supervisor.activo');
            let supervisorActivo = 'TODOS';
            
            if (botonActivo) {{
                const textoBoton = botonActivo.textContent.trim();
                if (textoBoton !== '👥 TODOS LOS EQUIPOS') {{
                    supervisorActivo = textoBoton;
                }}
            }}
            
            cargarDatosAsistencias(supervisorActivo);
        }}
    }}

    function calcularStepSize(valorMaximo) {{
        // Calcula un stepSize apropiado según el valor máximo
        if (valorMaximo <= 10) return 1;
        if (valorMaximo <= 50) return 5;
        if (valorMaximo <= 100) return 10;
        if (valorMaximo <= 200) return 20;
        if (valorMaximo <= 500) return 50;
        return 100;
    }}
    
    // ========== FUNCIONES PARA RANKING ==========
    function aplicarFiltroRanking(supervisor) {{
      const mesSeleccionado = document.getElementById('selectorMes').value;
      const añoSeleccionado = document.getElementById('selectorAño').value;
      const periodoCompleto = `${{mesSeleccionado}}_${{añoSeleccionado}}`;
    
      if (!datosMeses[periodoCompleto]) return;
    
      let asesoresBase = datosMeses[periodoCompleto] || [];
    
      if (supervisor !== 'TODOS') {{
        asesoresBase = asesoresBase.filter(a => a.supervisor === supervisor);
      }}
    
      // ✅ Ranking vuelve a su vista normal (mes seleccionado)
      actualizarLista(asesoresBase, '>100%', 'lista-100');
      actualizarLista(asesoresBase, '>70%', 'lista-70');
      actualizarLista(asesoresBase, '>40%', 'lista-40');
      actualizarLista(asesoresBase, '>0%', 'lista-0');
      actualizarEstadisticas(asesoresBase);
    }}
    
    function actualizarLista(asesores, clasificacion, idLista) {{
        const lista = document.getElementById(idLista);
        const asesoresFiltrados = asesores.filter(a => a.clasificacion === clasificacion);
        
        if (asesoresFiltrados.length === 0) {{
            lista.innerHTML = '<div class="asesor-item">No hay asesores en esta categoría</div>';
            return;
        }}
        
        // Ordenar por porcentaje (mayor a menor)
        asesoresFiltrados.sort((a, b) => b.porcentaje - a.porcentaje);
        
        let html = '';
        asesoresFiltrados.forEach(asesor => {{
            const claseGradiente = `gradiente-${{clasificacion.replace('%', '').replace('>', '')}}`;
            const clasePorcentaje = `porcentaje-${{clasificacion.replace('%', '').replace('>', '')}}`;
            html += `<div class="asesor-item ${{claseGradiente}}">
                <div class="asesor-nombre">${{asesor.nombre}}</div>
                <div class="asesor-porcentaje ${{clasePorcentaje}}">${{asesor.porcentaje}}%</div>
            </div>`;
        }});
        
        lista.innerHTML = html;
    }}
    
    function actualizarEstadisticas(asesores) {{
        const contadores = {{
            ">100%": asesores.filter(a => a.clasificacion === ">100%").length,
            ">70%": asesores.filter(a => a.clasificacion === ">70%").length,
            ">40%": asesores.filter(a => a.clasificacion === ">40%").length,
            ">0%": asesores.filter(a => a.clasificacion === ">0%").length
        }};
        
        // Actualizar estadísticas si los elementos existen
        const elem100 = document.getElementById('cantidad-100');
        const elem70 = document.getElementById('cantidad-70');
        const elem40 = document.getElementById('cantidad-40');
        const elem0 = document.getElementById('cantidad-0');
        
        if (elem100) elem100.textContent = contadores[">100%"];
        if (elem70) elem70.textContent = contadores[">70%"];
        if (elem40) elem40.textContent = contadores[">40%"];
        if (elem0) elem0.textContent = contadores[">0%"];
    }}

    // ========== FUNCIONES PARA BUSQUEDA Y COMPARACIÓN DE ASESORES ==========
    function agregarAsesor() {{
        const input = document.getElementById('inputBusqueda');
        const nombreAsesor = input.value.trim();
        
        if (nombreAsesor && !asesoresSeleccionados.has(nombreAsesor)) {{
            asesoresSeleccionados.add(nombreAsesor);
            actualizarAsesoresSeleccionados();
            input.value = '';
        }}
    }}
    
    function eliminarAsesor(nombre) {{
        asesoresSeleccionados.delete(nombre);
        actualizarAsesoresSeleccionados();
    }}
    
    function actualizarAsesoresSeleccionados() {{
        const container = document.getElementById('asesoresSeleccionados');
        if (!container) return;
        
        container.innerHTML = '';
        
        asesoresSeleccionados.forEach(asesor => {{
            const tag = document.createElement('div');
            tag.className = 'tag-elemento';
            tag.innerHTML = `
                ${{asesor}}
                <button class="eliminar-elemento" onclick="eliminarAsesor('${{asesor}}')">×</button>
            `;
            container.appendChild(tag);
        }});
    }}
    
    function compararAsesores() {{
        if (asesoresSeleccionados.size === 0) {{
            alert('Por favor selecciona al menos un asesor para comparar.');
            return;
        }}
        
        const resultadosDiv = document.getElementById('resultadosComparacion');
        const tablaDiv = document.getElementById('tablaComparacion');
        
        if (!resultadosDiv || !tablaDiv) return;
        
        // Mostrar los controles de meses
        const controlesHTML = `
            <div style="margin: 20px 0; text-align: center;">
                <div class="controles-grafica">
                    <button class="btn-comparacion" onclick="cambiarTipoGrafica(3)">
                        3 meses
                    </button>
                    <button class="btn-comparacion activo" onclick="cambiarTipoGrafica(6)">
                        6 meses
                    </button>
                    <button class="btn-comparacion" onclick="cambiarTipoGrafica(12)">
                        12 meses
                    </button>
                </div>
            </div>
        `;
        
        // Insertar controles ANTES de la tabla
        resultadosDiv.innerHTML = controlesHTML;
        
        // Crear div para tabla
        const tablaContainer = document.createElement('div');
        tablaContainer.id = 'tablaComparacion';
        resultadosDiv.appendChild(tablaContainer);
        
        // Crear div para gráfica (se creará dinámicamente después)
        const graficaContainer = document.createElement('div');
        graficaContainer.id = 'graficaContainerAsesores';
        resultadosDiv.appendChild(graficaContainer);
        
        // Actualizar tabla y gráfica
        actualizarTablaComparacionAsesores();
        actualizarGraficaComparacionAsesores();
        
        resultadosDiv.style.display = 'block';
    }}
    
    function generarGraficaComparacionAsesores(meses, asesores) {{
        // Crear contenedor para el gráfico si no existe
        let graficaDiv = document.getElementById('graficaComparacion');
        if (!graficaDiv) {{
            const resultadosDiv = document.getElementById('resultadosComparacion');
            if (resultadosDiv) {{
                graficaDiv = document.createElement('div');
                graficaDiv.id = 'graficaComparacion';
                graficaDiv.className = 'grafica-container';
                graficaDiv.innerHTML = `
                    <h3>📈 Evolución de Desempeño</h3>
                    <canvas id="graficaDesempenio" width="1200" height="400"></canvas>
                `;
                resultadosDiv.appendChild(graficaDiv);
            }}
        }}
        
        const canvas = document.getElementById('graficaDesempenio');
        if (!canvas) return;
        
        const ctx = canvas.getContext('2d');
        
        // Destruir gráfica anterior si existe
        if (chartInstance) {{
            chartInstance.destroy();
            chartInstance = null;
        }}
        
        // LIMPIAR EL CANVAS COMPLETAMENTE
        ctx.clearRect(0, 0, canvas.width, canvas.height);
        
        // Preparar datos para la gráfica
        const datasets = [];
        const colores = [
            '#FF6384', '#36A2EB', '#FFCE56', '#8AC926', 
            '#9966FF', '#FF9F40', '#4BC0C0', '#1982C4'
        ];
        
        let maxValor = 0;
        let minValor = 0;
        
        asesores.forEach((asesor, index) => {{
            const datosAsesor = [];
            const borderColor = colores[index % colores.length];
            const backgroundColor = borderColor + '40'; // 40% de opacidad para área
            
            meses.forEach(mes => {{
                const datosMes = datosMeses[mes];
                const asesorData = datosMes.find(a => a.nombre === asesor);
                const valor = asesorData ? asesorData.porcentaje : null;
                datosAsesor.push(valor);
                
                // Calcular máximo y mínimo
                if (valor !== null && valor !== undefined && !isNaN(valor)) {{
                    maxValor = Math.max(maxValor, valor);
                    minValor = Math.min(minValor, valor);
                }}
            }});
            
            // GRÁFICA DE LÍNEAS CON ÁREA Y LÍNEAS DISCONTINUAS
            datasets.push({{
                label: asesor,
                data: datosAsesor,
                borderColor: borderColor,
                backgroundColor: backgroundColor,
                borderWidth: 3,
                fill: true,
                tension: 0.4,
                pointRadius: 6,
                pointHoverRadius: 8,
                type: 'line',
                spanGaps: true,
                // LÍNEAS DISCONTINUAS PARA DATOS FALTANTES
                segment: {{
                    borderColor: ctx => {{
                        const p0Null = ctx.p0.parsed.y === null;
                        const p1Null = ctx.p1.parsed.y === null;
                        return (p0Null || p1Null) ? borderColor + '60' : borderColor;
                    }},
                    borderDash: ctx => {{
                        const p0Null = ctx.p0.parsed.y === null;
                        const p1Null = ctx.p1.parsed.y === null;
                        return (p0Null || p1Null) ? [5, 3] : []; // Discontinuo para datos faltantes
                    }}
                }},
                // PUNTOS DIFERENCIADOS PARA DATOS VÁLIDOS VS NULOS
                pointBackgroundColor: datosAsesor.map(valor => 
                    valor === null ? '#95a5a6' : borderColor
                ),
                pointBorderColor: datosAsesor.map(valor => 
                    valor === null ? '#95a5a6' : '#ffffff'
                ),
                pointBorderWidth: 2
            }});
        }});
        
        // Calcular límites inteligentes para el eje Y
        let yMin = 0;
        let yMax = 100; // Por defecto 100%
        
        if (maxValor > 0) {{
            // Redondear al múltiplo de 10 más cercano hacia arriba
            yMax = Math.max(100, Math.ceil(maxValor / 10) * 10);
        }}
        
        // Si hay valores negativos (raro, pero por si acaso)
        if (minValor < 0) {{
            yMin = Math.floor(minValor / 10) * 10;
        }}
        
        // Formatear labels de meses para mejor visualización
        const labels = meses.map(mes => {{
            const [mesNombre, año] = mes.split('_');
            return `${{mesNombre.substring(0, 3)}} ${{año}}`;
        }});
        
        // Crear la gráfica SOLO DE LÍNEAS
        chartInstance = new Chart(ctx, {{
            type: 'line',
            data: {{
                labels: labels,
                datasets: datasets
            }},
            options: {{
                responsive: false,
                maintainAspectRatio: false,
                plugins: {{
                    title: {{
                        display: false,
                        text: 'Evolución del Porcentaje de Alcance - Asesores',
                        font: {{ size: 16 }}
                    }},
                    tooltip: {{
                        mode: 'index',
                        intersect: false,
                        backgroundColor: 'rgba(0, 0, 0, 0.8)',
                        titleFont: {{ size: 14, weight: 'bold' }},
                        bodyFont: {{ size: 13, weight: 'normal' }},
                        padding: 12,
                        cornerRadius: 8,
                        callbacks: {{
                            label: function(context) {{
                                const asesor = context.dataset.label;
                                const porcentaje = context.parsed.y;
                                
                                if (porcentaje === null) return `${{asesor}}: No participó`;
                                
                                const label = context.chart.data.labels[context.dataIndex];
                                const [mesAbrev, año] = label.split(' ');
                                
                                // Mapeo de meses en MAYÚSCULAS
                                const mesesMap = {{
                                    'ENE':'ENERO','FEB':'FEBRERO','MAR':'MARZO','ABR':'ABRIL',
                                    'MAY':'MAYO','JUN':'JUNIO','JUL':'JULIO','AGO':'AGOSTO',
                                    'SEP':'SETIEMBRE','OCT':'OCTUBRE','NOV':'NOVIEMBRE','DIC':'DICIEMBRE'
                                }};
                                
                                const mesKey = `${{mesesMap[mesAbrev]}}_${{año}}`;
                                const datos = datosMeses[mesKey]?.find(a => a.nombre === asesor);
                                
                                // Función para formatear con separador de miles
                                const formatoMiles = (valor) => {{
                                    return 'S/ ' + valor.toLocaleString('es-PE', {{
                                        minimumFractionDigits: 2,
                                        maximumFractionDigits: 2
                                    }});
                                }};
                                
                                return [
                                    `${{asesor}}: ${{porcentaje.toFixed(2)}}%`,
                                    `Meta: ${{formatoMiles(datos?.meta || 0)}}`,
                                    `Recupero: ${{formatoMiles(datos?.recupero || 0)}}`
                                ];
                            }}
                        }}
                    }},
                    legend: {{
                        position: 'top',
                        labels: {{ font: {{ size: 14 }} }},
                        onClick: function (e, legendItem, legend) {{
                            const index = legendItem.datasetIndex;
                            const ci = legend.chart;
                            if (ci.isDatasetVisible(index)) {{
                                ci.hide(index);
                            }} else {{
                                ci.show(index);
                            }}
                        }}
                    }}
                }},
                scales: {{
                    y: {{
                        beginAtZero: true,
                        title: {{
                            display: false,
                            text: 'Porcentaje de Alcance (%)'
                        }},
                        ticks: {{
                            font: {{ size: 16 }},
                            callback: function(value) {{
                                return value + '%';
                            }},
                            stepSize: calcularStepSize(yMax)
                        }},
                        // ¡LIMITES EXPLÍCITOS Y FIJO!
                        min: yMin,
                        max: yMax,
                        suggestedMin: yMin,
                        suggestedMax: yMax
                    }},
                    x: {{
                        title: {{
                            display: false,
                            text: 'Meses'
                        }},
                        ticks: {{
                            font: {{ size: 15 }}
                        }}
                    }}
                }},
                interaction: {{
                    mode: 'nearest',
                    axis: 'x',
                    intersect: false
                }}
            }}
        }});
    }}

    function cambiarTipoGrafica(cantidadMeses) {{
        mesesAMostrar = cantidadMeses;
        
        // Actualizar botones activos
        document.querySelectorAll('#resultadosComparacion .btn-comparacion').forEach(btn => {{
            btn.classList.remove('activo');
        }});
        event.target.classList.add('activo');
        
        // Regenerar tabla Y gráfica si hay datos
        if (asesoresSeleccionados.size > 0) {{
            // Primero regenerar tabla
            actualizarTablaComparacionAsesores();
            // Luego regenerar gráfica
            actualizarGraficaComparacionAsesores();
        }}
    }}

    function actualizarGraficaComparacionAsesores() {{
        if (asesoresSeleccionados.size === 0) return;
        
        // Obtener meses ordenados
        let mesesOrdenados = Object.keys(datosMeses).sort((a, b) => {{
            const [mesA, añoA] = a.split('_');
            const [mesB, añoB] = b.split('_');
            const fechaA = new Date(añoA, obtenerNumeroMes(mesA));
            const fechaB = new Date(añoB, obtenerNumeroMes(mesB));
            return fechaB - fechaA;
        }});
        
        // Filtrar por cantidad de meses
        if (mesesAMostrar > 0 && mesesOrdenados.length > mesesAMostrar) {{
            mesesOrdenados = mesesOrdenados.slice(0, mesesAMostrar);
        }}
        
        const mesesParaGrafica = [...mesesOrdenados].reverse();
        generarGraficaComparacionAsesores(mesesParaGrafica, Array.from(asesoresSeleccionados));
    }}

    // ========== FUNCION PARA ACTUALIZAR TABLA ASESORES ==========
    function actualizarTablaComparacionAsesores() {{
        if (asesoresSeleccionados.size === 0) return;
        
        const tablaDiv = document.getElementById('tablaComparacion');
        if (!tablaDiv) return;
        
        // Obtener meses ordenados (MÁS RECIENTE PRIMERO)
        let mesesOrdenados = Object.keys(datosMeses).sort((a, b) => {{
            const [mesA, añoA] = a.split('_');
            const [mesB, añoB] = b.split('_');
            const fechaA = new Date(añoA, obtenerNumeroMes(mesA));
            const fechaB = new Date(añoB, obtenerNumeroMes(mesB));
            return fechaB - fechaA;
        }});
        
        // Filtrar por cantidad de meses (los N más recientes)
        if (mesesAMostrar > 0 && mesesOrdenados.length > mesesAMostrar) {{
            mesesOrdenados = mesesOrdenados.slice(0, mesesAMostrar);
        }}
        
        // Invertir para mostrar de más antiguo a más reciente
        const mesesParaTabla = [...mesesOrdenados].reverse();
        
        // Crear tabla de comparación
        let html = '<div class="contenedor-tabla-scroll"><table class="tabla-comparacion">';
        
        // Encabezado
        html += '<tr><th>Asesor</th>';
        mesesParaTabla.forEach(mes => {{
            html += `<th>${{mes.replace('_', ' ')}}</th>`;
        }});
        html += '</tr>';
        
        // Filas por cada asesor
        asesoresSeleccionados.forEach(asesor => {{
            // Fila 1: Nombre del asesor y porcentajes (ALTURA NORMAL)
            html += `<tr><td><strong>${{asesor}}</strong></td>`;
            
            mesesParaTabla.forEach(mes => {{
                const datosMes = datosMeses[mes];
                const asesorData = datosMes.find(a => a.nombre === asesor);
                
                if (asesorData) {{
                    const clasePorcentaje = `porcentaje-${{asesorData.clasificacion.replace('%', '').replace('>', '')}}`;
                    html += `<td><span class="asesor-porcentaje ${{clasePorcentaje}}">${{asesorData.porcentaje}}%</span></td>`;
                }} else {{
                    html += '<td style="color: #999; font-style: italic;">No participó</td>';
                }}
            }});
            
            html += '</tr>';
            
            // Fila 2: Supervisor de cada mes - CON ALTURA REDUCIDA
            html += `<tr class="fila-secundaria-asesor"><td class="label-secundario-asesor">Supervisor:</td>`;
            
            mesesParaTabla.forEach(mes => {{
                const datosMes = datosMeses[mes];
                const asesorData = datosMes.find(a => a.nombre === asesor);
                
                if (asesorData) {{
                    const supervisor = asesorData.supervisor || 'Sin Supervisor';
                    
                    // Determinar clase CSS basada en el supervisor
                    let supervisorClass = 'supervisor-normal';
                    if (supervisor === 'Sin Supervisor') supervisorClass = 'supervisor-sin';
                    else if (supervisor.toUpperCase().includes('SUPER') || supervisor.toUpperCase().includes('JEFE')) {{
                        supervisorClass = 'supervisor-jefe';
                    }}
                    
                    html += `<td class="dato-secundario-asesor"><span class="${{supervisorClass}}">${{supervisor}}</span></td>`;
                }} else {{
                    html += '<td class="dato-secundario-asesor supervisor-vacio">-</td>';
                }}
            }});
            
            html += '</tr>';
            
        }});
        
        html += '</table></div>';
        tablaDiv.innerHTML = html;
    }}
    
    function limpiarBusqueda() {{
        asesoresSeleccionados.clear();
        actualizarAsesoresSeleccionados();
        
        // Destruir gráfica
        if (chartInstance) {{
            chartInstance.destroy();
            chartInstance = null;
        }}
        
        const resultadosDiv = document.getElementById('resultadosComparacion');
        if (resultadosDiv) resultadosDiv.style.display = 'none';
    }}
    
    // ========== FUNCIONES PARA BUSQUEDA Y COMPARACIÓN DE SUPERVISORES ==========
    function agregarSupervisor() {{
        const input = document.getElementById('inputBusquedaSupervisor');
        const nombreSupervisor = input.value.trim();
        
        if (nombreSupervisor && !supervisoresSeleccionados.has(nombreSupervisor)) {{
            supervisoresSeleccionados.add(nombreSupervisor);
            actualizarSupervisoresSeleccionados();
            input.value = '';
        }}
    }}
    
    function eliminarSupervisor(nombre) {{
        supervisoresSeleccionados.delete(nombre);
        actualizarSupervisoresSeleccionados();
    }}
    
    function actualizarSupervisoresSeleccionados() {{
        const container = document.getElementById('supervisoresSeleccionados');
        if (!container) return;
        
        container.innerHTML = '';
        
        supervisoresSeleccionados.forEach(supervisor => {{
            const tag = document.createElement('div');
            tag.className = 'tag-elemento';
            tag.innerHTML = `
                ${{supervisor}}
                <button class="eliminar-elemento" onclick="eliminarSupervisor('${{supervisor}}')">×</button>
            `;
            container.appendChild(tag);
        }});
    }}
    
    function compararSupervisores() {{
        if (supervisoresSeleccionados.size === 0) {{
            alert('Por favor selecciona al menos un supervisor para comparar.');
            return;
        }}
        
        const resultadosDiv = document.getElementById('resultadosComparacionSupervisores');
        
        if (!resultadosDiv) return;
        
        // Mostrar los controles de meses
        const controlesHTML = `
            <div style="margin: 20px 0; text-align: center;">
                <div class="controles-grafica">
                    <button class="btn-comparacion" onclick="cambiarTipoGraficaSupervisores(3)">
                        3 meses
                    </button>
                    <button class="btn-comparacion activo" onclick="cambiarTipoGraficaSupervisores(6)">
                        6 meses
                    </button>
                    <button class="btn-comparacion" onclick="cambiarTipoGraficaSupervisores(12)">
                        12 meses
                    </button>
                </div>
            </div>
        `;
        
        // Insertar controles ANTES de la tabla
        resultadosDiv.innerHTML = controlesHTML;
        
        // Crear div para tabla
        const tablaContainer = document.createElement('div');
        tablaContainer.id = 'tablaComparacionSupervisores';
        resultadosDiv.appendChild(tablaContainer);
        
        // Crear div para gráfica
        const graficaContainer = document.createElement('div');
        graficaContainer.id = 'graficaContainerSupervisores';
        resultadosDiv.appendChild(graficaContainer);
        
        // Actualizar tabla y gráfica
        actualizarTablaComparacionSupervisores();
        actualizarGraficaComparacionSupervisores();
        
        resultadosDiv.style.display = 'block';
    }}
    
    function generarGraficaComparacionSupervisores(meses, supervisores) {{
        // Crear contenedor para el gráfico si no existe
        let graficaDiv = document.getElementById('graficaComparacionSupervisores');
        if (!graficaDiv) {{
            const resultadosDiv = document.getElementById('resultadosComparacionSupervisores');
            if (resultadosDiv) {{
                graficaDiv = document.createElement('div');
                graficaDiv.id = 'graficaComparacionSupervisores';
                graficaDiv.className = 'grafica-container';
                graficaDiv.innerHTML = `
                    <h3>📈 Evolución de Desempeño</h3>
                    <canvas id="graficaDesempenioSupervisores" width="1200" height="400"></canvas>
                `;
                resultadosDiv.appendChild(graficaDiv);
            }}
        }}
        
        const canvas = document.getElementById('graficaDesempenioSupervisores');
        if (!canvas) return;
        
        const ctx = canvas.getContext('2d');
        
        // Destruir gráfica anterior si existe
        if (chartInstanceSupervisores) {{
            chartInstanceSupervisores.destroy();
            chartInstanceSupervisores = null;
        }}
        
        // LIMPIAR EL CANVAS COMPLETAMENTE
        ctx.clearRect(0, 0, canvas.width, canvas.height);
        
        // Preparar datos para la gráfica
        const datasets = [];
        const colores = [
            '#FF6384', '#36A2EB', '#FFCE56', '#8AC926', 
            '#9966FF', '#FF9F40', '#4BC0C0', '#1982C4'
        ];
        
        let maxValor = 0;
        let minValor = 0;
        
        supervisores.forEach((supervisor, index) => {{
            const datosSupervisor = [];
            const borderColor = colores[index % colores.length];
            const backgroundColor = borderColor + '40'; // 40% de opacidad para área
            
            meses.forEach(mes => {{
                const datosSupervisorMes = datosSupervisores[supervisor];
                if (datosSupervisorMes && datosSupervisorMes[mes]) {{
                    const valor = datosSupervisorMes[mes].porcentaje_vs_meta_super;
                    const valorNum = (valor !== null && valor !== undefined && !isNaN(valor)) 
                        ? parseFloat(valor) 
                        : null;
                    datosSupervisor.push(valorNum);
                    
                    // Calcular máximo y mínimo
                    if (valorNum !== null && !isNaN(valorNum)) {{
                        maxValor = Math.max(maxValor, valorNum);
                        minValor = Math.min(minValor, valorNum);
                    }}
                }} else {{
                    datosSupervisor.push(null);
                }}
            }});
            
            // GRÁFICA DE LÍNEAS CON ÁREA Y LÍNEAS DISCONTINUAS
            datasets.push({{
                label: supervisor,
                data: datosSupervisor,
                borderColor: borderColor,
                backgroundColor: backgroundColor,
                borderWidth: 3,
                fill: true, // ¡ÁREA COLOREA!
                tension: 0.4,
                pointRadius: 6,
                pointHoverRadius: 8,
                type: 'line',
                spanGaps: true,
                // LÍNEAS DISCONTINUAS PARA DATOS FALTANTES
                segment: {{
                    borderColor: ctx => {{
                        const p0Null = ctx.p0.parsed.y === null;
                        const p1Null = ctx.p1.parsed.y === null;
                        return (p0Null || p1Null) ? borderColor + '60' : borderColor;
                    }},
                    borderDash: ctx => {{
                        const p0Null = ctx.p0.parsed.y === null;
                        const p1Null = ctx.p1.parsed.y === null;
                        return (p0Null || p1Null) ? [5, 3] : [];
                    }}
                }},
                // PUNTOS DIFERENCIADOS
                pointBackgroundColor: datosSupervisor.map(valor => 
                    valor === null ? '#95a5a6' : borderColor
                ),
                pointBorderColor: datosSupervisor.map(valor => 
                    valor === null ? '#95a5a6' : '#ffffff'
                ),
                pointBorderWidth: 2
            }});
        }});
        
        // Agregar línea de meta (100%)
        datasets.push({{
            label: 'Meta (100%)',
            data: Array(meses.length).fill(100),
            borderColor: '#2ecc71',
            backgroundColor: 'transparent',
            borderWidth: 2,
            borderDash: [5, 5],
            pointRadius: 0,
            fill: false,
            tension: 0
        }});
        
        // Asegurar que la meta esté incluida en el cálculo
        maxValor = Math.max(maxValor, 100);
        
        // Calcular límites inteligentes para el eje Y
        let yMin = 0;
        let yMax = 100; // Por defecto 100%
        
        if (maxValor > 0) {{
            // Redondear al múltiplo de 10 más cercano hacia arriba
            yMax = Math.max(100, Math.ceil(maxValor / 10) * 10);
        }}
        
        // Si hay valores negativos (raro, pero por si acaso)
        if (minValor < 0) {{
            yMin = Math.floor(minValor / 10) * 10;
        }}
        
        // Formatear labels de meses para mejor visualización
        const labels = meses.map(mes => {{
            const [mesNombre, año] = mes.split('_');
            return `${{mesNombre.substring(0, 3)}} ${{año}}`;
        }});
        
        // Crear la gráfica SOLO DE LÍNEAS
        chartInstanceSupervisores = new Chart(ctx, {{
            type: 'line',
            data: {{
                labels: labels,
                datasets: datasets
            }},
            options: {{
                responsive: false,
                maintainAspectRatio: false,
                plugins: {{
                    title: {{
                        display: false,
                        text: 'Evolución del Porcentaje vs Meta Supervisor - Equipos',
                        font: {{ size: 16 }}
                    }},
                    tooltip: {{
                        mode: 'index',
                        intersect: false,
                        backgroundColor: 'rgba(0, 0, 0, 0.8)',
                        titleFont: {{
                            size: 14,
                            weight: 'bold'
                        }},
                        bodyFont: {{
                            size: 13,
                            weight: 'normal'
                        }},
                        padding: 12,
                        cornerRadius: 8,
                        callbacks: {{
                            label: function(context) {{
                                const supervisor = context.dataset.label;
                                const valor = context.parsed.y;
                                
                                // Ocultar línea de meta del tooltip
                                if (supervisor === 'Meta (100%)') return null;
                                if (valor === null) return `${{supervisor}}: Sin datos`;
                                
                                const [mesAbrev, año] = context.chart.data.labels[context.dataIndex].split(' ');
                                const meses = {{'ENE':'ENERO','FEB':'FEBRERO','MAR':'MARZO','ABR':'ABRIL','MAY':'MAYO','JUN':'JUNIO','JUL':'JULIO','AGO':'AGOSTO','SEP':'SETIEMBRE','OCT':'OCTUBRE','NOV':'NOVIEMBRE','DIC':'DICIEMBRE'}};
                                const mesKey = `${{meses[mesAbrev]}}_${{año}}`;
                                const datos = datosSupervisores[supervisor]?.[mesKey];
                                
                                const formato = (v) => 'S/ ' + (v||0).toLocaleString('es-PE',{{minimumFractionDigits:2}});
                                
                                return [
                                    `${{supervisor}}: ${{valor.toFixed(2)}}%`,
                                    `Meta: ${{formato(datos?.meta_super)}}`,
                                    `Recupero: ${{formato(datos?.total_recupero)}}`
                                ];
                            }}
                        }}
                    }},
                    legend: {{
                        position: 'top',
                        labels: {{ font: {{ size: 14 }} }},
                        onClick: function (e, legendItem, legend) {{
                            const index = legendItem.datasetIndex;
                            const ci = legend.chart;
                            if (ci.isDatasetVisible(index)) {{
                                ci.hide(index);
                            }} else {{
                                ci.show(index);
                            }}
                        }}
                    }}
                }},
                scales: {{
                    y: {{
                        beginAtZero: true,
                        title: {{
                            display: false,
                            text: 'Porcentaje vs Meta Supervisor (%)'
                        }},
                        ticks: {{
                            font: {{ size: 16 }},
                            callback: function(value) {{
                                return value + '%';
                            }},
                            stepSize: calcularStepSize(yMax)
                        }},
                        // ¡LIMITES EXPLÍCITOS Y FIJO!
                        min: yMin,
                        max: yMax,
                        suggestedMin: yMin,
                        suggestedMax: yMax
                    }},
                    x: {{
                        title: {{
                            display: false,
                            text: 'Meses'
                        }},
                        ticks: {{
                            font: {{ size: 15 }}
                        }}
                    }}
                }},
                interaction: {{
                    mode: 'nearest',
                    axis: 'x',
                    intersect: false
                }}
            }}
        }});
    }}

    function cambiarTipoGraficaSupervisores(cantidadMeses) {{
        mesesAMostrar = cantidadMeses;
        
        // Actualizar botones activos
        document.querySelectorAll('#resultadosComparacionSupervisores .btn-comparacion').forEach(btn => {{
            btn.classList.remove('activo');
        }});
        event.target.classList.add('activo');
        
        // Regenerar tabla Y gráfica si hay datos
        if (supervisoresSeleccionados.size > 0) {{
            // Primero regenerar tabla
            actualizarTablaComparacionSupervisores();
            // Luego regenerar gráfica
            actualizarGraficaComparacionSupervisores();
        }}
    }}

    function actualizarGraficaComparacionSupervisores() {{
        if (supervisoresSeleccionados.size === 0) return;
        
        // Obtener meses ordenados
        let mesesOrdenados = Object.keys(datosMeses).sort((a, b) => {{
            const [mesA, añoA] = a.split('_');
            const [mesB, añoB] = b.split('_');
            const fechaA = new Date(añoA, obtenerNumeroMes(mesA));
            const fechaB = new Date(añoB, obtenerNumeroMes(mesB));
            return fechaB - fechaA;
        }});
        
        // Filtrar por cantidad de meses
        if (mesesAMostrar > 0 && mesesOrdenados.length > mesesAMostrar) {{
            mesesOrdenados = mesesOrdenados.slice(0, mesesAMostrar);
        }}
        
        const mesesParaGrafica = [...mesesOrdenados].reverse();
        generarGraficaComparacionSupervisores(mesesParaGrafica, Array.from(supervisoresSeleccionados));
    }}

    // ========== FUNCIONES PARA ACTUALIZAR TABLAS ==========
    function actualizarTablaComparacionSupervisores() {{
        if (supervisoresSeleccionados.size === 0) return;
        
        const tablaDiv = document.getElementById('tablaComparacionSupervisores');
        if (!tablaDiv) return;
        
        // Obtener meses ordenados (más reciente primero)
        let mesesOrdenados = Object.keys(datosMeses).sort((a, b) => {{
            const [mesA, añoA] = a.split('_');
            const [mesB, añoB] = b.split('_');
            const fechaA = new Date(añoA, obtenerNumeroMes(mesA));
            const fechaB = new Date(añoB, obtenerNumeroMes(mesB));
            return fechaB - fechaA;
        }});
        
        // Filtrar por cantidad de meses
        if (mesesAMostrar > 0 && mesesOrdenados.length > mesesAMostrar) {{
            mesesOrdenados = mesesOrdenados.slice(0, mesesAMostrar);
        }}
        
        const mesesParaTabla = [...mesesOrdenados].reverse();
        
        // Ordenar supervisores alfabéticamente
        const supervisoresOrdenados = Array.from(supervisoresSeleccionados).sort();
        
        // Crear tabla de comparación
        let html = '<div class="contenedor-tabla-scroll"><table class="tabla-comparacion">';
        
        // Encabezado
        html += '<tr><th>Supervisor</th>';
        mesesParaTabla.forEach(mes => {{
            html += `<th>${{mes.replace('_', ' ')}}</th>`;
        }});
        html += '</tr>';
        
        // Filas por cada supervisor ORDENADO
        supervisoresOrdenados.forEach((supervisor, index) => {{
            // **FILA PRINCIPAL: Porcentaje de alcance** (altura normal)
            html += `<tr><td class="supervisor-name">${{supervisor}}</td>`;
            
            mesesParaTabla.forEach(mes => {{
                const datosSupervisor = datosSupervisores[supervisor];
                if (datosSupervisor && datosSupervisor[mes]) {{
                    const datosMes = datosSupervisor[mes];
                    const porcentaje = datosMes.porcentaje_vs_meta_super || 0;
                    const clasePorcentaje = `porcentaje-${{datosMes.clasificacion.replace('%', '').replace('>', '')}}`;
                    html += `<td><span class="asesor-porcentaje ${{clasePorcentaje}}">${{porcentaje.toFixed(2)}}%</span></td>`;
                }} else {{
                    html += '<td style="color: #999; font-style: italic;">Sin datos</td>';
                }}
            }});
            
            html += '</tr>';
            
            // **FILA 2: Total de Asesores** (altura reducida)
            html += `<tr class="fila-equipo"><td class="equipo-label">Asesores:</td>`;
            
            mesesParaTabla.forEach(mes => {{
                const datosSupervisor = datosSupervisores[supervisor];
                if (datosSupervisor && datosSupervisor[mes]) {{
                    const datosMes = datosSupervisor[mes];
                    const cantidad = datosMes.cantidad_asesores || 0;
                    html += `<td class="equipo-supervisor">${{cantidad}}</td>`;
                }} else {{
                    html += '<td class="sin-equipo">-</td>';
                }}
            }});
            
            html += '</tr>';
            
            // **FILA 3: Segmento/Cartera** (altura reducida)
            html += `<tr class="fila-equipo"><td class="equipo-label">Cartera:</td>`;
            
            mesesParaTabla.forEach(mes => {{
                const datosSupervisor = datosSupervisores[supervisor];
                if (datosSupervisor && datosSupervisor[mes]) {{
                    const datosMes = datosSupervisor[mes];
                    const cartera = datosMes.cartera || 'No definida';
                    
                    // Determinar clase de segmento según el tipo
                    let segmentoClass = '';
                    if (cartera.includes('PREMIER') || cartera.includes('PRIME') || cartera.includes('PREMIUM')) {{
                        segmentoClass = 'segmento-premier';
                    }} else if (cartera.includes('EMPRESARIAL') || cartera.includes('BUSINESS')) {{
                        segmentoClass = 'segmento-empresarial';
                    }} else if (cartera.includes('MASIVO') || cartera.includes('MASS')) {{
                        segmentoClass = 'segmento-masivo';
                    }} else if (cartera.includes('PYME') || cartera.includes('SME')) {{
                        segmentoClass = 'segmento-pyme';
                    }}
                    
                    html += `<td class="equipo-supervisor ${{segmentoClass}}">${{cartera}}</td>`;
                }} else {{
                    html += '<td class="sin-equipo">-</td>';
                }}
            }});
            
            html += '</tr>';
        }});
        
        html += '</table></div>';
        tablaDiv.innerHTML = html;
    }}

    function limpiarBusquedaSupervisores() {{
        supervisoresSeleccionados.clear();
        actualizarSupervisoresSeleccionados();
        
        // Destruir gráfica si existe
        if (chartInstanceSupervisores) {{
            chartInstanceSupervisores.destroy();
            chartInstanceSupervisores = null;
        }}
        
        // Ocultar resultados
        const resultadosDiv = document.getElementById('resultadosComparacionSupervisores');
        if (resultadosDiv) resultadosDiv.style.display = 'none';
    }}
    
    // ========== FUNCIONES PARA PERIODO DE PRUEBA ==========
    function agregarAsesorPeriodo() {{
        const input = document.getElementById('inputBusquedaPeriodo');
        const nombreAsesor = input.value.trim();
        
        if (nombreAsesor && !asesoresSeleccionadosPeriodo.has(nombreAsesor)) {{
            asesoresSeleccionadosPeriodo.add(nombreAsesor);
            actualizarAsesoresSeleccionadosPeriodo();
            input.value = '';
        }}
    }}
    
    function eliminarAsesorPeriodo(nombre) {{
        asesoresSeleccionadosPeriodo.delete(nombre);
        actualizarAsesoresSeleccionadosPeriodo();
    }}
    
    function actualizarAsesoresSeleccionadosPeriodo() {{
        const container = document.getElementById('asesoresSeleccionadosPeriodo');
        if (!container) return;
        
        container.innerHTML = '';
        
        asesoresSeleccionadosPeriodo.forEach(asesor => {{
            const tag = document.createElement('div');
            tag.className = 'tag-elemento';
            tag.innerHTML = `
                ${{asesor}}
                <button class="eliminar-elemento" onclick="eliminarAsesorPeriodo('${{asesor}}')">×</button>
            `;
            container.appendChild(tag);
        }});
    }}
    
    function limpiarPeriodoPrueba() {{
        // Limpiar selects
        const selects = ['mes1', 'año1', 'mes2', 'año2'];
        selects.forEach(id => {{
            const select = document.getElementById(id);
            if (select) select.value = '';
        }});
        
        // Limpiar porcentaje mínimo
        const porcentajeInput = document.getElementById('porcentajeMinimo');
        if (porcentajeInput) porcentajeInput.value = '70';
        
        // Limpiar input de búsqueda
        const inputBusqueda = document.getElementById('inputBusquedaPeriodo');
        if (inputBusqueda) inputBusqueda.value = '';
        
        // Limpiar asesores seleccionados
        asesoresSeleccionadosPeriodo.clear();
        actualizarAsesoresSeleccionadosPeriodo();
        
        // Ocultar resultados
        const resultadosDiv = document.getElementById('resultadosPeriodo');
        if (resultadosDiv) resultadosDiv.style.display = 'none';
    }}
    
    function calcularPeriodoPrueba() {{
        const mes1 = document.getElementById('mes1').value;
        const año1 = document.getElementById('año1').value;
        const mes2 = document.getElementById('mes2').value;
        const año2 = document.getElementById('año2').value;
        const porcentajeMinimo = parseFloat(document.getElementById('porcentajeMinimo').value) || 70;
        
        if (!mes1 || !año1 || !mes2 || !año2) {{
            alert('Por favor selecciona ambos meses y años para calcular el periodo de prueba.');
            return;
        }}
        
        const periodo1 = `${{mes1}}_${{año1}}`;
        const periodo2 = `${{mes2}}_${{año2}}`;
        
        if (!datosMeses[periodo1] || !datosMeses[periodo2]) {{
            alert('No hay datos disponibles para uno de los periodos seleccionados.');
            return;
        }}
        
        const resultadosDiv = document.getElementById('resultadosPeriodo');
        const tablaDiv = document.getElementById('tablaPeriodo');
        
        if (!resultadosDiv || !tablaDiv) return;
        
        // Obtener asesores a evaluar (seleccionados o todos)
        const asesoresAEvaluar = asesoresSeleccionadosPeriodo.size > 0 ? 
            Array.from(asesoresSeleccionadosPeriodo) : 
            [...new Set([...datosMeses[periodo1].map(a => a.nombre), ...datosMeses[periodo2].map(a => a.nombre)])];
        
        // Función para formatear números con separadores de miles y 2 decimales
        const formatoNumero = (numero) => {{
            return numero.toLocaleString('es-PE', {{
                minimumFractionDigits: 2,
                maximumFractionDigits: 2
            }});
        }};
        
        // Función para formatear números enteros
        const formatoEntero = (numero) => {{
            return numero.toLocaleString('es-PE', {{
                minimumFractionDigits: 0,
                maximumFractionDigits: 0
            }});
        }};
        
        let html = '<table class="tabla-comparacion">';
        html += '<tr><th>Asesor</th><th>Periodo 1</th><th>Periodo 2</th><th>Total Recupero</th><th>Total Meta</th><th>% Calculado</th><th>Estado</th><th>Falta Recuperar</th></tr>';
        
        asesoresAEvaluar.forEach(asesor => {{
            const data1 = datosMeses[periodo1].find(a => a.nombre === asesor);
            const data2 = datosMeses[periodo2].find(a => a.nombre === asesor);
            
            if (data1 && data2 && data1.recupero !== undefined && data1.meta !== undefined && 
                data2.recupero !== undefined && data2.meta !== undefined) {{
                
                const totalRecupero = data1.recupero + data2.recupero;
                const totalMeta = data1.meta + data2.meta;
                const porcentajePrueba = totalMeta > 0 ? (totalRecupero / totalMeta) * 100 : 0;
                const porcentajeFormateado = porcentajePrueba.toFixed(2);
                
                const estado = porcentajePrueba >= porcentajeMinimo ? 
                    '<span style="color: #27ae60; font-weight: bold;">✓ Supera</span>' : 
                    '<span style="color: #e74c3c; font-weight: bold;">✗ No supera</span>';
                
                const faltaRecuperar = porcentajePrueba < porcentajeMinimo ? 
                    (totalMeta * (porcentajeMinimo / 100) - totalRecupero) : 0;
                
                html += `<tr>
                    <td style="text-align: left; padding-left: 15px;"><strong>${{asesor}}</strong></td>
                    <td>S/ ${{formatoNumero(data1.recupero)}} / ${{formatoNumero(data1.meta)}}</td>
                    <td>S/ ${{formatoNumero(data2.recupero)}} / ${{formatoNumero(data2.meta)}}</td>
                    <td><strong style="color: #27ae60;">S/ ${{formatoNumero(totalRecupero)}}</strong></td>
                    <td><strong style="color: #3498db;">S/ ${{formatoNumero(totalMeta)}}</strong></td>
                    <td><strong>${{porcentajeFormateado}}%</strong></td>
                    <td>${{estado}}</td>
                    <td style="color: ${{porcentajePrueba < porcentajeMinimo ? '#e74c3c' : '#27ae60'}}; font-weight: ${{porcentajePrueba < porcentajeMinimo ? 'bold' : 'normal'}};">
                        S/ ${{formatoNumero(faltaRecuperar)}}
                    </td>
                </tr>`;
            }}
        }});
        
        html += '</table>';
        tablaDiv.innerHTML = html;
        resultadosDiv.style.display = 'block';
        
        // Scroll suave a la tabla de resultados
        setTimeout(() => {{
            resultadosDiv.scrollIntoView({{
                behavior: 'smooth',
                block: 'start'
            }});
        }}, 300);
    }}
    
    function generarGraficaIncrementoTotal(periodoCompleto) {{
        const canvas = document.getElementById('graficaIncrementoTotal');
        if (!canvas) {{
            console.log('Canvas no encontrado');
            return;
        }}
        
        const ctx = canvas.getContext('2d');
        
        // Destruir gráfica anterior si existe
        if (window.chartIncrementoTotal) {{
            window.chartIncrementoTotal.destroy();
            window.chartIncrementoTotal = null;
        }}
        
        // Limpiar canvas
        ctx.clearRect(0, 0, canvas.width, canvas.height);
        
        // ========== 1. CALCULAR META TOTAL DEL MES ==========
        let metaTotalMes = 0;
        const supervisoresUnicos = new Set();
        
        Object.keys(datosSupervisores).forEach(supervisor => {{
            const datosMes = datosSupervisores[supervisor][periodoCompleto];
            if (datosMes && datosMes.meta_super !== undefined && !supervisoresUnicos.has(supervisor)) {{
                const metaSupervisor = parseFloat(datosMes.meta_super) || 0;
                metaTotalMes += metaSupervisor;
                supervisoresUnicos.add(supervisor);
            }}
        }});
        
        // Si no hay meta total, no hay datos para graficar
        if (metaTotalMes === 0) {{
            console.log('No hay meta total para el periodo:', periodoCompleto);
            mostrarMensajeSinDatos(canvas, periodoCompleto);
            return;
        }}
        
        // ========== 2. RECOPILAR TODAS LAS FECHAS ÚNICAS ==========
        const todasFechas = new Set();
        const recuperoAcumuladoPorFecha = {{}};
        
        Object.keys(datosSupervisores).forEach(supervisor => {{
            const datosMes = datosSupervisores[supervisor][periodoCompleto];
            if (!datosMes) return;
            
            const datosDiarios = datosMes.datos_diarios_supervisor || {{}};
            
            Object.keys(datosDiarios).forEach(fecha => {{
                todasFechas.add(fecha);
                
                const recuperoSupervisor = parseFloat(datosDiarios[fecha]) || 0;
                
                if (!recuperoAcumuladoPorFecha[fecha]) {{
                    recuperoAcumuladoPorFecha[fecha] = 0;
                }}
                
                recuperoAcumuladoPorFecha[fecha] += recuperoSupervisor;
            }});
        }});
        
        // Si no hay fechas con datos, no hay nada que graficar
        if (todasFechas.size === 0) {{
            console.log('No hay datos diarios para el periodo:', periodoCompleto);
            mostrarMensajeSinDatos(canvas, periodoCompleto);
            return;
        }}
        
        // ========== 3. ORDENAR FECHAS CRONOLÓGICAMENTE ==========
        const fechasOrdenadas = Array.from(todasFechas).sort((a, b) => 
            convertirFechaDiaria(a) - convertirFechaDiaria(b)
        );
        
        console.log(`📊 Total fechas disponibles: ${{fechasOrdenadas.length}}`);
        
        // ========== 4. CALCULAR ALCANCE ACUMULADO ==========
        const datosValidosParaGrafica = [];  // Datos para la línea
        const alcancesPorcentaje = [];
        
        // Inicializar todas las fechas
        fechasOrdenadas.forEach(fecha => {{
            const recuperoAcumulado = recuperoAcumuladoPorFecha[fecha] || 0;
            
            const alcanceDia = metaTotalMes > 0 ? (recuperoAcumulado / metaTotalMes) * 100 : 0;
            
            // SOLO agregar a datos válidos si hay datos reales (recupero > 0)
            if (recuperoAcumulado > 0) {{
                datosValidosParaGrafica.push(alcanceDia);
                alcancesPorcentaje.push(alcanceDia);
            }} else {{
                // Para días sin datos, poner null (la línea no se dibujará aquí)
                datosValidosParaGrafica.push(null);
                alcancesPorcentaje.push(0);
            }}
        }});
        
        // ========== 5. ENCONTRAR EL ÚLTIMO DÍA CON DATOS REALES ==========
        let ultimoDiaConDatos = -1;
        for (let i = fechasOrdenadas.length - 1; i >= 0; i--) {{
            const fecha = fechasOrdenadas[i];
            const recuperoDia = recuperoAcumuladoPorFecha[fecha] || 0;
            if (recuperoDia > 0) {{
                ultimoDiaConDatos = i;
                break;
            }}
        }}
        
        if (ultimoDiaConDatos === -1) {{
            ultimoDiaConDatos = fechasOrdenadas.length - 1;
        }}
        
        console.log(`📊 Fechas con datos > 0: hasta día ${{ultimoDiaConDatos + 1}}`);
        
        // ========== 6. CREAR LA GRÁFICA DE LÍNEA CON ÁREA ==========
        canvas.width = 1200;
        canvas.height = 700;
        canvas.style.display = 'block';
        canvas.style.margin = '0 auto';
        canvas.style.maxWidth = '100%';
        
        // USAR TODAS LAS FECHAS para las etiquetas (CAMBIO PRINCIPAL)
        const labelsParaGrafica = fechasOrdenadas.map(fecha => {{
            if (fecha.includes('-')) {{
                return fecha.split('-')[0];
            }}
            return fecha;
        }});
        
        // COLORES PARA GRÁFICA DE LÍNEA (MORADO)
        const colorLinea = '#9b59b6';
        const colorArea = 'rgba(155, 89, 182, 0.2)';
        
        // PLUGIN PARA ETIQUETAS DE DATOS (PORCENTAJE DE ALCANCE TOTAL)
        const dataLabelsPluginTotal = {{
            id: 'dataLabelsTotal',
            afterDatasetsDraw(chart, args, options) {{
                const {{ctx}} = chart;
                const meta = chart.getDatasetMeta(0);
                
                meta.data.forEach((point, index) => {{
                    const value = chart.data.datasets[0].data[index];
                    if (value === null || value === undefined) return;
                    
                    // Solo mostrar etiqueta si hay datos > 0
                    if (value > 0) {{
                        ctx.save();
                        
                        const x = point.x;
                        const y = point.y - 20;
                        
                        const porcentajeAlcance = value;
                        const formattedValue = porcentajeAlcance.toFixed(1) + '%';
                        
                        ctx.font = 'bold 15px Arial';
                        ctx.textAlign = 'center';
                        ctx.textBaseline = 'bottom';
                        
                        let textColor = '#2c3e50';
                        if (porcentajeAlcance === 0) textColor = '#95a5a6';
                        else if (porcentajeAlcance < 40) textColor = '#e74c3c';
                        else if (porcentajeAlcance < 70) textColor = '#f39c12';
                        else if (porcentajeAlcance < 100) textColor = '#27ae60';
                        else textColor = '#2ecc71';
                        
                        ctx.fillStyle = textColor;
                        
                        ctx.shadowColor = 'rgba(255, 255, 255, 0.95)';
                        ctx.shadowBlur = 8;
                        ctx.shadowOffsetX = 0;
                        ctx.shadowOffsetY = 0;
                        
                        ctx.fillText(formattedValue, x, y);
                        
                        ctx.restore();
                    }}
                }});
            }}
        }};
        
        try {{
            window.chartIncrementoTotal = new Chart(ctx, {{
                type: 'line',
                data: {{
                    labels: labelsParaGrafica,  // TODAS las fechas
                    datasets: [{{
                        label: 'Alcance Acumulado (%)',
                        data: datosValidosParaGrafica,  // Datos: válidos donde hay > 0, null donde no
                        backgroundColor: colorArea,
                        borderColor: colorLinea,
                        borderWidth: 3,
                        fill: true,
                        tension: 0.4,
                        pointRadius: function(context) {{
                            // Puntos más grandes para datos válidos, invisibles para nulos
                            const value = context.dataset.data[context.dataIndex];
                            return value === null || value === 0 ? 0 : 6;
                        }},
                        pointHoverRadius: function(context) {{
                            const value = context.dataset.data[context.dataIndex];
                            return value === null || value === 0 ? 0 : 10;
                        }},
                        pointBackgroundColor: function(context) {{
                            const value = context.dataset.data[context.dataIndex];
                            return value === null || value === 0 ? 'transparent' : colorLinea;
                        }},
                        pointBorderColor: function(context) {{
                            const value = context.dataset.data[context.dataIndex];
                            return value === null || value === 0 ? 'transparent' : '#ffffff';
                        }},
                        pointBorderWidth: 2,
                        spanGaps: false  // No conectar puntos nulos
                    }}]
                }},
                options: {{
                    responsive: false,
                    maintainAspectRatio: false,
                    plugins: {{
                        title: {{
                            display: true,
                            text: [`            `,`            `],
                            font: {{ size: 22, weight: 'bold' }},
                            padding: {{ bottom: 25 }}
                        }},
                        tooltip: {{
                            mode: 'nearest',
                            intersect: false,
                            backgroundColor: 'rgba(0, 0, 0, 0.9)',
                            titleFont: {{ size: 16, weight: 'bold' }},
                            bodyFont: {{ size: 14 }},
                            padding: 15,
                            cornerRadius: 10,
                            callbacks: {{
                                title: function(context) {{
                                    const index = context[0].dataIndex;
                                    return 'Día ' + (index + 1) + ': ' + fechasOrdenadas[index];
                                }},
                                label: function(context) {{
                                    const index = context.dataIndex;
                                    const alcance = context.parsed.y;
                                    const recuperoAcumulado = recuperoAcumuladoPorFecha[fechasOrdenadas[index]] || 0;
                                    
                                    // Si no hay datos para este día
                                    if (index > ultimoDiaConDatos || alcance === null) {{
                                        return '📭 Sin datos de recupero';
                                    }}
                                    
                                    // Calcular incremento del día
                                    let incrementoDia = 0;
                                    if (index > 0) {{
                                        const alcanceAnterior = context.chart.data.datasets[0].data[index - 1] || 0;
                                        incrementoDia = (alcance - alcanceAnterior);
                                    }} else {{
                                        incrementoDia = alcance;
                                    }}
                                    
                                    return [
                                        '🎯 Alcance Acumulado: ' + alcance.toFixed(2) + '%',
                                        '📈 Alcance del día: ' + incrementoDia.toFixed(2) + '%',
                                        '💰 Recupero Acumulado: ' + recuperoAcumulado.toLocaleString('es-PE', {{ 
                                            minimumFractionDigits: 0,
                                            maximumFractionDigits: 0 
                                        }})
                                    ];
                                }}
                            }}
                        }},
                        legend: {{ display: false }}
                    }},
                    scales: {{
                        x: {{
                            title: {{ 
                                display: false, 
                                text: 'Días del Mes',
                                font: {{ size: 14, weight: 'bold' }}
                            }},
                            ticks: {{ 
                                font: {{ size: 16 }},
                                maxRotation: 0,
                                minRotation: 0,
                                autoSkip: false,
                                callback: function(value, index, values) {{
                                    // Colores diferentes para días con/sin datos
                                    const tieneDatos = datosValidosParaGrafica[index] !== null && 
                                                      datosValidosParaGrafica[index] > 0;
                                    this.fontColor = tieneDatos ? '#2c3e50' : '#bdc3c7';
                                    return this.getLabelForValue(value);
                                }}
                            }},
                            grid: {{ 
                                display: true,
                                color: 'rgba(0, 0, 0, 0.05)'
                            }}
                        }},
                        y: {{
                            beginAtZero: true,
                            title: {{ 
                                display: false, 
                                text: 'Alcance Acumulado (%)',
                                font: {{ size: 16, weight: 'bold' }}
                            }},
                            ticks: {{
                                callback: function(value) {{ return value + '%'; }},
                                stepSize: 10,
                                font: {{ size: 16 }}
                            }},
                            min: 0,
                            suggestedMax: function() {{
                                // Calcular máximo solo de los datos válidos
                                const valoresValidos = datosValidosParaGrafica.filter(v => v !== null && v > 0);
                                if (valoresValidos.length === 0) return 100;
                                const maxValor = Math.max(...valoresValidos);
                                return Math.max(100, Math.ceil(maxValor / 10) * 10);
                            }}(),
                            grid: {{
                                display: true,
                                color: 'rgba(0, 0, 0, 0.05)'
                            }}
                        }}
                    }},
                    interaction: {{
                        mode: 'nearest',
                        axis: 'x',
                        intersect: false
                    }},
                    elements: {{
                        line: {{
                            tension: 0.4  // Suavizado de la línea
                        }},
                        point: {{
                            hoverBackgroundColor: function(context) {{
                                const value = context.dataset.data[context.dataIndex];
                                return value === null || value === 0 ? 'transparent' : colorLinea;
                            }}
                        }}
                    }}
                }},
                plugins: [dataLabelsPluginTotal]
            }});
            
        }} catch (error) {{
            console.error("Error al crear la gráfica:", error);
            // Mostrar mensaje de error en el canvas
            ctx.fillStyle = '#e74c3c';
            ctx.font = 'bold 20px Arial';
            ctx.textAlign = 'center';
            ctx.fillText('Error al generar gráfica', canvas.width / 2, canvas.height / 2);
        }}
    }}
    
    // Función auxiliar para mostrar mensaje cuando no hay datos
    function mostrarMensajeSinDatos(canvas, periodoCompleto) {{
        const ctx = canvas.getContext('2d');
        ctx.clearRect(0, 0, canvas.width, canvas.height);
        
        // Configurar el canvas para mensaje
        canvas.width = 1300;
        canvas.height = 200;
        canvas.style.display = 'block';
        canvas.style.margin = '0 auto';
        canvas.style.maxWidth = '100%';
        
        // Dibujar mensaje
        ctx.fillStyle = '#95a5a6';
        ctx.font = 'bold 24px Arial';
        ctx.textAlign = 'center';
        ctx.fillText('📊 No hay datos de recupero acumulado disponibles', canvas.width / 2, canvas.height / 2 - 20);
        
        ctx.fillStyle = '#7f8c8d';
        ctx.font = '18px Arial';
        ctx.fillText(`Periodo: ${{periodoCompleto.replace('_', ' ')}}`, canvas.width / 2, canvas.height / 2 + 20);
    }}

    function actualizarVistaRecuperos() {{
        const mesSeleccionado = document.getElementById('selectorMes').value;
        const añoSeleccionado = document.getElementById('selectorAño').value;
        const periodoCompleto = `${{mesSeleccionado}}_${{añoSeleccionado}}`;
        
        if (!datosMeses[periodoCompleto]) {{
            document.getElementById('graficas-recuperos').innerHTML = 
                '<div class="seccion-vacia"><h3>No hay datos disponibles para este periodo</h3></div>';
            return;
        }}
        
        // Calcular y mostrar estadísticas
        calcularEstadisticasRecuperos(periodoCompleto);
        
        // Generar gráficos
        generarGraficasRecuperos(periodoCompleto);
    }}

    // CALCULO DE DATA ESTADÍSTICA PARA ALCANCES
    function calcularEstadisticasRecuperos(periodoCompleto) {{
        // 1. OBTENER DATOS DE SUPERVISORES
        const supervisoresData = datosSupervisores;
        
        // Filtrar supervisores que tienen datos para este periodo
        const supervisoresConDatos = Object.keys(supervisoresData).filter(supervisor => 
            supervisoresData[supervisor] && supervisoresData[supervisor][periodoCompleto]
        );
        
        if (supervisoresConDatos.length === 0) {{
            console.log(`⚠️ No hay datos de supervisores para: ${{periodoCompleto}}`);
            
            // Mostrar estadísticas vacías o con valores cero
            const [mes, año] = periodoCompleto.split('_');
            
            const statsElement = document.getElementById('estadisticas-recuperos');
            if (!statsElement) return;
            
            statsElement.innerHTML = `
                <div style="text-align: center; padding: 40px; background: #f8f9fa; border-radius: 12px; border: 2px dashed #95a5a6; margin: 20px 0;">
                    <div style="font-size: 2.5rem; color: #95a5a6; margin-bottom: 20px;">📭</div>
                    <div style="font-size: 1.5rem; font-weight: bold; color: #7f8c8d; margin-bottom: 10px;">
                        No hay datos de supervisores disponibles
                    </div>
                    <div style="font-size: 1.1rem; color: #95a5a6;">
                        Periodo: ${{mes}} ${{año}}
                    </div>
                </div>
            `;
            return;
        }}
        
        // 2. CALCULAR META TOTAL DEL MES (SUMA DE METAS DE SUPERVISORES)
        let metaTotalMes = 0;
        let recuperoTotalActual = 0;
        
        supervisoresConDatos.forEach(supervisor => {{
            const datosSupervisor = supervisoresData[supervisor][periodoCompleto];
            if (datosSupervisor) {{
                metaTotalMes += datosSupervisor.meta_super || 0;
                recuperoTotalActual += datosSupervisor.total_recupero || 0;
            }}
        }});
        
        // 3. OBTENER DÍAS LABORABLES (de los datos diarios de SUPERVISORES)
        const todasFechas = new Set();
        
        // Recorrer todos los supervisores para obtener fechas únicas
        supervisoresConDatos.forEach(supervisor => {{
            const datosSupervisor = supervisoresData[supervisor][periodoCompleto];
            if (datosSupervisor && datosSupervisor.datos_diarios_supervisor) {{
                Object.keys(datosSupervisor.datos_diarios_supervisor).forEach(fecha => {{
                    // Solo agregar fechas que tengan valor > 0 o que sean relevantes
                    const valor = datosSupervisor.datos_diarios_supervisor[fecha];
                    if (valor !== null && valor !== undefined) {{
                        todasFechas.add(fecha);
                    }}
                }});
            }}
        }});
        
        // Si no hay fechas en datos diarios, intentar con alcance acumulado
        if (todasFechas.size === 0) {{
            supervisoresConDatos.forEach(supervisor => {{
                const datosSupervisor = supervisoresData[supervisor][periodoCompleto];
                if (datosSupervisor && datosSupervisor.alcance_acumulado_diario) {{
                    Object.keys(datosSupervisor.alcance_acumulado_diario).forEach(fecha => {{
                        todasFechas.add(fecha);
                    }});
                }}
            }});
        }}
        
        const fechasOrdenadas = Array.from(todasFechas).sort((a, b) => 
            convertirFechaDiaria(a) - convertirFechaDiaria(b)
        );
        
        const diasLaborables = fechasOrdenadas.length;
        
        // 4. CALCULAR DÍAS TRABAJADOS (con recupero > 0 en datos diarios de supervisores)
        let diasTrabajados = 0;
        
        if (diasLaborables > 0) {{
            // Para cada fecha, verificar si hubo recupero en algún supervisor
            fechasOrdenadas.forEach(fecha => {{
                let totalRecuperoDia = 0;
                
                // Sumar recupero de todos los supervisores para esta fecha
                supervisoresConDatos.forEach(supervisor => {{
                    const datosSupervisor = supervisoresData[supervisor][periodoCompleto];
                    if (datosSupervisor && datosSupervisor.datos_diarios_supervisor) {{
                        // Obtener el INCREMENTO del día, no el acumulado
                        // Para esto necesitamos calcular la diferencia
                        const recuperoDia = datosSupervisor.datos_diarios_supervisor[fecha] || 0;
                        
                        // Si es acumulado, calcular diferencia con el día anterior
                        if (fechasOrdenadas.indexOf(fecha) > 0) {{
                            const fechaAnterior = fechasOrdenadas[fechasOrdenadas.indexOf(fecha) - 1];
                            const recuperoAnterior = datosSupervisor.datos_diarios_supervisor[fechaAnterior] || 0;
                            totalRecuperoDia += (recuperoDia - recuperoAnterior);
                        }} else {{
                            // Primer día
                            totalRecuperoDia += recuperoDia;
                        }}
                    }}
                }});
                
                // Si hubo recupero en algún supervisor, contar como día trabajado
                if (totalRecuperoDia > 0) {{
                    diasTrabajados++;
                }}
            }});
        }}
        
        // 5. CALCULAR ALCANCE ACTUAL
        const alcanceActual = metaTotalMes > 0 ? 
            parseFloat(((recuperoTotalActual / metaTotalMes) * 100).toFixed(2)) : 0;
        
        // 6. CALCULAR META DIARIA
        const metaDiaria = diasLaborables > 0 ? metaTotalMes / diasLaborables : 0;
        
        // 7. CALCULAR PROMEDIO DIARIO REAL
        const promedioDiarioReal = diasTrabajados > 0 ? recuperoTotalActual / diasTrabajados : 0;
        
        // 8. CALCULAR PROYECCIONES
        const diasRestantes = Math.max(0, diasLaborables - diasTrabajados);
        const recuperoProyectado = promedioDiarioReal * diasRestantes;
        const recuperoTotalProyectado = recuperoTotalActual + recuperoProyectado;
        const eficienciaDiaria = metaTotalMes > 0 ? (recuperoTotalProyectado / metaTotalMes) * 100 : 0;
        
        // 9. Calcular eficiencia vs meta diaria
        const eficienciaVsMetaDiaria = metaDiaria > 0 ? (promedioDiarioReal / metaDiaria) * 100 : 0;
        
        // 10. MOSTRAR EN INTERFAZ
        mostrarEstadisticasUnificada(
            periodoCompleto,
            {{
                metaTotalMes,
                recuperoTotalActual,
                alcanceActual,
                eficienciaDiaria,
                diasLaborables,
                diasTrabajados,
                metaDiaria,
                promedioDiarioReal,
                eficienciaVsMetaDiaria,
                recuperoProyectado,
                recuperoTotalProyectado,
                diasRestantes
            }},
            'global'
        );
    }}
    
    function generarGraficasRecuperos(periodoCompleto) {{
        const graficasContainer = document.getElementById('graficas-recuperos');
        if (!graficasContainer) return;
        
            graficasContainer.innerHTML = `
              <div style="text-align: center; margin-bottom: 30px; display:flex; justify-content:center; gap:12px; flex-wrap:wrap;">
                  <button class="boton-exportar-excel" onclick="exportarTablaDetalleExcelAvanzado()">
                      📊 Exportar Recupero Diario
                  </button>
            
                  <button class="boton-exportar-excel" onclick="exportarRecupero4MesesDiaSupervisor('TODOS')">
                      📈 Exportar Recupero 4M (Día x Mes)
                  </button>
              </div>
    
            <!-- CONTENEDOR PARA LAS CUATRO GRÁFICAS CON TOGGLE -->
            <div class="grafica-container" id="contenedorGraficasToggle" style="position: relative;">
                <!-- BARRA DE CONTROLES CON 4 BOTONES -->
                <div style="display: flex; justify-content: center; gap: 20px; margin-bottom: 20px; position: absolute; top: 15px; left: 0; right: 0; z-index: 10;">
                    <button id="btnRecuperoAcumulado" class="btn-toggle-grafica activo" onclick="cambiarGrafica('acumulado')" style="
                        padding: 10px 25px;
                        background: linear-gradient(135deg, #9b59b6, #8e44ad);
                        color: white;
                        border: none;
                        border-radius: 25px;
                        cursor: pointer;
                        font-size: 1rem;
                        font-weight: 600;
                        transition: all 0.3s ease;
                        box-shadow: 0 4px 10px rgba(52, 152, 219, 0.3);
                    ">
                        📈 Recupero Acumulado
                    </button>
                    
                    <button id="btnRecuperoDiario" class="btn-toggle-grafica" onclick="cambiarGrafica('diario')" style="
                        padding: 10px 25px;
                        background: linear-gradient(135deg, #9b59b6, #8e44ad);
                        color: white;
                        border: none;
                        border-radius: 25px;
                        cursor: pointer;
                        font-size: 1rem;
                        font-weight: 600;
                        transition: all 0.3s ease;
                        box-shadow: 0 4px 10px rgba(155, 89, 182, 0.3);
                    ">
                        📅 Recupero Diario
                    </button>
                    
                    <button id="btnRecuperoEquipos" class="btn-toggle-grafica" onclick="cambiarGrafica('equipos')" style="
                        padding: 10px 25px;
                        background: linear-gradient(135deg, #9b59b6, #8e44ad);
                        color: white;
                        border: none;
                        border-radius: 25px;
                        cursor: pointer;
                        font-size: 1rem;
                        font-weight: 600;
                        transition: all 0.3s ease;
                        box-shadow: 0 4px 10px rgba(46, 204, 113, 0.3);
                    ">
                        👥 Recupero por Equipos
                    </button>
                    
                    <button id="btnRecuperoAnual" class="btn-toggle-grafica" onclick="cambiarGrafica('anual')" style="
                        padding: 10px 25px;
                        background: linear-gradient(135deg, #9b59b6, #8e44ad);
                        color: white;
                        border: none;
                        border-radius: 25px;
                        cursor: pointer;
                        font-size: 1rem;
                        font-weight: 600;
                        transition: all 0.3s ease;
                        box-shadow: 0 4px 10px rgba(230, 126, 34, 0.3);
                    ">
                        📊 Recupero Anual
                    </button>
                </div>
                
                <!-- CANVAS PARA LAS GRÁFICAS (SOLO UNO VISIBLE A LA VEZ) -->
                <div id="graficaAcumuladoContainer" style="display: block;">
                    <canvas id="graficaIncrementoTotal" width="1200" height="700"></canvas>
                </div>
                
                <div id="graficaDiarioContainer" style="display: none;">
                    <canvas id="graficaRecuperoDiario" width="1200" height="700"></canvas>
                </div>
                
                <div id="graficaEquiposContainer" style="display: none;">
                    <canvas id="graficaAlcanceEquipos" width="1200" height="700"></canvas>
                </div>
                
                <div id="graficaAnualContainer" style="display: none;">
                    <canvas id="graficaEvolucionAnualRecuperos" width="1200" height="700"></canvas>
                </div>
            </div>
        `;
        
        // Generar gráficos después de un breve delay
        setTimeout(() => {{
            // Generar ambas gráficas inicialmente (pero solo mostrar una)
            generarGraficaIncrementoTotal(periodoCompleto);
            generarGraficaRecuperoDiario(periodoCompleto);
            
            // ✅ SIEMPRE generar gráfica de equipos
            generarGraficaAlcanceEquiposRecuperos(periodoCompleto);
        }}, 100);
    }}

    // ========== FUNCIÓN PARA CAMBIAR ENTRE GRÁFICAS ==========
    function cambiarGrafica(tipo) {{
        const btnAcumulado = document.getElementById('btnRecuperoAcumulado');
        const btnDiario = document.getElementById('btnRecuperoDiario');
        const btnEquipos = document.getElementById('btnRecuperoEquipos');
        const btnAnual = document.getElementById('btnRecuperoAnual');
        const containerAcumulado = document.getElementById('graficaAcumuladoContainer');
        const containerDiario = document.getElementById('graficaDiarioContainer');
        const containerEquipos = document.getElementById('graficaEquiposContainer');
        const containerAnual = document.getElementById('graficaAnualContainer');
        
        if (!btnAcumulado || !btnDiario || !btnEquipos || !btnAnual || 
            !containerAcumulado || !containerDiario || !containerEquipos || !containerAnual) return;
        
        // Remover clase activa de todos los botones
        btnAcumulado.classList.remove('activo');
        btnDiario.classList.remove('activo');
        btnEquipos.classList.remove('activo');
        btnAnual.classList.remove('activo');
        
        // Ocultar todos los contenedores
        containerAcumulado.style.display = 'none';
        containerDiario.style.display = 'none';
        containerEquipos.style.display = 'none';
        containerAnual.style.display = 'none';
        
        // Restaurar estilos de todos los botones
        const estilosInactivo = `
            background: linear-gradient(135deg, #9b59b6, #8e44ad);
            box-shadow: 0 4px 10px rgba(155, 89, 182, 0.3);
        `;
        
        btnAcumulado.style.cssText = btnAcumulado.style.cssText.replace(/background[^;]+;/, '').replace(/box-shadow[^;]+;/, '') + estilosInactivo;
        btnDiario.style.cssText = btnDiario.style.cssText.replace(/background[^;]+;/, '').replace(/box-shadow[^;]+;/, '') + estilosInactivo;
        btnEquipos.style.cssText = btnEquipos.style.cssText.replace(/background[^;]+;/, '').replace(/box-shadow[^;]+;/, '') + estilosInactivo;
        btnAnual.style.cssText = btnAnual.style.cssText.replace(/background[^;]+;/, '').replace(/box-shadow[^;]+;/, '') + estilosInactivo;
        
        // Mostrar la gráfica seleccionada y activar su botón
        const estilosActivo = `
            background: linear-gradient(135deg, #2ecc71, #27ae60);
            box-shadow: 0 0 0 3px white, 0 0 0 6px #2ecc71;
        `;
        
        if (tipo === 'acumulado') {{
            btnAcumulado.classList.add('activo');
            containerAcumulado.style.display = 'block';
            btnAcumulado.style.cssText = btnAcumulado.style.cssText.replace(/background[^;]+;/, '').replace(/box-shadow[^;]+;/, '') + estilosActivo;
        }} 
        else if (tipo === 'diario') {{
            btnDiario.classList.add('activo');
            containerDiario.style.display = 'block';
            btnDiario.style.cssText = btnDiario.style.cssText.replace(/background[^;]+;/, '').replace(/box-shadow[^;]+;/, '') + estilosActivo;
        }}
        else if (tipo === 'equipos') {{
            btnEquipos.classList.add('activo');
            containerEquipos.style.display = 'block';
            btnEquipos.style.cssText = btnEquipos.style.cssText.replace(/background[^;]+;/, '').replace(/box-shadow[^;]+;/, '') + estilosActivo;
        }}
        else if (tipo === 'anual') {{
            btnAnual.classList.add('activo');
            containerAnual.style.display = 'block';
            btnAnual.style.cssText = btnAnual.style.cssText.replace(/background[^;]+;/, '').replace(/box-shadow[^;]+;/, '') + estilosActivo;
        }}
    }}

    // Función para convertir fecha diaria (ej: "3-Nov") a número para ordenar
    function convertirFechaDiaria(fechaStr) {{
        try {{
            const partes = fechaStr.split('-');
            if (partes.length === 2) {{
                const dia = parseInt(partes[0]);
                const mesAbrev = partes[1].toUpperCase();
                
                const meses = {{
                    'ENE': 1, 'FEB': 2, 'MAR': 3, 'ABR': 4, 'MAY': 5, 'JUN': 6,
                    'JUL': 7, 'AGO': 8, 'SEP': 9, 'OCT': 10, 'NOV': 11, 'DIC': 12
                }};
                
                for (const [mesKey, mesNum] of Object.entries(meses)) {{
                    if (mesAbrev.startsWith(mesKey)) {{
                        return mesNum * 100 + dia;
                    }}
                }}
            }}
            return 0;
        }} catch (e) {{
            console.error("Error convirtiendo fecha:", fechaStr, e);
            return 0;
        }}
    }}

    // ========== EXPORTAR EXCEL ==========
    async function exportarTablaDetalleExcelAvanzado(supervisorFiltro = null) {{
        const mesSeleccionado = document.getElementById('selectorMes').value;
        const añoSeleccionado = document.getElementById('selectorAño').value;
        const periodoCompleto = `${{mesSeleccionado}}_${{añoSeleccionado}}`;
        
        if (!datosMeses[periodoCompleto]) {{
            alert('No hay datos para exportar');
            return;
        }}
        
        const fechaHoy = new Date();
        const año = fechaHoy.getFullYear();
        const mes = String(fechaHoy.getMonth() + 1).padStart(2, '0');
        const dia = String(fechaHoy.getDate()).padStart(2, '0');
        const fechaFormato = `${{año}}${{mes}}${{dia}}`;
        
        // Nombre del archivo según filtro
        let nombreArchivo = '';
        if (supervisorFiltro && supervisorFiltro !== 'TODOS') {{
            nombreArchivo = `Recupero y alcance por asesor declarado_${{supervisorFiltro}}_${{mesSeleccionado}}_${{fechaFormato}}.xlsx`;
        }} else {{
            nombreArchivo = `Recupero y alcance por asesor declarado_${{mesSeleccionado}}_${{fechaFormato}}.xlsx`;
        }}
        
        // Obtener datos del periodo
        let asesores = datosMeses[periodoCompleto] || [];
        
        // Filtrar por supervisor si se especifica
        if (supervisorFiltro && supervisorFiltro !== 'TODOS') {{
            asesores = asesores.filter(a => a.supervisor === supervisorFiltro);
        }}
        
        // ========== 3. DETERMINAR FECHAS CON DATOS REALES ==========
        // Obtener TODAS las fechas únicas de los datos diarios
        const todasFechas = new Set();
        asesores.forEach(asesor => {{
            Object.keys(asesor.datos_diarios_asesor || {{}}).forEach(fecha => {{
                todasFechas.add(fecha);
            }});
        }});
        
        // Ordenar fechas
        let todasFechasOrdenadas = Array.from(todasFechas).sort((a, b) => {{
            return convertirFechaDiaria(a) - convertirFechaDiaria(b);
        }});
        
        // ========== NUEVO: FILTRAR FECHAS SIN DATOS ==========
        // Verificar qué fechas tienen al menos UN dato mayor a 0
        const fechasConDatos = [];
        
        todasFechasOrdenadas.forEach(fecha => {{
            let tieneDatos = false;
            
            // Verificar si al menos un asesor tiene datos > 0 para esta fecha
            for (const asesor of asesores) {{
                const valor = asesor.datos_diarios_asesor ? (asesor.datos_diarios_asesor[fecha] || 0) : 0;
                if (valor > 0) {{
                    tieneDatos = true;
                    break;
                }}
            }}
            
            // Solo incluir fechas con datos reales (> 0)
            if (tieneDatos) {{
                fechasConDatos.push(fecha);
            }}
        }});
        
        console.log(`📅 Total fechas disponibles: ${{todasFechasOrdenadas.length}}`);
        console.log(`📅 Fechas con datos (>0): ${{fechasConDatos.length}}`);
        
        // Usar SOLO fechas con datos para la exportación
        const fechasOrdenadas = fechasConDatos;
        
        // Si no hay fechas con datos, mostrar mensaje
        if (fechasOrdenadas.length === 0) {{
            alert('⚠️ No hay datos de recupero diario para exportar. Todas las fechas tienen valor 0.');
            return;
        }}
        
        // Crear workbook con ExcelJS
        const workbook = new ExcelJS.Workbook();
        const worksheet = workbook.addWorksheet('Detalle Diario');
        
        // ========== CONSTRUIR LOS DATOS ==========
        const datosDetalle = [];
        const encabezados = ['EQUIPO', 'ASESOR'];
        
        // Agregar columnas SOLO de fechas con datos
        fechasOrdenadas.forEach(fecha => {{
            encabezados.push(fecha);
        }});
        
        encabezados.push('RECUPERO TOTAL', 'META', '% ALCANCE');
        
        // Datos de asesores
        asesores.forEach(asesor => {{
            const filaDatos = [];
            
            // Equipo y Asesor
            filaDatos.push(asesor.supervisor || 'Sin Supervisor');
            filaDatos.push(asesor.nombre);
            
            // Datos diarios
            fechasOrdenadas.forEach(fecha => {{
                const valor = asesor.datos_diarios_asesor ? (asesor.datos_diarios_asesor[fecha] || 0) : 0;
                filaDatos.push(valor);
            }});
            
            // Recupero total y % Alcance
            filaDatos.push(asesor.recupero || 0);
            
            // ¡IMPORTANTE! Asegurar 2 decimales en el porcentaje
            const metaAsesor = asesor.meta || 0;
            filaDatos.push(metaAsesor);
            
            // ¡IMPORTANTE! Asegurar 2 decimales en el porcentaje
            const porcentajeConDosDecimales = parseFloat((asesor.porcentaje || 0).toFixed(2));
            filaDatos.push(porcentajeConDosDecimales);
            
            datosDetalle.push(filaDatos);
        }});
        
        // ========== APLICAR FORMATOS PROFESIONALES ==========
        
        // 1. ENCABEZADOS
        const headerRow = worksheet.addRow(encabezados);
        headerRow.font = {{
            name: 'Aptos Narrow',
            size: 12,
            bold: true,
            color: {{ argb: 'FFFFFFFF' }}
        }};
        headerRow.fill = {{
            type: 'pattern',
            pattern: 'solid',
            fgColor: {{ argb: 'FF2C3E50' }}
        }};
        headerRow.alignment = {{ horizontal: 'center', vertical: 'middle' }};
        headerRow.border = {{
            top: {{ style: 'thin', color: {{ argb: 'FFFFFFFF' }} }},
            bottom: {{ style: 'thin', color: {{ argb: 'FFFFFFFF' }} }},
            left: {{ style: 'thin', color: {{ argb: 'FFFFFFFF' }} }},
            right: {{ style: 'thin', color: {{ argb: 'FFFFFFFF' }} }}
        }};
        
        // 2. DATOS DE ASESORES
        datosDetalle.forEach((fila, rowIndex) => {{
            const dataRow = worksheet.addRow(fila);
            
            // Aplicar formatos por columna
            dataRow.eachCell((cell, colNumber) => {{
                cell.font = {{ 
                    name: 'Aptos Narrow',
                    size: 10 
                }};
                cell.border = {{
                    top: {{ style: 'thin', color: {{ argb: 'FFD0D0D0' }} }},
                    bottom: {{ style: 'thin', color: {{ argb: 'FFD0D0D0' }} }},
                    left: {{ style: 'thin', color: {{ argb: 'FFD0D0D0' }} }},
                    right: {{ style: 'thin', color: {{ argb: 'FFD0D0D0' }} }}
                }};
                
                // Formato para EQUIPO (columna 1)
                if (colNumber === 1) {{
                    cell.font = {{ 
                        name: 'Aptos Narrow', 
                        size: 10, 
                        bold: true 
                    }};
                    cell.fill = {{
                        type: 'pattern',
                        pattern: 'solid',
                        fgColor: {{ argb: 'FFFFFFFF' }}
                    }};
                    cell.alignment = {{ horizontal: 'left' }};
                }}
                // Formato para ASESOR (columna 2)
                else if (colNumber === 2) {{
                    cell.fill = {{
                        type: 'pattern',
                        pattern: 'solid',
                        fgColor: {{ argb: 'FFFFFF' }}
                    }};
                    cell.alignment = {{ horizontal: 'left' }};
                }}
                // Formato para DÍAS (columnas 3 a n-2) 
                else if (colNumber > 2 && colNumber <= encabezados.length - 3) {{
                    cell.alignment = {{ horizontal: 'right' }};
                    cell.numFmt = '#,##0.00';
                    cell.font = {{ 
                        name: 'Aptos Narrow', 
                        size: 10,
                        color: {{ argb: 'FF000000' }}
                    }};
                    
                    // Nuevos colores condicionales para valores diarios
                    const valor = cell.value;
                    if (valor > 7000) {{
                        cell.fill = {{
                            type: 'pattern',
                            pattern: 'solid',
                            fgColor: {{ argb: 'FF99E7BA' }}  // >7000
                        }};
                    }} else if (valor > 1000) {{
                        cell.fill = {{
                            type: 'pattern',
                            pattern: 'solid',
                            fgColor: {{ argb: 'FFD5F5E3' }}  // >1000
                        }};
                    }} else if (valor > 0) {{
                        cell.fill = {{
                            type: 'pattern',
                            pattern: 'solid',
                            fgColor: {{ argb: 'FFFFFFFF' }}  // >0
                        }};
                    }} else {{
                        cell.fill = {{
                            type: 'pattern',
                            pattern: 'solid',
                            fgColor: {{ argb: 'FFFDEDEC' }}  // =0
                        }};
                    }}
                }}
                // Formato para TOTAL R y META (columnas numéricas finales)
                else if (colNumber === encabezados.length - 3 || colNumber === encabezados.length - 1) {{
                    // Formato estándar para columnas numéricas
                    cell.font = {{ 
                        name: 'Aptos Narrow', 
                        size: 10
                    }};
                    cell.alignment = {{ horizontal: 'right' }};
                    cell.numFmt = '#,##0.00';
                    cell.fill = {{
                        type: 'pattern',
                        pattern: 'solid',
                        fgColor: {{ argb: 'FFFFFFFF' }}
                    }};
                    // Bordes estándar
                    cell.border = {{
                        top: {{ style: 'thin', color: {{ argb: 'FFD0D0D0' }} }},
                        bottom: {{ style: 'thin', color: {{ argb: 'FFD0D0D0' }} }},
                        left: {{ style: 'thin', color: {{ argb: 'FFD0D0D0' }} }},
                        right: {{ style: 'thin', color: {{ argb: 'FFD0D0D0' }} }}
                    }};
                }}
                // Formato para % ALCANCE (última columna)
                else if (colNumber === encabezados.length) {{
                    const fontConfig = {{
                        name: 'Aptos Narrow',
                        size: 12,
                        bold: true,
                        color: {{ argb: 'FF000000' }}
                    }};
                    
                    cell.font = fontConfig;
                    cell.alignment = {{ horizontal: 'center' }};
                    // Solo formato, los datos ya tienen 2 decimales
                    cell.numFmt = '0.00"%";';
                    cell.border.right = {{ style: 'medium', color: {{ argb: 'FFF39C12' }} }};
                    
                    // Nuevos colores según porcentaje (SOLO EL FONDO)
                    const valor = cell.value;
                    if (valor >= 100) {{
                        cell.fill = {{ 
                            type: 'pattern', 
                            pattern: 'solid', 
                            fgColor: {{ argb: 'FF99E7BA' }}
                        }};
                    }} else if (valor >= 70) {{
                        cell.fill = {{ 
                            type: 'pattern', 
                            pattern: 'solid', 
                            fgColor: {{ argb: 'FFFCF3CF' }} 
                        }};
                    }} else if (valor >= 40) {{
                        cell.fill = {{ 
                            type: 'pattern', 
                            pattern: 'solid', 
                            fgColor: {{ argb: 'FFFAE5D3' }} 
                        }};
                    }} else {{
                        cell.fill = {{ 
                            type: 'pattern', 
                            pattern: 'solid', 
                            fgColor: {{ argb: 'FFFADBD8' }} 
                        }};
                    }}
                }}
            }});
        }});
        
        // 3. TOTALES CON SUMAS POR DÍA
        worksheet.addRow([]);
        const totalesFila = ['TOTALES', ''];
        
        // Calcular totales por día
        for (let col = 2; col < encabezados.length - 3; col++) {{
            let totalDia = 0;
            datosDetalle.forEach(fila => {{
                totalDia += fila[col] || 0;
            }});
            totalesFila.push(totalDia);
        }}
        
        // Calcular SOLO total recupero (NO calcular total meta)
        let totalRecupero = 0;
        
        datosDetalle.forEach(fila => {{
            totalRecupero += fila[fila.length - 3] || 0; 

        }});
        
        // SOLO agregar total recupero, NO agregar total meta ni % alcance
        totalesFila.push(totalRecupero, '', '');  // TOTAL R, META vacío, % ALCANCE vacío
        
        const totalRow = worksheet.addRow(totalesFila);
        totalRow.eachCell((cell, colNumber) => {{
            cell.font = {{ 
                name: 'Aptos Narrow', 
                size: 11,
                bold: true, 
                color: {{ argb: 'FF000000' }}
            }};
            cell.fill = {{
                type: 'pattern',
                pattern: 'solid',
                fgColor: {{ argb: 'FFD6DBDF' }}
            }};
            cell.border = {{
                top: {{ style: 'thin', color: {{ argb: 'FF2C3E50' }} }},
                bottom: {{ style: 'thin', color: {{ argb: 'FF2C3E50' }} }}
            }};
            
            if (colNumber > 2 && colNumber <= encabezados.length - 3) {{  // MODIFICADO: -3 en lugar de -2
                cell.numFmt = '#,##0.00';
            }} 
            // Formato SOLO para columna TOTAL R (la única con total)
            else if (colNumber === encabezados.length - 3) {{
                cell.numFmt = '#,##0.00';
                cell.font = {{
                    name: 'Aptos Narrow', 
                    size: 10,
                    bold: false
                }};
                cell.alignment = {{ horizontal: 'right' }};
                cell.fill = {{
                    type: 'pattern',
                    pattern: 'solid',
                    fgColor: {{ argb: 'FFFFFFFF' }}
                }};
                cell.font.color = {{ argb: 'FF000000' }};
                // Asegurar que el valor sea numérico para el formato
                if (cell.value !== null && cell.value !== undefined && cell.value !== '') {{
                    // Convertir a número si es necesario
                    const valor = parseFloat(cell.value);
                    if (!isNaN(valor)) {{
                        cell.value = valor;
                    }}
                }}
            }}
        }});
        
        // ========== AJUSTAR ANCHOS DE COLUMNA ==========
        worksheet.columns = [
            {{ width: 12 }}, // Equipo
            {{ width: 30 }}, // Asesor
            ...Array(fechasOrdenadas.length).fill({{ width: 12 }}), // Días
            {{ width: 23 }}, // TOTAL R
            {{ width: 23 }}, // META (NUEVA)
            {{ width: 15 }}  // % Alcance
        ];
        
        // Congelar paneles
        worksheet.views = [
            {{ state: 'frozen', xSplit: 2, ySplit: 1, activeCell: 'C2' }}
        ];
        
        // ========== GUARDAR ARCHIVO ==========
        try {{
            const buffer = await workbook.xlsx.writeBuffer();
            const blob = new Blob([buffer], {{ type: 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet' }});
            saveAs(blob, nombreArchivo);
            
            console.log('Excel con formato exportado exitosamente');
            alert('✅ Archivo Excel exportado exitosamente: ' + nombreArchivo);
        }} catch (error) {{
            console.error('Error al exportar:', error);
            alert('❌ Error al exportar el archivo. Verifica la consola para más detalles.');
        }}
    }}

    // ========== EXPORTAR RECUPERO 4 MESES (DIA x MES) ==========
    async function exportarRecupero4MesesDiaSupervisor(supervisorParam = '') {{
      const mesSeleccionado = document.getElementById('selectorMes')?.value;
      const añoSeleccionado = document.getElementById('selectorAño')?.value;
      const periodoActual = `${{mesSeleccionado}}_${{añoSeleccionado}}`;
    
      if (!mesSeleccionado || !añoSeleccionado) {{
        alert('Selecciona mes y año');
        return;
      }}
    
      // ✅ TOMAR LA FUENTE REAL DE SUPERVISORES (tu variable es datosSupervisores)
      const fuenteSupervisores =
        (typeof datosSupervisores !== 'undefined' && datosSupervisores) ? datosSupervisores :
        (typeof supervisoresData !== 'undefined' && supervisoresData) ? supervisoresData :
        (window && window.supervisoresData) ? window.supervisoresData :
        null;
    
      if (!fuenteSupervisores || typeof fuenteSupervisores !== 'object') {{
        alert('No hay datos de supervisores disponibles (datosSupervisores)');
        return;
      }}
    
      let supervisorFiltro = (supervisorParam || '').trim();
    
      if (!supervisorFiltro) {{
        const barraFiltrosGlobal = document.getElementById('barraFiltrosSupervisoresGlobal');
        if (barraFiltrosGlobal) {{
          const botonActivoGlobal = barraFiltrosGlobal.querySelector('.filtro-supervisor.activo[data-supervisor]');
          if (botonActivoGlobal) {{
            supervisorFiltro = botonActivoGlobal.getAttribute('data-supervisor') || '';
          }}
        }}
      }}
    
      if (!supervisorFiltro) supervisorFiltro = 'TODOS';
    
      const periodos = _getPeriodosAtras(mesSeleccionado, añoSeleccionado, 4, true);
      if (!periodos.length) {{
        alert('No se pudieron calcular los 4 periodos');
        return;
      }}
    
      const _tituloMes = (periodo) => {{
        const p = String(periodo).split('_');
        const mes = (p[0] || '').toLowerCase();
        return mes ? (mes.charAt(0).toUpperCase() + mes.slice(1)) : periodo;
      }};
    
      const _anioPeriodo = (periodo) => {{
        const p = String(periodo).split('_');
        return p[1] || '';
      }};
    
      const _diaDeFechaStr = (fechaStr) => {{
        const s = String(fechaStr || '').trim();
        const m = s.match(/^(\d{{1,2}})\s*-/);
        return m ? parseInt(m[1], 10) : null;
      }};
    
      const _forwardFill31 = (mapDiaToValor) => {{
        const out = new Array(32).fill(null);
        for (let d = 1; d <= 31; d++) {{
          const v = mapDiaToValor[d];
          if (v === undefined || v === null || Number(v) === 0) {{
            out[d] = (d === 1) ? 0 : (out[d - 1] ?? 0);
          }} else {{
            out[d] = Number(v) || 0;
          }}
        }}
        return out;
      }};
    
      const supervisoresAExportar =
        (supervisorFiltro && supervisorFiltro !== 'TODOS')
          ? [supervisorFiltro]
          : Object.keys(fuenteSupervisores || {{}});
    
      if (!supervisoresAExportar.length) {{
        alert('No hay supervisores para exportar');
        return;
      }}
    
      const wb = new ExcelJS.Workbook();
    
      for (const sup of supervisoresAExportar) {{
        const sheetName = String(sup || 'TODOS').substring(0, 31);
        const ws = wb.addWorksheet(sheetName);
    
        const row1 = ['', ...periodos.map(p => _anioPeriodo(p))];
        const row2 = ['Dia', ...periodos.map(p => _tituloMes(p))];
    
        ws.addRow(row1);
        ws.addRow(row2);
    
        const colValoresPorPeriodo = {{}};
    
        for (const periodo of periodos) {{
          const dataPeriodo = fuenteSupervisores?.[sup]?.[periodo];
          const diarios = dataPeriodo?.datos_diarios_supervisor || {{}};
    
          const mapDia = {{}};
          Object.keys(diarios).forEach((k) => {{
            const d = _diaDeFechaStr(k);
            if (!d || d < 1 || d > 31) return;
            mapDia[d] = Number(diarios[k]) || 0;
          }});
    
          colValoresPorPeriodo[periodo] = _forwardFill31(mapDia);
        }}
    
        for (let d = 1; d <= 31; d++) {{
          const row = [d];
          for (const periodo of periodos) {{
            row.push(colValoresPorPeriodo[periodo][d] ?? 0);
          }}
          ws.addRow(row);
        }}
    
        ws.getRow(1).font = {{ bold: true, color: {{ argb: 'FFFFFFFF' }} }};
        ws.getRow(1).alignment = {{ horizontal: 'center', vertical: 'middle' }};
        ws.getRow(1).fill = {{
          type: 'pattern',
          pattern: 'solid',
          fgColor: {{ argb: 'FF0D47A1' }}
        }};
    
        ws.getRow(2).font = {{ bold: true, color: {{ argb: 'FFFFFFFF' }} }};
        ws.getRow(2).alignment = {{ horizontal: 'center', vertical: 'middle' }};
        ws.getRow(2).fill = {{
          type: 'pattern',
          pattern: 'solid',
          fgColor: {{ argb: 'FF2C3E50' }}
        }};
    
        ws.views = [{{ state: 'frozen', ySplit: 2, xSplit: 1 }}];
    
        for (let c = 2; c <= (1 + periodos.length); c++) {{
          ws.getColumn(c).numFmt = '#,##0';
        }}
    
        ws.getColumn(1).width = 6;
        for (let c = 2; c <= (1 + periodos.length); c++) {{
          ws.getColumn(c).width = 16;
        }}
      }}
    
      const fechaHoy = new Date();
      const yy = fechaHoy.getFullYear();
      const mm = String(fechaHoy.getMonth() + 1).padStart(2, '0');
      const dd = String(fechaHoy.getDate()).padStart(2, '0');
      const stamp = `${{yy}}${{mm}}${{dd}}`;
    
      const nombreArchivo = `Recupero_4M_Dia_x_Mes_${{supervisorFiltro}}_${{periodoActual}}_${{stamp}}.xlsx`;
      const buffer = await wb.xlsx.writeBuffer();
    
      saveAs(
        new Blob([buffer], {{
          type: "application/vnd.openxmlformats-officedocument.spreadsheetml.sheet"
        }}),
        nombreArchivo
      );
    }}
    
    function generarGraficaAlcanceEquiposRecuperos(periodoCompleto) {{
        const supervisores = Object.keys(datosSupervisores).filter(supervisor => 
            datosSupervisores[supervisor] && datosSupervisores[supervisor][periodoCompleto]
        );
        
        if (supervisores.length === 0) return;
        
        // ========== OBTENER TODAS LAS FECHAS ÚNICAS ==========
        const todasFechas = new Set();
        
        supervisores.forEach(supervisor => {{
            const datosMes = datosSupervisores[supervisor][periodoCompleto];
            const alcanceDiario = datosMes.alcance_acumulado_diario || {{}};
            
            Object.keys(alcanceDiario).forEach(fecha => {{
                if (alcanceDiario[fecha] !== undefined && alcanceDiario[fecha] !== null) {{
                    todasFechas.add(fecha);
                }}
            }});
        }});
        
        // ========== ORDENAR TODAS LAS FECHAS CRONOLÓGICAMENTE ==========
        const todasFechasArray = Array.from(todasFechas).sort((a, b) => 
            convertirFechaDiaria(a) - convertirFechaDiaria(b)
        );
        
        console.log(`📊 Gráfica equipos - Total fechas: ${{todasFechasArray.length}}`);
        
        // ========== PREPARAR DATOS CON TODAS LAS FECHAS ==========
        const datasets = [];
        const colores = ['#FF6384', '#36A2EB', '#FFCE56', '#8AC926', '#9966FF', '#FF9F40'];
        
        supervisores.forEach((supervisor, index) => {{
            const datosMes = datosSupervisores[supervisor][periodoCompleto];
            const alcanceDiario = datosMes.alcance_acumulado_diario || {{}};
            
            // USAR TODAS LAS FECHAS
            const datosAlcance = todasFechasArray.map(fecha => {{
                const valor = alcanceDiario[fecha];
                // Si no hay valor o es 0, poner null (la línea se detendrá)
                return (valor !== undefined && valor !== null && valor > 0) ? valor : null;
            }});
            
            // Verificar que haya datos válidos (no todos null)
            const tieneDatos = datosAlcance.some(valor => valor !== null);
            
            if (tieneDatos) {{
                const color = colores[index % colores.length];
                
                datasets.push({{
                    label: supervisor,
                    data: datosAlcance,
                    borderColor: color,
                    backgroundColor: color + '20',
                    borderWidth: 3,
                    fill: false,
                    tension: 0.4,
                    pointRadius: function(context) {{
                        // Puntos solo para datos válidos
                        const value = context.dataset.data[context.dataIndex];
                        return value === null ? 0 : 6;
                    }},
                    pointHoverRadius: function(context) {{
                        const value = context.dataset.data[context.dataIndex];
                        return value === null ? 0 : 10;
                    }},
                    pointBackgroundColor: function(context) {{
                        const value = context.dataset.data[context.dataIndex];
                        return value === null ? 'transparent' : color;
                    }},
                    pointBorderColor: function(context) {{
                        const value = context.dataset.data[context.dataIndex];
                        return value === null ? 'transparent' : '#ffffff';
                    }},
                    pointBorderWidth: 2,
                    spanGaps: false  // IMPORTANTE: No conectar puntos nulos
                }});
            }}
        }});
        
        // ========== VERIFICAR QUE HAYA DATASETS VÁLIDOS ==========
        if (datasets.length === 0) {{
            const canvas = document.getElementById('graficaAlcanceEquipos');
            if (!canvas) return;
            
            const ctx = canvas.getContext('2d');
            ctx.clearRect(0, 0, canvas.width, canvas.height);
            
            canvas.width = 1200;
            canvas.height = 300;
            canvas.style.display = 'block';
            canvas.style.margin = '0 auto';
            canvas.style.maxWidth = '100%';
            
            ctx.fillStyle = '#95a5a6';
            ctx.font = 'bold 24px Arial';
            ctx.textAlign = 'center';
            ctx.fillText('📊 No hay datos de equipos disponibles', canvas.width / 2, canvas.height / 2 - 20);
            
            ctx.fillStyle = '#7f8c8d';
            ctx.font = '18px Arial';
            ctx.fillText(`Periodo: ${{periodoCompleto.replace('_', ' ')}}`, canvas.width / 2, canvas.height / 2 + 20);
            
            return;
        }}
        
        // ========== AGREGAR LÍNEA DE META ==========
        datasets.push({{
            label: 'Meta 100%',
            data: Array(todasFechasArray.length).fill(100),
            borderColor: '#2ecc71',
            backgroundColor: 'transparent',
            borderWidth: 2,
            borderDash: [5, 5],
            fill: false,
            tension: 0,
            pointRadius: 0,
            order: 999
        }});
        
        // ========== CONFIGURAR CANVAS ==========
        const canvas = document.getElementById('graficaAlcanceEquipos');
        canvas.width = 1200;
        canvas.height = 700;
        canvas.style.display = 'block';
        canvas.style.margin = '0 auto';
        canvas.style.maxWidth = '100%';
    
        if (!canvas) return;
        
        const ctx = canvas.getContext('2d');
        
        // ========== DESTRUIR GRÁFICA ANTERIOR ==========
        if (chartAlcanceEquipos) {{
            chartAlcanceEquipos.destroy();
        }}
        
        // ========== OBTENER TÍTULO ==========
        const [mes, año] = periodoCompleto.split('_');
        const tituloGrafica = [`        `,`        `];
        
        // ========== CREAR GRÁFICA CON TODAS LAS FECHAS ==========
        chartAlcanceEquipos = new Chart(ctx, {{
            type: 'line',
            data: {{
                labels: todasFechasArray.map(fecha => {{
                    // Mostrar solo el número del día
                    if (fecha.includes('-')) {{
                        return fecha.split('-')[0];
                    }}
                    return fecha;
                }}),
                datasets: datasets
            }},
            options: {{
                responsive: false,
                maintainAspectRatio: false,
                plugins: {{
                    title: {{
                        display: true,
                        text: tituloGrafica,
                        font: {{ size: 28, weight: 'bold' }}
                    }},
                    legend: {{
                        display: true,
                        position: 'bottom',
                        labels: {{ 
                            font: {{ 
                                size: 18, 
                                weight: 'bold' 
                            }} 
                        }}
                    }},
                    tooltip: {{
                        enabled: true,
                        mode: 'nearest',
                        intersect: false,
                        padding: 15,
                        cornerRadius: 10,
                        backgroundColor: 'rgba(0, 0, 0, 0.9)',
                        titleFont: {{ size: 18, weight: 'bold' }},
                        bodyFont: {{ size: 18 }},
                        callbacks: {{
                            title: function(tooltipItems) {{
                                const index = tooltipItems[0].dataIndex;
                                return todasFechasArray[index] || '';
                            }},
                            label: function(context) {{
                                const labelDataset = context.dataset.label || '';
                            
                                // Ocultar solo "Meta 100%" en el tooltip
                                if (labelDataset === 'Meta 100%') {{
                                    return null;
                                }}
                            
                                let label = labelDataset ? labelDataset + ': ' : '';
                            
                                if (context.parsed.y !== null) {{
                                    label += context.parsed.y.toFixed(2) + '%';
                                }} else {{
                                    return `${{labelDataset}}: Sin datos`;
                                }}
                            
                                return label;
                            }}
                        }}
                    }}
                }},
                interaction: {{
                    mode: 'nearest',
                    axis: 'x',
                    intersect: false
                }},
                scales: {{
                    x: {{
                        title: {{
                            display: false,
                            text: 'Días del Mes',
                            font: {{ size: 12, weight: 'bold' }}
                        }},
                        ticks: {{
                            font: {{ size: 16 }},
                            callback: function(value, index, values) {{
                                // Colores diferentes para días con/sin datos en al menos un supervisor
                                const tieneDatosEnAlguno = datasets.some(dataset => 
                                    dataset.data[index] !== null && 
                                    dataset.label !== 'Meta 100%'
                                );
                                this.fontColor = tieneDatosEnAlguno ? '#2c3e50' : '#bdc3c7';
                                return this.getLabelForValue(value);
                            }}
                        }},
                        grid: {{
                            drawOnChartArea: true
                        }}
                    }},
                    y: {{
                        type: 'linear',
                        display: true,
                        position: 'left',
                        title: {{
                            display: false,
                            text: '% Alcance Acumulado',
                            font: {{ size: 12, weight: 'bold' }}
                        }},
                        ticks: {{
                            stepSize: 10,
                            callback: function(value) {{
                                return value + '%';
                            }},
                            font: {{ size: 16 }}
                        }},
                        min: 0,
                        suggestedMax: 100
                    }}
                }}
            }}
        }});
    }}
    
    function generarGraficaRecuperoDiario(periodoCompleto) {{
        const asesores = datosMeses[periodoCompleto] || [];
        const todasFechas = new Set();
        
        // Obtener todas las fechas únicas de los datos diarios
        asesores.forEach(asesor => {{
            Object.keys(asesor.datos_diarios_asesor || {{}}).forEach(fecha => {{
                todasFechas.add(fecha);
            }});
        }});
        
        if (todasFechas.size === 0) {{
            // Limpiar el canvas
            const canvas = document.getElementById('graficaRecuperoDiario');
            canvas.width = 1200;
            canvas.height = 700;
            canvas.style.display = 'block';
            canvas.style.margin = '0 auto';
            canvas.style.maxWidth = '100%';
            const ctx = canvas.getContext('2d');
            ctx.clearRect(0, 0, canvas.width, canvas.height);
            
            // Destruir gráfica anterior si existe
            if (chartTendencia) {{
                chartTendencia.destroy();
                chartTendencia = null;
            }}
            return;
        }}
        
        const fechasOrdenadas = Array.from(todasFechas).sort((a, b) => {{
            return convertirFechaDiaria(a) - convertirFechaDiaria(b);
        }});
        
        // ========== NUEVO: CALCULAR META DIARIA ==========
        // Obtener meta total del mes
        let metaTotalMes = 0;
        const supervisoresProcesados = new Set();
        
        // Calcular meta total de todos los supervisores (sin duplicados)
        asesores.forEach(asesor => {{
            const supervisor = asesor.supervisor;
            if (supervisor && supervisor !== 'Sin Supervisor' && !supervisoresProcesados.has(supervisor)) {{
                metaTotalMes += asesor.meta_super || 0;
                supervisoresProcesados.add(supervisor);
            }}
        }});
        
        // Obtener días laborables (cantidad de fechas únicas con columnas)
        const diasLaborables = fechasOrdenadas.length;
        
        // Calcular meta diaria
        const metaDiaria = diasLaborables > 0 ? metaTotalMes / diasLaborables : 0;
        
        console.log(`📊 Meta diaria calculada: Meta total: S/ ${{metaTotalMes.toLocaleString()}}, Días laborables: ${{diasLaborables}}, Meta diaria: S/ ${{metaDiaria.toLocaleString()}}`);
        
        // ========== CALCULAR ALCANCE DIARIO ==========
        const alcanceDiario = fechasOrdenadas.map(fecha => {{
            let totalRecuperoDia = 0;
            asesores.forEach(asesor => {{
                totalRecuperoDia += (asesor.datos_diarios_asesor || {{}})[fecha] || 0;
            }});
            
            // Calcular porcentaje de alcance diario
            // Si metaDiaria es 0 (no hay meta definida), mostrar 0%
            const porcentajeDiario = metaDiaria > 0 ? (totalRecuperoDia / metaDiaria) * 100 : 0;
            
            return {{
                fecha: fecha,
                recupero: totalRecuperoDia,
                porcentaje: porcentajeDiario
            }};
        }});
        
        const canvas = document.getElementById('graficaRecuperoDiario');
        canvas.width = 1300;
        canvas.height = 700;
        canvas.style.display = 'block';
        canvas.style.margin = '0 auto';
        canvas.style.maxWidth = '100%';
        const ctx = canvas.getContext('2d');
    
        // Destruir gráfica anterior si existe
        if (chartTendencia) {{
            chartTendencia.destroy();
            chartTendencia = null;
        }}
        
        // LIMPIAR EL CANVAS COMPLETAMENTE
        ctx.clearRect(0, 0, canvas.width, canvas.height);
        
        // Preparar datos para la gráfica
        const porcentajes = alcanceDiario.map(dia => dia.porcentaje);
        
        // Crear gradiente de colores según el alcance diario
        const coloresBarras = porcentajes.map(porcentaje => {{
            if (porcentaje <= 0) return '#F3E5F5'; // Morado muy claro
            if (porcentaje <= 20) return '#E1BEE7'; // Lila claro
            if (porcentaje <= 40) return '#BA68C8'; // Lila
            if (porcentaje <= 60) return '#8E24AA'; // Morado claro
            if (porcentaje <= 80) return '#6A1B9A'; // Morado
            if (porcentaje <= 100) return '#4A148C'; // Morado oscuro
            return '#38006B'; // Morado muy oscuro (más de 100%)
        }});
        
        // Obtener mes y año para el título
        const [mes, año] = periodoCompleto.split('_');

        // PLUGIN PARA ETIQUETAS DE DATOS (RECUPERO DIARIO - TODOS)
        const dataLabelsPluginDiarioTotal = {{
            id: 'dataLabelsDiarioTotal',
            afterDatasetsDraw(chart, args, options) {{
                const {{ctx}} = chart;
                const meta = chart.getDatasetMeta(0);
                
                meta.data.forEach((bar, index) => {{
                    const value = chart.data.datasets[0].data[index];
                    if (value === null || value === undefined) return;
                    
                    ctx.save();
                    
                    const x = bar.x;
                    const y = bar.y - 20; // Posicionar más arriba de la barra
                    
                    // Obtener el recupero DIARIO (no acumulado)
                    const recuperoDiario = alcanceDiario[index]?.recupero || 0;
                    
                    // Formatear el valor COMPLETO con separador de miles
                    const formattedValue = recuperoDiario.toLocaleString('es-PE', {{
                        minimumFractionDigits: 0,
                        maximumFractionDigits: 0
                    }});
                    
                    // TAMAÑO DE FUENTE AUMENTADO (15px)
                    ctx.font = 'bold 15px Arial';
                    ctx.textAlign = 'center';
                    ctx.textBaseline = 'bottom';
                    
                    // Color de texto según el valor - SIN FONDO, SOLO TEXTO
                    let textColor = '#2c3e50';
                    if (recuperoDiario === 0) textColor = '#95a5a6';
                    else if (recuperoDiario < metaDiaria * 0.5) textColor = '#e74c3c';
                    else if (recuperoDiario < metaDiaria) textColor = '#f39c12';
                    else if (recuperoDiario < metaDiaria * 1.2) textColor = '#27ae60';
                    else textColor = '#2ecc71';
                    
                    ctx.fillStyle = textColor;
                    
                    // Sombra sutil para mejor legibilidad - SIN FONDO BLANCO
                    ctx.shadowColor = 'rgba(255, 255, 255, 0.95)';
                    ctx.shadowBlur = 8;
                    ctx.shadowOffsetX = 0;
                    ctx.shadowOffsetY = 0;
                    
                    // DIBUJAR SOLO EL TEXTO - SIN FONDO NI MARCO
                    ctx.fillText(formattedValue, x, y);
                    
                    ctx.restore();
                }});
            }}
        }};
        // Crear la gráfica de BARRAS con porcentaje
        chartTendencia = new Chart(ctx, {{
            type: 'bar',
            data: {{
                labels: fechasOrdenadas.map(fecha => {{
                    // Mostrar solo el número del día
                    if (fecha.includes('-')) {{
                        return fecha.split('-')[0];
                    }}
                    return fecha;
                }}),
                datasets: [{{
                    label: 'Alcance Diario (%)',
                    data: porcentajes,
                    backgroundColor: coloresBarras,
                    borderColor: coloresBarras.map(color => color + 'CC'),
                    borderWidth: 2,
                    borderRadius: 6,
                    borderSkipped: false
                }}]
            }},
            options: {{
                responsive: false,
                maintainAspectRatio: false,
                interaction: {{
                    mode: 'index',
                    intersect: false
                }},
                plugins: {{
                    title: {{
                        display: true,
                        text: [`            `,`            `],
                        font: {{ size: 24, weight: 'bold' }}
                    }},
                    tooltip: {{
                        mode: 'nearest',
                        intersect: false,
                        backgroundColor: 'rgba(0, 0, 0, 0.8)',
                        titleFont: {{ size: 16, weight: 'bold' }},
                        bodyFont: {{ size: 14 }},
                        padding: 15,
                        cornerRadius: 10,
                        callbacks: {{
                            title: function(context) {{
                                const index = context[0].dataIndex;
                                return `Día ${{index + 1}}: ${{fechasOrdenadas[index]}}`;
                            }},
                            label: function(context) {{
                                const index = context.dataIndex;
                                const dia = alcanceDiario[index];
                                
                                return [
                                    `Alcance: ${{dia.porcentaje.toFixed(2)}}%`,
                                    `Recupero: S/ ${{dia.recupero.toLocaleString('es-PE', {{ 
                                        minimumFractionDigits: 2,
                                        maximumFractionDigits: 2 
                                    }})}}`,
                                    `Meta diaria: S/ ${{metaDiaria.toLocaleString('es-PE', {{ 
                                        minimumFractionDigits: 2,
                                        maximumFractionDigits: 2 
                                    }})}}`
                                ];
                            }},
                            afterLabel: function(context) {{
                                const index = context.dataIndex;
                                const dia = alcanceDiario[index];
                                
                                if (dia.porcentaje >= 100) {{
                                    return '✅ Superó la meta diaria';
                                }} else if (dia.porcentaje >= 70) {{
                                    return '🟡 Cerca de la meta';
                                }} else if (dia.porcentaje >= 40) {{
                                    return '🟠 Bajo rendimiento';
                                }} else {{
                                    return '🔴 Rendimiento crítico';
                                }}
                            }}
                        }}
                    }},
                    legend: {{
                        display: false
                    }}
                }},
                scales: {{
                    x: {{
                        title: {{
                            display: false,
                            text: 'Días del Mes',
                            font: {{ size: 14, weight: 'bold' }}
                        }},
                        ticks: {{
                            font: {{ size: 16 }},
                            maxRotation: 0,
                            minRotation: 0
                        }},
                        grid: {{
                            display: false
                        }}
                    }},
                    y: {{
                        type: 'linear',
                        display: true,
                        position: 'left',
                        title: {{
                            display: false,
                            text: 'Alcance Diario (%)',
                            font: {{ size: 14, weight: 'bold' }}
                        }},
                        ticks: {{
                            callback: function(value) {{
                                return value + '%';
                            }},
                            font: {{ size: 16 }},
                            stepSize: 10
                        }},
                        // Ajustar escala según el máximo alcance
                        min: 0,
                        suggestedMax: Math.max(100, Math.ceil(Math.max(...porcentajes) / 10) * 10)
                    }}
                }},
                // Agregar línea horizontal en 100% (meta)
                elements: {{
                    line: {{
                        borderWidth: 0 // No dibujar líneas ya que es gráfica de barras
                    }}
                }}
            }},
            plugins: [dataLabelsPluginDiarioTotal]  // ← AGREGAR EL PLUGIN AQUÍ
        }});
    }}

    // FUNCIONES AUXILIARES

    function sincronizarSelectoresGlobal() {{
        // Obtener valores actuales de los selectores del header
        const mesSeleccionado = document.getElementById('selectorMes').value;
        const añoSeleccionado = document.getElementById('selectorAño').value;
        
        // Actualizar estado global
        estadoGlobal.mesActual = mesSeleccionado;
        estadoGlobal.añoActual = añoSeleccionado;
        estadoGlobal.periodoActual = `${{mesSeleccionado}}_${{añoSeleccionado}}`;
        
        console.log(`🔄 Estado global actualizado: ${{estadoGlobal.periodoActual}}`);
    }}
    
    function obtenerNumeroMes(mes) {{
        const meses = {{
            'ENERO': 0, 'FEBRERO': 1, 'MARZO': 2, 'ABRIL': 3, 'MAYO': 4, 'JUNIO': 5,
            'JULIO': 6, 'AGOSTO': 7, 'SETIEMBRE': 8, 'SEPTIEMBRE': 8, 
            'OCTUBRE': 9, 'NOVIEMBRE': 10, 'DICIEMBRE': 11
        }};
        return meses[mes.toUpperCase()] || 0;
    }}
    
    function convertirFechaDiaria(fechaStr) {{
        try {{
            const partes = fechaStr.split('-');
            if (partes.length === 2) {{
                const dia = parseInt(partes[0]);
                const mesAbrev = partes[1].toUpperCase();
    
                const meses = {{
                    'ENE': 1, 'FEB': 2, 'MAR': 3, 'ABR': 4, 'MAY': 5, 'JUN': 6,
                    'JUL': 7, 'AGO': 8, 'SEP': 9, 'OCT': 10, 'NOV': 11, 'DIC': 12
                }};
    
                for (const [mesKey, mesNum] of Object.entries(meses)) {{
                    if (mesAbrev.startsWith(mesKey)) {{
                        return mesNum * 100 + dia;
                    }}
                }}
            }}
            return 0;
        }} catch (e) {{
            console.error("Error convirtiendo fecha:", fechaStr, e);
            return 0;
        }}
    }}
    
    // Event listeners para slider e input (sin llamadas a funciones inexistentes)
    document.addEventListener('DOMContentLoaded', function () {{
        const slider = document.getElementById('alcanceObjetivoSlider');
        const input = document.getElementById('alcanceObjetivoInput');
    
        if (slider && input) {{
            slider.addEventListener('input', function () {{
                input.value = this.value;
            }});
    
            input.addEventListener('change', function () {{
                let valor = parseInt(this.value);
                if (isNaN(valor)) valor = 0;
                if (valor < 0) valor = 0;
                if (valor > 200) valor = 200;
                this.value = valor;
                slider.value = valor;
            }});
        }}
    }});

    // ========== FUNCIONES TOP 10 ==========
    function mostrarTop10() {{
        const mesSeleccionado = document.getElementById('selectorMes').value;
        const añoSeleccionado = document.getElementById('selectorAño').value;
        const periodoCompleto = `${{mesSeleccionado}}_${{añoSeleccionado}}`;
        
        // Mostrar el modal directamente
        mostrarModalTop10(periodoCompleto);
    }}
    
    function mostrarModalTop10(periodoCompleto) {{
        // Cerrar modal existente si hay uno
        const modalExistente = document.getElementById('modalTop10');
        if (modalExistente) {{
            cerrarModalTop10();
        }}
        
        // Crear el modal
        const modal = document.createElement('div');
        modal.id = 'modalTop10';
        modal.className = 'modal-top10';
        modal.innerHTML = `
            <div class="modal-overlay" onclick="cerrarModalTop10()"></div>
            <div class="modal-content">
                <canvas id="graficaTop10Modal" width="1400" height="850"></canvas>
            </div>
        `;
        
        document.body.appendChild(modal);
        document.body.style.overflow = 'hidden';  // Bloquear scroll
        document.addEventListener('keydown', handleModalEscape);  // Listener ESC
        
        // Agregar estilos si no existen
        if (!document.getElementById('modal-styles-top10')) {{
            agregarEstilosModalMinimalista();
        }}
        
        // Bloquear scroll del body
        document.body.style.overflow = 'hidden';
        
        // Agregar event listener para ESC
        document.addEventListener('keydown', handleModalEscape);
        
        // Mostrar modal
        modal.style.display = 'block';
        
        // Generar la gráfica
        setTimeout(() => {{
            generarGraficaTop10Modal(periodoCompleto);
        }}, 50);
    }}
    
    function agregarEstilosModalMinimalista() {{
        const style = document.createElement('style');
        style.textContent = `
            /* MODAL MINIMALISTA */
            .modal-top10 {{
                display: none;
                position: fixed;
                top: 0;
                left: 0;
                width: 100%;
                height: 100%;
                z-index: 9999;
            }}
            
            .modal-overlay {{
                position: absolute;
                top: 0;
                left: 0;
                width: 100%;
                height: 100%;
                background: rgba(0, 0, 0, 0.92);
                backdrop-filter: blur(5px);
                -webkit-backdrop-filter: blur(5px);
                cursor: pointer;
                animation: fadeIn 0.2s ease-out;
            }}
            
            .modal-content {{
                position: absolute;
                top: 50%;
                left: 50%;
                transform: translate(-50%, -50%);
                background: transparent;
                width: 95%;
                max-width: 1400px;
                max-height: 95vh;
                animation: slideIn 0.3s ease-out;
                display: flex;
                justify-content: center;
                align-items: center;
                padding: 0;
                border: none;
                box-shadow: none;
            }}
            
            #graficaTop10Modal {{
                max-width: 100%;
                max-height: 95vh;
                height: auto !important;
                background: white;
                border-radius: 12px;
                padding: 40px 30px 30px 30px; /* Más padding superior para título */
                box-shadow: 0 20px 60px rgba(0, 0, 0, 0.4);
                cursor: default;
            }}
            
            /* ANIMACIONES SUAVES */
            @keyframes fadeIn {{
                from {{ opacity: 0; }}
                to {{ opacity: 1; }}
            }}
            
            @keyframes slideIn {{
                from {{
                    opacity: 0;
                    transform: translate(-50%, -55%) scale(0.95);
                }}
                to {{
                    opacity: 1;
                    transform: translate(-50%, -50%) scale(1);
                }}
            }}
            
            @keyframes fadeOut {{
                from {{ opacity: 1; }}
                to {{ opacity: 0; }}
            }}
            
            @keyframes slideOut {{
                from {{
                    opacity: 1;
                    transform: translate(-50%, -50%) scale(1);
                }}
                to {{
                    opacity: 0;
                    transform: translate(-50%, -45%) scale(0.95);
                }}
            }}
            
            /* EFECTO DE CIERRE */
            .modal-top10.closing .modal-overlay {{
                animation: fadeOut 0.2s ease-in forwards;
            }}
            
            .modal-top10.closing .modal-content {{
                animation: slideOut 0.2s ease-in forwards;
            }}
            
            /* RESPONSIVE */
            @media (max-width: 1600px) {{
                #graficaTop10Modal {{
                    width: 1300px !important;
                    height: 800px !important;
                    padding: 35px 25px 25px 25px;
                }}
            }}
            
            @media (max-width: 1400px) {{
                #graficaTop10Modal {{
                    width: 1200px !important;
                    height: 750px !important;
                    padding: 30px 20px 20px 20px;
                }}
            }}
            
            @media (max-width: 1200px) {{
                #graficaTop10Modal {{
                    width: 1100px !important;
                    height: 700px !important;
                    padding: 25px 15px 15px 15px;
                }}
            }}
            
            @media (max-width: 992px) {{
                .modal-content {{
                    width: 98%;
                }}
                
                #graficaTop10Modal {{
                    width: 1000px !important;
                    height: 650px !important;
                    padding: 20px 15px 15px 15px;
                }}
            }}
            
            @media (max-width: 768px) {{
                #graficaTop10Modal {{
                    width: 95% !important;
                    height: 550px !important;
                    padding: 15px 10px 10px 10px;
                    border-radius: 8px;
                }}
            }}
            
            @media (max-width: 576px) {{
                #graficaTop10Modal {{
                    width: 98% !important;
                    height: 500px !important;
                    padding: 12px 8px 8px 8px;
                    border-radius: 6px;
                }}
            }}
            
            /* MEJORAR TEXTO EN GRÁFICA */
            .chartjs-render-monitor text {{
                font-family: 'Segoe UI', -apple-system, BlinkMacSystemFont, sans-serif !important;
            }}
        `;
        document.head.appendChild(style);
    }}
    
    function cerrarModalTop10() {{
        const modal = document.getElementById('modalTop10');
        
        if (!modal) return;
        
        // Destruir gráfica si existe
        if (chartTop10Modal) {{
            try {{
                chartTop10Modal.destroy();
                chartTop10Modal = null;
            }} catch (e) {{
                // Ignorar error
            }}
        }}

        document.body.style.overflow = 'auto';

        document.removeEventListener('keydown', handleModalEscape);

        if (modal.parentNode) {{
            modal.parentNode.removeChild(modal);
        }}

        document.body.focus();
    }}

    function handleModalEscape(event) {{
        if (event.key === 'Escape' || event.keyCode === 27) {{
            cerrarModalTop10();
        }}
    }}
    
    function generarGraficaTop10Modal(periodoCompleto) {{
        if (!datosMeses[periodoCompleto]) {{
            alert('No hay datos disponibles para este periodo.');
            cerrarModalTop10();
            return;
        }}
        
        const asesores = datosMeses[periodoCompleto] || [];
        const asesoresOrdenados = [...asesores].sort((a, b) => b.porcentaje - a.porcentaje);
        const top10 = asesoresOrdenados.slice(0, 10);
        
        if (top10.length === 0) {{
            alert('No hay asesores para mostrar en el Top 10.');
            cerrarModalTop10();
            return;
        }}
        
        // Nombres más cortos
        const nombres = top10.map((asesor, index) => {{
            let nombre = asesor.nombre;
            if (nombre.length > 30) {{
                nombre = nombre.substring(0, 28) + '...';
            }}
            return nombre;
        }});
        
        const porcentajes = top10.map(asesor => asesor.porcentaje);
        const clasificaciones = top10.map(asesor => asesor.clasificacion);
        
        const coloresPorClasificacion = {{
            '>100%': '#27ae60',
            '>70%': '#f39c12',
            '>40%': '#e67e22',
            '>0%': '#e74c3c'
        }};
        
        const coloresBarras = clasificaciones.map(clas => coloresPorClasificacion[clas] || '#95a5a6');
        const coloresNombres = top10.map((asesor, index) => {{
            if (index === 0) return '#B8860B';
            if (index === 1) return '#696969';
            if (index === 2) return '#8B4513';
            return '#2c3e50';
        }});
        
        const canvas = document.getElementById('graficaTop10Modal');
        const ctx = canvas.getContext('2d');
        
        // Destruir gráfica anterior
        if (chartTop10Modal) {{
            chartTop10Modal.destroy();
            chartTop10Modal = null;
        }}
        
        // Plugin para etiquetas de porcentaje (LOCAL)
        const dataLabelsPlugin = {{
            id: 'dataLabelsModal',
            afterDatasetsDraw(chart, args, options) {{
                const {{ctx}} = chart;
                const meta = chart.getDatasetMeta(0);
                
                meta.data.forEach((bar, index) => {{
                    const value = chart.data.datasets[0].data[index];
                    if (value === null || value === undefined) return;
                    
                    ctx.save();
                    
                    const x = bar.x - 25;
                    const y = bar.y;
                    const formattedValue = value.toFixed(2) + '%';
                    
                    ctx.font = 'bold 22px Segoe UI';
                    ctx.textAlign = 'right';
                    ctx.textBaseline = 'middle';
                    ctx.fillStyle = '#ffffff';
                    
                    ctx.shadowColor = 'rgba(0, 0, 0, 0.8)';
                    ctx.shadowBlur = 5;
                    ctx.shadowOffsetX = 2;
                    ctx.shadowOffsetY = 2;
                    
                    ctx.fillText(formattedValue, x, y);
                    
                    ctx.restore();
                }});
            }}
        }};
        
        // Plugin para números de posición (LOCAL)
        const positionNumbersPlugin = {{
            id: 'positionNumbersModal',
            afterDatasetsDraw(chart, args, options) {{
                const {{ctx}} = chart;
                const meta = chart.getDatasetMeta(0);
                
                meta.data.forEach((bar, index) => {{
                    ctx.save();
                    
                    const x = 15;
                    const y = bar.y;
                    
                    let textColor;
                    if (index === 0) textColor = '#B8860B';
                    else if (index === 1) textColor = '#696969';
                    else if (index === 2) textColor = '#8B4513';
                    else textColor = '#2c3e50';
                    
                    ctx.font = `bold 20px Segoe UI`;
                    ctx.textAlign = 'left';
                    ctx.textBaseline = 'middle';
                    ctx.fillStyle = textColor;
                    
                    let positionText = (index + 1) + '.';
                    if (index === 0) positionText = '🥇 ' + positionText;
                    else if (index === 1) positionText = '🥈 ' + positionText;
                    else if (index === 2) positionText = '🥉 ' + positionText;
                    
                    ctx.fillText(positionText, x, y);
                    
                    ctx.restore();
                }});
            }}
        }};
        
        // Crear la gráfica con los plugins locales
        chartTop10Modal = new Chart(ctx, {{
            type: 'bar',
            data: {{
                labels: nombres,
                datasets: [{{
                    label: 'Porcentaje de Alcance',
                    data: porcentajes,
                    backgroundColor: coloresBarras,
                    borderColor: coloresBarras.map(color => color + 'CC'),
                    borderWidth: 3,
                    borderRadius: 10,
                    borderSkipped: false
                }}]
            }},
            options: {{
                indexAxis: 'y',
                responsive: false,
                maintainAspectRatio: false,
                
                plugins: {{
                    title: {{
                        display: true,
                        text: `🏆 TOP 10 ASESORES - ${{periodoCompleto.replace('_', ' ')}}`,
                        font: {{
                            size: 32,
                            weight: 'bold',
                            family: 'Segoe UI'
                        }},
                        color: '#2c3e50',
                        padding: {{
                            top: 10,
                            bottom: 30
                        }}
                    }},
                    
                    tooltip: {{
                        backgroundColor: 'rgba(0, 0, 0, 0.9)',
                        titleFont: {{
                            size: 20,
                            weight: 'bold',
                            family: 'Segoe UI'
                        }},
                        bodyFont: {{
                            size: 18,
                            family: 'Segoe UI'
                        }},
                        padding: 20,
                        cornerRadius: 10,
                        displayColors: false,
                        callbacks: {{
                            title: function(context) {{
                                const index = context[0].dataIndex;
                                const asesor = top10[index];
                                const icon = index === 0 ? '🥇 ' : index === 1 ? '🥈 ' : index === 2 ? '🥉 ' : '';
                                return icon + asesor.nombre;
                            }},
                            label: function(context) {{
                                const index = context.dataIndex;
                                const asesor = top10[index];
                                return [
                                    `Porcentaje: ${{context.parsed.x}}%`,
                                    `Clasificación: ${{asesor.clasificacion}}`,
                                    `Supervisor: ${{asesor.supervisor || 'Sin Supervisor'}}`
                                ];
                            }}
                        }}
                    }},
                    legend: {{ display: false }}
                }},
                
                scales: {{
                    x: {{
                        beginAtZero: true,
                        grid: {{
                            display: false,
                            drawBorder: false
                        }},
                        ticks: {{
                            callback: function(value) {{
                                return value + '%';
                            }},
                            font: {{
                                size: 18,
                                family: 'Segoe UI',
                                weight: 'bold'
                            }},
                            color: '#34495e'
                        }},
                        // LIMITAR EJE X
                        min: 0,
                        max: Math.ceil(Math.max(...porcentajes) / 10) * 10,
                        suggestedMax: 100
                    }},
                    y: {{
                        afterFit: function(scale) {{
                            scale.width = 420;
                            scale.left = 60;
                        }},
                        grid: {{
                            display: false,
                            drawBorder: false
                        }},
                        ticks: {{
                            color: function(context) {{
                                return coloresNombres[context.index];
                            }},
                            font: function(context) {{
                                const index = context.index;
                                return {{ 
                                    weight: index <= 2 ? 'bold' : '600',
                                    size: index <= 2 ? 30 : 27,
                                    family: 'Segoe UI'
                                }};
                            }},
                            autoSkip: false,
                            maxRotation: 0,
                            padding: 15
                        }}
                    }}
                }},
                
                animation: {{
                    duration: 1200,
                    easing: 'easeOutQuart'
                }}
            }},
            // Pasar plugins directamente a esta instancia
            plugins: [dataLabelsPlugin, positionNumbersPlugin]
        }});
        
        chartTop10Modal.update();
    }}

    // ========== FUNCIONES PARA MODAL DE CUMPLEAÑOS SIMPLIFICADO ==========
    function mostrarModalCumpleaños() {{
        const mesSeleccionado = document.getElementById('selectorMes').value;
        const añoSeleccionado = document.getElementById('selectorAño').value;
        const periodoCompleto = `${{mesSeleccionado}}_${{añoSeleccionado}}`;
        
        // Mostrar el modal directamente
        mostrarModalCumpleañosSimple(periodoCompleto, mesSeleccionado);
    }}
    
    function mostrarModalCumpleañosSimple(periodoCompleto, mesNombre) {{
        // Cerrar modal existente si hay uno
        const modalExistente = document.getElementById('modalCumpleaños');
        if (modalExistente) {{
            cerrarModalCumpleaños();
        }}
        
        // Cerrar también el modal de Top 10 si está abierto
        const modalTop10 = document.getElementById('modalTop10');
        if (modalTop10) {{
            cerrarModalTop10();
        }}
        
        // Obtener datos de cumpleaños para este periodo
        const todosLosCumpleaños = datosCumpleaños[periodoCompleto] || [];
        
        // Filtrar por mes actual (coincidencia exacta)
        const mesFiltroUpper = mesNombre.toUpperCase();
        const cumpleañosDelMes = todosLosCumpleaños.filter(persona => {{
            return persona.mes === mesFiltroUpper;
        }});
        
        // Ordenar por día del mes
        cumpleañosDelMes.sort((a, b) => a.dia - b.dia);
        
        // Guardar datos globalmente para las funciones de descarga
        window.cumpleañosData = {{
            asesores: cumpleañosDelMes.filter(p => p.puesto === 'ASESOR').sort((a, b) => a.dia - b.dia),
            staff: cumpleañosDelMes.filter(p => p.puesto === 'STAFF').sort((a, b) => a.dia - b.dia),
            mes: mesNombre
        }};
        
        // Asegurar que los estilos estén cargados (FORZAR recarga)
        if (document.getElementById('modal-cumpleaños-styles')) {{
            // Si ya existe, removerlo y recrearlo
            document.getElementById('modal-cumpleaños-styles').remove();
        }}
        agregarEstilosModalCumpleañosSimple();
        
        // SOLO UNA DECLARACIÓN DE MODAL - Con botones de descarga
        const modal = document.createElement('div');
        modal.id = 'modalCumpleaños';
        modal.innerHTML = `
            <div class="modal-overlay-cumple" onclick="cerrarModalCumpleaños()"></div>
            <div class="modal-content-cumple">
                <div class="modal-header-cumple">
                    <div style="display: flex; justify-content: space-between; align-items: center; width: 100%;">
                        <h2>🎂 CUMPLEAÑOS - ${{mesNombre.toUpperCase()}}</h2>
                        <div style="display: flex; gap: 10px;">
                            <button class="btn-descargar-cumple" onclick="descargarTarjetasAsesores()" 
                                    style="background: linear-gradient(135deg, #2ecc71, #27ae60);">
                                📥 Asesores
                            </button>
                            <button class="btn-descargar-cumple" onclick="descargarTarjetasStaff()" 
                                    style="background: linear-gradient(135deg, #3498db, #2980b9);">
                                📥 Staff
                            </button>
                        </div>
                    </div>
                </div>
                <div class="modal-body-cumple">
                    ${{generarContenidoCumpleañosSimple(cumpleañosDelMes, mesNombre)}}
                </div>
            </div>
        `;
        
        document.body.appendChild(modal);
        document.body.style.overflow = 'hidden';
        document.addEventListener('keydown', handleModalCumpleañosEscape);
        
        // Mostrar modal después de un pequeño delay
        setTimeout(() => {{
            modal.style.display = 'block';
        }}, 10);
    }}
    
    function generarContenidoCumpleañosSimple(cumpleañosData, mesNombre) {{
        if (cumpleañosData.length === 0) {{
            return `
                <div style="text-align: center; padding: 50px; color: #666;">
                    <div style="font-size: 3rem; margin-bottom: 20px;">🎂</div>
                    <div style="font-size: 1.5rem; font-weight: bold; margin-bottom: 10px;">
                        No hay cumpleaños este mes
                    </div>
                    <div style="font-size: 1rem;">
                        No se encontraron cumpleaños para ${{mesNombre}}
                    </div>
                </div>
            `;
        }}
        
        // Separar asesores vs staff
        const asesores = cumpleañosData.filter(p => p.puesto === 'ASESOR').sort((a, b) => a.dia - b.dia);
        const staff = cumpleañosData.filter(p => p.puesto === 'STAFF').sort((a, b) => a.dia - b.dia);
        
        // Función para generar una columna de cumpleañeros
        const generarColumna = (personas, titulo, color) => {{
            if (personas.length === 0) {{
                return `
                    <div style="text-align: center; padding: 30px; color: #95a5a6;">
                        <div style="font-size: 2rem; margin-bottom: 10px;">👥</div>
                        <div style="font-weight: bold;">No hay ${{titulo.toLowerCase()}}</div>
                    </div>
                `;
            }}
            
            let html = `
                <div style="margin-bottom: 20px;">
                    <div style="background: ${{color}}; color: white; padding: 8px 15px; 
                         border-radius: 8px; font-weight: bold; margin-bottom: 15px; 
                         text-align: center; font-size: 1.1rem;">
                        ${{titulo}} (${{personas.length}})
                    </div>
                    
                    <div style="max-height: 50vh; overflow-y: auto; padding: 0 5px;">
            `;
            
        personas.forEach((persona) => {{
            // MODIFICADO: Mostrar puesto_real en lugar de alias
            const infoAdicional = persona.puesto_real || persona.alias || '';
            
            html += `
                <div style="display: flex; align-items: center; padding: 12px; margin-bottom: 8px; 
                            background: white; border-radius: 8px; border-left: 4px solid ${{color}};
                            box-shadow: 0 2px 4px rgba(0,0,0,0.05);">
                    <div style="font-size: 1.2rem; margin-right: 12px; color: ${{color}};">
                        👤
                    </div>
                    <div style="flex: 1;">
                        <div style="font-weight: bold; font-size: 1.5rem; color: #2c3e50;">
                            ${{persona.nombre}}
                        </div>
                        <!-- MODIFICADO: Mostrar PUESTO REAL en lugar de alias -->
                        <div style="color: #666; font-size: 0.85rem; font-style: italic;">
                            ${{infoAdicional}}
                        </div>
                    </div>
                    <div style="text-align: right;">
                        <div style="font-size: 1.5rem; font-weight: bold; color: #e74c3c;">
                            ${{persona.dia}}
                        </div>
                        <div style="color: #666; font-size: 1rem;">
                            de ${{persona.mes}}
                        </div>
                    </div>
                </div>
            `;
        }});
        
        html += `
                </div>
            </div>
        `;
        
        return html;
        }};
        
        // Crear layout de dos columnas
        let html = `
        <div style="padding: 20px;">
            <div style="display: grid; grid-template-columns: 1fr 1fr; gap: 25px; min-height: 400px;">
                <!-- COLUMNA ASESORES -->
                <div>
                    ${{generarColumna(asesores, 'ASESORES', '#2ecc71')}}
                </div>
                
                <!-- COLUMNA STAFF -->
                <div>
                    ${{generarColumna(staff, 'STAFF', '#3498db')}}
                </div>
            </div>
        </div>
        `;
        
        return html;
    }}
        
    function agregarEstilosModalCumpleañosSimple() {{
        const style = document.createElement('style');
        style.id = 'modal-cumpleaños-styles';
        style.textContent = `
            /* MODAL DE CUMPLEAÑOS - CON ANIMACIONES */
            #modalCumpleaños {{
                display: none;
                position: fixed;
                top: 0;
                left: 0;
                width: 100%;
                height: 100%;
                z-index: 9999;
            }}
            
            #modalCumpleaños .modal-overlay-cumple {{
                position: absolute;
                top: 0;
                left: 0;
                width: 100%;
                height: 100%;
                background: rgba(0, 0, 0, 0.8);
                backdrop-filter: blur(4px);
                -webkit-backdrop-filter: blur(4px);
                cursor: pointer;
                opacity: 0;
                animation: fadeInCumple 0.3s ease-out forwards;
            }}
            
            #modalCumpleaños .modal-content-cumple {{
                position: absolute;
                top: 50%;
                left: 50%;
                transform: translate(-50%, -50%) scale(0.95);
                background: white;
                width: 90%;
                max-width: 1000px;
                max-height: 90vh;
                border-radius: 15px;
                overflow: hidden;
                box-shadow: 0 25px 60px rgba(0, 0, 0, 0.35);
                opacity: 0;
                animation: slideInCumple 0.4s ease-out 0.1s forwards;
            }}
            
            /* ANIMACIONES ESPECÍFICAS PARA CUMPLEAÑOS */
            @keyframes fadeInCumple {{
                from {{ 
                    opacity: 0; 
                    backdrop-filter: blur(0px);
                    -webkit-backdrop-filter: blur(0px);
                }}
                to {{ 
                    opacity: 1; 
                    backdrop-filter: blur(4px);
                    -webkit-backdrop-filter: blur(4px);
                }}
            }}
            
            @keyframes slideInCumple {{
                from {{
                    opacity: 0;
                    transform: translate(-50%, -50%) scale(0.9);
                    box-shadow: 0 10px 30px rgba(0, 0, 0, 0.1);
                }}
                50% {{
                    opacity: 1;
                    transform: translate(-50%, -52%) scale(1.02);
                    box-shadow: 0 30px 70px rgba(0, 0, 0, 0.4);
                }}
                to {{
                    opacity: 1;
                    transform: translate(-50%, -50%) scale(1);
                    box-shadow: 0 25px 60px rgba(0, 0, 0, 0.35);
                }}
            }}
            
            /* ANIMACIÓN DE CIERRE */
            @keyframes fadeOutCumple {{
                from {{ 
                    opacity: 1; 
                    backdrop-filter: blur(4px);
                    -webkit-backdrop-filter: blur(4px);
                }}
                to {{ 
                    opacity: 0; 
                    backdrop-filter: blur(0px);
                    -webkit-backdrop-filter: blur(0px);
                }}
            }}
            
            @keyframes slideOutCumple {{
                from {{
                    opacity: 1;
                    transform: translate(-50%, -50%) scale(1);
                }}
                to {{
                    opacity: 0;
                    transform: translate(-50%, -45%) scale(0.95);
                }}
            }}
            
            /* CLASE PARA CIERRE ANIMADO */
            #modalCumpleaños.cerrando .modal-overlay-cumple {{
                animation: fadeOutCumple 0.25s ease-in forwards !important;
            }}
            
            #modalCumpleaños.cerrando .modal-content-cumple {{
                animation: slideOutCumple 0.25s ease-in forwards !important;
            }}
            
            /* ESTILOS DEL CONTENIDO (igual que antes) */
            #modalCumpleaños .modal-header-cumple {{
                background: linear-gradient(135deg, #FF6B6B, #EE5A24);
                padding: 20px 30px;
                display: flex;
                justify-content: space-between;
                align-items: center;
            }}
            
            #modalCumpleaños .modal-header-cumple h2 {{
                margin: 0;
                color: white;
                font-size: 2rem;
                font-weight: 800;
            }}
            
            #modalCumpleaños .btn-cerrar-modal-cumple {{
                background: rgba(255, 255, 255, 0.2);
                border: none;
                color: white;
                font-size: 2rem;
                width: 45px;
                height: 45px;
                border-radius: 50%;
                cursor: pointer;
                display: flex;
                align-items: center;
                justify-content: center;
                transition: all 0.3s ease;
            }}
            
            #modalCumpleaños .btn-cerrar-modal-cumple:hover {{
                background: rgba(255, 255, 255, 0.3);
                transform: rotate(90deg);
            }}
            
            #modalCumpleaños .modal-body-cumple {{
                padding: 0;
                max-height: calc(90vh - 85px);
                overflow-y: auto;
            }}
            
            /* RESPONSIVE */
            @media (max-width: 1100px) {{
                #modalCumpleaños .modal-content-cumple {{
                    width: 95%;
                    max-width: 95%;
                }}
            }}
            
            @media (max-width: 768px) {{
                #modalCumpleaños .modal-content-cumple {{
                    width: 98%;
                    max-width: 98%;
                    animation: slideInCumple 0.35s ease-out 0.1s forwards !important;
                }}
                
                @keyframes slideInCumple {{
                    from {{
                        opacity: 0;
                        transform: translate(-50%, -50%) scale(0.95);
                    }}
                    to {{
                        opacity: 1;
                        transform: translate(-50%, -50%) scale(1);
                    }}
                }}
                
                #modalCumpleaños .modal-body-cumple > div > div {{
                    grid-template-columns: 1fr !important;
                }}
            }}

            /* Botones de descarga */
            .btn-descargar-cumple {{
                padding: 10px 20px;
                color: white;
                border: none;
                border-radius: 8px;
                cursor: pointer;
                font-size: 14px;
                font-weight: 600;
                transition: all 0.3s ease;
                display: flex;
                align-items: center;
                gap: 8px;
                box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1);
            }}
            
            .btn-descargar-cumple:hover {{
                transform: translateY(-2px);
                box-shadow: 0 6px 12px rgba(0, 0, 0, 0.15);
            }}
            
            .btn-descargar-cumple:active {{
                transform: translateY(0);
            }}
            
            /* Responsive para botones */
            @media (max-width: 768px) {{
                .btn-descargar-cumple {{
                    padding: 8px 15px;
                    font-size: 12px;
                }}
            }}
        `;
        document.head.appendChild(style);
    }}
    
    function cerrarModalCumpleaños() {{
        const modal = document.getElementById('modalCumpleaños');
        
        if (!modal) return;
        
        // RESTAURAR SCROLL DEL BODY
        document.body.style.overflow = 'auto';
        
        // Eliminar event listener de ESC
        document.removeEventListener('keydown', handleModalCumpleañosEscape);
        
        // Eliminar modal
        if (modal.parentNode) {{
            modal.parentNode.removeChild(modal);
        }}
    }}
    
    function handleModalCumpleañosEscape(event) {{
        if (event.key === 'Escape' || event.keyCode === 27) {{
            cerrarModalCumpleaños();
        }}
    }}

    // ========== FUNCIONES PARA GRÁFICA DE EVOLUCIÓN ANUAL ==========
    function actualizarGraficaEvolucionAnual() {{
        const añoSeleccionado = document.getElementById('selectorAño').value;
        const contenedorGrafica = document.getElementById('grafica-evolucion-anual');
        const añoElement = document.getElementById('año-grafica-evolucion');
        
        if (!añoSeleccionado) {{
            contenedorGrafica.style.display = 'none';
            return;
        }}
        
        // Mostrar contenedor y actualizar año
        contenedorGrafica.style.display = 'block';
        añoElement.textContent = añoSeleccionado;
        
        // SOLO generar gráfica de tendencia
        generarGraficaEvolucionAnualTendencia(añoSeleccionado);
    }}
    
    function generarGraficaEvolucionAnualTendencia(año) {{
        console.log(`🚀 Generando gráfica de tendencia anual para año: ${{año}}`);
        
        // Obtener todos los meses disponibles para este año
        const mesesDelAño = obtenerMesesDelAño(año);
        if (mesesDelAño.length === 0) {{
            mostrarMensajeSinDatosAnual('graficaEvolucionAnualTendencia', año);
            return;
        }}
        
        // Calcular alcance total por mes
        const alcancesPorMes = mesesDelAño.map(mes => {{
            const periodoCompleto = `${{mes}}_${{año}}`;  // ¡CORRECTO!
            return calcularAlcanceTotalMes(periodoCompleto);
        }});
        
        const canvas = document.getElementById('graficaEvolucionAnualTendencia');
        if (!canvas) return;
        
        const ctx = canvas.getContext('2d');
        
        // Destruir gráfica anterior si existe
        if (window.chartEvolucionAnualTendencia) {{
            window.chartEvolucionAnualTendencia.destroy();
            window.chartEvolucionAnualTendencia = null;
        }}
        
        // Limpiar canvas
        ctx.clearRect(0, 0, canvas.width, canvas.height);
        
        canvas.width = 1200;
        canvas.height = 700;
        canvas.style.display = 'block';
        canvas.style.margin = '0 auto';
        canvas.style.maxWidth = '100%';
        
        // Abreviar nombres de meses
        const mesesAbreviados = mesesDelAño.map(mes => {{
            return mes.substring(0, 3).toUpperCase();
        }});
        
        // Calcular línea de tendencia (regresión lineal simple)
        const tendencia = calcularLineaTendencia(alcancesPorMes);
        
        try {{
            window.chartEvolucionAnualTendencia = new Chart(ctx, {{
                type: 'line',
                data: {{
                    labels: mesesAbreviados,
                    datasets: [
                        {{
                            label: 'Alcance Real',
                            data: alcancesPorMes,
                            borderColor: '#3498db',
                            backgroundColor: 'rgba(52, 152, 219, 0.1)',
                            borderWidth: 3,
                            fill: true,
                            tension: 0.4,
                            pointRadius: 6,
                            pointHoverRadius: 10,
                            pointBackgroundColor: '#3498db',
                            pointBorderColor: '#ffffff',
                            pointBorderWidth: 2
                        }},
                        {{
                            label: 'Línea de Tendencia',
                            data: tendencia,
                            borderColor: '#e74c3c',
                            backgroundColor: 'transparent',
                            borderWidth: 2,
                            borderDash: [5, 5],
                            fill: false,
                            tension: 0,
                            pointRadius: 0
                        }},
                        {{
                            label: 'Meta 100%',
                            data: Array(mesesDelAño.length).fill(100),
                            borderColor: '#2ecc71',
                            backgroundColor: 'transparent',
                            borderWidth: 1,
                            borderDash: [3, 3],
                            fill: false,
                            tension: 0,
                            pointRadius: 0
                        }}
                    ]
                }},
                options: {{
                    responsive: false,
                    maintainAspectRatio: false,
                    plugins: {{
                        title: {{
                            display: true,
                            text: `Tendencia de Alcances Anual - Año ${{año}}`,
                            font: {{ size: 18, weight: 'bold' }},
                            padding: {{ bottom: 20 }}
                        }},
                        tooltip: {{
                            mode: 'index',
                            intersect: false,
                            backgroundColor: 'rgba(0, 0, 0, 0.9)',
                            titleFont: {{ size: 14, weight: 'bold' }},
                            bodyFont: {{ size: 13 }},
                            padding: 12,
                            cornerRadius: 8,
                            callbacks: {{
                                label: function(context) {{
                                    const label = context.dataset.label;
                        
                                    if (label === 'Meta 100%' || label === 'Línea de Tendencia') {{
                                        return null;
                                    }}
                        
                                    let texto = label ? label + ': ' : '';
                        
                                    if (context.parsed.y !== null) {{
                                        texto += context.parsed.y.toFixed(2) + '%';
                                    }}
                        
                                    return texto;
                                }}
                            }}
                        }},
                        legend: {{
                            position: 'top',
                            labels: {{ font: {{ size: 12 }} }}
                        }}
                    }},
                    scales: {{
                        x: {{
                            title: {{ 
                                display: true, 
                                text: 'Meses del Año',
                                font: {{ size: 14, weight: 'bold' }}
                            }},
                            ticks: {{ 
                                font: {{ size: 13 }}
                            }}
                        }},
                        y: {{
                            beginAtZero: true,
                            title: {{ 
                                display: true, 
                                text: 'Alcance Total (%)',
                                font: {{ size: 14, weight: 'bold' }}
                            }},
                            ticks: {{
                                callback: function(value) {{ return value + '%'; }},
                                stepSize: 10,
                                font: {{ size: 13 }}
                            }},
                            min: 0,
                            suggestedMax: function() {{
                                const valoresValidos = alcancesPorMes.filter(v => v !== null);
                                if (valoresValidos.length === 0) return 100;
                                const maxValor = Math.max(...valoresValidos);
                                return Math.max(100, Math.ceil(maxValor / 10) * 10);
                            }}()
                        }}
                    }},
                    interaction: {{
                        mode: 'nearest',
                        axis: 'x',
                        intersect: false
                    }}
                }}
            }});
        }} catch (error) {{
            console.error("❌ Error al crear la gráfica de tendencia:", error);
        }}
    }}

    function obtenerMesesDelAño(año) {{
        // Versión mejorada que maneja variaciones en nombres de meses
        const mesesDisponibles = Object.keys(datosMeses || {{}});
        
        console.log(`🔍 Buscando meses para año ${{año}}`);
        console.log(`📊 Meses disponibles:`, mesesDisponibles);
        
        // Filtrar meses que corresponden al año seleccionado
        const mesesDelAño = mesesDisponibles
            .filter(periodo => {{
                // Extraer año del periodo (última parte después del _)
                const partes = periodo.split('_');
                if (partes.length < 2) return false;
                
                const añoPeriodo = partes[partes.length - 1];
                return añoPeriodo === año.toString();
            }})
            .map(periodo => {{
                // Extraer nombre del mes (todo antes del último _)
                const partes = periodo.split('_');
                partes.pop(); // Remover el año
                return partes.join('_'); // Unir en caso de que haya más de un _
            }})
            .filter(mes => mes && mes !== '');
        
        console.log(`📈 Meses encontrados para ${{año}}:`, mesesDelAño);
        
        // Ordenar meses cronológicamente
        const ordenMeses = {{
            'ENERO': 1, 'FEBRERO': 2, 'MARZO': 3, 'ABRIL': 4, 
            'MAYO': 5, 'JUNIO': 6, 'JULIO': 7, 'AGOSTO': 8, 
            'SETIEMBRE': 9, 'OCTUBRE': 10, 
            'NOVIEMBRE': 11, 'DICIEMBRE': 12
        }};
        
        return mesesDelAño.sort((a, b) => {{
            const mesA = a.toUpperCase();
            const mesB = b.toUpperCase();
            
            // Manejar sinónimos (SETIEMBRE/SEPTIEMBRE)
            const numA = ordenMeses[mesA] || (mesA.includes('SETIEMBRE') ? 9 : 0);
            const numB = ordenMeses[mesB] || (mesB.includes('SEPTIEMBRE') ? 9 : 0);
            
            return numA - numB;
        }});
    }}
    
    function calcularAlcanceTotalMes(periodoCompleto) {{
        console.log(`📊 Calculando alcance TOTAL para: ${{periodoCompleto}}`);
        
        if (!datosMeses || !datosMeses[periodoCompleto]) {{
            console.warn(`⚠️ No hay datos para: ${{periodoCompleto}}`);
            return null;
        }}
        
        const asesores = datosMeses[periodoCompleto] || [];
        console.log(`👥 ${{asesores.length}} asesores encontrados`);
        
        // USAR SOLO DATOS DE SUPERVISORES (sin duplicados)
        const supervisoresUnicos = new Map(); // Usar Map para evitar duplicados
        
        // 1. RECOLECTAR DATOS ÚNICOS DE SUPERVISORES
        asesores.forEach(asesor => {{
            const supervisor = asesor.supervisor;
            
            if (supervisor && supervisor !== 'Sin Supervisor') {{
                // Solo guardar una vez por supervisor
                if (!supervisoresUnicos.has(supervisor)) {{
                    supervisoresUnicos.set(supervisor, {{
                        meta_super: asesor.meta_super || 0,
                        recupero_supervisor: asesor.recupero_supervisor || 0
                    }});
                }}
            }}
        }});
        
        // 2. SUMAR METAS Y RECUPEROS DE SUPERVISORES
        let metaTotal = 0;
        let recuperoTotal = 0;
        
        supervisoresUnicos.forEach((datos, supervisor) => {{
            metaTotal += datos.meta_super;
            recuperoTotal += datos.recupero_supervisor;
            console.log(`👨‍💼 ${{supervisor}}: Meta S/ ${{datos.meta_super.toLocaleString()}}, Recupero S/ ${{datos.recupero_supervisor.toLocaleString()}}`);
        }});
        
        console.log(`💰 Meta total (supervisores únicos): S/ ${{metaTotal.toLocaleString()}}`);
        console.log(`💰 Recupero total (supervisores): S/ ${{recuperoTotal.toLocaleString()}}`);
        
        // 3. CALCULAR ALCANCE
        if (metaTotal > 0) {{
            const alcance = (recuperoTotal / metaTotal) * 100;
            console.log(`🎯 Alcance calculado: ${{alcance.toFixed(2)}}%`);
            return alcance;
        }}
        
        console.warn(`❌ Meta total es 0 para ${{periodoCompleto}}`);
        return 0;
    }}
    
    function calcularLineaTendencia(datos) {{
        // Regresión lineal simple: y = mx + b
        const n = datos.length;
        const indices = datos.map((_, i) => i);
        
        // Filtrar datos nulos
        const datosValidos = datos.filter((d, i) => d !== null);
        const indicesValidos = indices.filter((_, i) => datos[i] !== null);
        
        if (datosValidos.length < 2) return datos;
        
        // Calcular promedios
        const sumX = indicesValidos.reduce((a, b) => a + b, 0);
        const sumY = datosValidos.reduce((a, b) => a + b, 0);
        const sumXY = indicesValidos.reduce((sum, x, i) => sum + x * datosValidos[i], 0);
        const sumX2 = indicesValidos.reduce((sum, x) => sum + x * x, 0);
        
        const m = (datosValidos.length * sumXY - sumX * sumY) / (datosValidos.length * sumX2 - sumX * sumX);
        const b = (sumY - m * sumX) / datosValidos.length;
        
        // Calcular valores de la línea de tendencia
        return indices.map(x => m * x + b);
    }}
    
    function mostrarMensajeSinDatosAnual(canvasId, año) {{
        const canvas = document.getElementById(canvasId);
        if (!canvas) return;
        
        const ctx = canvas.getContext('2d');
        ctx.clearRect(0, 0, canvas.width, canvas.height);
        
        canvas.width = 1200;
        canvas.height = 200;
        canvas.style.display = 'block';
        canvas.style.margin = '0 auto';
        canvas.style.maxWidth = '100%';
        
        ctx.fillStyle = '#95a5a6';
        ctx.font = 'bold 20px Arial';
        ctx.textAlign = 'center';
        ctx.fillText('📊 No hay datos disponibles para este año', canvas.width / 2, canvas.height / 2 - 20);
        
        ctx.fillStyle = '#7f8c8d';
        ctx.font = '16px Arial';
        ctx.fillText(`Año: ${{año}}`, canvas.width / 2, canvas.height / 2 + 20);
    }}

    // ========== FUNCIONES PARA ASISTENCIAS ==========

    function actualizarAsistencias() {{
        // Esta función se llama automáticamente al mostrar la sección o cambiar filtros
        cargarDatosAsistencias();
    }}

    function aplicarFiltroAsistencias(supervisor) {{
        cargarDatosAsistencias(supervisor);
    }}
    
    function cargarDatosAsistencias(supervisorFiltro = 'TODOS') {{
        console.log(`📊 Cargando datos de asistencias para supervisor: ${{supervisorFiltro}}`);
        
        // Obtener mes y año del header principal
        const mesSeleccionado = document.getElementById('selectorMes').value;
        const añoSeleccionado = document.getElementById('selectorAño').value;
        const periodoCompleto = `${{mesSeleccionado}}_${{añoSeleccionado}}`;
        
        console.log(`📅 Periodo seleccionado: ${{periodoCompleto}}`);
        
        // DEBUG: Mostrar todas las claves disponibles
        console.log('🔍 Claves en datosAsistencias:', Object.keys(datosAsistencias));
        
        // Verificar si hay datos de asistencias para este periodo
        if (!datosAsistencias || !datosAsistencias[periodoCompleto]) {{
            console.warn(`⚠️ No hay datos de asistencias para el periodo ${{periodoCompleto}}`);
            
            // Mostrar mensaje en la UI
            const estadisticasDiv = document.getElementById('estadisticas-recuperos');
            if (estadisticasDiv) {{
                estadisticasDiv.innerHTML = `
                    <div style="text-align: center; padding: 40px; background: #f8f9fa; border-radius: 12px; border: 2px dashed #95a5a6; margin: 20px 0;">
                        <div style="font-size: 2.5rem; color: #95a5a6; margin-bottom: 20px;">📭</div>
                        <div style="font-size: 1.5rem; font-weight: bold; color: #7f8c8d; margin-bottom: 10px;">
                            No hay datos de asistencias disponibles
                        </div>
                        <div style="font-size: 1.1rem; color: #95a5a6;">
                            Periodo: ${{periodoCompleto.replace('_', ' ')}}
                        </div>
                        <div style="font-size: 0.9rem; color: #95a5a6; margin-top: 15px;">
                            Periodos disponibles: ${{Object.keys(datosAsistencias).join(', ')}}
                        </div>
                    </div>
                `;
            }}
            return;
        }}
        
        // Cargar datos del periodo
        const datosPeriodo = datosAsistencias[periodoCompleto];
        const personasData = datosPeriodo.personas || {{}};
        const diasLaborables = datosPeriodo.dias_laborables || 0;
        
        // Actualizar estadística de días laborables
        const diasLaborablesElement = document.getElementById('estadistica-dias-laborables');
        if (diasLaborablesElement) {{
            diasLaborablesElement.textContent = diasLaborables;
        }}
        
        // Filtrar personas por supervisor
        let personasFiltradas = {{}};
        
        if (supervisorFiltro === 'TODOS') {{
            personasFiltradas = personasData;
            console.log(`📊 Mostrando TODAS las personas: ${{Object.keys(personasFiltradas).length}}`);
        }} else {{
            // Filtrar personas que pertenecen a este equipo/supervisor
            Object.keys(personasData).forEach(persona => {{
                const personaData = personasData[persona];
                if (personaData.supervisor === supervisorFiltro) {{
                    personasFiltradas[persona] = personaData;
                }}
            }});
            console.log(`📊 Filtrando por supervisor ${{supervisorFiltro}}: ${{Object.keys(personasFiltradas).length}} personas`);
        }}
        
        // Si no hay personas filtradas, mostrar mensaje
        if (Object.keys(personasFiltradas).length === 0) {{
            const estadisticasDiv = document.getElementById('estadisticas-recuperos');
            if (estadisticasDiv) {{
                estadisticasDiv.innerHTML = `
                    <div style="text-align: center; padding: 40px; background: #f8f9fa; border-radius: 12px; border: 2px dashed #95a5a6; margin: 20px 0;">
                        <div style="font-size: 2.5rem; color: #95a5a6; margin-bottom: 20px;">👥</div>
                        <div style="font-size: 1.5rem; font-weight: bold; color: #7f8c8d; margin-bottom: 10px;">
                            No hay personas en este equipo
                        </div>
                        <div style="font-size: 1.1rem; color: #95a5a6;">
                            Supervisor: ${{supervisorFiltro}}
                        </div>
                        <div style="font-size: 0.9rem; color: #95a5a6; margin-top: 15px;">
                            ${{Object.keys(personasData).length}} personas en total
                        </div>
                    </div>
                `;
            }}
            
            // Limpiar gráficas y tabla
            ['graficaTopFaltas', 'graficaTopTardanzaTiempo', 'graficaTopTardanzas'].forEach(id => {{
                const canvas = document.getElementById(id);
                if (canvas) {{
                    const ctx = canvas.getContext('2d');
                    ctx.clearRect(0, 0, canvas.width, canvas.height);
                }}
            }});
            
            const tbody = document.getElementById('tbodyDetalleAsistencias');
            if (tbody) tbody.innerHTML = '';
            
            return;
        }}
        
        // Calcular estadísticas
        calcularEstadisticasAsistencias(personasFiltradas);
        
        // Generar gráficas
        generarGraficasAsistencias(personasFiltradas);
        
        // Generar tabla de detalles
        generarTablaDetalleAsistencias(personasFiltradas);
    }}
    
    function calcularEstadisticasAsistencias(personas) {{
        console.log('📈 Calculando estadísticas de asistencias');
        
        let totalAsesores = 0;
        let totalSupervisores = 0;
        let totalFaltas = 0;
        let totalTardanzaSegundos = 0;
        let totalRegistrosTardanza = 0;
        
        // Calcular por persona
        Object.values(personas).forEach(persona => {{
            // Contar por tipo
            if (persona.tipo === 'ASESOR') {{
                totalAsesores++;
            }} else if (persona.tipo === 'SUPERVISOR') {{
                totalSupervisores++;
            }}
            
            // Sumar faltas
            totalFaltas += persona.dias_faltados || 0;
            
            // Sumar tardanza
            totalTardanzaSegundos += persona.tardanza_total_segundos || 0;
            totalRegistrosTardanza += persona.cantidad_tardanzas || 0;
        }});
        
        // Actualizar UI
        document.getElementById('estadistica-asesores').textContent = totalAsesores;
        document.getElementById('estadistica-supervisores').textContent = totalSupervisores;
        document.getElementById('estadistica-faltas').textContent = totalFaltas;
        document.getElementById('estadistica-tardanzas-totales').textContent = totalRegistrosTardanza;
        
        // Formatear tiempo de tardanza
        const horas = Math.floor(totalTardanzaSegundos / 3600);
        const minutos = Math.floor((totalTardanzaSegundos % 3600) / 60);
        const segundos = totalTardanzaSegundos % 60;
        const tiempoTardanza = `${{horas.toString().padStart(2, '0')}}:${{minutos.toString().padStart(2, '0')}}:${{segundos.toString().padStart(2, '0')}}`;
        document.getElementById('estadistica-tardanza-acumulada').textContent = tiempoTardanza;
    }}
    
    function generarGraficasAsistencias(personas) {{
        console.log('📊 Generando gráficas de asistencias');
        
        // Preparar datos para las gráficas
        const datosPersonas = Object.entries(personas).map(([nombre, datos]) => ({{
            nombre,
            dias_faltados: datos.dias_faltados || 0,
            tardanza_total_segundos: datos.tardanza_total_segundos || 0,
            cantidad_tardanzas: datos.cantidad_tardanzas || 0,
            tipo: datos.tipo || 'ASESOR'
        }}));
        
        // 1. Top 7 asesores con más faltas
        const topFaltas = [...datosPersonas]
            .filter(p => p.dias_faltados > 0)
            .sort((a, b) => b.dias_faltados - a.dias_faltados)
            .slice(0, 7);
        
        generarGraficaDona(
            'graficaTopFaltas',
            topFaltas.map(p => p.nombre),
            topFaltas.map(p => p.dias_faltados),
            '❌ Faltas',
            ['#FF4444', '#FF6B6B', '#FF8E8E', '#FFB1B1', '#FFD4D4', '#FFE8E8', '#FFF5F5']
        );
        
        // 2. Top 7 asesores con más tiempo de tardanza
        const topTardanzaTiempo = [...datosPersonas]
            .filter(p => p.tardanza_total_segundos > 0)
            .sort((a, b) => b.tardanza_total_segundos - a.tardanza_total_segundos)
            .slice(0, 7);
        
        // Convertir segundos a minutos para la gráfica
        const tardanzaMinutos = topTardanzaTiempo.map(p => Math.round(p.tardanza_total_segundos / 60));
        
        generarGraficaDona(
            'graficaTopTardanzaTiempo',
            topTardanzaTiempo.map(p => p.nombre),
            tardanzaMinutos,
            '⏰ Minutos',
            ['#FFA726', '#FFB74D', '#FFCC80', '#FFE0B2', '#FFF3E0', '#FFF8E1', '#FFFDE7']
        );
        
        // 3. Top 7 asesores con más registros de tardanza
        const topTardanzas = [...datosPersonas]
            .filter(p => p.cantidad_tardanzas > 0)
            .sort((a, b) => b.cantidad_tardanzas - a.cantidad_tardanzas)
            .slice(0, 7);
        
        generarGraficaDona(
            'graficaTopTardanzas',
            topTardanzas.map(p => p.nombre),
            topTardanzas.map(p => p.cantidad_tardanzas),
            '📊 Registros',
            ['#42A5F5', '#64B5F6', '#90CAF9', '#BBDEFB', '#E3F2FD', '#E8F4FD', '#F0F8FF']
        );
    }}
    
    function generarGraficaDona(canvasId, labels, data, label, colores) {{
        const canvas = document.getElementById(canvasId);
        if (!canvas) return;
        
        // ESTABLECER DIMENSIONES FIJAS (manteniendo el tamaño actual)
        canvas.width = 430;
        canvas.height = 340;
        canvas.style.width = '430px';
        canvas.style.height = '340px';
        canvas.style.maxWidth = '100%';
        
        const ctx = canvas.getContext('2d');
        
        // DESTRUIR GRÁFICA ANTERIOR si existe
        if (canvas.chartInstance) {{
            try {{
                canvas.chartInstance.destroy();
            }} catch (e) {{
                console.warn('Error al destruir gráfica anterior:', e);
            }}
            canvas.chartInstance = null;
        }}
        
        // LIMPIAR CANVAS
        ctx.clearRect(0, 0, canvas.width, canvas.height);
        
        // SI NO HAY DATOS, MOSTRAR MENSAJE
        if (!data || data.length === 0 || data.every(val => val === 0)) {{
            ctx.fillStyle = '#95a5a6';
            ctx.font = 'bold 14px Arial';
            ctx.textAlign = 'center';
            ctx.textBaseline = 'middle';
            ctx.fillText('No hay datos disponibles', canvas.width / 2, canvas.height / 2);
            return;
        }}
        
        try {{
            // PLUGIN PERSONALIZADO - VERSIÓN SIMPLIFICADA
            const doughnutLabelsPlugin = {{
                id: 'doughnutLabelsSimple',
                afterDraw: function(chart) {{
                    const ctx = chart.ctx;
                    
                    // Centro de la dona
                    const meta = chart.getDatasetMeta(0);
                    const arc = meta.data[0];
                    const centerX = arc.x;
                    const centerY = arc.y;
                    
                    // Configurar estilos de texto
                    ctx.save();
                    ctx.textAlign = 'center';
                    ctx.textBaseline = 'middle';
                    
                    // Mostrar etiquetas en cada segmento
                    meta.data.forEach((slice, index) => {{
                        // Obtener el ángulo medio del segmento
                        const startAngle = slice.startAngle;
                        const endAngle = slice.endAngle;
                        const midAngle = startAngle + (endAngle - startAngle) / 2;
                        
                        // Calcular posición fuera de la dona
                        const radius = slice.outerRadius + 25; // Distancia más cercana
                        const x = centerX + Math.cos(midAngle) * radius;
                        const y = centerY + Math.sin(midAngle) * radius;
                        
                        // Mostrar etiqueta completa
                        const textoEtiqueta = labels[index] || '';
                        
                        const valueText = data[index];
                        
                        // Configurar estilos según el tipo de gráfica
                        ctx.font = 'bold 14px Segoe UI';
                        
                        // Color del texto
                        ctx.fillStyle = '#2c3e50';
                        
                        // Fondo para mejor legibilidad
                        ctx.shadowColor = 'rgba(255, 255, 255, 0.8)';
                        ctx.shadowBlur = 6;
                        
                        // 🔴 Dibujar SOLO el primer nombre
                        ctx.fillText(textoEtiqueta, x, y - 8);
                        
                        // Dibujar valor con formato especial
                        ctx.font = 'bold 13px Segoe UI';
                        
                        // Formato según el tipo de dato
                        let displayValue = valueText;
                        let suffix = '';
                        
                        if (label.includes('Minutos')) {{
                            suffix = ' min';
                            displayValue = Math.round(valueText);
                        }} else if (label.includes('Faltas') || label.includes('Registros')) {{
                            suffix = '';
                            displayValue = Math.round(valueText);
                        }}
                        
                        // Color según el valor
                        if (valueText === Math.max(...data)) {{
                            ctx.fillStyle = '#e74c3c'; // Rojo para el máximo
                        }} else if (valueText > Math.max(...data) * 0.5) {{
                            ctx.fillStyle = '#f39c12'; // Naranja para valores altos
                        }} else {{
                            ctx.fillStyle = '#27ae60'; // Verde para valores bajos
                        }}
                        
                        ctx.fillText(displayValue + suffix, x, y + 8);
                        
                        // Línea conectora más corta
                        ctx.beginPath();
                        ctx.moveTo(
                            centerX + Math.cos(midAngle) * (slice.outerRadius + 3),
                            centerY + Math.sin(midAngle) * (slice.outerRadius + 3)
                        );
                        ctx.lineTo(
                            centerX + Math.cos(midAngle) * (slice.outerRadius + 20),
                            centerY + Math.sin(midAngle) * (slice.outerRadius + 20)
                        );
                        ctx.strokeStyle = colores[index] + '60'; // Más transparente
                        ctx.lineWidth = 1;
                        ctx.stroke();
                    }});
                    
                    ctx.restore();
                }}
            }};
            
            canvas.chartInstance = new Chart(ctx, {{
                type: 'doughnut',
                data: {{
                    labels: labels,
                    datasets: [{{
                        label: label,
                        data: data,
                        backgroundColor: colores,
                        borderColor: colores.map(color => color + 'CC'),
                        borderWidth: 2,
                        hoverOffset: 15,
                        circumference: 360,
                        rotation: -90,
                        radius: '75%'
                    }}]
                }},
                options: {{
                    responsive: false,
                    maintainAspectRatio: false,
                    cutout: '50%', // Hueco un poco más grande
                    plugins: {{
                        legend: {{
                            display: false,
                        }},
                        tooltip: {{
                            callbacks: {{
                                label: function(context) {{
                                    const label = context.label || '';
                                    const value = context.parsed;
                                    const total = context.dataset.data.reduce((a, b) => a + b, 0);
                                    const percentage = Math.round((value / total) * 100);
                                    
                                    // En el tooltip mostrar nombre completo
                                    if (label.includes('Minutos')) {{
                                        return `${{label}}: ${{value}} min (${{percentage}}%)`;
                                    }}
                                    return `${{label}}: ${{value}} (${{percentage}}%)`;
                                }}
                            }},
                            bodyFont: {{ 
                                size: 14, family: 'Segoe UI'
                            }},
                        }}
                    }},
                    layout: {{
                        padding: {{
                            top: 2,    // Menos padding
                            bottom: 2,
                            left: 2,
                            right: 2
                        }}
                    }}
                }},
                plugins: [doughnutLabelsPlugin]
            }});
        }} catch (error) {{
            console.error(`❌ Error creando gráfica ${{canvasId}}:`, error);
            
            // Mostrar mensaje de error en el canvas
            ctx.fillStyle = '#e74c3c';
            ctx.font = '12px Arial';
            ctx.textAlign = 'center';
            ctx.fillText('Error al generar gráfica', canvas.width / 2, canvas.height / 2);
        }}
    }}
        
    function generarTablaDetalleAsistencias(personas) {{
        console.log('📋 Generando tabla de detalles de asistencias');
        
        const tbody = document.getElementById('tbodyDetalleAsistencias');
        if (!tbody) return;
        
        let html = '';
        
        // Ordenar por días faltados (mayor a menor) por defecto
        const personasOrdenadas = Object.entries(personas)
            .map(([nombre, datos]) => ({{ nombre, ...datos }}))
            .sort((a, b) => b.dias_faltados - a.dias_faltados);
        
        // Guardar los datos globalmente para ordenamiento
        window.datosAsistenciasTabla = personasOrdenadas;
        
        personasOrdenadas.forEach(persona => {{
            // Calcular porcentajes
            const porcentajeAsistencia = persona.dias_deberia_trabajar > 0 ? 
                Math.round((persona.dias_asistidos / persona.dias_deberia_trabajar) * 100) : 0;
            
            const porcentajePuntualidad = persona.dias_asistidos > 0 ?
                Math.round(((persona.dias_asistidos - persona.cantidad_tardanzas) / persona.dias_asistidos) * 100) : 0;
            
            // Formatear fecha de ingreso
            const fechaIngreso = persona.fecha_ingreso ? 
                new Date(persona.fecha_ingreso).toLocaleDateString('es-PE') : 'No registrada';
            
            // Formatear tiempo de tardanza
            const horas = Math.floor(persona.tardanza_total_segundos / 3600);
            const minutos = Math.floor((persona.tardanza_total_segundos % 3600) / 60);
            const segundos = persona.tardanza_total_segundos % 60;
            const tiempoTardanza = `${{horas.toString().padStart(2, '0')}}:${{minutos.toString().padStart(2, '0')}}:${{segundos.toString().padStart(2, '0')}}`;
            
            // Obtener supervisor del asesor (de datos de desempeño)
            const supervisor = obtenerSupervisorAsesor(persona.nombre);
            
            // Determinar estado
            let estado = '';
            let estadoColor = '';
            if (persona.dias_faltados === 0) {{
                estado = 'Perfecto';
                estadoColor = '#27ae60';
            }} else if (persona.dias_faltados <= 2) {{
                estado = 'Aceptable';
                estadoColor = '#f39c12';
            }} else if (persona.dias_faltados <= 5) {{
                estado = 'Preocupante';
                estadoColor = '#e67e22';
            }} else {{
                estado = 'Crítico';
                estadoColor = '#e74c3c';
            }}
            
            // Color para supervisores
            const esSupervisor = persona.tipo === 'SUPERVISOR';
            const estiloFila = esSupervisor ? 'style="background-color: #f8f9fa;"' : '';
            
            html += `
                <tr ${{estiloFila}} data-persona="${{persona.nombre}}" 
                    data-tipo="${{persona.tipo}}"
                    data-equipo="${{persona.equipo || ''}}"
                    data-fecha-ingreso="${{persona.fecha_ingreso || ''}}"
                    data-dias-deberia="${{persona.dias_deberia_trabajar}}"
                    data-dias-asistidos="${{persona.dias_asistidos}}"
                    data-dias-faltados="${{persona.dias_faltados}}"
                    data-tardanza-segundos="${{persona.tardanza_total_segundos}}"
                    data-cantidad-tardanzas="${{persona.cantidad_tardanzas}}"
                    data-porcentaje-asistencia="${{porcentajeAsistencia}}"
                    data-porcentaje-puntualidad="${{porcentajePuntualidad}}"
                    data-estado="${{estado}}"
                    data-supervisor="${{supervisor || ''}}">
                    <td><strong>${{persona.nombre}}</strong></td>
                    <td><span style="font-weight: 600; color: ${{esSupervisor ? '#3498db' : '#27ae60'}};">${{persona.tipo}}</span></td>
                    <td>${{persona.equipo || 'Sin equipo'}}</td>
                    <td>${{fechaIngreso}}</td>
                    <td>${{persona.dias_deberia_trabajar}}</td>
                    <td>${{persona.dias_asistidos}}</td>
                    <td style="color: ${{persona.dias_faltados > 0 ? '#e74c3c' : '#27ae60'}}; font-weight: ${{persona.dias_faltados > 0 ? 'bold' : 'normal'}};">${{persona.dias_faltados}}</td>
                    <td style="color: ${{persona.tardanza_total_segundos > 0 ? '#f39c12' : '#27ae60'}};">${{tiempoTardanza}}</td>
                    <td>${{persona.cantidad_tardanzas}}</td>
                    <td>
                        <span style="font-weight: bold; color: ${{porcentajeAsistencia >= 90 ? '#27ae60' : porcentajeAsistencia >= 70 ? '#f39c12' : '#e74c3c'}}">
                            ${{porcentajeAsistencia}}%
                        </span>
                    </td>
                    <td>
                        <span style="font-weight: bold; color: ${{porcentajePuntualidad >= 90 ? '#27ae60' : porcentajePuntualidad >= 70 ? '#f39c12' : '#e74c3c'}}">
                            ${{porcentajePuntualidad}}%
                        </span>
                    </td>
                    <td>
                        <button class="btn-historico" onclick="mostrarOcultarHistorico('${{persona.nombre}}')" 
                                style="padding: 6px 12px; background: linear-gradient(135deg, #3498db, #2980b9); color: white; 
                                       border: none; border-radius: 6px; cursor: pointer; font-size: 0.85rem; font-weight: 600;
                                       display: flex; align-items: center; gap: 5px; transition: all 0.3s ease;">
                            Histórico
                        </button>
                    </td>
                </tr>
                <tr id="historico-${{persona.nombre.replace(/[^a-zA-Z0-9]/g, '-')}}" class="fila-historico" style="display: none;">
                    <td colspan="12" style="padding: 0;">
                        <div id="contenido-historico-${{persona.nombre.replace(/[^a-zA-Z0-9]/g, '-')}}" class="contenedor-historico">
                            <!-- El contenido del histórico se cargará dinámicamente -->
                            <div style="text-align: center; padding: 15px; color: #95a5a6;">
                                Cargando histórico...
                            </div>
                        </div>
                    </td>
                </tr>
            `;
        }});
        
        tbody.innerHTML = html;
        
        // 🔴 REINICIALIZAR ORDENAMIENTO DESPUÉS DE GENERAR LA TABLA
        setTimeout(() => {{
            reinicializarOrdenamientoTablaAsistencias();
        }}, 200);
        
        // Mostrar el contenedor de la tabla
        document.getElementById('tablaDetalleAsistencias').style.display = 'table';
    }}

    // ========== FUNCIÓN NUEVA PARA ASIGNAR ORDENAMIENTO ==========
    function asignarOrdenamientoTablaAsistencias() {{
        console.log('🔄 Asignando ordenamiento a tabla de asistencias');
        
        const thead = document.querySelector('#tablaDetalleAsistencias thead');
        if (!thead) {{
            console.warn('⚠️ No se encontró thead para ordenamiento');
            return;
        }}
        
        const headers = thead.querySelectorAll('th');
        const encabezadosOriginales = [
            'Persona', 'Tipo', 'Equipo', 'Fecha Ingreso', 'Días Debería',
            'Días Asistidos', 'Días Faltados', 'Tiempo Tardanza', 'Cant. Tardanzas',
            '% Asistencia', '% Puntualidad', 'Histórico'
        ];
        
        // Primero limpiar completamente los encabezados
        headers.forEach((header, index) => {{
            // Verificar si ya tiene el formato de ordenamiento
            const tieneContenedor = header.querySelector('div[style*="display: flex"]');
            
            if (tieneContenedor) {{
                // Limpiar completamente
                header.innerHTML = '';
            }}
            
            // Crear NUEVO contenedor para el encabezado con icono
            const originalText = encabezadosOriginales[index] || header.textContent;
            const divContenedor = document.createElement('div');
            divContenedor.style.cssText = 'display: flex; align-items: center; justify-content: space-between; cursor: pointer;';
            
            const spanTexto = document.createElement('span');
            spanTexto.textContent = originalText;
            
            const spanIcono = document.createElement('span');
            spanIcono.className = 'sort-icon';
            spanIcono.style.cssText = 'font-size: 0.8em; opacity: 0.5;';
            spanIcono.textContent = '↕️';
            
            divContenedor.appendChild(spanTexto);
            divContenedor.appendChild(spanIcono);
            
            // Limpiar header y agregar nuevo contenido
            header.innerHTML = '';
            header.appendChild(divContenedor);
            
            // Agregar event listener directo al header (no al div)
            header.style.cursor = 'pointer';
            
            // Remover cualquier listener previo
            const nuevoHeader = header.cloneNode(true);
            header.parentNode.replaceChild(nuevoHeader, header);
            
            // Agregar nuevo listener al header actualizado
            nuevoHeader.addEventListener('click', () => {{
                ordenarTablaAsistenciasPorColumna(index, originalText);
            }});
            
            // Efectos hover simples
            nuevoHeader.addEventListener('mouseenter', () => {{
                nuevoHeader.style.backgroundColor = '#2c3e50';
                const icono = nuevoHeader.querySelector('.sort-icon');
                if (icono) icono.style.opacity = '1';
            }});
            
            nuevoHeader.addEventListener('mouseleave', () => {{
                if (!nuevoHeader.hasAttribute('data-sort')) {{
                    nuevoHeader.style.backgroundColor = '';
                }}
                const icono = nuevoHeader.querySelector('.sort-icon');
                if (icono && !nuevoHeader.hasAttribute('data-sort')) {{
                    icono.style.opacity = '0.5';
                }}
            }});
        }});
        
        console.log('✅ Ordenamiento asignado a encabezados de asistencias');
    }}

    function obtenerSupervisorAsesor(nombreAsesor) {{
        // Buscar supervisor en los datos de desempeño
        const mesSeleccionado = document.getElementById('selectorMes').value;
        const añoSeleccionado = document.getElementById('selectorAño').value;
        const periodoCompleto = `${{mesSeleccionado}}_${{añoSeleccionado}}`;
        
        if (datosMeses && datosMeses[periodoCompleto]) {{
            const asesor = datosMeses[periodoCompleto].find(a => a.nombre === nombreAsesor);
            return asesor ? asesor.supervisor : 'No encontrado';
        }}
        return 'No disponible';
    }}
        
    async function cargarHistoricoPersona(nombrePersona, idSeguro) {{
        const contenidoHistorico = document.getElementById(`contenido-historico-${{idSeguro}}`);
        if (!contenidoHistorico) return;
        
        // Mostrar loader simple
        contenidoHistorico.innerHTML = `
            <div style="text-align: center; padding: 20px;">
                <div style="display: inline-block; width: 40px; height: 40px; border: 3px solid #f3f3f3; border-top: 3px solid #3498db; border-radius: 50%; animation: spin 1s linear infinite;"></div>
                <p style="margin-top: 10px; color: #666; font-size: 0.9rem;">Cargando evolución...</p>
            </div>
            <style>
                @keyframes spin {{ 0% {{ transform: rotate(0deg); }} 100% {{ transform: rotate(360deg); }} }}
            </style>
        `;
        
        try {{
            // ¡IMPORTANTE! Para asistencias, usar SOLO datosAsistencias
            // Obtener TODOS los meses disponibles en datosAsistencias
            const mesesDisponiblesAsistencias = Object.keys(datosAsistencias || {{}});
            
            if (mesesDisponiblesAsistencias.length === 0) {{
                contenidoHistorico.innerHTML = `
                    <div style="text-align: center; padding: 30px; color: #95a5a6;">
                        <div style="font-size: 2.5rem; margin-bottom: 10px;">📭</div>
                        <div style="font-weight: bold; margin-bottom: 5px;">No hay datos de asistencias</div>
                        <div>No se encontraron registros históricos para ${{nombrePersona}}</div>
                    </div>
                `;
                return;
            }}
            
            console.log(`📊 Histórico de asistencias para ${{nombrePersona}}: ${{mesesDisponiblesAsistencias.length}} meses disponibles`);
            console.log(`📅 Meses en asistencias: ${{mesesDisponiblesAsistencias.join(', ')}}`);
            
            // Ordenar meses cronológicamente (más antiguo primero)
            const mesesOrdenados = mesesDisponiblesAsistencias.sort((a, b) => {{
                const [mesA, añoA] = a.split('_');
                const [mesB, añoB] = b.split('_');
                const fechaA = new Date(añoA, obtenerNumeroMes(mesA));
                const fechaB = new Date(añoB, obtenerNumeroMes(mesB));
                return fechaA - fechaB; // Más antiguo primero
            }});
            
            // TOMAR SOLO LOS ÚLTIMOS 6 MESES DISPONIBLES EN ASISTENCIAS
            const maxMeses = 6;
            const mesesParaMostrar = mesesOrdenados.slice(-maxMeses); // Últimos N meses
            
            console.log(`📈 Mostrando últimos ${{mesesParaMostrar.length}} meses de asistencias`);
            
            // Preparar datos históricos SOLO de asistencias
            const datosHistoricos = [];
            
            mesesParaMostrar.forEach(mesCompleto => {{
                const [mes, año] = mesCompleto.split('_');
                
                // Datos de asistencias para esta persona en este mes
                let supervisor = 'No disponible';
                let cartera = 'No definida';
                let porcentajeAsistencia = null;
                let porcentajePuntualidad = null;
                let tardanzaAcumulada = '00:00:00';
                let tieneDatos = false;
                
                // Verificar si hay datos de asistencias para este mes
                if (datosAsistencias && datosAsistencias[mesCompleto]) {{
                    const personaData = datosAsistencias[mesCompleto].personas?.[nombrePersona];
                    
                    if (personaData) {{
                        tieneDatos = true;
                        supervisor = personaData.supervisor || 'No disponible';
                        
                        // Obtener datos básicos
                        const diasDeberia = personaData.dias_deberia_trabajar || 0;
                        const diasAsistidos = personaData.dias_asistidos || 0;
                        const cantidadTardanzas = personaData.cantidad_tardanzas || 0;
                        
                        // SOLO calcular porcentaje si hay datos válidos
                        if (diasDeberia > 0 && diasAsistidos >= 0) {{
                            porcentajeAsistencia = Math.round((diasAsistidos / diasDeberia) * 100);
                        }}
                        
                        // SOLO calcular puntualidad si hay días asistidos
                        if (diasAsistidos > 0) {{
                            porcentajePuntualidad = Math.round(((diasAsistidos - cantidadTardanzas) / diasAsistidos) * 100);
                        }}
                        
                        // Formatear tardanza acumulada
                        const tardanzaSegundos = personaData.tardanza_total_segundos || 0;
                        const horas = Math.floor(tardanzaSegundos / 3600);
                        const minutos = Math.floor((tardanzaSegundos % 3600) / 60);
                        const segundos = tardanzaSegundos % 60;
                        tardanzaAcumulada = `${{horas.toString().padStart(2, '0')}}:${{minutos.toString().padStart(2, '0')}}:${{segundos.toString().padStart(2, '0')}}`;
                        
                        // Intentar obtener cartera del asesor desde datosMeses (si existe)
                        if (datosMeses && datosMeses[mesCompleto]) {{
                            const asesorData = datosMeses[mesCompleto].find(a => a.nombre === nombrePersona);
                            if (asesorData) {{
                                cartera = asesorData.cartera || 'No definida';
                            }}
                        }}
                    }}
                }}
                
                // SOLO agregar si hay datos de asistencias
                if (tieneDatos) {{
                    datosHistoricos.push({{
                        mesCompleto,
                        mes,
                        año,
                        supervisor,
                        cartera,
                        porcentajeAsistencia,
                        porcentajePuntualidad,
                        tardanzaAcumulada,
                        tieneDatos: true
                    }});
                }} else {{
                    console.log(`⚠️ No hay datos de asistencias para ${{nombrePersona}} en ${{mesCompleto}}`);
                }}
            }});
            
            // Verificar que haya datos
            if (datosHistoricos.length === 0) {{
                contenidoHistorico.innerHTML = `
                    <div style="text-align: center; padding: 30px; color: #95a5a6;">
                        <div style="font-size: 2.5rem; margin-bottom: 10px;">📭</div>
                        <div style="font-weight: bold; margin-bottom: 5px;">No hay datos históricos de asistencias</div>
                        <div>No se encontraron registros para ${{nombrePersona}}</div>
                        <div style="font-size: 0.8rem; margin-top: 10px;">
                            Meses disponibles en el sistema: ${{mesesDisponiblesAsistencias.length}}
                        </div>
                    </div>
                `;
                return;
            }}
            
            console.log(`✅ Datos históricos de asistencias válidos: ${{datosHistoricos.length}} meses`);
            console.log(`📊 Datos:`, datosHistoricos.map(d => ({{
                mes: d.mesCompleto,
                asistencia: d.porcentajeAsistencia,
                puntualidad: d.porcentajePuntualidad
            }})));
            
            // Generar HTML del histórico con nueva estructura
            contenidoHistorico.innerHTML = generarHTMLHistoricoAsistencias(nombrePersona, datosHistoricos);
            
        }} catch (error) {{
            console.error('Error cargando histórico de asistencias:', error);
            contenidoHistorico.innerHTML = `
                <div style="text-align: center; padding: 20px; color: #e74c3c;">
                    <div style="font-weight: bold; margin-bottom: 10px;">Error al cargar histórico</div>
                    <div style="font-size: 0.9rem; color: #666;">${{error.message}}</div>
                    <div style="font-size: 0.8rem; color: #999; margin-top: 5px;">
                        Intente nuevamente más tarde
                    </div>
                </div>
            `;
        }}
    }}
        
    function generarHTMLHistoricoAsistencias(nombrePersona, datosHistoricos) {{
        if (datosHistoricos.length === 0) {{
            return `
                <div style="text-align: center; padding: 30px; color: #95a5a6;">
                    <div style="font-size: 2.5rem; margin-bottom: 10px;">📭</div>
                    <div style="font-weight: bold; margin-bottom: 5px;">No hay datos históricos</div>
                    <div>No se encontraron registros para ${{nombrePersona}}</div>
                </div>
            `;
        }}
        
        // Invertir orden: de más antiguo a más reciente (para progresión)
        const datosOrdenados = [...datosHistoricos];
        
        // Formatear meses para mostrar
        const mesesAbreviados = {{
            'ENERO': 'Ene', 'FEBRERO': 'Feb', 'MARZO': 'Mar', 'ABRIL': 'Abr',
            'MAYO': 'May', 'JUNIO': 'Jun', 'JULIO': 'Jul', 'AGOSTO': 'Ago',
            'SETIEMBRE': 'Set', 'SEPTIEMBRE': 'Set', 'OCTUBRE': 'Oct', 'NOVIEMBRE': 'Nov', 'DICIEMBRE': 'Dic'
        }};
        
        // Preparar datos para tabla invertida
        const mesesLabels = datosOrdenados.map(dato => {{
            const mesAbrev = mesesAbreviados[dato.mes.toUpperCase()] || dato.mes.substring(0, 3);
            return `${{mesAbrev}} ${{dato.año}}`;
        }});
        
        // CALCULAR SUMA DE PORCENTAJES - ¡SOLO USAR DATOS VÁLIDOS (no null)!
        const sumaPorcentajes = datosOrdenados.map(d => {{
            const asistencia = d.porcentajeAsistencia !== null ? d.porcentajeAsistencia : 0;
            const puntualidad = d.porcentajePuntualidad !== null ? d.porcentajePuntualidad : 0;
            return asistencia + puntualidad;
        }});
        
        // CALCULAR PROMEDIOS HISTÓRICOS - ¡IGNORAR VALORES NULL!
        const datosAsistenciaValidos = datosOrdenados.filter(d => d.porcentajeAsistencia !== null);
        const promedioAsistencia = datosAsistenciaValidos.length > 0 
            ? Math.round(datosAsistenciaValidos.reduce((sum, d) => sum + d.porcentajeAsistencia, 0) / datosAsistenciaValidos.length)
            : 0;
        
        const datosPuntualidadValidos = datosOrdenados.filter(d => d.porcentajePuntualidad !== null);
        const promedioPuntualidad = datosPuntualidadValidos.length > 0
            ? Math.round(datosPuntualidadValidos.reduce((sum, d) => sum + d.porcentajePuntualidad, 0) / datosPuntualidadValidos.length)
            : 0;
        
        // Calcular promedio de tardanzas (convertir a segundos)
        let totalTardanzaSegundos = 0;
        let mesesConTardanza = 0;
        
        datosOrdenados.forEach(d => {{
            const tiempo = d.tardanzaAcumulada;
            if (tiempo && tiempo !== '00:00:00') {{
                const [horas, minutos, segundos] = tiempo.split(':').map(Number);
                totalTardanzaSegundos += (horas * 3600 + minutos * 60 + segundos);
                mesesConTardanza++;
            }}
        }});
        
        const promedioTardanzaSegundos = mesesConTardanza > 0 ? Math.round(totalTardanzaSegundos / mesesConTardanza) : 0;
        
        // Formatear promedio de tardanza
        const horasProm = Math.floor(promedioTardanzaSegundos / 3600);
        const minutosProm = Math.floor((promedioTardanzaSegundos % 3600) / 60);
        const segundosProm = promedioTardanzaSegundos % 60;
        const promedioTardanza = `${{horasProm.toString().padStart(2, '0')}}:${{minutosProm.toString().padStart(2, '0')}}:${{segundosProm.toString().padStart(2, '0')}}`;
        
        // CALCULAR VARIACIONES Y ETIQUETAS
        const variaciones = [];
        for (let i = 1; i < sumaPorcentajes.length; i++) {{
            const variacion = sumaPorcentajes[i] - sumaPorcentajes[i-1];
            const porcentajeVariacion = (Math.abs(variacion) / sumaPorcentajes[i-1]) * 100;
            variaciones.push({{
                valor: variacion,
                porcentaje: porcentajeVariacion,
                esPositiva: variacion > 0,
                esFuerte: porcentajeVariacion > 20
            }});
        }}
        
        // Contar variaciones
        const variacionesPositivas = variaciones.filter(v => v.esPositiva).length;
        const variacionesNegativas = variaciones.filter(v => !v.esPositiva).length;
        const variacionesFuertes = variaciones.filter(v => v.esFuerte).length;
        
        // DETERMINAR ETIQUETA
        let etiqueta = '';
        let colorEtiqueta = '';
        let iconoEtiqueta = '';
        
        if (variacionesFuertes <= 2) {{
            etiqueta = 'Estable';
            colorEtiqueta = '#2ecc71';
            iconoEtiqueta = '⚖️';
        }} else {{
            etiqueta = 'Inestable';
            colorEtiqueta = '#e74c3c';
            iconoEtiqueta = '🌊';
        }}
        
        // Ajustar etiqueta según tendencia
        if (variacionesPositivas > variacionesNegativas) {{
            if (etiqueta === 'Estable') {{
                etiqueta = 'Estable en Mejora';
                iconoEtiqueta = '📈⚖️';
            }} else {{
                etiqueta = 'Inestable en Mejora';
                iconoEtiqueta = '📈🌊';
            }}
        }} else if (variacionesNegativas > variacionesPositivas) {{
            if (etiqueta === 'Estable') {{
                etiqueta = 'Estable Sin Mejora';
                iconoEtiqueta = '📉⚖️';
            }} else {{
                etiqueta = 'Inestable Sin Mejora';
                iconoEtiqueta = '📉🌊';
            }}
        }}
        
        // ✅ MODIFICADO: Solo 3 estadísticas ahora (eliminado SUPERVISOR y CARTERA)
        const estadisticas = [
            {{
                nombre: '% Asistencia',
                datos: datosOrdenados.map(d => d.porcentajeAsistencia),
                color: '#2ecc71',
                tipo: 'porcentaje'
            }},
            {{
                nombre: '% Puntualidad',
                datos: datosOrdenados.map(d => d.porcentajePuntualidad),
                color: '#f39c12',
                tipo: 'porcentaje'
            }},
            {{
                nombre: 'Tardanza',
                datos: datosOrdenados.map(d => d.tardanzaAcumulada),
                color: '#e74c3c',
                tipo: 'tiempo'
            }}
        ];
        
        let html = `
            <div class="historico-container">
                <div style="background: #f8f9fa; padding: 15px 20px; border-bottom: 1px solid #ddd;">
                    <h3 style="margin: 0; font-size: 1.1rem; color: #2c3e50;">
                        📊 Evolución de Asistencias - ${{nombrePersona}}
                    </h3>
                    <div style="font-size: 0.85rem; color: #7f8c8d; margin-top: 5px;">
                        Período: ${{datosOrdenados.length}} meses (de ${{mesesLabels[0]}} a ${{mesesLabels[mesesLabels.length-1]}})
                    </div>
                </div>
                
                <!-- Gráfica de Pulso Cardiaco -->
                <div style="padding: 20px; border-bottom: 1px solid #eee;">
                    <div style="margin-bottom: 15px; display: flex; justify-content: space-between; align-items: center;">
                        <div style="font-weight: 600; color: #2c3e50;">
                            📈 Pulso de Asistencia (%Asistencia + %Puntualidad)
                        </div>
                        <div style="font-size: 0.85rem; color: #7f8c8d;">
                            Meta: 200%
                        </div>
                    </div>
                    <div style="height: 120px; position: relative;">
                        <canvas id="graficaPulso-${{nombrePersona.replace(/[^a-zA-Z0-9]/g, '-')}}" 
                                style="width: 100%; height: 100%;"></canvas>
                    </div>
                </div>
                
                <div class="tabla-historico-contenedor">
                    <table class="tabla-historico">
                        <thead>
                            <tr>
                                <th style="background: #f8f9fa; color: #2c3e50; border-bottom: 2px solid #ddd; min-width: 120px;">Métrica</th>
        `;
        
        // Encabezados de meses
        mesesLabels.forEach((mes, index) => {{
            const esUltimoMes = index === mesesLabels.length - 1;
            html += `
                    <th style="background: ${{esUltimoMes ? '#e8f4fd' : '#f8f9fa'}}; 
                               color: ${{esUltimoMes ? '#3498db' : '#2c3e50'}}; 
                               font-weight: ${{esUltimoMes ? 'bold' : 'normal'}};
                               border-bottom: 2px solid #ddd; 
                               font-size: 0.8rem; 
                               min-width: 80px;">
                        ${{mes}}
                    </th>`;
        }});
        
        html += `
                            </tr>
                        </thead>
                        <tbody>
        `;
        
        // Filas de estadísticas (ahora solo 3)
        estadisticas.forEach((estadistica, index) => {{
            html += `
                <tr ${{index % 2 === 0 ? 'style="background-color: #fafbfc;"' : ''}}>
                    <td style="font-weight: 600; color: ${{estadistica.color}}; text-align: left; padding-left: 15px; position: sticky; left: 0; background: inherit; border-right: 1px solid #eee;">
                        ${{estadistica.nombre}}
                    </td>
            `;
            
            // Datos por mes
            estadistica.datos.forEach((dato, i) => {{
                const esUltimoMes = i === estadistica.datos.length - 1;
                let celdaHTML = '';
                let estiloCelda = esUltimoMes ? 'background-color: #e8f4fd; font-weight: 500;' : '';
                
                switch(estadistica.tipo) {{
                    case 'porcentaje':
                        // MANEJAR VALORES NULL - mostrar "-" en lugar de 0%
                        if (dato === null) {{
                            celdaHTML = `<span style="font-weight: ${{esUltimoMes ? 'bold' : 'normal'}}; color: #95a5a6">-</span>`;
                        }} else {{
                            // Color condicional según valor
                            let colorPorcentaje = '#95a5a6';
                            if (dato >= 90) colorPorcentaje = '#2ecc71';
                            else if (dato >= 70) colorPorcentaje = '#f39c12';
                            else if (dato >= 40) colorPorcentaje = '#e67e22';
                            else if (dato > 0) colorPorcentaje = '#e74c3c';
                            
                            celdaHTML = `<span style="font-weight: ${{esUltimoMes ? 'bold' : 'normal'}}; color: ${{colorPorcentaje}}">${{dato}}%</span>`;
                        }}
                        break;
                        
                    case 'tiempo':
                        // Color para tardanza
                        let colorTardanza = '#95a5a6';
                        if (dato !== '00:00:00') {{
                            // Extraer horas de la cadena tiempo
                            const horas = parseInt(dato.split(':')[0]);
                            if (horas > 5) colorTardanza = '#e74c3c';
                            else if (horas > 2) colorTardanza = '#f39c12';
                            else if (horas > 0) colorTardanza = '#f1c40f';
                        }}
                        
                        celdaHTML = `<span style="font-weight: ${{esUltimoMes ? 'bold' : 'normal'}}; color: ${{colorTardanza}}">${{dato}}</span>`;
                        break;
                }}
                
                html += `<td style="text-align: center; ${{estiloCelda}} border-right: 1px solid #f0f0f0;">${{celdaHTML}}</td>`;
            }});
            
            html += `</tr>`;
        }});
        
        html += `
                            </tbody>
                        </table>
                    </div>  
                    <!-- PROMEDIOS HISTÓRICOS Y EVALUACIÓN -->
                    <div style="background: #f8f9fa; padding: 15px 20px; border-top: 1px solid #ddd;">
                        <div style="display: grid; grid-template-columns: 1fr 1fr; gap: 20px;">
                            <!-- COLUMNA 1: Promedios Históricos EN UNA SOLA FILA -->
                            <div>
                                <div style="font-weight: 600; color: #2c3e50; margin-bottom: 10px; display: flex; align-items: center; gap: 8px;">
                                    <span>📊</span> Promedios Históricos
                                </div>
                                
                                <!-- UNA SOLA FILA CON 3 MÉTRICAS -->
                                <div style="display: grid; grid-template-columns: repeat(3, 1fr); gap: 10px; margin-bottom: 15px;">
                                    <!-- Promedio Asistencia -->
                                    <div style="background: white; padding: 15px; border-radius: 8px; border-left: 4px solid #2ecc71; box-shadow: 0 2px 4px rgba(0,0,0,0.05);">
                                        <div style="font-size: 0.9rem; color: #2ecc71; margin-bottom: 8px; font-weight: 600; display: flex; align-items: center; gap: 5px;">
                                            <span>✓</span> Asistencia
                                        </div>
                                        <div style="font-size: 1.8rem; font-weight: bold; color: #2ecc71; margin-bottom: 5px;">
                                            ${{promedioAsistencia}}%
                                        </div>
                                        <div style="font-size: 0.75rem; color: #7f8c8d;">
                                            ${{datosAsistenciaValidos.length}} meses
                                        </div>
                                    </div>
                                    
                                    <!-- Promedio Puntualidad -->
                                    <div style="background: white; padding: 15px; border-radius: 8px; border-left: 4px solid #f39c12; box-shadow: 0 2px 4px rgba(0,0,0,0.05);">
                                        <div style="font-size: 0.9rem; color: #f39c12; margin-bottom: 8px; font-weight: 600; display: flex; align-items: center; gap: 5px;">
                                            <span>⏰</span> Puntualidad
                                        </div>
                                        <div style="font-size: 1.8rem; font-weight: bold; color: #f39c12; margin-bottom: 5px;">
                                            ${{promedioPuntualidad}}%
                                        </div>
                                        <div style="font-size: 0.75rem; color: #7f8c8d;">
                                            ${{datosPuntualidadValidos.length}} meses
                                        </div>
                                    </div>
                                    
                                    <!-- Promedio Tardanza -->
                                    <div style="background: white; padding: 15px; border-radius: 8px; border-left: 4px solid #e74c3c; box-shadow: 0 2px 4px rgba(0,0,0,0.05);">
                                        <div style="font-size: 0.9rem; color: #e74c3c; margin-bottom: 8px; font-weight: 600; display: flex; align-items: center; gap: 5px;">
                                            <span>⏱️</span> Tardanza
                                        </div>
                                        <div style="font-size: 1.4rem; font-weight: bold; color: #e74c3c; margin-bottom: 5px;">
                                            ${{promedioTardanza}}
                                        </div>
                                        <div style="font-size: 0.75rem; color: #7f8c8d;">
                                            ${{mesesConTardanza}} meses
                                        </div>
                                    </div>
                                </div>
                            </div>
                            
                            <!-- COLUMNA 2: Evaluación Final -->
                            <div>
                                <div style="font-weight: 600; color: #2c3e50; margin-bottom: 10px; display: flex; align-items: center; gap: 8px;">
                                    <span>🎯</span> Evaluación Final
                                </div>
                                
                                <!-- Etiqueta Principal -->
                                <div style="background: white; padding: 15px; border-radius: 8px; margin-bottom: 15px; border: 2px solid ${{colorEtiqueta}}; box-shadow: 0 2px 4px rgba(0,0,0,0.05);">
                                    <div style="display: flex; align-items: center; justify-content: space-between;">
                                        <div>
                                            <div style="font-size: 0.9rem; color: #7f8c8d; margin-bottom: 5px;">Etiqueta de Evaluación</div>
                                            <div style="font-size: 1.4rem; font-weight: bold; color: ${{colorEtiqueta}};">
                                                ${{iconoEtiqueta}} ${{etiqueta}}
                                            </div>
                                            <div style="font-size: 0.8rem; color: #95a5a6; margin-top: 5px;">
                                                Basado en ${{datosOrdenados.length}} meses
                                            </div>
                                        </div>
                                        <div style="font-size: 3rem; color: ${{colorEtiqueta}}; opacity: 0.7;">
                                            ${{iconoEtiqueta.charAt(0)}}
                                        </div>
                                    </div>
                                </div>
                            </div>
                        </div>
                    </div>
                </div>
                
                <style>
                    .historico-container {{
                        border: 1px solid #ddd;
                        border-radius: 6px;
                        overflow: hidden;
                        margin: 5px;
                        background: white;
                    }}
                    
                    .tabla-historico-contenedor {{
                        overflow-x: auto;
                        max-height: 300px;
                        border-bottom: 1px solid #eee;
                    }}
                    
                    .tabla-historico {{
                        width: 100%;
                        border-collapse: collapse;
                        font-size: 0.85rem;
                        min-width: 600px;
                    }}
                    
                    .tabla-historico th {{
                        padding: 8px 6px;
                        text-align: center;
                        font-weight: 500;
                        position: sticky;
                        top: 0;
                        z-index: 20;
                        white-space: nowrap;
                    }}
                    
                    .tabla-historico th:first-child {{
                        position: sticky;
                        left: 0;
                        z-index: 30;
                        min-width: 120px;
                        background: #f8f9fa;
                    }}
                    
                    .tabla-historico td {{
                        padding: 6px 4px;
                        text-align: center;
                        border-bottom: 1px solid #eee;
                        vertical-align: middle;
                        min-width: 80px;
                    }}
                    
                    .tabla-historico td:first-child {{
                        position: sticky;
                        left: 0;
                        background: inherit;
                        z-index: 10;
                        font-weight: 600;
                        text-align: left;
                        min-width: 120px;
                    }}
                    
                    .tabla-historico tr:hover td {{
                        background-color: #f5f7fa !important;
                    }}
                    
                    .tabla-historico tr:hover td:first-child {{
                        background-color: #f5f7fa !important;
                    }}
                </style>
            `;
            
            // Generar la gráfica después de que el HTML se haya insertado
            setTimeout(() => {{
                generarGraficaPulsoCardiaco(nombrePersona, mesesLabels, sumaPorcentajes);
            }}, 100);
            
            return html;
        }}
        
    function generarGraficaPulsoCardiaco(nombrePersona, mesesLabels, sumaPorcentajes) {{
        const canvasId = `graficaPulso-${{nombrePersona.replace(/[^a-zA-Z0-9]/g, '-')}}`;
        const canvas = document.getElementById(canvasId);
        if (!canvas) return;
    
        const ctx = canvas.getContext('2d');
    
        // Limpiar gráfica previa
        if (canvas.chartInstance) {{
            try {{ canvas.chartInstance.destroy(); }} catch (e) {{}}
            canvas.chartInstance = null;
        }}
    
        // Dimensiones
        canvas.width = canvas.offsetWidth;
        canvas.height = 120;
    
        // Padding interno
        const paddingX = 80;
        const paddingY = 15;
    
        const usableWidth = canvas.width - paddingX * 2;
        const usableHeight = canvas.height - paddingY * 2;
    
        // Calcular puntos (escala máxima 300%)
        const puntos = sumaPorcentajes.map((valor, index) => ({{
            x: paddingX + (index / (sumaPorcentajes.length - 1)) * usableWidth,
            y: paddingY + (usableHeight - ((valor / 300) * usableHeight))
        }}));
    
        // Fondo de área
        ctx.beginPath();
        ctx.moveTo(paddingX, canvas.height - paddingY);
        puntos.forEach(p => ctx.lineTo(p.x, p.y));
        ctx.lineTo(canvas.width - paddingX, canvas.height - paddingY);
        ctx.closePath();
    
        const areaGradient = ctx.createLinearGradient(0, paddingY, 0, canvas.height - paddingY);
        areaGradient.addColorStop(0, 'rgba(52, 152, 219, 0.2)');
        areaGradient.addColorStop(1, 'rgba(52, 152, 219, 0.05)');
        ctx.fillStyle = areaGradient;
        ctx.fill();
    
        // Línea de pulso (curvas suaves)
        ctx.beginPath();
        ctx.moveTo(puntos[0].x, puntos[0].y);
    
        for (let i = 1; i < puntos.length; i++) {{
            const prev = puntos[i - 1];
            const curr = puntos[i];
    
            const cp1x = prev.x + (curr.x - prev.x) * 0.3;
            const cp1y = prev.y;
            const cp2x = prev.x + (curr.x - prev.x) * 0.7;
            const cp2y = curr.y;
    
            ctx.bezierCurveTo(cp1x, cp1y, cp2x, cp2y, curr.x, curr.y);
        }}
    
        ctx.lineWidth = 3;
        ctx.strokeStyle = '#3498db';
        ctx.lineCap = 'round';
        ctx.lineJoin = 'round';
        ctx.stroke();
    
        // Puntos y etiquetas
        ctx.fillStyle = '#2c3e50';
        ctx.font = 'bold 12px Arial';
        ctx.textBaseline = 'bottom';
    
        puntos.forEach((punto, index) => {{
            // Punto
            ctx.beginPath();
            ctx.arc(punto.x, punto.y, 5, 0, Math.PI * 2);
            ctx.fillStyle = '#3498db';
            ctx.fill();
            ctx.strokeStyle = 'white';
            ctx.lineWidth = 2;
            ctx.stroke();
    
            // Alineación inteligente para extremos
            ctx.textAlign =
                index === 0 ? 'left' :
                index === puntos.length - 1 ? 'right' :
                'center';
    
            ctx.fillStyle = '#2c3e50';
            ctx.fillText(`${{sumaPorcentajes[index]}}%`, punto.x, punto.y - 10);
        }});

        // Etiquetas eje X (meses)
        ctx.font = '11px Arial';
        ctx.fillStyle = '#7f8c8d';
        ctx.textAlign = 'center';
        ctx.textBaseline = 'top';
        
        const yLabelsX = canvas.height - paddingY + 2;
        
        mesesLabels.forEach((mes, index) => {{
            const x =
                paddingX +
                (index / (mesesLabels.length - 1)) * usableWidth;
        
            ctx.fillText(mes, x, yLabelsX);
        }});
    
        // Línea de meta 200% (sin texto)
        const yMeta = paddingY + (usableHeight - ((200 / 300) * usableHeight));
    
        ctx.beginPath();
        ctx.moveTo(paddingX, yMeta);
        ctx.lineTo(canvas.width - paddingX, yMeta);
        ctx.strokeStyle = 'rgba(231, 76, 60, 0.5)';
        ctx.lineWidth = 2;
        ctx.setLineDash([5, 3]);
        ctx.stroke();
        ctx.setLineDash([]);
    }}
    
    // ========== FUNCIÓN ESPECÍFICA PARA ASISTENCIAS ==========
    function ordenarTablaAsistenciasPorColumna(colIndex, colName) {{
        console.log(`📊 Ordenando tabla por columna: ${{colIndex}} - ${{colName}}`);
        
        const tbody = document.getElementById('tbodyDetalleAsistencias');
        if (!tbody) return;
        
        // 1. Obtener todas las filas (principales e históricas)
        const allRows = Array.from(tbody.children);
        
        // 2. Separar en pares [filaPrincipal, filaHistorico]
        const filasPrincipales = [];
        const filasHistoricos = new Map(); // Mapa para encontrar histórico por persona
        
        allRows.forEach(fila => {{
            if (fila.classList.contains('fila-historico')) {{
                // Es una fila histórico - extraer ID de persona del id de la fila
                const idFila = fila.id;
                const nombrePersona = idFila.replace('historico-', '').replace(/-/g, ' ');
                filasHistoricos.set(nombrePersona, fila);
            }} else {{
                // Es una fila principal
                filasPrincipales.push(fila);
            }}
        }});
        
        // 3. Verificar si ya está ordenando por esta columna (alternar ascendente/descendente)
        const thElement = document.querySelector(`#tablaDetalleAsistencias th:nth-child(${{colIndex + 1}})`);
        if (!thElement) return;
        
        let direccion = 'asc';
        if (thElement.hasAttribute('data-sort')) {{
            direccion = thElement.getAttribute('data-sort') === 'asc' ? 'desc' : 'asc';
            thElement.removeAttribute('data-sort');
        }}
        thElement.setAttribute('data-sort', direccion);
        
        // 4. Ordenar las filas principales según la columna
        filasPrincipales.sort((a, b) => {{
            // Obtener valores de las celdas
            const valorA = a.cells[colIndex]?.textContent?.trim() || '';
            const valorB = b.cells[colIndex]?.textContent?.trim() || '';
            
            // Convertir a números si es posible
            const numA = parseFloat(valorA.replace(/[^0-9.-]+/g, ''));
            const numB = parseFloat(valorB.replace(/[^0-9.-]+/g, ''));
            
            let resultado = 0;
            
            if (!isNaN(numA) && !isNaN(numB)) {{
                // Ambos son números
                resultado = numA - numB;
            }} else {{
                // Comparación de texto
                resultado = valorA.localeCompare(valorB, 'es', {{ 
                    sensitivity: 'base',
                    numeric: true 
                }});
            }}
            
            // Aplicar dirección
            return direccion === 'asc' ? resultado : -resultado;
        }});
        
        // 5. Actualizar el DOM manteniendo filas históricas
        // Limpiar el tbody
        tbody.innerHTML = '';
        
        // Insertar filas ordenadas con sus históricos
        filasPrincipales.forEach(filaPrincipal => {{
            // Insertar fila principal
            tbody.appendChild(filaPrincipal);
            
            // Buscar y insertar su fila histórico si existe
            const personaCell = filaPrincipal.cells[0]?.textContent?.trim();
            if (personaCell && filasHistoricos.has(personaCell)) {{
                const filaHistorico = filasHistoricos.get(personaCell);
                tbody.appendChild(filaHistorico);
            }}
        }});
        
        // 6. Actualizar iconos de ordenamiento
        document.querySelectorAll('#tablaDetalleAsistencias th').forEach((th, idx) => {{
            const icono = th.querySelector('.sort-icon');
            if (icono) {{
                if (idx === colIndex) {{
                    icono.textContent = direccion === 'asc' ? '↑' : '↓';
                    icono.style.opacity = '1';
                    th.style.backgroundColor = '#e8f4fd';
                }} else {{
                    icono.textContent = '↕️';
                    icono.style.opacity = '0.5';
                    th.style.backgroundColor = '';
                    th.removeAttribute('data-sort');
                }}
            }}
        }});
        
        console.log(`✅ Tabla ordenada por ${{colName}} (${{direccion}})`);
    }}

    function reinicializarOrdenamientoTablaAsistencias() {{
        console.log('🔄 Reinicializando ordenamiento de tabla asistencias');
        
        const thead = document.querySelector('#tablaDetalleAsistencias thead');
        if (!thead) return;
        
        const headers = thead.querySelectorAll('th');
        
        headers.forEach((header, index) => {{
            // Solo procesar hasta columna 11 (Histórico no es ordenable)
            if (index < 11) {{
                // Remover todos los listeners previos clonando el header
                const nuevoHeader = header.cloneNode(true);
                header.parentNode.replaceChild(nuevoHeader, header);
                
                // Asignar nueva funcionalidad
                nuevoHeader.style.cursor = 'pointer';
                
                nuevoHeader.addEventListener('click', (e) => {{
                    e.stopPropagation();
                    const colName = nuevoHeader.querySelector('span:first-child')?.textContent || '';
                    ordenarTablaAsistenciasPorColumna(index, colName);
                }});
                
                // Efectos hover
                nuevoHeader.addEventListener('mouseenter', () => {{
                    if (!nuevoHeader.hasAttribute('data-sort')) {{
                        nuevoHeader.style.backgroundColor = '#2c3e50';
                    }}
                }});
                
                nuevoHeader.addEventListener('mouseleave', () => {{
                    if (!nuevoHeader.hasAttribute('data-sort')) {{
                        nuevoHeader.style.backgroundColor = '';
                    }}
                }});
            }}
        }});
    }}
    
    // MODIFICAR LA FUNCIÓN mostrarOcultarHistorico para mantener el ordenamiento
    function mostrarOcultarHistorico(nombrePersona) {{
        const idSeguro = nombrePersona.replace(/[^a-zA-Z0-9]/g, '-');
        const filaHistorico = document.getElementById(`historico-${{idSeguro}}`);
        const contenidoHistorico = document.getElementById(`contenido-historico-${{idSeguro}}`);
        const boton = event.target;
        
        if (!filaHistorico || !contenidoHistorico) return;
        
        // Alternar visibilidad
        if (filaHistorico.style.display === 'none') {{
            filaHistorico.style.display = 'table-row';
            boton.innerHTML = '✕ Cerrar';
            boton.style.background = 'linear-gradient(135deg, #e74c3c, #c0392b)';
            
            // Si es la primera vez que se abre, cargar datos
            if (contenidoHistorico.innerHTML.includes('Cargando histórico')) {{
                cargarHistoricoPersona(nombrePersona, idSeguro);
            }}
        }} else {{
            filaHistorico.style.display = 'none';
            boton.innerHTML = 'Histórico';
            boton.style.background = 'linear-gradient(135deg, #3498db, #2980b9)';
            
            // 🔴 REINICIALIZAR ORDENAMIENTO después de cerrar
            setTimeout(() => {{
                reinicializarOrdenamientoTablaAsistencias();
            }}, 50);
        }}
    }}
    
    // ========== EXPORTACIÓN EXCEL DE ASISTENCIAS ==========
    async function exportarAsistenciasExcelCompleto() {{
        console.log('📊 Exportando asistencias detalladas a Excel');
        
        // Obtener mes y año
        const mesSeleccionado = document.getElementById('selectorMes').value;
        const añoSeleccionado = document.getElementById('selectorAño').value;
        const periodoCompleto = `${{mesSeleccionado}}_${{añoSeleccionado}}`;
        
        // Verificar si hay datos de asistencias
        if (!datosAsistencias || !datosAsistencias[periodoCompleto]) {{
            alert('⚠️ No hay datos de asistencias para exportar en este periodo');
            return;
        }}
        
        // 1. DETERMINAR EL SUPERVISOR FILTRADO ACTUALMENTE
        let supervisorFiltro = 'TODOS';
        
        // Buscar el supervisor activo en la barra de filtros GLOBAL
        const barraFiltrosGlobal = document.getElementById('barraFiltrosSupervisoresGlobal');
        if (barraFiltrosGlobal) {{
            const botonActivoGlobal = barraFiltrosGlobal.querySelector('.filtro-supervisor.activo[data-supervisor]');
            if (botonActivoGlobal) {{
                supervisorFiltro = botonActivoGlobal.getAttribute('data-supervisor');
            }}
        }}
        
        console.log(`👥 Filtro activo: ${{supervisorFiltro}}`);
        
        // Crear nombre del archivo (incluir supervisor si está filtrado)
        const fechaHoy = new Date();
        const año = fechaHoy.getFullYear();
        const mes = String(fechaHoy.getMonth() + 1).padStart(2, '0');
        const dia = String(fechaHoy.getDate()).padStart(2, '0');
        const fechaFormato = `${{año}}${{mes}}${{dia}}`;
        
        let nombreArchivo = '';
        if (supervisorFiltro === 'TODOS') {{
            nombreArchivo = `Asistencias_detalladas_${{mesSeleccionado}}_${{fechaFormato}}.xlsx`;
        }} else {{
            // Limpiar el nombre del supervisor para el archivo
            const supervisorLimpio = supervisorFiltro.replace(/[^a-zA-Z0-9]/g, '_');
            nombreArchivo = `Asistencias_${{supervisorLimpio}}_${{mesSeleccionado}}_${{fechaFormato}}.xlsx`;
        }}
        
        try {{
            // Obtener datos del periodo
            const datosPeriodo = datosAsistencias[periodoCompleto];
            const personas = datosPeriodo.personas || {{}};
            
            // 2. FILTRAR PERSONAS SEGÚN EL SUPERVISOR
            let personasFiltradas = {{}};
            
            if (supervisorFiltro === 'TODOS') {{
                personasFiltradas = personas;
                console.log(`📊 Exportando TODAS las personas: ${{Object.keys(personasFiltradas).length}}`);
            }} else {{
                // Filtrar personas que pertenecen a este equipo/supervisor
                Object.keys(personas).forEach(persona => {{
                    const personaData = personas[persona];
                    if (personaData.supervisor === supervisorFiltro) {{
                        personasFiltradas[persona] = personaData;
                    }}
                }});
                console.log(`📊 Exportando filtrado por ${{supervisorFiltro}}: ${{Object.keys(personasFiltradas).length}} personas`);
            }}
            
            // Verificar si hay datos después del filtro
            if (Object.keys(personasFiltradas).length === 0) {{
                alert(`⚠️ No hay datos de asistencias para el equipo: ${{supervisorFiltro}}`);
                return;
            }}
            
            // 3. OBTENER TODAS LAS FECHAS ÚNICAS DE LAS PERSONAS FILTRADAS
            const todasFechas = new Set();
            
            // Recorrer solo las personas filtradas para obtener fechas únicas
            Object.values(personasFiltradas).forEach(persona => {{
                Object.keys(persona.asistencias_detalle || {{}}).forEach(fecha => {{
                    todasFechas.add(fecha);
                }});
            }});
            
            // Convertir a array y ordenar de menor a mayor
            const fechasOrdenadas = Array.from(todasFechas)
                .map(fecha => {{
                    // Asegurar formato de fecha
                    try {{
                        const [año, mes, dia] = fecha.split('-');
                        return new Date(año, mes - 1, dia);
                    }} catch {{
                        return new Date(fecha);
                    }}
                }})
                .sort((a, b) => a - b)  // Orden ascendente (más antigua primero)
                .map(fecha => {{
                    // Convertir de nuevo a formato DD/MM/YYYY
                    const año = fecha.getFullYear();
                    const mes = String(fecha.getMonth() + 1).padStart(2, '0');
                    const dia = String(fecha.getDate()).padStart(2, '0');
                    return `${{dia}}/${{mes}}/${{año}}`;
                }});
            
            console.log(`📅 Fechas únicas encontradas: ${{fechasOrdenadas.length}}`);
            
            // 4. PREPARAR ENCABEZADOS (incluyendo nueva columna al final)
            const encabezados = ['PERSONA', ...fechasOrdenadas, 'TOTAL TARDANZA'];
            
            // 5. PREPARAR DATOS POR PERSONA FILTRADA
            const datosExportar = [];
            
            // Ordenar personas alfabéticamente
            const personasOrdenadas = Object.entries(personasFiltradas)
                .map(([nombre, datos]) => ({{ nombre, ...datos }}))
                .sort((a, b) => a.nombre.localeCompare(b.nombre));
            
            personasOrdenadas.forEach(persona => {{
                const filaDatos = [persona.nombre];
                let totalTardanzaSegundos = persona.tardanza_total_segundos || 0;
                
                // Para cada fecha, buscar la hora de inicio si existe
                fechasOrdenadas.forEach(fechaExcel => {{
                    // Buscar en formato YYYY-MM-DD
                    const fechaKey = convertirFechaExcelAFechaKey(fechaExcel);
                    const registro = persona.asistencias_detalle?.[fechaKey];
                    
                    if (registro && registro.hora_inicio) {{
                        // Formatear hora a HH:MM:SS
                        const horaFormateada = formatearHora(registro.hora_inicio);
                        filaDatos.push(horaFormateada);
                    }} else {{
                        // Celda vacía si no hay registro
                        filaDatos.push('');
                    }}
                }});
                
                // AGREGAR COLUMNA DE TOTAL TARDANZA AL FINAL
                const totalTardanzaFormateada = segundosAHorasMinutosSegundos(totalTardanzaSegundos);
                filaDatos.push(totalTardanzaFormateada);
                
                datosExportar.push(filaDatos);
            }});
            
            console.log(`👥 Datos preparados: ${{datosExportar.length}} personas`);
            
            // 6. CREAR WORKBOOK Y WORKSHEET
            const workbook = new ExcelJS.Workbook();
            const worksheet = workbook.addWorksheet('Detalle Asistencias');
            
            
            // 8. AGREGAR ENCABEZADOS CON FORMATO
            const headerRow = worksheet.addRow(encabezados);
            
            // Formato para encabezados
            headerRow.font = {{
                name: 'Aptos Narrow',
                size: 12,
                bold: true,
                color: {{ argb: 'FFFFFFFF' }}
            }};
            
            headerRow.fill = {{
                type: 'pattern',
                pattern: 'solid',
                fgColor: {{ argb: 'FF2C3E50' }}
            }};
            
            headerRow.alignment = {{ 
                horizontal: 'center', 
                vertical: 'middle',
                wrapText: true
            }};
            
            headerRow.border = {{
                top: {{ style: 'thin', color: {{ argb: 'FFFFFFFF' }} }},
                bottom: {{ style: 'thin', color: {{ argb: 'FFFFFFFF' }} }},
                left: {{ style: 'thin', color: {{ argb: 'FFFFFFFF' }} }},
                right: {{ style: 'thin', color: {{ argb: 'FFFFFFFF' }} }}
            }};
            
            headerRow.height = 30;
            
            // 9. AGREGAR DATOS CON FORMATOS SIMPLIFICADOS
            datosExportar.forEach((fila, rowIndex) => {{
                const dataRow = worksheet.addRow(fila);
                
                // Formato para cada celda
                dataRow.eachCell((cell, colNumber) => {{
                    // Primera columna (PERSONA) - SOLO NEGRITA
                    if (colNumber === 1) {{
                        cell.font = {{ 
                            name: 'Aptos Narrow', 
                            size: 10, 
                            bold: true  // 🔴 SOLO NEGRITA
                        }};
                        // 🔴 QUITAR FONDO COLORIDO
                        cell.fill = {{
                            type: 'pattern',
                            pattern: 'solid',
                            fgColor: {{ argb: 'FFFFFFFF' }}  // FONDO BLANCO
                        }};
                        cell.alignment = {{ 
                            horizontal: 'left',
                            vertical: 'middle'
                        }};
                        // Mantener borde izquierdo especial
                        cell.border = {{
                            left: {{ style: 'medium', color: {{ argb: 'FF3498DB' }} }},
                            top: {{ style: 'thin', color: {{ argb: 'FFD0D0D0' }} }},
                            bottom: {{ style: 'thin', color: {{ argb: 'FFD0D0D0' }} }},
                            right: {{ style: 'thin', color: {{ argb: 'FFD0D0D0' }} }}
                        }};
                    }} 
                    // Última columna (TARDANZA TOTAL) - SOLO NEGRITA
                    else if (colNumber === encabezados.length) {{
                        cell.font = {{ 
                            name: 'Aptos Narrow', 
                            size: 10,
                            bold: true  // 🔴 SOLO NEGRITA
                        }};
                        cell.alignment = {{ 
                            horizontal: 'center',
                            vertical: 'middle'
                        }};
                        cell.numFmt = 'hh:mm:ss';
                        
                        // 🔴 QUITAR FORMATO CONDICIONAL DE COLORES
                        cell.fill = {{
                            type: 'pattern',
                            pattern: 'solid',
                            fgColor: {{ argb: 'FFFFFFFF' }}  // FONDO BLANCO
                        }};
                        cell.font.color = {{ argb: 'FF000000' }}; // TEXTO NEGRO
                    }}
                    // Columnas de fechas (horas) - MANTENER FORMATOS ORIGINALES
                    else if (colNumber > 1 && colNumber < encabezados.length) {{
                        cell.font = {{ 
                            name: 'Aptos Narrow', 
                            size: 10 
                        }};
                        cell.alignment = {{ 
                            horizontal: 'center',
                            vertical: 'middle'
                        }};
                        cell.numFmt = 'hh:mm:ss';
                        
                        // Bordes estándar
                        cell.border = {{
                            top: {{ style: 'thin', color: {{ argb: 'FFD0D0D0' }} }},
                            bottom: {{ style: 'thin', color: {{ argb: 'FFD0D0D0' }} }},
                            left: {{ style: 'thin', color: {{ argb: 'FFD0D0D0' }} }},
                            right: {{ style: 'thin', color: {{ argb: 'FFD0D0D0' }} }}
                        }};
                        
                        // 🔴 MANTENER FORMATO CONDICIONAL SOLO PARA FECHAS
                        const valorHora = cell.value;
                        if (valorHora && typeof valorHora === 'string') {{
                            const horaSegundos = convertirHoraASegundos(valorHora);
                            const horaReferenciaSegundos = 7 * 3600; // 07:00:00
                            
                            const [horaStr, minStr, segStr] = valorHora.split(':');
                            const horaNum = parseInt(horaStr || 0);
                            const minNum = parseInt(minStr || 0);
                            const segNum = parseInt(segStr || 0);
                            
                            if (horaNum === 7 && minNum === 0 && segNum <= 59) {{
                                // Rango 07:00:00 - 07:00:59 → NARANJA TENUE
                                cell.fill = {{
                                    type: 'pattern',
                                    pattern: 'solid',
                                    fgColor: {{ argb: 'FFFFF5EB' }}
                                }};
                            }} else if (horaSegundos > horaReferenciaSegundos + 59) {{
                                // Después de 07:00:59 → ROJO TENUE
                                cell.fill = {{
                                    type: 'pattern',
                                    pattern: 'solid',
                                    fgColor: {{ argb: 'FFFFC7CE' }}
                                }};
                            }} else {{
                                // Dentro del horario normal → SIN COLOR
                                cell.fill = {{
                                    type: 'pattern',
                                    pattern: 'solid',
                                    fgColor: {{ argb: 'FFFFFFFF' }}
                                }};
                            }}
                        }} else {{
                            // Celda vacía → GRIS MUY CLARO
                            cell.fill = {{
                                type: 'pattern',
                                pattern: 'solid',
                                fgColor: {{ argb: 'FFF5F5F5' }}
                            }};
                        }}
                    }}
                    
                    // 🔴 AGREGAR ESTE ELSE PARA CELDAS INTERMEDIAS (por si acaso)
                    else {{
                        cell.font = {{ 
                            name: 'Aptos Narrow', 
                            size: 10 
                        }};
                        cell.fill = {{
                            type: 'pattern',
                            pattern: 'solid',
                            fgColor: {{ argb: 'FFFFFFFF' }}
                        }};
                        cell.border = {{
                            top: {{ style: 'thin', color: {{ argb: 'FFD0D0D0' }} }},
                            bottom: {{ style: 'thin', color: {{ argb: 'FFD0D0D0' }} }},
                            left: {{ style: 'thin', color: {{ argb: 'FFD0D0D0' }} }},
                            right: {{ style: 'thin', color: {{ argb: 'FFD0D0D0' }} }}
                        }};
                    }}
                }});
                
                // Altura de fila
                dataRow.height = 14;
            }});
            
            // 10. AGREGAR FILA DE RESUMEN
            worksheet.addRow([]);  // Fila vacía de separación
            
            // Calcular total general de tardanzas para las personas filtradas
            let totalGeneralSegundos = 0;
            personasOrdenadas.forEach(persona => {{
                totalGeneralSegundos += persona.tardanza_total_segundos || 0;
            }});
            
            const totalesFila = ['TARDANZA ' + (supervisorFiltro !== 'TODOS' ? 'DEL EQUIPO' : 'DEL MES'), 
                               ...Array(fechasOrdenadas.length).fill(''), 
                               segundosAHorasMinutosSegundos(totalGeneralSegundos)];
            
            const totalRow = worksheet.addRow(totalesFila);
            
            // Formato especial para fila de resumen
            totalRow.eachCell((cell, colNumber) => {{
                if (colNumber === 1) {{
                    cell.font = {{ 
                        name: 'Aptos Narrow', 
                        size: 11,
                        bold: true, 
                        color: {{ argb: 'FF2C3E50' }}
                    }};
                    cell.fill = {{
                        type: 'pattern',
                        pattern: 'solid',
                        fgColor: {{ argb: 'FFD6DBDF' }}
                    }};
                    cell.alignment = {{ horizontal: 'center' }};
                }} else if (colNumber === encabezados.length) {{
                    cell.font = {{ 
                        name: 'Aptos Narrow', 
                        size: 11,
                        bold: true, 
                        color: {{ argb: 'FFE74C3C' }}
                    }};
                    cell.fill = {{
                        type: 'pattern',
                        pattern: 'solid',
                        fgColor: {{ argb: 'FFFADBD8' }}
                    }};
                    cell.alignment = {{ horizontal: 'center' }};
                    cell.numFmt = 'hh:mm:ss';
                }}
                
                cell.border = {{
                    top: {{ style: 'thin', color: {{ argb: 'FF2C3E50' }} }},
                    bottom: {{ style: 'thin', color: {{ argb: 'FF2C3E50' }} }}
                }};
            }});
            
            // 11. AJUSTAR ANCHOS DE COLUMNA
            const anchosColumnas = [
                {{ width: 29 }},  // Columna PERSONA
                ...fechasOrdenadas.map(() => ({{ width: 12 }})),  // Columnas de fechas
                {{ width: 15 }}  // Columna TOTAL TARDANZA
            ];
            
            worksheet.columns = anchosColumnas;
            
            // 12. CONGELAR PANELES (PERSONA siempre visible)
            // Ajustar la fila inicial según si hay título
            const filaInicialCongelado = supervisorFiltro !== 'TODOS' ? 3 : 2;
            worksheet.views = [
                {{ 
                    state: 'frozen', 
                    xSplit: 1, 
                    ySplit: filaInicialCongelado, 
                    activeCell: `B${{filaInicialCongelado + 1}}` 
                }}
            ];
            
            // 13. GUARDAR ARCHIVO
            const buffer = await workbook.xlsx.writeBuffer();
            const blob = new Blob([buffer], {{ type: 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet' }});
            saveAs(blob, nombreArchivo);
            
            console.log('✅ Excel exportado exitosamente');
            
            // Mensaje personalizado según el filtro
            if (supervisorFiltro === 'TODOS') {{
                alert(`✅ Archivo Excel exportado (todos los equipos): ${{nombreArchivo}}`);
            }} else {{
                alert(`✅ Archivo Excel exportado (equipo ${{supervisorFiltro}}): ${{nombreArchivo}}`);
            }}
            
        }} catch (error) {{
            console.error('❌ Error al exportar Excel:', error);
            alert('❌ Error al exportar el archivo. Verifica la consola para detalles.');
        }}
    }}

    // ========== FUNCIONES PARA SECCION EVALUACION ========== 
    function setVistaEvaluacion(tipo) {{
      vistaEvaluacion = tipo;
    
      // Activo visual solo en los botones de evaluación
      document.querySelectorAll('#controlesEvaluacionMeses .btn-comparacion').forEach(b => {{
        b.classList.remove('activo');
      }});

      const btn3 = document.getElementById('btnEval3Meses');
      const btn6 = document.getElementById('btnEval6Meses');

      if (tipo === '3M') btn3?.classList.add('activo');
      if (tipo === '6M') btn6?.classList.add('activo');

    }}

    // ===================== HELPERS DE FECHAS / PERIODOS =====================
    
    // MES (nombre) -> número
    function _mesToNum(mesNombre) {{
      const m = String(mesNombre || '').toUpperCase();
      const map = {{
        'ENERO': 1, 'FEBRERO': 2, 'MARZO': 3, 'ABRIL': 4, 'MAYO': 5, 'JUNIO': 6,
        'JULIO': 7, 'AGOSTO': 8, 'SETIEMBRE': 9, 'OCTUBRE': 10, 'NOVIEMBRE': 11, 'DICIEMBRE': 12
      }};
      return map[m] || 0;
    }}
    
    // número -> MES (nombre)
    function _numToMes(n) {{
      const arr = ['', 'ENERO','FEBRERO','MARZO','ABRIL','MAYO','JUNIO',
                   'JULIO','AGOSTO','SETIEMBRE','OCTUBRE','NOVIEMBRE','DICIEMBRE'];
      return arr[n] || '';
    }}
    
    // Retroceder un mes
    function _retroMes(m, y) {{
      let mm = m - 1;
      let yy = y;
      if (mm <= 0) {{ mm = 12; yy = y - 1; }}
      return {{ m: mm, y: yy }};
    }}
    
    // "MES_AÑO" -> {{ y, m }}
    function _periodoToYM(periodo) {{
      const p = String(periodo || '').split('_');
      if (p.length !== 2) return {{ y: 0, m: 0 }};
      const m = _mesToNum(p[0]);
      const y = parseInt(p[1], 10);
      return {{ y: y || 0, m: m || 0 }};
    }}
    
    // y,m -> YYYYMM
    function _ymToNum(y, m) {{
      return (y * 100) + m;
    }}
    
    // ===================== FECHA DE INGRESO =====================
    
    // "dd/mm/yyyy" o "yyyy-mm-dd" -> YYYYMM
    function _fechaIngresoToYMNum(fechaIngresoStr) {{
      const s = String(fechaIngresoStr || '').trim();
      if (!s) return 0;
    
      // yyyy-mm-dd
      const m1 = s.match(/^(\d{{4}})-(\d{{2}})-(\d{{2}})/);
      if (m1) {{
        const y = parseInt(m1[1], 10);
        const m = parseInt(m1[2], 10);
        return _ymToNum(y, m);
      }}
    
      // dd/mm/yyyy
      const m2 = s.match(/^(\d{{2}})\/(\d{{2}})\/(\d{{4}})/);
      if (m2) {{
        const y = parseInt(m2[3], 10);
        const m = parseInt(m2[2], 10);
        return _ymToNum(y, m);
      }}
    
      return 0;
    }}
    
    // ===================== PERIODOS HACIA ATRÁS =====================
    
    // Devuelve N periodos hacia atrás
    // incluirActual = true  -> incluye mes seleccionado
    // incluirActual = false -> empieza desde el mes anterior
    function _getPeriodosAtras(mesNombre, anioStr, cantidad, incluirActual) {{
      const m0 = _mesToNum(mesNombre);
      const y0 = parseInt(anioStr, 10);
      if (!m0 || !y0) return [];
    
      let m = m0;
      let y = y0;
    
      if (!incluirActual) {{
        const r = _retroMes(m, y);
        m = r.m;
        y = r.y;
      }}
    
      const out = [];
      while (out.length < cantidad) {{
        // Omitir setiembre (regla del negocio)
        if (m !== 9) {{
          out.push(`${{_numToMes(m)}}_${{y}}`);
        }}
        const r2 = _retroMes(m, y);
        m = r2.m;
        y = r2.y;
      }}
    
      return out.reverse(); // antiguo -> reciente
    }}
    
    // ===================== MAPAS POR ALIAS =====================
    
    // Mapea datosMeses[periodo] por alias_crr
    function _mapPeriodoPorAlias(periodo) {{
      const arr = datosMeses?.[periodo] || [];
      const map = new Map();
      arr.forEach(a => {{
        const alias = (a.alias_crr || '').trim();
        if (alias) map.set(alias, a);
      }});
      return map;
    }}
    
    // Mapea baseAsesoresAnalisis[periodo] por alias_crr
    function _mapBasePeriodoPorAlias(periodo) {{
      const arr = window.baseAsesoresAnalisis?.[periodo] || [];
      const map = new Map();
      arr.forEach(p => {{
        const alias = (p.alias_crr || '').trim();
        if (alias) map.set(alias, p);
      }});
      return map;
    }}
    
    // ===================== CÁLCULOS =====================
    
    // Promedio seguro
    function _avg(nums) {{
      if (!nums || !nums.length) return null;
      const s = nums.reduce((acc, x) => acc + x, 0);
      return s / nums.length;
    }}
    
    // Clasificación por porcentaje
    function _clasificarPorcentaje(pct) {{
      const v = Number(pct || 0);
      if (v > 100) return '>100%';
      if (v > 70) return '>70%';
      if (v > 40) return '>40%';
      return '>0%';
    }}

    // ===================== EXPORTAR ANALISIS (EVALUACIÓN) =====================
    async function exportarAnalisisEvaluacion() {{
      const mesSeleccionado = document.getElementById('selectorMes')?.value;
      const añoSeleccionado = document.getElementById('selectorAño')?.value;
      const periodoFiltrado = `${{mesSeleccionado}}_${{añoSeleccionado}}`;
    
      if (!datosMeses?.[periodoFiltrado]) {{
        alert('No hay datos para exportar');
        return;
      }}
    
      // ✅ Vista viene de Evaluación, NO de Ranking
      // (solo debe estar en '3M' o '6M' porque el botón se habilita solo ahí)
      const n = (vistaEvaluacion === '3M') ? 3 : 6;
    
      // Columnas de meses: N anteriores + mes seleccionado = N+1 (3->4, 6->7)
      const periodosCols = _getPeriodosAtras(mesSeleccionado, añoSeleccionado, n + 1, true);
    
      // Promedio CERRADO: N meses anteriores (excluye mes seleccionado)
      const periodosCerrado = _getPeriodosAtras(mesSeleccionado, añoSeleccionado, n, false);
    
      // Promedio ACTUAL: N meses incluyendo mes seleccionado (N-1 anteriores + mes seleccionado)
      const periodosActual = _getPeriodosAtras(mesSeleccionado, añoSeleccionado, n, true);
    
      // Mapas por periodo para lookup rápido
      const mapsPorPeriodo = {{}};
      [...new Set([...periodosCols, ...periodosCerrado, ...periodosActual])].forEach(p => {{
        mapsPorPeriodo[p] = _mapBasePeriodoPorAlias(p);
      }});
    
      // Base de asesores: desde el mes filtrado (lista viene de baseAsesoresAnalisis)
      const baseAsesores = (window.baseAsesoresAnalisis?.[periodoFiltrado] || []).slice();
    
      // Colores encabezado
      const COLOR1 = 'FF0D47A1'; // azul oscuro
      const COLOR2 = 'FFFCCF10'; // amarillo
      const COLOR3 = 'FFFFB300'; // ambar
    
      const wb = new ExcelJS.Workbook();
      const ws = wb.addWorksheet('ANALISIS');
    
      // Encabezados
      const headersFijos = ['DNI', 'ALIAS', 'Fecha de Ingreso', 'Estatus'];
      const headersMeses = periodosCols;
    
      // ✅ Opcional: título dinámico según 3M/6M
      const headersCalc = [
        `ALCANCE_${{n}}M_CERRADO`,
        `ALCANCE_${{n}}M_ACTUAL`
      ];
    
      const headersFinal = ['CARTERA', 'EQUIPO'];
      const encabezados = [...headersFijos, ...headersMeses, ...headersCalc, ...headersFinal];
      ws.addRow(encabezados);
    
      // Estilo encabezados por grupo
      const headerRow = ws.getRow(1);
      headerRow.eachCell((cell, colNumber) => {{
        cell.font = {{ bold: true, color: {{ argb: 'FFFFFFFF' }} }};
        cell.alignment = {{ vertical: 'middle', horizontal: 'center' }};
        cell.border = {{
          top: {{ style: 'thin', color: {{ argb: 'FFD0D0D0' }} }},
          left: {{ style: 'thin', color: {{ argb: 'FFD0D0D0' }} }},
          bottom: {{ style: 'thin', color: {{ argb: 'FFD0D0D0' }} }},
          right: {{ style: 'thin', color: {{ argb: 'FFD0D0D0' }} }}
        }};
    
        const idxMesInicio = 5;
        const idxMesFin = 4 + headersMeses.length;
        const idxCalcInicio = idxMesFin + 1;
        const idxCalcFin = idxMesFin + headersCalc.length;
        const idxFinalInicio = idxCalcFin + 1;
    
        let fillColor = COLOR1;
        if (colNumber >= idxMesInicio && colNumber <= idxMesFin) fillColor = COLOR2;
        else if (colNumber >= idxCalcInicio && colNumber <= idxCalcFin) fillColor = COLOR3;
        else if (colNumber >= idxFinalInicio) fillColor = COLOR1;
    
        const textoNegro = (fillColor === COLOR2);
        cell.font = {{ bold: true, color: {{ argb: textoNegro ? 'FF000000' : 'FFFFFFFF' }} }};
    
        cell.fill = {{
          type: 'pattern',
          pattern: 'solid',
          fgColor: {{ argb: fillColor }}
        }};
      }});
    
      // Filas
      baseAsesores.forEach(asesorBase => {{
        const alias = String(asesorBase.alias_crr || '').trim();
        const dni = String(asesorBase.dni || '').trim();
    
        let fechaIngreso = String(asesorBase.fecha_ingreso || '').trim();
        if (fechaIngreso.includes(' ')) fechaIngreso = fechaIngreso.split(' ')[0];
        if (/^\d{{4}}-\d{{2}}-\d{{2}}$/.test(fechaIngreso)) {{
          const [y, m, d] = fechaIngreso.split('-');
          fechaIngreso = `${{d}}/${{m}}/${{y}}`;
        }}
    
        const supervisor = String(asesorBase.supervisor || '').trim();
    
        let cartera = '';
        try {{
          cartera = String(
            datosSupervisores?.[supervisor]?.[periodoFiltrado]?.cartera || ''
          ).trim();
        }} catch (e) {{
          cartera = '';
        }}
    
        const ingresoYM = _fechaIngresoToYMNum(fechaIngreso);
    
        // Valores por mes (solo si existe dato para ese alias y respeta fecha ingreso)
        const alcances = [];
        periodosCols.forEach(p => {{
          const {{ y, m }} = _periodoToYM(p);
          const ymNum = _ymToNum(y, m);
    
          if (ingresoYM && ymNum < ingresoYM) {{
            alcances.push("No participó");
            return;
          }}
    
          const reg = mapsPorPeriodo[p]?.get(alias);
          if (!reg) {{
            alcances.push("No participó");
            return;
          }}
    
          const val = reg.alcance;
          alcances.push((val === null || val === undefined) ? "No participó" : Number(val));
        }});
    
        // Promedio cerrado
        const valsCerrado = [];
        periodosCerrado.forEach(p => {{
          const {{ y, m }} = _periodoToYM(p);
          const ymNum = _ymToNum(y, m);
          if (ingresoYM && ymNum < ingresoYM) return;
    
          const reg = mapsPorPeriodo[p]?.get(alias);
          if (!reg) return;
          if (reg.alcance === null || reg.alcance === undefined) return;
          valsCerrado.push(Number(reg.alcance));
        }});
        const promCerrado = _avg(valsCerrado);
    
        // Promedio actual
        const valsActual = [];
        periodosActual.forEach(p => {{
          const {{ y, m }} = _periodoToYM(p);
          const ymNum = _ymToNum(y, m);
          if (ingresoYM && ymNum < ingresoYM) return;
    
          const reg = mapsPorPeriodo[p]?.get(alias);
          if (!reg) return;
          if (reg.alcance === null || reg.alcance === undefined) return;
          valsActual.push(Number(reg.alcance));
        }});
        const promActual = _avg(valsActual);
    
        const row = [
          dni,
          alias,
          fechaIngreso,
          'CALL',
          ...alcances,
          promCerrado,
          promActual,
          cartera,
          supervisor
        ];
    
        ws.addRow(row);
      }});
    
      // Formatos de columnas
      ws.getColumn(1).numFmt = '@'; // DNI como texto
    
      const colMesInicio = 5;
      const colMesFin = 4 + periodosCols.length;
      for (let c = colMesInicio; c <= colMesFin; c++) {{
        ws.getColumn(c).numFmt = '0.00%';
      }}
      ws.getColumn(colMesFin + 1).numFmt = '0.00%';
      ws.getColumn(colMesFin + 2).numFmt = '0.00%';
    
      // Auto ancho simple
      ws.columns.forEach(col => {{
        col.width = 18;
      }});
      ws.getColumn(2).width = 28; // alias
      ws.getColumn(3).width = 16; // fecha
      ws.getColumn(9 + periodosCols.length).width = 22;
    
      // Guardar
      const fechaHoy = new Date();
      const yy = fechaHoy.getFullYear();
      const mm = String(fechaHoy.getMonth() + 1).padStart(2, '0');
      const dd = String(fechaHoy.getDate()).padStart(2, '0');
      const stamp = `${{yy}}${{mm}}${{dd}}`;
    
      const nombreArchivo = `Analisis_Evaluacion_${{periodoFiltrado}}_${{stamp}}.xlsx`;
    
      const buffer = await wb.xlsx.writeBuffer();
      saveAs(new Blob([buffer], {{
        type: "application/vnd.openxmlformats-officedocument.spreadsheetml.sheet"
      }}), nombreArchivo);
    }}

    async function exportarRecuperoYMetasEvaluacion() {{
      const mesSeleccionado = document.getElementById('selectorMes')?.value;
      const añoSeleccionado = document.getElementById('selectorAño')?.value;
      const periodoFiltrado = `${{mesSeleccionado}}_${{añoSeleccionado}}`;
    
      if (!datosMeses?.[periodoFiltrado]) {{
        alert('No hay datos para exportar');
        return;
      }}
    
      // 3M o 6M según Evaluación
      const n = (vistaEvaluacion === '3M') ? 3 : 6;
    
      // ✅ SOLO meses anteriores (excluye mes seleccionado) e ignora Setiembre automáticamente
      // Ej: DIC-2025 con 3M => normalmente SEP/OCT/NOV, pero omite SEP => AGO/OCT/NOV
      const periodos = _getPeriodosAtras(mesSeleccionado, añoSeleccionado, n, false);
    
      // Mapas por periodo para encontrar rápido por alias_crr
        const mapsBase = {{}};
        periodos.forEach(p => {{
          mapsBase[p] = _mapBasePeriodoPorAlias(p);
        }});
    
      // Base (misma lista que EXPORTAR ANALISIS)
      const baseAsesores = (window.baseAsesoresAnalisis?.[periodoFiltrado] || []).slice();
    
      if (!baseAsesores.length) {{
        alert('No hay base de asesores para este periodo');
        return;
      }}
    
      const wb = new ExcelJS.Workbook();
      const ws = wb.addWorksheet('RECUPERO_METAS');
    
      // Encabezados
      const headersFijos = ['DNI', 'ALIAS', 'Fecha de Ingreso'];
      const headersMeses = [];
      periodos.forEach(p => {{
        headersMeses.push(`RECUPERO_${{p}}`);
        headersMeses.push(`META_${{p}}`);
      }});
      const headersFinal = ['SUPERVISOR'];
      const COLOR_RECUPERO = 'FFFF9800'; // naranja
      const COLOR_META = 'FFD32F2F';     // rojo
    
      ws.addRow([ ...headersFijos, ...headersMeses, ...headersFinal ]);
      ws.getRow(1).eachCell((cell, colNumber) => {{
          const text = String(cell.value || '').toUpperCase();
        
          if (text.startsWith('RECUPERO_')) {{
            cell.fill = {{
              type: 'pattern',
              pattern: 'solid',
              fgColor: {{ argb: COLOR_RECUPERO }}
            }};
            cell.font = {{ bold: true, color: {{ argb: 'FFFFFFFF' }} }};
          }}
        
          if (text.startsWith('META_')) {{
            cell.fill = {{
              type: 'pattern',
              pattern: 'solid',
              fgColor: {{ argb: COLOR_META }}
            }};
            cell.font = {{ bold: true, color: {{ argb: 'FFFFFFFF' }} }};
          }}
      }});
    
      // Estilo encabezado simple
      ws.getRow(1).font = {{ bold: true, color: {{ argb: 'FFFFFFFF' }} }};
      ws.getRow(1).alignment = {{ horizontal: 'center', vertical: 'middle' }};
      ws.getRow(1).fill = {{
        type: 'pattern',
        pattern: 'solid',
        fgColor: {{ argb: 'FF0D47A1' }}
      }};
    
      // Filas
      baseAsesores.forEach(p => {{
        const alias = (p.alias_crr || '').trim();
        if (!alias) return;
    
        const dni = (p.dni || '').toString();
        const fechaIngreso = (p.fecha_ingreso || '').toString();
        const supervisorBase = (p.supervisor || '').toString();
    
        const valoresMeses = [];
        let cartera = '';
        let supervisor = supervisorBase;
    
        periodos.forEach(periodo => {{
          const regBase = mapsBase[periodo]?.get(alias);

          if (!_mesEsValidoParaIngreso(periodo, fechaIngreso)) {{
            valoresMeses.push('');
            valoresMeses.push('');
            return;
          }}

          const rec = regBase ? Number(regBase.recupero || 0) : 0;
          const met = regBase ? Number(regBase.meta || 0) : 0;
    
          valoresMeses.push(rec);
          valoresMeses.push(met);
        }});
    
        const row = [
          dni,
          alias,
          fechaIngreso,
          ...valoresMeses,
          supervisor || ''
        ];
    
        ws.addRow(row);

      }});
    
      // Formatos
      ws.getColumn(1).numFmt = '@'; // DNI texto
      ws.getColumn(2).width = 28;
      ws.getColumn(3).width = 16;
    
      // Columnas numéricas (recupero/meta)
      const colNumInicio = 4;
      const colNumFin = 3 + headersMeses.length;
      for (let c = colNumInicio; c <= colNumFin; c++) {{
        ws.getColumn(c).numFmt = '#,##0.00';
        ws.getColumn(c).width = 16;
      }}
    
      // Supervisor / Cartera
      ws.getColumn(colNumFin + 1).width = 22;
    
      // Guardar
      const fechaHoy = new Date();
      const yy = fechaHoy.getFullYear();
      const mm = String(fechaHoy.getMonth() + 1).padStart(2, '0');
      const dd = String(fechaHoy.getDate()).padStart(2, '0');
      const stamp = `${{yy}}${{mm}}${{dd}}`;
    
      const nombreArchivo = `Recupero_Metas_${{n}}M_${{periodoFiltrado}}_${{stamp}}.xlsx`;
    
      const buffer = await wb.xlsx.writeBuffer();
      saveAs(new Blob([buffer], {{
        type: "application/vnd.openxmlformats-officedocument.spreadsheetml.sheet"
      }}), nombreArchivo);
    }}

    function _parseFechaIngreso(fechaStr) {{
    if (!fechaStr) return null;
    const s = String(fechaStr).trim();
    
    // Caso: "2023-12-21 00:00:00" o "2023-12-21"
    const iso = s.match(/^(\d{{4}})-(\d{{2}})-(\d{{2}})/);
    if (iso) {{
    const y = Number(iso[1]), m = Number(iso[2]), d = Number(iso[3]);
    const dt = new Date(y, m - 1, d);
    return isNaN(dt.getTime()) ? null : dt;
    }}
    
    // Caso: "dd/mm/yyyy"
    const latam = s.match(/^(\d{{1,2}})\/(\d{{1,2}})\/(\d{{4}})$/);
    if (latam) {{
    const d = Number(latam[1]), m = Number(latam[2]), y = Number(latam[3]);
    const dt = new Date(y, m - 1, d);
    return isNaN(dt.getTime()) ? null : dt;
    }}
    
    const dt = new Date(s);
    return isNaN(dt.getTime()) ? null : dt;
    }}
    
    function _finDeMes(periodo) {{
    // periodo: "DICIEMBRE_2025"
    const [mesTxt, anioTxt] = String(periodo || '').split('_');
    const y = Number(anioTxt);
    
    const meses = {{
    'ENERO': 0, 'FEBRERO': 1, 'MARZO': 2, 'ABRIL': 3, 'MAYO': 4, 'JUNIO': 5,
    'JULIO': 6, 'AGOSTO': 7, 'SETIEMBRE': 8, 'SEPTIEMBRE': 8, 'OCTUBRE': 9,
    'NOVIEMBRE': 10, 'DICIEMBRE': 11
    }};
    
    const m = meses[String(mesTxt || '').toUpperCase()];
    if (m === undefined || isNaN(y)) return null;
    
    // último día del mes: (mes+1, día 0)
    return new Date(y, m + 1, 0, 23, 59, 59);
    }}
    
    function _mesEsValidoParaIngreso(periodo, fechaIngresoStr) {{
    const ingreso = _parseFechaIngreso(fechaIngresoStr);
    if (!ingreso) return true; // si no hay fecha, no bloqueamos
    
    const finMes = _finDeMes(periodo);
    if (!finMes) return true;
    
    // “mostrar info si el asesor ya estaba contratado en ese mes”
    return ingreso <= finMes;
    }}
    
    // Hooks (EVALUACIÓN)
    
    document.getElementById('btnEval3Meses')
      ?.addEventListener('click', () => setVistaEvaluacion('3M'));
    
    document.getElementById('btnEval6Meses')
      ?.addEventListener('click', () => setVistaEvaluacion('6M'));
    
    document.getElementById('chkIncluirMesSeleccionadoEval')
      ?.addEventListener('change', (e) => {{
        incluirMesSeleccionadoEval = !!e.target.checked;
      }});
    
    document.getElementById('btnEvalExportarAnalisis')
      ?.addEventListener('click', async () => {{
        await exportarAnalisisEvaluacion();
      }});

    document.getElementById('btnEvalExportarRecuperoMetas')
      ?.addEventListener('click', async () => {{
        await exportarRecuperoYMetasEvaluacion();
      }});
    
    // Estado inicial
    setVistaEvaluacion('3M');
    
    // ========== NUEVA FUNCIÓN AUXILIAR ==========
    function segundosAHorasMinutosSegundos(segundosTotales) {{
        // Convierte segundos totales a formato HH:MM:SS
        const horas = Math.floor(segundosTotales / 3600);
        const minutos = Math.floor((segundosTotales % 3600) / 60);
        const segundos = Math.floor(segundosTotales % 60);
        
        return `${{String(horas).padStart(2, '0')}}:${{String(minutos).padStart(2, '0')}}:${{String(segundos).padStart(2, '0')}}`;
    }}
    
    // ========== FUNCIONES AUXILIARES EXISTENTES ==========
    
    function convertirFechaExcelAFechaKey(fechaExcel) {{
        // Convierte formato DD/MM/YYYY a YYYY-MM-DD
        try {{
            const [dia, mes, año] = fechaExcel.split('/');
            return `${{año}}-${{mes.padStart(2, '0')}}-${{dia.padStart(2, '0')}}`;
        }} catch {{
            return fechaExcel;
        }}
    }}
    
    function formatearHora(hora) {{
        // Asegura que la hora esté en formato HH:MM:SS
        if (!hora) return '';
        
        // Si ya está en formato HH:MM:SS, devolverlo
        if (typeof hora === 'string' && hora.includes(':')) {{
            const partes = hora.split(':');
            if (partes.length >= 3) {{
                // Asegurar 2 dígitos
                const hh = partes[0].padStart(2, '0');
                const mm = partes[1].padStart(2, '0');
                const ss = partes[2].padStart(2, '0');
                return `${{hh}}:${{mm}}:${{ss}}`;
            }}
        }}
        
        // Si es un objeto Date
        if (hora instanceof Date) {{
            const hh = String(hora.getHours()).padStart(2, '0');
            const mm = String(hora.getMinutes()).padStart(2, '0');
            const ss = String(hora.getSeconds()).padStart(2, '0');
            return `${{hh}}:${{mm}}:${{ss}}`;
        }}
        
        return hora.toString();
    }}
    
    function convertirHoraASegundos(horaStr) {{
        // Convierte HH:MM:SS a segundos totales
        try {{
            const [horas, minutos, segundos] = horaStr.split(':').map(Number);
            return horas * 3600 + minutos * 60 + segundos;
        }} catch {{
            return 0;
        }}
    }}
    
    // ========== INICIALIZACIÓN SIMPLIFICADA ==========
    document.addEventListener('DOMContentLoaded', function() {{
        
        // 1. INICIALIZAR FILTROS GLOBALES UNA VEZ
        const mesSeleccionado = document.getElementById('selectorMes').value;
        const añoSeleccionado = document.getElementById('selectorAño').value;
        const periodoCompleto = `${{mesSeleccionado}}_${{añoSeleccionado}}`;
        
        if (datosMeses[periodoCompleto]) {{
            actualizarFiltrosGlobales();
        }}
        
        // 2. OBSERVADOR SIMPLIFICADO PARA ASISTENCIAS
        const seccionAsistencias = document.getElementById('seccion-asistencias');
        if (seccionAsistencias) {{
            const observer = new MutationObserver(function(mutations) {{
                mutations.forEach(function(mutation) {{
                    if (mutation.type === 'attributes' && mutation.attributeName === 'class') {{
                        if (seccionAsistencias.classList.contains('activa')) {{
                            console.log('📊 Sección de asistencias activada');
                            
                            // Aplicar filtro actual
                            setTimeout(() => {{
                                aplicarFiltroSupervisor(supervisorFiltroActual);
                            }}, 100);
                        }}
                    }}
                }});
            }});
            
            observer.observe(seccionAsistencias, {{ attributes: true }});
        }}
        
        // 3. EVENTOS PARA LOS SELECTORES DEL HEADER
        const selectorMes = document.getElementById('selectorMes');
        const selectorAño = document.getElementById('selectorAño');
        
        if (selectorMes && selectorAño) {{
            const actualizarTodo = function() {{
                const mes = selectorMes.value;
                const año = selectorAño.value;
                const periodo = `${{mes}}_${{año}}`;
                
                if (datosMeses[periodo]) {{
                    // Actualizar filtros globales
                    actualizarFiltrosGlobales();
                    
                    // Aplicar filtro actual a la sección activa
                    aplicarFiltroSupervisor(supervisorFiltroActual);
                }}
            }};
            
            selectorMes.addEventListener('change', actualizarTodo);
            selectorAño.addEventListener('change', actualizarTodo);
        }}
        
        // 4. INICIALIZAR GRÁFICA DE EVOLUCIÓN ANUAL SI ESTAMOS EN RECUPEROS
        const seccionRecuperos = document.getElementById('seccion-recuperos');
        if (seccionRecuperos && seccionRecuperos.classList.contains('activa')) {{
            setTimeout(() => {{
                actualizarGraficaEvolucionAnual();
            }}, 300);
        }}
    }});

    // ========== INICIALIZACIÓN ==========
    document.addEventListener('DOMContentLoaded', function() {{
        // Inicializar la sección de ranking por defecto
        const mesSeleccionado = document.getElementById('selectorMes').value;
        const añoSeleccionado = document.getElementById('selectorAño').value;
        
        actualizarPeriodo();
        
        // Inicializar la sección de ranking por defecto (primera carga)
        if (datosMeses[estadoGlobal.periodoActual]) {{
            const asesores = datosMeses[estadoGlobal.periodoActual] || [];
            actualizarBotonesSupervisores(asesores);
        }}
    
        // Permitir agregar asesor con Enter
        const inputBusqueda = document.getElementById('inputBusqueda');
        if (inputBusqueda) {{
            inputBusqueda.addEventListener('keypress', function(e) {{
                if (e.key === 'Enter') {{
                    agregarAsesor();
                }}
            }});
        }}
        
        // Permitir agregar supervisor con Enter
        const inputBusquedaSupervisor = document.getElementById('inputBusquedaSupervisor');
        if (inputBusquedaSupervisor) {{
            inputBusquedaSupervisor.addEventListener('keypress', function(e) {{
                if (e.key === 'Enter') {{
                    agregarSupervisor();
                }}
            }});
        }}
        
        // Permitir agregar asesor periodo con Enter
        const inputBusquedaPeriodo = document.getElementById('inputBusquedaPeriodo');
        if (inputBusquedaPeriodo) {{
            inputBusquedaPeriodo.addEventListener('keypress', function(e) {{
                if (e.key === 'Enter') {{
                    agregarAsesorPeriodo();
                }}
            }});
        }}
    
        // Asegurar que los selectores del header están sincronizados
        document.getElementById('selectorMes').addEventListener('change', sincronizarSelectoresGlobal);
        document.getElementById('selectorAño').addEventListener('change', sincronizarSelectoresGlobal);
    
        // DETECTORES DE CAMBIO A LOS SELECTORES DEL HEADER
        const selectorMes = document.getElementById('selectorMes');
        const selectorAño = document.getElementById('selectorAño');
        
        if (selectorMes) {{
            selectorMes.addEventListener('change', function() {{
                console.log('📅 Selector de mes cambiado');
                actualizarPeriodo();
            }});
            
            // También para entrada de teclado
            selectorMes.addEventListener('input', function() {{
                setTimeout(() => actualizarPeriodo(), 100);
            }});
        }}
        
        if (selectorAño) {{
            selectorAño.addEventListener('change', function() {{
                console.log('📅 Selector de año cambiado');
                actualizarPeriodo();
            }});
            
            selectorAño.addEventListener('input', function() {{
                setTimeout(() => actualizarPeriodo(), 100);
            }});
        }}
        // NUEVO: Inicializar gráfica de evolución anual si estamos en recuperos
            const seccionRecuperos = document.getElementById('seccion-recuperos');
            if (seccionRecuperos && seccionRecuperos.classList.contains('activa')) {{
                setTimeout(() => {{
                    actualizarGraficaEvolucionAnual();
                }}, 300);
            }}
        }});

        // ========== FUNCIONES PARA DESCARGAR TARJETAS DE CUMPLEAÑOS ==========
        function descargarTarjetasAsesores() {{
            if (!window.cumpleañosData || !window.cumpleañosData.asesores) {{
                alert('No hay datos de asesores para descargar');
                return;
            }}
            
            descargarTarjetasSeccion('asesores', window.cumpleañosData.asesores, 'Asesores');
        }}
        
        function descargarTarjetasStaff() {{
            if (!window.cumpleañosData || !window.cumpleañosData.staff) {{
                alert('No hay datos de staff para descargar');
                return;
            }}
            
            descargarTarjetasSeccion('staff', window.cumpleañosData.staff, 'Staff');
        }}
        
        function descargarTarjetasSeccion(tipo, personas, nombreSeccion) {{
            if (!personas || personas.length === 0) {{
                alert(`No hay ${{nombreSeccion.toLowerCase()}} para descargar`);
                return;
            }}
            
            const mensajeProcesando = document.createElement('div');
            mensajeProcesando.style.cssText = `
                position: fixed;
                top: 50%;
                left: 50%;
                transform: translate(-50%, -50%);
                background: rgba(0, 0, 0, 0.9);
                color: white;
                padding: 20px 30px;
                border-radius: 10px;
                z-index: 99999;
                text-align: center;
                font-size: 16px;
            `;
            mensajeProcesando.innerHTML = `🔄 Generando imagen...`;
            document.body.appendChild(mensajeProcesando);
            
            // Crear contenedor mínimo sin fondos externos
            const contenedorTemp = document.createElement('div');
            contenedorTemp.id = 'contenedor-temp-cumple';
            contenedorTemp.style.cssText = `
                position: absolute;
                left: -9999px;
                top: -9999px;
                background: transparent !important;
            `;
            
            // Color según el tipo
            const colorPrincipal = tipo === 'asesores' ? '#2ecc71' : '#3498db';
            const colorPrincipalTransparente = tipo === 'asesores' ? 'rgba(46, 204, 113, 0.95)' : 'rgba(52, 152, 219, 0.95)';
            
            // Crear tarjetas individuales con 70% de opacidad
            personas.forEach(persona => {{
                const tarjeta = document.createElement('div');
                tarjeta.style.cssText = `
                    width: 760px;
                    min-height: 100px;
                    background: rgba(255, 255, 255, 0.95); /* FONDO BLANCO 70% OPACO */
                    border-radius: 10px;
                    padding: 20px;
                    display: flex;
                    align-items: center;
                    box-shadow: 0 4px 12px rgba(0,0,0,0.1);
                    border-left: 6px solid ${{colorPrincipalTransparente}}; /* BORDE 70% OPACO */
                    margin-bottom: 15px;
                    box-sizing: border-box;
                    backdrop-filter: blur(2px); /* Efecto de vidrio opcional */
                `;
                
                tarjeta.innerHTML = `
                    <div style="margin-right: 15px; flex-shrink: 0;">
                        <div style="font-size: 36px; color: ${{colorPrincipalTransparente}}; opacity: 0.7;">👤</div>
                    </div>
                    <div style="flex: 1; min-width: 0;">
                        <div style="font-size: 40px; font-weight: bold; color: rgba(44, 62, 80, 1); margin-bottom: 5px;">
                            ${{persona.nombre}}
                        </div>
                        <div style="color: rgba(102, 102, 102, 0.7); font-size: 22px; font-style: italic;">
                            ${{persona.puesto_real || persona.alias || ''}}
                        </div>
                    </div>
                    <div style="text-align: right; flex-shrink: 0; margin-left: 15px;">
                        <div style="font-size: 48px; font-weight: bold; color: rgba(231, 76, 60, 1); line-height: 1;">
                            ${{persona.dia}}
                        </div>
                        <div style="color: rgba(102, 102, 102, 0.7); font-size: 22px;">
                            ${{persona.mes}}
                        </div>
                    </div>
                `;
                
                contenedorTemp.appendChild(tarjeta);
            }});
            
            document.body.appendChild(contenedorTemp);
            
            setTimeout(() => {{
                html2canvas(contenedorTemp, {{
                    scale: 2,
                    backgroundColor: null, // FONDO TRANSPARENTE
                    logging: false,
                    useCORS: true
                }}).then(canvas => {{
                    // Obtener imagen con fondo transparente
                    const imagen = canvas.toDataURL('image/png');
                    
                    const enlace = document.createElement('a');
                    enlace.href = imagen;
                    enlace.download = `Cumpleaños_${{window.cumpleañosData.mes}}_${{nombreSeccion}}_translucido.png`;
                    document.body.appendChild(enlace);
                    enlace.click();
                    document.body.removeChild(enlace);
                    
                    // Limpiar
                    document.body.removeChild(contenedorTemp);
                    document.body.removeChild(mensajeProcesando);
                    
                }}).catch(error => {{
                    console.error('Error:', error);
                    alert('Error al generar la imagen');
                    document.body.removeChild(contenedorTemp);
                    document.body.removeChild(mensajeProcesando);
                }});
            }}, 500);
        }}
        
        // Verificar si html2canvas está disponible, si no, cargarlo dinámicamente
        function cargarHtml2CanvasSiEsNecesario() {{
            if (typeof html2canvas === 'undefined') {{
                const script = document.createElement('script');
                script.src = 'https://cdnjs.cloudflare.com/ajax/libs/html2canvas/1.4.1/html2canvas.min.js';
                script.integrity = 'sha512-BNaRQnYJYiPSqHHDb58B0yaPfCu+Wgds8Gp/gU33kqBtgNS4tSPHuGibyoeqMV/TJlSKda6FXzoEyYGjTe+vXA==';
                script.crossOrigin = 'anonymous';
                script.referrerPolicy = 'no-referrer';
                
                script.onload = function() {{
                    console.log('html2canvas cargado exitosamente');
                }};
                
                script.onerror = function() {{
                    alert('Error al cargar html2canvas. No se pueden generar imágenes.');
                }};
                
                document.head.appendChild(script);
            }}
        }}
        
        // Cargar html2canvas automáticamente cuando se carga la página
        document.addEventListener('DOMContentLoaded', function() {{
            cargarHtml2CanvasSiEsNecesario();
        }});

</script>
</body>
</html>'''
    
    with open(ruta_html, 'w', encoding='utf-8') as f:
        f.write(html_content)
    
    print(f"HTML modular generado y guardado en: {ruta_html}")

def generar_seccion_ranking(asesores_data, contadores, opciones_asesores, opciones_supervisores, meses_unicos, años_unicos):
    """Genera el contenido de la sección RANKING"""

    # Listas de asesores por clasificación
    lista_100 = generar_lista_asesores(asesores_data, ">100%")
    lista_70 = generar_lista_asesores(asesores_data, ">70%")
    lista_40 = generar_lista_asesores(asesores_data, ">40%")
    lista_0 = generar_lista_asesores(asesores_data, ">0%")

    # Generar opciones para selects de periodo de prueba (si los usas más abajo)
    opciones_mes_periodo = ""
    for mes in meses_unicos:
        opciones_mes_periodo += f'<option value="{mes}">{mes.title()}</option>\n'

    opciones_año_periodo = ""
    for año in años_unicos:
        opciones_año_periodo += f'<option value="{año}">{año}</aoption>\n'

    opciones_asesores_periodo = "".join(
        [f'<option value="{asesor}">' for asesor in sorted(set([a["nombre"] for a in asesores_data]))]
    )

    return f'''
        <div class="clasificaciones">
            <div class="clasificacion">
                <div class="clasificacion-header clasificacion-100">100% – ↑</div>
                <div class="asesores-lista" id="lista-100">
                    {lista_100}
                </div>
            </div>

            <div class="clasificacion">
                <div class="clasificacion-header clasificacion-70">70% – 100%</div>
                <div class="asesores-lista" id="lista-70">
                    {lista_70}
                </div>
            </div>

            <div class="clasificacion">
                <div class="clasificacion-header clasificacion-40">40% – 70%</div>
                <div class="asesores-lista" id="lista-40">
                    {lista_40}
                </div>
            </div>

            <div class="clasificacion">
                <div class="clasificacion-header clasificacion-0">0% – 40%</div>
                <div class="asesores-lista" id="lista-0">
                    {lista_0}
                </div>
            </div>
        </div>
        
        <!-- Estadísticas -->
        <div style="display: grid; grid-template-columns: repeat(4, 1fr); gap: 20px; margin: 40px 0;">
            <div style="text-align: center; background: #e8f6f3; padding: 25px; border-radius: 12px; border: 2px solid #27ae60;">
                <div style="font-size: 1.1rem; color: #27ae60; margin-bottom: 10px;">Superaron el 100%</div>
                <div id="cantidad-100" style="font-size: 3rem; font-weight: bold; color: #27ae60;">{contadores[">100%"]}</div>
                <div style="font-size: 0.9rem; color: #666; margin-top: 10px;">Meta superada</div>
            </div>
            
            <div style="text-align: center; background: #fef9e7; padding: 25px; border-radius: 12px; border: 2px solid #f39c12;">
                <div style="font-size: 1.1rem; color: #f39c12; margin-bottom: 10px;">Superaron el 70%</div>
                <div id="cantidad-70" style="font-size: 3rem; font-weight: bold; color: #f39c12;">{contadores[">70%"]}</div>
                <div style="font-size: 0.9rem; color: #666; margin-top: 10px;">Alto rendimiento</div>
            </div>
            
            <div style="text-align: center; background: #fdebd0; padding: 25px; border-radius: 12px; border: 2px solid #e67e22;">
                <div style="font-size: 1.1rem; color: #e67e22; margin-bottom: 10px;">Superaron el 40%</div>
                <div id="cantidad-40" style="font-size: 3rem; font-weight: bold; color: #e67e22;">{contadores[">40%"]}</div>
                <div style="font-size: 0.9rem; color: #666; margin-top: 10px;">Rendimiento medio</div>
            </div>
            
            <div style="text-align: center; background: #fadbd8; padding: 25px; border-radius: 12px; border: 2px solid #e74c3c;">
                <div style="font-size: 1.1rem; color: #e74c3c; margin-bottom: 10px;">Entre 0% a 40%</div>
                <div id="cantidad-0" style="font-size: 3rem; font-weight: bold; color: #e74c3c;">{contadores[">0%"]}</div>
                <div style="font-size: 0.9rem; color: #666; margin-top: 10px;">Por mejorar</div>
            </div>
        </div>
        
        <!-- Búsqueda y Comparación de Asesores -->
        <div class="seccion-busqueda">
            <h2>🔍 Búsqueda y Comparación de Asesores</h2>
            
            <div class="controles-busqueda">
                <input type="text" id="inputBusqueda" class="input-busqueda" placeholder="Escribe el nombre del asesor..." list="listaAsesores">
                <datalist id="listaAsesores">
                    {opciones_asesores}
                </datalist>
                <button class="boton-busqueda" onclick="agregarAsesor()">Agregar Asesor</button>
                <button class="boton-busqueda verde" onclick="compararAsesores()">Comparar Seleccionados</button>
                <button class="boton-busqueda rojo" onclick="limpiarBusqueda()">Limpiar</button>
            </div>
            
            <div class="elementos-seleccionados" id="asesoresSeleccionados">
                <!-- Los asesores seleccionados aparecerán aquí -->
            </div>
            
            <div id="resultadosComparacion" style="display: none; margin-top: 30px;">
                <div id="tablaComparacion"></div>
            </div>
        </div>
        
        <!-- Búsqueda y Comparación de Supervisores -->
        <div class="seccion-busqueda">
            <h2>👥 Búsqueda y Comparación de Supervisores</h2>
            
            <div class="controles-busqueda">
                <input type="text" id="inputBusquedaSupervisor" class="input-busqueda" placeholder="Escribe el nombre del supervisor..." list="listaSupervisores">
                <datalist id="listaSupervisores">
                    {opciones_supervisores}
                </datalist>
                <button class="boton-busqueda" onclick="agregarSupervisor()">Agregar Supervisor</button>
                <button class="boton-busqueda verde" onclick="compararSupervisores()">Comparar Equipos</button>
                <button class="boton-busqueda rojo" onclick="limpiarBusquedaSupervisores()">Limpiar</button>
            </div>
            
            <div class="elementos-seleccionados" id="supervisoresSeleccionados">
                <!-- Los supervisores seleccionados aparecerán aquí -->
            </div>
            
            <div id="resultadosComparacionSupervisores" style="display: none; margin-top: 30px;">
                <h3 style="color: #2c3e50; margin-bottom: 20px;">📊 Comparación de Equipos</h3>
                <div id="tablaComparacionSupervisores"></div>
            </div>
        </div>
        
        <!-- Periodo de Prueba -->
        <div class="seccion-busqueda">
            <h2>📈 Evaluación de Periodo de Prueba</h2>
            
            <div class="selectores-periodo-prueba">
                <div class="selector-item">
                    <label for="mes1" class="selector-label">📅 Mes 1</label>
                    <select id="mes1">
                        <option value="">Seleccione mes...</option>
                        {opciones_mes_periodo}
                    </select>
                </div>
                
                <div class="selector-item">
                    <label for="año1" class="selector-label">📅 Año 1</label>
                    <select id="año1">
                        <option value="">Seleccione año...</option>
                        {opciones_año_periodo}
                    </select>
                </div>
                
                <div class="selector-item">
                    <label for="mes2" class="selector-label">📅 Mes 2</label>
                    <select id="mes2">
                        <option value="">Seleccione mes...</option>
                        {opciones_mes_periodo}
                    </select>
                </div>
                
                <div class="selector-item">
                    <label for="año2" class="selector-label">📅 Año 2</label>
                    <select id="año2">
                        <option value="">Seleccione año...</option>
                        {opciones_año_periodo}
                    </select>
                </div>
                
                <div class="selector-item">
                    <label for="porcentajeMinimo" class="selector-label">🎯 % Mínimo</label>
                    <input type="number" id="porcentajeMinimo" placeholder="Ej: 70" value="70" min="0" max="100" step="1">
                </div>
            </div>
            
            <div class="controles-busqueda">
                <input type="text" id="inputBusquedaPeriodo" class="input-busqueda" placeholder="Escribe el nombre del asesor..." list="listaAsesoresPeriodo">
                <datalist id="listaAsesoresPeriodo">
                    {opciones_asesores}
                </datalist>
                <button class="boton-busqueda" onclick="agregarAsesorPeriodo()">Agregar Asesor</button>
                <button class="boton-busqueda verde" onclick="calcularPeriodoPrueba()">Calcular Periodo</button>
                <button class="boton-busqueda rojo" onclick="limpiarPeriodoPrueba()">Limpiar</button>
            </div>
            
            <div class="elementos-seleccionados" id="asesoresSeleccionadosPeriodo">
                <!-- Los asesores para periodo de prueba aparecerán aquí -->
            </div>
            
            <div id="resultadosPeriodo" style="display: none; margin-top: 30px;">
                <div id="tablaPeriodo"></div>
            </div>
        </div>
    '''
    
def generar_seccion_asistencias():
    """Genera el contenido de la sección ASISTENCIAS con nueva estructura"""
    return '''

        <!-- Cuadro Estadístico -->
        <div id="cuadro-estadistico-asistencias" style="margin-bottom: 40px;">
            <div style="display: grid; grid-template-columns: repeat(5, 1fr); gap: 15px; margin-bottom: 30px;">
                <!-- DÍAS LABORABLES -->
                <div style="text-align: center; background: #E8F4FD; padding: 20px; border-radius: 10px; border: 2px solid #3498db;">
                    <div style="font-size: 0.9rem; color: #3498db; margin-bottom: 8px; font-weight: 600;">DÍAS REGISTRADOS</div>
                    <div id="estadistica-dias-laborables" style="font-size: 2rem; font-weight: bold; color: #3498db;">0</div>
                    <div style="font-size: 0.8rem; color: #666; margin-top: 5px;">Fechas únicas con registros</div>
                </div>
                
                <!-- ASESORES/SUPERVISORES -->
                <div style="text-align: center; background: #F4ECF7; padding: 20px; border-radius: 10px; border: 2px solid #9b59b6;">
                    <div style="font-size: 0.9rem; color: #9b59b6; margin-bottom: 8px; font-weight: 600;">PERSONAS</div>
                    <div style="display: flex; justify-content: center; gap: 15px;">
                        <div>
                            <div id="estadistica-asesores" style="font-size: 1.8rem; font-weight: bold; color: #27ae60;">0</div>
                            <div style="font-size: 0.7rem; color: #666;">Asesores</div>
                        </div>
                        <div>
                            <div id="estadistica-supervisores" style="font-size: 1.8rem; font-weight: bold; color: #3498db;">0</div>
                            <div style="font-size: 0.7rem; color: #666;">Supervisores</div>
                        </div>
                    </div>
                </div>
                
                <!-- FALTAS TOTALES -->
                <div style="text-align: center; background: #FADBD8; padding: 20px; border-radius: 10px; border: 2px solid #e74c3c;">
                    <div style="font-size: 0.9rem; color: #e74c3c; margin-bottom: 8px; font-weight: 600;">FALTAS TOTALES</div>
                    <div id="estadistica-faltas" style="font-size: 2rem; font-weight: bold; color: #e74c3c;">0</div>
                    <div style="font-size: 0.8rem; color: #666; margin-top: 5px;">Personas con faltas</div>
                </div>
                
                <!-- TIEMPO TARDANZAS ACUMULADAS -->
                <div style="text-align: center; background: #FDEBD0; padding: 20px; border-radius: 10px; border: 2px solid #f39c12;">
                    <div style="font-size: 0.9rem; color: #f39c12; margin-bottom: 8px; font-weight: 600;">TARDANZA ACUMULADA</div>
                    <div id="estadistica-tardanza-acumulada" style="font-size: 1.6rem; font-weight: bold; color: #f39c12;">00:00:00</div>
                    <div style="font-size: 0.8rem; color: #666; margin-top: 5px;">Total tiempo</div>
                </div>
                
                <!-- TARDANZAS TOTALES -->
                <div style="text-align: center; background: #D5F5E3; padding: 20px; border-radius: 10px; border: 2px solid #2ecc71;">
                    <div style="font-size: 0.9rem; color: #2ecc71; margin-bottom: 8px; font-weight: 600;">TARDANZAS TOTALES</div>
                    <div id="estadistica-tardanzas-totales" style="font-size: 2rem; font-weight: bold; color: #2ecc71;">0</div>
                    <div style="font-size: 0.8rem; color: #666; margin-top: 5px;">Registros con tardanza</div>
                </div>
            </div>
        </div>

        <!-- Gráficas de Donas -->
        <div style="display: grid; grid-template-columns: repeat(3, 1fr); gap: 15px; margin-bottom: 30px;">
        
            <!-- Gráfica 1 -->
            <div style="background: white; border-radius: 10px; padding: 12px; box-shadow: 0 3px 8px rgba(0,0,0,0.08); display: flex; flex-direction: column; height: 420px;">
                <h3 style="text-align: center; color: #e74c3c; margin: 0 0 10px 0; font-size: 1rem; font-weight: 600; padding: 5px; flex-shrink: 0;">
                    ❌ TOP 7 - FALTAS
                </h3>
                <div style="flex: 1; display: flex; justify-content: center; align-items: center; min-height: 0;">
                    <canvas id="graficaTopFaltas"></canvas>
                </div>
            </div>
        
            <!-- Gráfica 2 -->
            <div style="background: white; border-radius: 10px; padding: 12px; box-shadow: 0 3px 8px rgba(0,0,0,0.08); display: flex; flex-direction: column; height: 420px;">
                <h3 style="text-align: center; color: #f39c12; margin: 0 0 10px 0; font-size: 1rem; font-weight: 600; padding: 5px; flex-shrink: 0;">
                    ⏰ TOP 7 - TIEMPO EN TARDANZAS (MIN)
                </h3>
                <div style="flex: 1; display: flex; justify-content: center; align-items: center; min-height: 0;">
                    <canvas id="graficaTopTardanzaTiempo"></canvas>
                </div>
            </div>
        
            <!-- Gráfica 3 -->
            <div style="background: white; border-radius: 10px; padding: 12px; box-shadow: 0 3px 8px rgba(0,0,0,0.08); display: flex; flex-direction: column; height: 420px;">
                <h3 style="text-align: center; color: #3498db; margin: 0 0 10px 0; font-size: 1rem; font-weight: 600; padding: 5px; flex-shrink: 0;">
                    📊 TOP 7 - CANTIDAD DE TARDANZAS
                </h3>
                <div style="flex: 1; display: flex; justify-content: center; align-items: center; min-height: 0;">
                    <canvas id="graficaTopTardanzas"></canvas>
                </div>
            </div>
        </div>

        <!-- Botón de Exportación CENTRADO -->
        <div style="text-align: center; margin: 30px 0;">
            <button class="boton-exportar-excel" onclick="exportarAsistenciasExcelCompleto()" 
                    style="padding: 12px 30px; font-size: 1.1rem; margin: 0 auto; display: inline-block;">
                📊 Exportar Asistencias
            </button>
        </div>

        <!-- Tabla de Detalles CON ORDENAMIENTO Y HISTÓRICO -->
        <div style="margin-top: 20px;">
            <div class="contenedor-tabla-scroll">
                <table class="tabla-comparacion" id="tablaDetalleAsistencias">
                    <thead>
                        <tr>
                            <th>Persona</th>
                            <th>Tipo</th>
                            <th>Equipo</th>
                            <th>Fecha Ingreso</th>
                            <th>Días Debería</th>
                            <th>Días Asistidos</th>
                            <th>Días Faltados</th>
                            <th>Tiempo Tardanza</th>
                            <th>Cant. Tardanzas</th>
                            <th>% Asistencia</th>
                            <th>% Puntualidad</th>
                            <th>Histórico</th>
                        </tr>
                    </thead>
                    <tbody id="tbodyDetalleAsistencias">
                        <!-- Los datos se cargarán dinámicamente -->
                    </tbody>
                </table>
            </div>
        </div> 
    <!-- ESTILOS ADICIONALES PARA HISTÓRICO Y ORDENAMIENTO -->
    <style>
        .sort-icon {{
            transition: all 0.2s ease;
            font-size: 0.8em;
        }}
        
        th:hover .sort-icon {{
            opacity: 1 !important;
        }}
        
        th[data-sort] {{
            background-color: #e8f4fd !important;
            position: relative;
        }}
        
        th[data-sort]::after {{
            content: "";
            position: absolute;
            bottom: 0;
            left: 0;
            right: 0;
            height: 3px;
            background: linear-gradient(90deg, #3498db, #2ecc71);
        }}
        
        /* ESTILOS ESPECÍFICOS PARA TABLA DE ASISTENCIAS */
        #tablaDetalleAsistencias th {{
            cursor: pointer;
            user-select: none;
            transition: background-color 0.2s ease;
        }}
        
        #tablaDetalleAsistencias th:hover {{
            background-color: #2c3e50!important;
        }}
        
        #tablaDetalleAsistencias th:active {{
            background-color: #e8f4fd !important;
        }}
        
        .btn-historico {{
            transition: all 0.3s ease;
        }}
        
        .btn-historico:hover {{
            transform: translateY(-2px);
            box-shadow: 0 4px 8px rgba(52, 152, 219, 0.3);
        }}
        
        .fila-historico {{
            background-color: #f8f9fa;
            border-top: 2px solid #3498db !important;
        }}
        
        .contenedor-historico {{
            padding: 10px;
            background: linear-gradient(135deg, #f8f9fa, #e8eaf1);
        }}
        
        .loader {{
            display: inline-block;
            width: 40px;
            height: 40px;
            border: 4px solid #f3f3f3;
            border-top: 4px solid #3498db;
            border-radius: 50%;
            animation: spin 1s linear infinite;
        }}
        
        @keyframes spin {{
            0% {{ transform: rotate(0deg); }}
            100% {{ transform: rotate(360deg); }}
        }}
    </style>
    
    <!-- SCRIPT ADICIONAL PARA ORDENAMIENTO -->
    <script>
        // Estas funciones ya están en el JavaScript principal
        // Se mantienen aquí por compatibilidad
    </script>
    '''
    
def generar_seccion_recuperos(mes_actual, año_actual):
    """Genera el contenido de la sección RECUPEROS Y ALCANCES con filtros por supervisor"""
    return f'''
        
        <!-- Estadísticas del mes -->
        <div id="estadisticas-recuperos">
            <!-- Las estadísticas se cargarán dinámicamente -->
        </div>
        
        <!-- Gráficos y exportación -->
        <div id="graficas-recuperos">
            <!-- Los gráficos se generarán dinámicamente -->
        </div>
    '''

def generar_lista_asesores(asesores_data, clasificacion):
    """Genera HTML para la lista de asesores de una clasificación específica"""
    asesores_filtrados = [a for a in asesores_data if a['clasificacion'] == clasificacion]
    
    if not asesores_filtrados:
        return '<div class="asesor-item">No hay asesores en esta categoría</div>'
    
    html_items = []
    for asesor in asesores_filtrados:
        clase_gradiente = f"gradiente-{clasificacion.replace('%', '').replace('>', '')}"
        clase_porcentaje = f"porcentaje-{clasificacion.replace('%', '').replace('>', '')}"
        
        item_html = f'''
        <div class="asesor-item {clase_gradiente}">
            <div class="asesor-nombre">{asesor['nombre']}</div>
            <div class="asesor-porcentaje {clase_porcentaje}">
                {asesor['porcentaje']}%
            </div>
        </div>'''
        html_items.append(item_html)
    
    return ''.join(html_items)

def obtener_mes_año_actual(meses_validos):
    """Obtiene el mes y año más reciente disponible"""
    if not meses_validos:
        return "NOVIEMBRE", "2025"
    
    # Ordenar por fecha (más reciente primero)
    meses_ordenados = sorted(meses_validos, key=lambda x: convertir_mes_a_fecha(x), reverse=True)
    mes, año = meses_ordenados[0].split('_')
    return mes, año

def convertir_mes_a_fecha(nombre_mes):
    """Convierte nombre de mes_año a fecha para ordenar"""
    try:
        mes, año = nombre_mes.split('_')
        # Mapear nombres de meses en español
        meses = {
            'ENERO': 1, 'FEBRERO': 2, 'MARZO': 3, 'ABRIL': 4, 'MAYO': 5, 'JUNIO': 6,
            'JULIO': 7, 'AGOSTO': 7, 'SETIEMBRE': 8, 'SEPTIEMBRE': 8, 
            'OCTUBRE': 9, 'NOVIEMBRE': 10, 'DICIEMBRE': 11
        }
        return datetime(int(año), meses.get(mes.upper(), 1), 1)
    except:
        return datetime(1900, 1, 1)

if __name__ == "__main__":
    procesar_excel_y_actualizar_html()

<>:11188: SyntaxWarning: invalid escape sequence '\d'
<>:11188: SyntaxWarning: invalid escape sequence '\s'
<>:11188: SyntaxWarning: invalid escape sequence '\d'
<>:11188: SyntaxWarning: invalid escape sequence '\d'
<>:11188: SyntaxWarning: invalid escape sequence '\d'
<>:11188: SyntaxWarning: invalid escape sequence '\d'
<>:11188: SyntaxWarning: invalid escape sequence '\/'
<>:11188: SyntaxWarning: invalid escape sequence '\/'
<>:11188: SyntaxWarning: invalid escape sequence '\d'
<>:11188: SyntaxWarning: invalid escape sequence '\d'
<>:11188: SyntaxWarning: invalid escape sequence '\d'
<>:11188: SyntaxWarning: invalid escape sequence '\d'
<>:11188: SyntaxWarning: invalid escape sequence '\d'
<>:11188: SyntaxWarning: invalid escape sequence '\d'
<>:11188: SyntaxWarning: invalid escape sequence '\d'
<>:11188: SyntaxWarning: invalid escape sequence '\/'
<>:11188: SyntaxWarning: invalid escape sequence '\/'
<>:11188: SyntaxWarning: invalid escape sequence '\d'
<>:11188: SyntaxWarning: inv


🎂 PROCESANDO HOJA DE CUMPLEAÑOS: INF

Procesando hoja: ENERO_2025

  📋 MAPEO DE COLUMNAS - TABLA 1 (ASESORES):
    📅 Fecha asesor: '2023-05-29 00:00:00' (col 8)
    ✅ AL_AC/ALCANCE: col 9
    ✅ CLASIFICACIÓN: col 10
    ✅ SUPERVISOR: col 11
    ✅ RECUPERO: col 12
    ✅ META: col 13
    ✅ ASER_: col 14
  📊 Total fechas asesores: 1

  📋 MAPEO DE COLUMNAS - TABLA 2 (SUPERVISORES):
    ✅ META SUPER: col 16
    ✅ RECUPERO SUPERVISOR: col 17
    ✅ ALCANCE SUPERVISOR: col 18
    ✅ SUPER (nombre supervisor): col 19
    📅 Fecha supervisor: '2-Ene' (col 20)
    📅 Fecha supervisor: '3-Ene' (col 21)
    📅 Fecha supervisor: '6-Ene' (col 22)
    📅 Fecha supervisor: '7-Ene' (col 23)
    📅 Fecha supervisor: '8-Ene' (col 24)
    📅 Fecha supervisor: '9-Ene' (col 25)
    📅 Fecha supervisor: '10-Ene' (col 26)
    📅 Fecha supervisor: '13-Ene' (col 27)
    📅 Fecha supervisor: '14-Ene' (col 28)
    📅 Fecha supervisor: '15-Ene' (col 29)
    📅 Fecha supervisor: '16-Ene' (col 30)
    📅 Fecha supervisor: '17-En

In [2]:
import subprocess
import os
from datetime import datetime

def realizar_git_operations():
    """
    Función separada para operaciones Git.
    Se puede llamar desde otra celda cuando se quiera hacer commit/push.
    """
    try:
        # Cambiar al directorio del proyecto
        base_path = r"C:\Users\Jorge Vasquez\Ranking"
        os.chdir(base_path)
        
        print("=" * 60)
        print("🚀 INICIANDO OPERACIONES GIT")
        print("=" * 60)
        
        # 1. VERIFICAR ESTADO DE GIT
        print("\n📋 Verificando estado del repositorio...")
        try:
            result = subprocess.run(["git", "status"], capture_output=True, text=True, check=True)
            print("   ✅ Repositorio Git detectado")
        except subprocess.CalledProcessError:
            print("   ❌ No se pudo ejecutar 'git status'")
            print("   ℹ️  El directorio podría no ser un repositorio Git")
            return
        
        # 2. AGREGAR ARCHIVO AL STAGING
        print("\n📁 Agregando cambios al staging area...")
        subprocess.run(["git", "add", "."], check=True)
        print("   ✅ Archivo 'index.html' agregado al staging")
        
        # 3. VERIFICAR SI HAY CAMBIOS PARA COMMIT
        print("\n🔍 Verificando cambios pendientes...")
        result = subprocess.run(["git", "status", "--porcelain"], capture_output=True, text=True, check=True)
        
        if not result.stdout.strip():
            print("   ℹ️  No hay cambios para commit (archivo ya actualizado)")
            return
        
        # 4. REALIZAR COMMIT
        print("\n💾 Creando commit...")
        commit_message = f"Actualización automática - {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"
        print(f"   📝 Mensaje: '{commit_message}'")
        
        commit_res = subprocess.run(
            ["git", "commit", "-m", commit_message],
            capture_output=True,
            text=True
        )
        
        if commit_res.returncode != 0:
            print("❌ Falló el commit")
            print("STDOUT:", commit_res.stdout)
            print("STDERR:", commit_res.stderr)
            return
        print("   ✅ Commit realizado exitosamente")
        
        # 5. REALIZAR PUSH
        print("\n📤 Enviando cambios al repositorio remoto...")
        push_result = subprocess.run(["git", "push"], capture_output=True, text=True)
        
        if push_result.returncode == 0:
            print("   ✅ Push completado exitosamente")
            
            # Mostrar información del push si está disponible
            if push_result.stdout:
                for line in push_result.stdout.strip().split('\n'):
                    if '->' in line or 'branch' in line.lower():
                        print(f"   📊 {line.strip()}")
        else:
            print("   ⚠️  Push encontró problemas")
            print(f"   📄 Salida: {push_result.stdout}")
            if push_result.stderr:
                print(f"   ❌ Error: {push_result.stderr.strip()}")
        
    except subprocess.CalledProcessError as e:
        print(f"\n❌ ERROR en operación Git (Código: {e.returncode})")
        print(f"   Comando: {e.cmd}")
        print(f"   Salida: {e.stdout}")
        print(f"   Error: {e.stderr}")
        
        print("\n💡 SOLUCIÓN RÁPIDA - Configurar Git:")
        print("   1. git config --global user.name 'Tu Nombre'")
        print("   2. git config --global user.email 'tu@email.com'")
        print("   3. git init  # Si no hay repositorio")
        print("   4. git remote add origin [url-del-repositorio]")
        
    except FileNotFoundError:
        print("\n❌ GIT NO ENCONTRADO")
        print("   Git no está instalado o no está en el PATH")
        print("   Descarga Git desde: https://git-scm.com/")
        
    except Exception as e:
        print(f"\n❌ ERROR INESPERADO: {e}")
        import traceback
        traceback.print_exc()

# Función auxiliar para solo hacer commit sin push
def solo_commit():
    """
    Solo hace commit sin push, útil para pruebas locales
    """
    try:
        base_path = r"C:\Users\Jorge Vasquez\Ranking"
        os.chdir(base_path)
        
        print("📁 Agregando archivo al staging...")
        subprocess.run(["git", "add", "."], check=True)
        
        print("💾 Creando commit local...")
        commit_message = f"Prueba local - {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"
        commit_res = subprocess.run(
            ["git", "commit", "-m", commit_message],
            capture_output=True,
            text=True
        )
        
        if commit_res.returncode != 0:
            print("❌ Falló el commit")
            print("STDOUT:", commit_res.stdout)
            print("STDERR:", commit_res.stderr)
            return
        
        print("✅ Commit local realizado")
        
    except Exception as e:
        print(f"❌ Error en commit local: {e}")

# Función para solo hacer push de commits existentes
def solo_push():
    """
    Solo hace push de commits existentes
    """
    try:
        base_path = r"C:\Users\Jorge Vasquez\Ranking"
        os.chdir(base_path)
        
        print("📤 Haciendo push de commits pendientes...")
        push_result = subprocess.run(["git", "push"], capture_output=True, text=True)
        
        if push_result.returncode == 0:
            print("✅ Push completado")
        else:
            print(f"⚠️  Error en push: {push_result.stderr}")
            
    except Exception as e:
        print(f"❌ Error en push: {e}")

# Función para ver estado sin modificar nada
def ver_estado_git():
    """
    Solo muestra el estado del repositorio
    """
    try:
        base_path = r"C:\Users\Jorge Vasquez\Ranking"
        os.chdir(base_path)
        
        print("📊 ESTADO DEL REPOSITORIO GIT")
        print("-" * 40)
        
        # Estado general
        subprocess.run(["git", "status"])
        
        print("\n📝 ÚLTIMOS COMMITS")
        print("-" * 40)
        subprocess.run(["git", "log", "--oneline", "-5"])
        
    except Exception as e:
        print(f"❌ Error al ver estado: {e}")

if __name__ == "__main__":
    realizar_git_operations()

🚀 INICIANDO OPERACIONES GIT

📋 Verificando estado del repositorio...
   ✅ Repositorio Git detectado

📁 Agregando cambios al staging area...
   ✅ Archivo 'index.html' agregado al staging

🔍 Verificando cambios pendientes...

💾 Creando commit...
   📝 Mensaje: 'Actualización automática - 2026-01-19 17:47:09'
   ✅ Commit realizado exitosamente

📤 Enviando cambios al repositorio remoto...
   ⚠️  Push encontró problemas
   📄 Salida: 
   ❌ Error: To https://github.com/jvasquez98/Ranking.git
 ! [rejected]        master -> master (fetch first)
error: failed to push some refs to 'https://github.com/jvasquez98/Ranking.git'
hint: Updates were rejected because the remote contains work that you do not
hint: have locally. This is usually caused by another repository pushing to
hint: the same ref. If you want to integrate the remote changes, use
hint: 'git pull' before pushing again.
hint: See the 'Note about fast-forwards' in 'git push --help' for details.
